#### PATRIC auto-generated model

Annotations converted via mackinac

In [1]:
# Import libraries - REQUIRES pip version 9.0.3
import pandas
import os
from os.path import join
import sys
import scipy.stats
import numpy
import math
import pickle
import copy
import time
import warnings
import gc

# Using Cobrapy 0.13.0
import cobra
import cobra.test
import cobra.flux_analysis.gapfilling
from cobra.io import write_sbml_model
from cobra.flux_analysis import  flux_variability_analysis
from cobra.flux_analysis.reaction import assess_component
from cobra.manipulation.delete import *
from cobra.flux_analysis.parsimonious import add_pfba
from cobra.medium import find_boundary_types
from cobra.util import solver as sutil
from cobra.flux_analysis.sampling import OptGPSampler

# Confidence levels in each annotation
#import probanno

# Import scripted python functions
sys.path.append('code/python/')
from read_excel import read_excel

# Estabish handler for logger
import logging
logging.basicConfig()
logger = logging.getLogger('logger')

# Verbose exception printing
%xmode

Exception reporting mode: Verbose


In [1]:
# Define functions

# Identify potentially gapfilled reactions
def findGapfilledRxn(model, exclude=[]):
    gapfilled = []
    transport = findTransports(model)
    if not type(exclude) is list:
        exclude = [exclude]
        
    for index in model.reactions:
        if len(list(index.genes)) == 0:
            if not index in model.boundary:
                if not index.id in exclude or not index.id in transport:
                    gapfilled.append(index.id)
    
    if len(gapfilled) > 0:
        print(str(len(gapfilled)) + ' metabolic reactions not associated with genes')
    
    return gapfilled

# Check for missing transport and exchange reactions
def missingRxns(model, extracellular='e'):

    transporters = set(findTransports(model))
    exchanges = set([x.id for x in model.exchanges])
    
    missing_exchanges = []
    missing_transports = []
    
    for metabolite in model.metabolites:
        if not metabolite.compartment == extracellular:
            continue

        curr_rxns = set([x.id for x in list(metabolite.reactions)])
        
        if not bool(curr_rxns & transporters):
            missing_transports.append(metabolite.id)
        if not bool(curr_rxns & exchanges):
            missing_exchanges.append(metabolite.id)
    
    if len(missing_transports) != 0:
        print(str(len(missing_transports)) + ' extracellular metabolites are missing transport reactions')
    if len(missing_exchanges) != 0:
        print(str(len(missing_exchanges)) + ' extracellular metabolites are missing exchange reactions')
        
    return missing_transports, missing_exchanges


# Checks which cytosolic metabolites are generated for free (bacteria only)
def checkFreeMass(raw_model, cytosol='cytosol'):
    
    with raw_model as model:
        
        # Close all exchanges
        for index in model.boundary:
            model.reactions.get_by_id(index.id).lower_bound = 0.
        
        # Identify all metabolites that are produced within the network
        demand_metabolites = [x.reactants[0].id for x in model.demands if len(x.reactants) > 0]
        demand_metabolites += [x.products[0].id for x in model.demands if len(x.products) > 0]

        free = []
        for index in model.metabolites: 
            if index.id in demand_metabolites:
                continue
            elif not index.compartment in cytosol:
                continue
            else:
                demand = model.add_boundary(index, type='demand')
                model.objective = demand
                obj_val = model.slim_optimize(error_value=0.)
                if obj_val > 1e-8:
                    free.append(index.id)
                model.remove_reactions([demand])
    
    if len(free) > 0:
        print(str(len(free)) + ' metabolites are generated for free')

    return(free)


# Check for mass and charge balance in reactions
def checkBalance(raw_model, exclude=[]):
    
    with raw_model as model:
        imbalanced = []
        mass_imbal = 0
        charge_imbal = 0
        elem_set = set()
        for metabolite in model.metabolites:
            try:
                elem_set |= set(metabolite.elements.keys())
            except:
                pass
        
        if len(elem_set) == 0:
            imbalanced = model.reactions
            mass_imbal = len(model.reactions)
            charge_imbal = len(model.reactions)
            print('No elemental data associated with metabolites!')
        
        else:
            if not type(exclude) is list: 
                exclude = [exclude]
            for index in model.reactions:
                if index in model.boundary or index.id in exclude:
                    continue

                else:
                    try:
                        test = index.check_mass_balance()
                    except ValueError:
                        continue

                    if len(list(test)) > 0:
                        imbalanced.append(index.id)

                        if 'charge' in test.keys():
                            charge_imbal += 1
                        if len(set(test.keys()).intersection(elem_set)) > 0:
                            mass_imbal += 1

    if mass_imbal != 0:
        print(str(mass_imbal) + ' reactions are mass imbalanced')
    if charge_imbal != 0:
        print(str(charge_imbal) + ' reactions are charge imbalanced')
    
    return(imbalanced)


# Checks the quality of models by a couple metrics and returns problems
def checkQuality(model, exclude=[], cytosol='c'):
    
    start_time = time.time()
    
    if model.name != None:
        model_name = model.name
    else:
        model_name = 'model'
    
    gaps = findGapfilledRxn(model, exclude)
    freemass = checkFreeMass(model, cytosol)
    balance = checkBalance(model, exclude)
    trans, exch = missingRxns(model)
    
    test = gaps + freemass + balance
    if len(test) == 0:
        print('No inconsistencies detected')
    
    duration = int(round(time.time() - start_time))
    print('Took ' + str(duration) + ' seconds to analyze ' + model_name) 

    return gaps, freemass, balance, trans, exch


# Trace back through reactions immediately adjacent to a given reaction to identify blocked precursor synthesis
def checkPrecursors(model, reaction):
    
    if isinstance(reaction, str) == True:
        reaction = model.reactions.get_by_id(reaction)
    
    model.objective = reaction
    obj_val = max(model.optimize(objective_sense='maximize').objective_value, abs(model.optimize(objective_sense='minimize').objective_value))

    if obj_val > 0.001:
        print('Able to produce all precursors for this reaction.')
        return None
    
    else:
        reactants = reaction.reactants
        check = 0
        
        for reactant in reactants:
            sub_reactions = list(reactant.reactions)
            
            for sub_reaction in sub_reactions:
                model.objective = sub_reaction
                obj_val = max(model.optimize(objective_sense='maximize').objective_value, abs(model.optimize(objective_sense='minimize').objective_value))

                if obj_val < 0.001 and reactant in sub_reaction.products:                    
                    print('Cannot acquire ' + str(reactant.id) + ' via ' + str(sub_reaction.id))
                elif obj_val < 0.001 and check < 1 and reactant in sub_reaction.reactants:
                    print(str(reactant.id) + ' not produced in any reactions.')
                    check += 1

#------------------------------------------------------------------------------------#

# Function to calculate doubling time from objective value
def doubling(model):
    
    with model as m:
        ov = m.slim_optimize()
        if ov < 1e-6:
            growth = 'No growth'
        else:
            growth = (1 / float(m.slim_optimize())) * 3600
        
            if growth < 60.0:
                growth = str(round(growth, 3)) + ' minutes'
            else:
                growth = growth / 60
                growth = str(round(growth, 3)) + ' hours'
            
    return growth


# Rough transcriptomic integration
def roughContextualize(model, transcript_profile, condition):
    
    orig_OV = model.optimize().objective_value

    model_context = copy.deepcopy(model)
    abundances = []
    
    with open(transcript_profile, 'r') as transcription:
        transcript_dict = {}
        abundances = []
        header = transcription.readline().strip().split(',')
        column = header.index(condition)
        for line in transcription:
            line = line.split(',')
            transcript_dict[line[0]] = float(line[column])
            abundances.append(float(line[column]))
    
    min_transcription = numpy.percentile(abundances, 50)
    penalty_bound = 10
    
    hits = 0
    fails = 0
    for gene in list(model_context.genes):
    
        gene = gene.name
    
        try:
            curr_rxns = list(model_context.genes.get_by_id(gene).reactions)
            hits += 1
        except KeyError:
            fails += 1
            continue
    
        try:
            curr_transcription = transcript_dict[gene]
        except KeyError:
            continue

        for reaction in curr_rxns:
            curr_id = reaction.id
        
            if curr_transcription >= min_transcription:            
                model_context.reactions.get_by_id(curr_id).lower_bound = -1000
                model_context.reactions.get_by_id(curr_id).upper_bound = 1000
            elif curr_transcription < min_transcription:
                if model_context.reactions.get_by_id(curr_id).lower_bound != 0:
                    model_context.reactions.get_by_id(curr_id).lower_bound = -penalty_bound
                if model_context.reactions.get_by_id(curr_id).upper_bound != 0:
                    model_context.reactions.get_by_id(curr_id).upper_bound = penalty_bound
                  

    #print('Gene hits across data types: ' + str(hits))
    #print('KeyErrors across data types: ' + str(fails) + '\n')
    new_OV = model_context.optimize().objective_value
    print('New objective value: ' + str(new_OV))
    print('Contextualized doubling time: ' + doubling(new_OV))
    
    return(model_context)


# Checks for availability of reactants of a given reaction
def availability(model, target_rxn):

    precursors = model.reactions.get_by_id(target_rxn).reactants
    
    total = 0
    unsuccessful = set()
    limited = set()
    for precursor in precursors:
        precursor_rxn = list(model.metabolites.get_by_id(precursor.id).reactions)
        
        for rxn in precursor_rxn:
            if rxn.id == target_rxn:
                continue
            elif precursor in model.reactions.get_by_id(rxn.id).reactants:
                
                model.objective = rxn
                obj_val = model.slim_optimize()
                if obj_val < 1e-8:
                    unsuccessful |= set([rxn.id])
                    limited |= set([precursor.id])
    
    print('Failed reactions: ' + str(len(unsuccessful)))
    print('Limiting reactants: ' + str(len(limited)))
    
    return unsuccessful, limited

# Removes all metabolites in a list of metabolite ids and all reactions associated with them
def removeAll(model, metabolite_list):
    
    new_model = copy.deepcopy(model)
    
    for metabolite in metabolite_list:
        try:
            metabolite = new_model.metabolites.get_by_id(metabolite)
            new_model.remove_reactions(metabolite.reactions)
            new_model.remove_metabolites([metabolite])
        except KeyError:
            print(metabolite + ' not found')
            continue
        
    return new_model


# Identify transport reactions (for any number compartments)
def findTransports(model):
    transporters = []
    compartments = set(list(model.compartments))
    if len(compartments) == 1:
        raise Exception('Model only has one compartment!')
    
    for reaction in model.reactions:
        
        reactant_compartments = set([x.compartment for x in reaction.reactants])
        product_compartments = set([x.compartment for x in reaction.products])
        reactant_baseID = set([x.id.split('_')[0] for x in reaction.reactants])
        product_baseID = set([x.id.split('_')[0] for x in reaction.products])
        
        if reactant_compartments == product_compartments and reactant_baseID != product_baseID:
            continue
        elif bool(compartments & reactant_compartments) == True and bool(compartments & product_compartments) == True:
            transporters.append(reaction.id)
        
    return(transporters)

# Removes a given percentage of reactions from a model, ignoring objective
def generate_gaps(model, percentage=0.2, prune=False, ignore=[]):
    number_to_remove = int(round(len(model.reactions) * percentage))
    rxn_ids = [x.id for x in model.reactions]
    random.shuffle(rxn_ids)
    rxns_to_remove = rxn_ids[-number_to_remove:]
        
    for rxn in ignore:
        try:
            rxns_to_remove.remove(rxn)
        except ValueError:
            continue
    
    truncated_model = copy.deepcopy(model)
    truncated_model.remove_reactions(rxns_to_remove)
    
    if prune == True:
        unused_cpds = prune_unused_metabolites(truncated_model)
    
    print('Reactions removed: ' + str(len(rxns_to_remove)))
    print('New objective value: ' + str(truncated_model.slim_optimize()))
    return truncated_model, rxns_to_remove


# Calculates the sum of fluxes for a given model
def sum_of_fluxes(model):
    
    with model as m:
        solution = m.optimize()
        flux_sum = sum(list(solution.fluxes))
        
    return flux_sum


# reports the reaction ids that are in 1 model but not the other
def rxn_diff(model_1, model_2):
    diff = set([str(x.id) for x in model_1.reactions]).difference([str(y.id) for y in model_2.reactions])
    return list(diff)

# Check for missing transport and exchange reactions
def missingRxn(model, extracellular='extracellular'):

    transporters = set(findTransports(model))
    exchanges = set([x.id for x in model.exchanges])
    
    missing_exchanges = []
    missing_transports = []
    
    for metabolite in model.metabolites:
        if not metabolite.compartment == extracellular:
            continue

        curr_rxns = set([x.id for x in list(metabolite.reactions)])
        
        if not bool(curr_rxns & transporters):
            missing_transports.append(metabolite.id)
        if not bool(curr_rxns & exchanges):
            missing_exchanges.append(metabolite.id)
    
    if len(missing_transports) != 0:
        print(str(len(missing_transports)) + ' extracellular metabolites are missing transport reactions')
    if len(missing_exchanges) != 0:
        print(str(len(missing_exchanges)) + ' extracellular metabolites are missing exchange reactions')
        
    return missing_transports, missing_exchanges

In [3]:
# Read in ModelSEED generated model
cd630_PATRIC = cobra.io.read_sbml_model('data/PATRIC/272563.8.xml')

In [4]:
# Set up counts for tracking changes
init_rxn = [str(x.id) for x in cd630_PATRIC.reactions]

new_genes = 0
new_rxns = 0
new_cpds = 0

In [5]:
cd630_PATRIC_gaps, cd630_PATRIC_free, cd630_PATRIC_imbal, cd630_PATRIC_trans, cd630_PATRIC_exch = checkQuality(cd630_PATRIC)

135 metabolic reactions not associated with genes
1 reactions are mass imbalanced
26 reactions are charge imbalanced
Took 27 seconds to analyze Clostridium difficile 630


In [6]:
# General curation

# Ensure oxygen exchange and transport exists, but leave anaerobic
cpd00532_e = cobra.Metabolite(
    'cpd00532_e',
    formula='O2',
    name='O2-', 
    compartment='extracellular')
cd630_PATRIC.add_boundary(cpd00532_e, type='exchange', reaction_id='EX_cpd00532_e', lb=0., ub=0.)
cpd00532_c = cd630_PATRIC.metabolites.get_by_id('cpd00532_c')
new_rxns += 1
new_cpds += 1

rxn09030_c = cobra.Reaction('rxn09030_c')
rxn09030_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED'}
rxn09030_c.name = 'O2 diffusion'
rxn09030_c.lower_bound = 0.
rxn09030_c.upper_bound = 1000.
rxn09030_c.add_metabolites({
    cpd00532_e: -1.0,
    cpd00532_c: 1.0
})
cd630_PATRIC.add_reactions([rxn09030_c])
new_rxns += 1

# Ensure free water diffusion across compartments
cd630_PATRIC.reactions.get_by_id('rxn05319_c').bounds = (-1000., 1000.)

# Rework ammonia correctly
cd630_PATRIC.metabolites.get_by_id('cpd00013_c').name = 'Ammonium'
#cd630_PATRIC.metabolites.get_by_id('cpd00013_c').formula = 'H3N'

# Remove old bio1 (generic Gram-positive Biomass function) and macromolecule demand reactions
cd630_PATRIC.remove_reactions(['rxn13783_c', 'rxn13784_c', 'rxn13782_c', 'bio1', 'SK_cpd11416_c'])

# Fix compartment labels to be more specific
for index in cd630_PATRIC.metabolites:
    index.notes = {'family': '', 'annotation': 'ModelSEED'}
    
    if index.compartment == 'c':
        index.compartment = 'cytosol'
    elif index.compartment == 'e':
        index.compartment = 'extracellular'

# Add structured note section to each gene
for index in cd630_PATRIC.genes:
    index.notes = {'annotation': 'PATRIC'}

# Parse and label reaction types (non-metabolic)
#likelihoods = pickle.load(open('data/PATRIC/272563.8.probs'))
for reaction in cd630_PATRIC.reactions:
    index.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
    
    if reaction in cd630_PATRIC.boundary:
        reaction.notes['type'] = 'exchange'
        continue
    substrates = [x.id.split('_')[-1] for x in reaction.products] + [x.id.split('_')[-1] for x in reaction.reactants]
    if 'e' in substrates and 'c' in substrates:
        reaction.notes['type'] = 'transport'
    
    #try:
    #    reaction.notes['confidence'] = likelihoods[reaction.id]
    #except:
    #    continue

# Save 2 transport reactions that should have genes associated 
cd630_PATRIC.reactions.get_by_id('rxn05669_c').gene_reaction_rule = 'CD630_12590' # Valine
rxn05669_c = cd630_PATRIC.reactions.get_by_id('rxn05669_c')
cd630_PATRIC.reactions.get_by_id('rxn05244_c').gene_reaction_rule = 'CD630_12590' # Isoleucine
rxn05244_c = cd630_PATRIC.reactions.get_by_id('rxn05244_c')
# Remove previously gapfilled metabolic reactions
cd630_PATRIC.remove_reactions(cd630_PATRIC_gaps)
cd630_PATRIC.add_reactions([rxn05669_c, rxn05244_c]) # Re-add saved reactions


cobra/core/model.py:613 UserWarning: bio1 not in 272563.8


In [7]:
# Deal with totally orphaned reactions that possess GPRs

#  Protein serine <=> [Protein]-D-serine
cd630_PATRIC.remove_reactions(['rxn06870_c'])

#  Ecgonine <=> Pseudoecgonine
cd630_PATRIC.remove_reactions(['rxn04561_c'])

#  Isopenicillin N <=> Penicillin N
cd630_PATRIC.remove_reactions(['rxn02896_c'])

#  Violaxanthin <=> Neoxanthin and 9-cis-Neoxanthin <=> 9-cis-Violaxanthin
cd630_PATRIC.remove_reactions(['rxn04757_c', 'rxn04760_c'])

# alpha-D-Hexose 1-phosphate <=> alpha-D-Hexose 6-phosphate (very generic)
cd630_PATRIC.remove_reactions(['rxn02379_c'])

# 3-Lysolecithin <=> 2-Lysolecithin
cd630_PATRIC.remove_reactions(['rxn02391_c'])

#-----------------------------------------------#

# Leaving 3 orphaned reactions with possible ties to known C. difficile metbolism

# Lantibiotic resistance
# rxn06517_c;  S-Alkyl-L-cysteine S-oxide <=> Dehydroalanine + Alkyl sulfenate

# Peptidoglycan synthesis
# rxn02975_c;  CDPparatose <=> CDPtyvelose
#rxn02737_c;  UDP-glucosamine <=> UDP-galactosamine

# Actually part of arabinose utilization
# araD; L-ribulose-5-phosphate 4-epimerase; CD630_22780
# rxn01292_c;  L-Arabinose <=> L-Ribulose

#-----------------------------------------------#

# Remove reactions with GPRs that are not actually present or detract from metabolic tasks

# L-galactose utilization
#cd630_PATRIC.remove_reactions(['rxn07493_c'])

# Selenium utiliztion
cd630_PATRIC.remove_reactions(['rxn03378_c','rxn02569_c'])

# Molybdenum import
cd630_PATRIC.remove_reactions(['rxn05619_c'])
#cpd11574_e = cd630_PATRIC.metabolites.get_by_id('cpd11574_e')
#cpd11574_c = cd630_PATRIC.metabolites.get_by_id('cpd11574_c')
#cd630_PATRIC.remove_metabolites([cpd11574_e, cpd11574_c]) # Molybdate

# Viomycin
cd630_PATRIC.remove_reactions(['rxn02576_c'])

#-----------------------------------------------#

# Useless macromolecules
cd630_PATRIC.remove_reactions(['rxn06197_c'])

In [8]:
# Components from minimal defined minimal media
# All need to feed directly into biomass

# Karasawa T, Ikoma S, Yamakawa K, Nakamura S. (1995).
# A defined growth medium for Clostridium difficile.
# Microbiology 141(2):371-375. 
# doi:10.1099/13500872-141-2-371

# D-Glucose
# cpd00027_e via rxn05226_c

#-------------------------------------------------------------------------#

# Minerals (already in place)

# cobalt; cpd00149_e (Co2+)
# manganese; cpd00030_e (Mn2+)
# magnesium, cpd00254_e (Mg)
# sodium; cpd00971_e (Na+)
# calcium; cpd00063_e (Ca2+)
# iron; cpd10515_e (Fe2+)
# potassium; cpd00205_e (K+)
# chloride, cpd00099_e (Cl-)

#-------------------------------------------------------------------------#

# Amino acids

# Handled in Stickland fermentation section
# isoleucine
# proline
# valine
# leucine

# Metabolites used in active tranport reactions
cpd00067_c = cd630_PATRIC.metabolites.get_by_id('cpd00067_c') # H+
cpd00002_c = cd630_PATRIC.metabolites.get_by_id('cpd00002_c') # ATP
cpd00008_c = cd630_PATRIC.metabolites.get_by_id('cpd00008_c') # ADP
cpd00001_c = cd630_PATRIC.metabolites.get_by_id('cpd00001_c') # H2O
cpd00009_c = cd630_PATRIC.metabolites.get_by_id('cpd00009_c') # Phosphate

# Cysteine
cpd00084_e = cobra.Metabolite(
    'cpd00084_e',
    formula='C3H7NO2S',
    name='L-Cysteine', 
    compartment='extracellular')
cd630_PATRIC.add_boundary(cpd00084_e, type='exchange', reaction_id='EX_cpd00084_e', lb=-1000.0, ub=1000.0)
cpd00084_c = cd630_PATRIC.metabolites.get_by_id('cpd00084_c') # L-Cysteine (cytosolic)
new_rxns += 1
new_cpds += 1

CD630_07500 = cobra.Gene('CD630_07500')
CD630_07500.name = 'ABC-type transport system, aminoacid-family extracellular solute-binding protein'
CD630_07500.notes = {'annotation': 'UniProt'}
new_genes += 1

rxn05237_c = cobra.Reaction('rxn05237_c')
rxn05237_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED', 'subsystem': 'ABC transporter'}
rxn05237_c.gene_reaction_rule = 'CD630_07500'
rxn05237_c.notes['citations'].append('Karasawa T, Ikoma S, Yamakawa K, Nakamura S. Microbiology 141(2):371-375 doi:10.1099/13500872-141-2-371')
rxn05237_c.name = 'ATP phosphohydrolase (polar-amino-acid-importing)'
rxn05237_c.lower_bound = 0.
rxn05237_c.upper_bound = 1000.
rxn05237_c.add_metabolites({
    cpd00001_c: -1.0, # H2O
    cpd00002_c: -1.0, # ATP
    cpd00084_e: -1.0,
    cpd00008_c: 1.0, # ADP
    cpd00009_c: 1.0, # Phosphate
    cpd00067_c: 1.0, # H+
    cpd00084_c: 1.0
})
cd630_PATRIC.add_reactions([rxn05237_c])
new_rxns += 1

# Trypotophan
cd630_PATRIC.reactions.get_by_id('rxn05663_c').lower_bound = 0.

# Methionine
# Correct in model already

#-------------------------------------------------------------------------#

# Vitamins

# Biotin (BIOT) - exchange and transport already in model
# cpd00104_e -> rxn05223_c -> cpd00104_c
cd630_PATRIC.remove_reactions([cd630_PATRIC.reactions.get_by_id('rxn08180_c')]) # remove sythesis step

# Pantothenate
cpd00644_e = cobra.Metabolite(
    'cpd00644_e',
    formula='C9H16NO5',
    name='PAN', # pantothenate
    compartment='extracellular')
cd630_PATRIC.add_boundary(cpd00644_e, type='exchange', reaction_id='EX_cpd00644_e', lb=-1000.0, ub=1000.0)
new_rxns += 1
cpd00644_c = cd630_PATRIC.metabolites.get_by_id('cpd00644_c') # pantothenate (cytosolic)

# transport via panT - http://jb.asm.org/content/191/1/42.full
R03018_c = cobra.Reaction('R03018_c')
R03018_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'KEGG', 'subsystem': 'ABC transporter'}
R03018_c.gene_reaction_rule = '272563.8.peg.164'
R03018_c.notes['citations'].append('Karasawa T, Ikoma S, Yamakawa K, Nakamura S. Microbiology 141(2):371-375 doi:10.1099/13500872-141-2-371')
R03018_c.name = 'ATP:pantothenate 4-phosphotransferase'
R03018_c.lower_bound = 0.
R03018_c.upper_bound = 1000.
R03018_c.add_metabolites({
    cpd00001_c: -1.0, # H2O
    cpd00002_c: -1.0, # ATP
    cpd00644_e: -1.0,
    cpd00008_c: 1.0, # ADP
    cpd00009_c: 1.0, # Phosphate
    cpd00067_c: 1.0, # H+
    cpd00644_c: 1.0
})
cd630_PATRIC.add_reactions([R03018_c])
new_rxns += 1
new_cpds += 1

# Pyridoxine
cpd00263_e = cobra.Metabolite(
    'cpd00263_e',
    formula='C8H11NO3',
    name='Pyridoxol', # Pyridoxine
    compartment='extracellular')
cd630_PATRIC.add_boundary(cpd00263_e, type='exchange', reaction_id='EX_cpd00263_e', lb=-1000.0, ub=1000.0)
new_rxns += 1
cpd00263_c = cd630_PATRIC.metabolites.get_by_id('cpd00263_c') # 

COG3601_c = cobra.Reaction('COG3601_c')
COG3601_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'COG', 'subsystem': 'ABC transporter'}
COG3601_c.gene_reaction_rule = '272563.8.peg.164'
COG3601_c.notes['citations'].append('Karasawa T, Ikoma S, Yamakawa K, Nakamura S. Microbiology 141(2):371-375 doi:10.1099/13500872-141-2-371')
COG3601_c.name = 'Pyridoxine active transporter'
COG3601_c.lower_bound = 0.
COG3601_c.upper_bound = 1000.
COG3601_c.add_metabolites({
    cpd00001_c: -1.0, # H2O
    cpd00002_c: -1.0, # ATP
    cpd00263_e: -1.0,
    cpd00008_c: 1.0, # ADP
    cpd00009_c: 1.0, # Phosphate
    cpd00067_c: 1.0, # H+
    cpd00263_c: 1.0
})
cd630_PATRIC.add_reactions([COG3601_c])
new_rxns += 1
new_cpds += 1
# cpd00478_c	Pyridoxine phosphate (blocked) intracellular

# Folate
cpd00393_e = cd630_PATRIC.metabolites.get_by_id('cpd00393_e')
cpd00393_c = cd630_PATRIC.metabolites.get_by_id('cpd00393_c')
cpd00067_c = cd630_PATRIC.metabolites.get_by_id('cpd00067_c') # H+
cpd00067_e = cd630_PATRIC.metabolites.get_by_id('cpd00067_e') # H+

rxn05255_c = cobra.Reaction('rxn05255_c')
rxn05255_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED'}
rxn05255_c.name = 'Folate transport via proton simport'
rxn05255_c.gene_reaction_rule = '272563.8.peg.164'
rxn05255_c.lower_bound = 0.
rxn05255_c.upper_bound = 1000.
rxn05255_c.add_metabolites({
    cpd00393_e: -1.0,
    cpd00067_e: -1.0,
    cpd00067_c: 1.0,
    cpd00393_c: 1.0
})
cd630_PATRIC.add_reactions([rxn05255_c])
new_rxns += 1


In [9]:
# TCA cycle - fragmented in strain 630
cpd00106_c = cd630_PATRIC.metabolites.get_by_id('cpd00106_c') # Fumarate
cpd00001_c = cd630_PATRIC.metabolites.get_by_id('cpd00001_c') # H2O
cpd00130_c = cd630_PATRIC.metabolites.get_by_id('cpd00130_c') # L-Malate

CD630_10030 = cobra.Gene('CD630_10030') # Define gene
CD630_10030.name = 'fumA; fumarate hydratase subunit A [KO:K01677] [EC:4.2.1.2]'
CD630_10030.notes = {'annotation': 'UniProt'}
CD630_10040 = cobra.Gene('CD630_10040') # Define gene
CD630_10040.name = 'fumA; fumarate hydratase subunit B [KO:K01678] [EC:4.2.1.2]'
CD630_10040.notes = {'annotation': 'UniProt'}
new_genes += 1
new_genes += 1

rxn00799_c = cobra.Reaction('rxn00799_c') 
rxn00799_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED', 'subsystem': 'TCA cycle'}
rxn00799_c.gene_reaction_rule = 'CD630_10030 and CD630_10040'
rxn00799_c.notes['citations'] += ['Dannheim, H., Will, S. E., Schomburg, D., & Neumann‐Schaal, M. (2017). Clostridioides difficile 630Δerm in silico and in vivo – quantitative growth and extensive polysaccharide secretion. FEBS Open Bio, 7(4), 602–615.', 'Dannheim, H., Riedel, T., Neumann-Schaal, M., Bunk, B., Schober, I., Spröer, C., & Schomburg, D. (2017). Manual curation and reannotation of the genomes of Clostridium difficile 630Δerm and C. difficile 630. Journal of Medical Microbiology.']
rxn00799_c.name = 'Fumarate hydratase'
rxn00799_c.lower_bound = -1000.
rxn00799_c.upper_bound = 1000.
rxn00799_c.add_metabolites({
    cpd00106_c: -1.0,
    cpd00001_c: -1.0,
    cpd00130_c: 1.0
})
cd630_PATRIC.add_reactions([rxn00799_c])
new_rxns += 1

cpd00041_c = cd630_PATRIC.metabolites.get_by_id('cpd00041_c') # L-aspartate
cpd00036_c = cd630_PATRIC.metabolites.get_by_id('cpd00036_c') # Succinate
cpd00067_c = cd630_PATRIC.metabolites.get_by_id('cpd00067_c') # H+
cpd03470_c = cd630_PATRIC.metabolites.get_by_id('cpd03470_c') # Iminoaspartate

CD630_23710 = cobra.Gene('CD630_23710') # Define gene
CD630_23710.name = 'L-aspartate oxidase (Quinolinate synthetase B)'
CD630_23710.notes = {'annotation': 'UniProt'}
new_genes += 1

rxn08168_c = cobra.Reaction('rxn08168_c')
rxn08168_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED', 'subsystem': 'TCA cycle'}
rxn08168_c.gene_reaction_rule = 'CD630_23710'
rxn08168_c.notes['citations'] += ['Dannheim, H., Will, S. E., Schomburg, D., & Neumann‐Schaal, M. (2017). Clostridioides difficile 630Δerm in silico and in vivo – quantitative growth and extensive polysaccharide secretion. FEBS Open Bio, 7(4), 602–615.', 'Dannheim, H., Riedel, T., Neumann-Schaal, M., Bunk, B., Schober, I., Spröer, C., & Schomburg, D. (2017). Manual curation and reannotation of the genomes of Clostridium difficile 630Δerm and C. difficile 630. Journal of Medical Microbiology.']
rxn08168_c.name = 'L-aspartate:fumarate oxidoreductase (deaminating)'
rxn08168_c.lower_bound = -1000.
rxn08168_c.upper_bound = 1000.
rxn08168_c.add_metabolites({
    cpd00041_c: -1.0,
    cpd00106_c: -1.0,
    cpd00067_c: -1.0,
    cpd00036_c: 1.0,
    cpd03470_c: 1.0
})
cd630_PATRIC.add_reactions([rxn08168_c])
new_rxns += 1


In [10]:
# Oxygen detoxification

# Turn off O2 exchange (anaerobc environment)
cd630_PATRIC.reactions.get_by_id('EX_cpd00007_e').lower_bound = 0. 

# Diffusion reaction: rxn05468_c

# Remove extraneous aerobic reactions 
cd630_PATRIC.remove_reactions(['rxn00333_c', 'rxn00262_c', 'rxn00338_c', 'rxn02303_c'])
# Keep 'rxn00206_c', 'rxn01600_c', and 'rxn00333_c' due to possible presence for detoxification

# Add other detoxification reactions
cpd00001_c = cd630_PATRIC.metabolites.get_by_id('cpd00001_c') # H2O
cpd00023_c = cd630_PATRIC.metabolites.get_by_id('cpd00023_c') # L-Glutamate
cpd00003_c = cd630_PATRIC.metabolites.get_by_id('cpd00003_c') # NAD
cpd00004_c = cd630_PATRIC.metabolites.get_by_id('cpd00004_c') # NADH
cpd00024_c = cd630_PATRIC.metabolites.get_by_id('cpd00024_c') # 2-Oxoglutarate
cpd00067_c = cd630_PATRIC.metabolites.get_by_id('cpd00067_c') # H+
cpd00013_c = cd630_PATRIC.metabolites.get_by_id('cpd00013_c') # Ammonia

CD630_01790 = cobra.Gene('CD630_01790')
CD630_01790.name = 'gluD; Glutamate dehydrogenase (NAD)'
CD630_01790.notes = {'annotation': 'UniProt'}
new_genes += 1

rxn30075_c = cobra.Reaction('rxn30075_c')
rxn30075_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED', 'subsystem': 'Oxygen detoxification'}
rxn30075_c.name = 'Glutamate dehydrogenase (NAD)'
rxn30075_c.gene_reaction_rule = 'CD630_01790'
rxn30075_c.lower_bound = -1000.
rxn30075_c.upper_bound = 1000.
rxn30075_c.notes['citations'] += 'Dannheim, H., Riedel, T., Neumann-Schaal, M., Bunk, B., Schober, I., Spröer, C., & Schomburg, D. (2017). Manual curation and reannotation of the genomes of Clostridium difficile 630Δerm and C. difficile 630. Journal of Medical Microbiology.'
rxn30075_c.add_metabolites({
    cpd00001_c: -1.0,
    cpd00003_c: -1.0,
    cpd00023_c: -1.0,
    cpd00024_c: 1.0,
    cpd00004_c: 1.0,
    cpd00067_c: 1.0,
    cpd00013_c: 1.0
})
cd630_PATRIC.add_reactions([rxn30075_c])
new_rxns += 1

In [11]:
# Nucleic acids

# Thymine biosynthesis
cpd00009_c = cd630_PATRIC.metabolites.get_by_id('cpd00009_c') # Phosphate
cpd00184_c = cd630_PATRIC.metabolites.get_by_id('cpd00184_c') # Thymidine
cpd00509_c = cd630_PATRIC.metabolites.get_by_id('cpd00509_c') # deoxyribose-1-phosphate
cpd00151_c = cobra.Metabolite(
    'cpd00151_c',
    formula='C5H6N2O2',
    name='Thymine',
    compartment='cytosol')

CD630_12250 = cobra.Gene('CD630_12250') 
CD630_12250.name = 'pdp; Pyrimidine-nucleoside phosphorylase'
CD630_12250.notes = {'annotation': 'UniProt'}

rxn27328_c = cobra.Reaction('rxn27328_c')
rxn27328_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED', 'subsystem': 'Thymine biosynthesis'}
rxn27328_c.gene_reaction_rule = 'CD630_12250'
rxn27328_c.name = 'Thymidine-orthophosphate deoxyribosyltransferase'
rxn27328_c.lower_bound = -1000.
rxn27328_c.upper_bound = 1000.
rxn27328_c.add_metabolites({
    cpd00009_c: -1.0,
    cpd00184_c: -1.0,
    cpd00509_c: 1.0,
    cpd00151_c: 1.0
})
cd630_PATRIC.add_reactions([rxn27328_c])
new_rxns += 1

# Future addition:
# Linkage between toxin production and purine biosynthesis in Clostridium difficile
# http://jmm.microbiologyresearch.org/content/journal/jmm/10.1099/0022-1317-51-1-34#tab2


In [12]:
# Host-derived glycans

# Stiemsma, L. T., Turvey, S. E., & Finlay, B. B. (2014). An antibiotic-altered microbiota 
# provides fuel for the enteric foe. Cell Research, 24(1), 5–6. http://doi.org/10.1038/cr.2013.142

# N-Acetyl neuraminic acid import
cpd00232_e = cobra.Metabolite(
    'cpd00232_e',
    formula='C11H18NO9',
    name='Neu5Ac',
    compartment='extracellular')
cd630_PATRIC.add_boundary(cpd00232_e, type='exchange', reaction_id='EX_cpd00232_e', lb=-1000.0, ub=1000.0)
new_rxns += 1
cpd00067_c = cd630_PATRIC.metabolites.get_by_id('cpd00067_c') # H+
cpd00067_e = cd630_PATRIC.metabolites.get_by_id('cpd00067_e') # H+
cpd00232_c = cd630_PATRIC.metabolites.get_by_id('cpd00232_c') # Neu5Ac (intracellular)

CD630_24170 = cobra.Gene('CD630_24170') # Define gene
CD630_24170.name = 'PTS system, Sorbitol-like IIB component'
CD630_24170.notes = {'annotation': 'UniProt'}
CD630_24180 = cobra.Gene('CD630_24180') # Define gene
CD630_24180.name = 'PTS system, Sorbitol-like IIC component'
CD630_24180.notes = {'annotation': 'UniProt'}

rxn29571_c = cobra.Reaction('rxn29571_c')
rxn29571_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED', 'subsystem': 'PTS system'}
rxn29571_c.gene_reaction_rule = 'CD630_24170 and CD630_24180'
rxn29571_c.name = 'Neu5Ac H+ symporter'
rxn29571_c.lower_bound = -1000.
rxn29571_c.upper_bound = 1000.
rxn29571_c.add_metabolites({
    cpd00067_e: -1.0,
    cpd00232_e: -1.0,
    cpd00067_c: 1.0,
    cpd00232_c: 1.0
})
cd630_PATRIC.add_reactions([rxn29571_c])
new_rxns += 1
# North RA, Kessans SA, Griffin MDW, et al. Cloning, expression, purification, crystallization 
# and preliminary X-ray diffraction analysis of N-acetylmannosamine-6-phosphate 2-epimerase 
# from methicillin-resistant Staphylococcus aureus. Acta Crystallogr F Struct Biol Commun. 
# 2014;70:650–655. doi: 10.1107/S2053230X14007171.

# Sialic Acid Catabolism in Staphylococcus aureus
# Michael E. Olson, Jessica M. King, Timothy L. Yahr and Alexander R. Horswill 
# http://jb.asm.org/content/195/8/1779.full
cpd00492_e = cobra.Metabolite(
    'cpd00492_e',
    formula='C8H15NO6',
    name='N-Acetyl-D-mannosamine',
    compartment='extracellular')
cd630_PATRIC.add_boundary(cpd00492_e, type='exchange', reaction_id='EX_cpd00492_e', lb=-1000.0, ub=1000.0)
new_rxns += 1
cpd00492_c = cd630_PATRIC.metabolites.get_by_id('cpd00492_c') # N-Acetylmannosamine

rxn05486_c = cobra.Reaction('rxn05486_c')
rxn05486_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED', 'subsystem': 'PTS system'}
rxn05486_c.gene_reaction_rule = 'CD630_24170 and CD630_24180'
rxn05486_c.name = 'N-Acetylmannosamine H+ symporter'
rxn05486_c.lower_bound = 0.
rxn05486_c.upper_bound = 1000.
rxn05486_c.add_metabolites({
    cpd00492_e: -1.0,
    cpd00067_e: -1.0,
    cpd00492_c: 1.0,
    cpd00067_c: 1.0
})
cd630_PATRIC.add_reactions([rxn05486_c])
new_rxns += 1
# # N-Acetyl-D-glucosamine -> GlcN conversion (nagA)
cpd00293_c = cd630_PATRIC.metabolites.get_by_id('cpd00293_c') # N-Acetyl-D-glucosamine 6-phosphate
cpd00288_c = cd630_PATRIC.metabolites.get_by_id('cpd00288_c') # D-Glucosamine 6-phosphate
cpd00029_c = cd630_PATRIC.metabolites.get_by_id('cpd00029_c') # Acetate
cpd00001_c = cd630_PATRIC.metabolites.get_by_id('cpd00001_c') # AH2O

CD630_10100 = cobra.Gene('CD630_10100') # Define gene
CD630_10100.name = 'nagA, N-Acetyl-D-glucosamine-6-phosphate amidohydrolase'
CD630_10100.notes = {'annotation': 'UniProt'}

rxn01484_c = cobra.Reaction('rxn01484_c') # GlcNAc 6-P deacetylase
rxn01484_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn01484_c.gene_reaction_rule = 'CD630_10100'
rxn01484_c.name = 'N-Acetyl-D-glucosamine-6-phosphate amidohydrolase'
rxn01484_c.lower_bound = 0.
rxn01484_c.upper_bound = 1000.
rxn01484_c.add_metabolites({
    cpd00001_c: -1.0,
    cpd00293_c: -1.0,
    cpd00288_c: 1.0,
    cpd00029_c: 1.0
})
cd630_PATRIC.add_reactions([rxn01484_c])
new_rxns += 1
# Engevik MA, Yacyshyn MB, Engevik KA, et al. Human Clostridium difficile infection: 
# altered mucus production and composition. American Journal of Physiology - 
# Gastrointestinal and Liver Physiology. 2015;308(6):G510-G524. doi:10.1152/ajpgi.00091.2014.


CD630_30130 = cobra.Gene('CD630_30130')
CD630_30130.name = 'PTS system, IIC component'
CD630_30130.notes = {'annotation': 'UniProt'}
CD630_30140 = cobra.Gene('CD630_30134')
CD630_30140.name = 'PTS system, IIB component'
CD630_30140.notes = {'annotation': 'UniProt'}
CD630_30150 = cobra.Gene('CD630_30130')
CD630_30150.name = 'PTS system, IIA component'
CD630_30150.notes = {'annotation': 'UniProt'}

cpd00751_e = cobra.Metabolite(
    'cpd00751_e',
    formula='C6H12O5',
    name='L-Fucose',
    compartment='extracellular')
cd630_PATRIC.add_boundary(cpd00751_e, type='exchange', reaction_id='EX_cpd00751_e', lb=-1000.0, ub=1000.0)
new_rxns += 1
cd630_PATRIC.reactions.get_by_id('rxn02262_c').remove_from_model() # remove incorrect transport reaction

cpd00002_c = cd630_PATRIC.metabolites.get_by_id('cpd00002_c') # ATP
cpd00008_c = cd630_PATRIC.metabolites.get_by_id('cpd00008_c') # ADP
cpd00067_c = cd630_PATRIC.metabolites.get_by_id('cpd00067_c') # H+
cpd01912_c = cd630_PATRIC.metabolites.get_by_id('cpd01912_c') # L-Fucose 1-phosphate

rxn02262_c = cobra.Reaction('rxn02262_c')
rxn02262_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn02262_c.gene_reaction_rule = '272563.8.peg.2384'
rxn02262_c.name = 'ATP:6-deoxy-L-galactose 1-phosphotransferase'
rxn02262_c.lower_bound = 0.
rxn02262_c.upper_bound = 1000.
rxn02262_c.add_metabolites({
    cpd00002_c: -1.0,
    cpd00751_e: -1.0,
    cpd00008_c: 1.0,
    cpd00067_c: 1.0,
    cpd01912_c: 1.0 
})
cd630_PATRIC.add_reactions([rxn02262_c])
new_rxns += 1
# Fucose utilization
cpd00334_c = cobra.Metabolite(
    'cpd00334_c',
    formula='C3H6O2',
    name='L-Lactaldehyde',
    compartment='cytosol')
cpd00095_c = cd630_PATRIC.metabolites.get_by_id('cpd00095_c') # Glycerone-phosphate

CD630_00450 = cobra.Gene('CD630_00450')
CD630_00450.name = 'L-fuculose-phosphate aldolase [EC:4.1.2.17]'
CD630_00450.notes = {'annotation': 'UniProt'}
CD630_22770 = cobra.Gene('CD630_22770')
CD630_22770.name = 'L-fuculose-phosphate aldolase [EC:4.1.2.17]'
CD630_22770.notes = {'annotation': 'UniProt'}

rxn01620_c = cobra.Reaction('rxn01620_c')
rxn01620_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn01620_c.gene_reaction_rule = 'CD630_00450 or CD630_22770'
rxn01620_c.name = 'L-Fuculose-1-phosphate lactaldehyde-lyase'
rxn01620_c.lower_bound = -1000.
rxn01620_c.upper_bound = 1000.
rxn01620_c.add_metabolites({
    cpd01912_c: -1.0,
    cpd00095_c: 1.0,
    cpd00334_c: 1.0 
})
cd630_PATRIC.add_reactions([rxn01620_c])
new_rxns += 1
cpd00003_c = cd630_PATRIC.metabolites.get_by_id('cpd00003_c') # NAD
cpd00004_c = cd630_PATRIC.metabolites.get_by_id('cpd00004_c') # NADH
cpd00428_c = cd630_PATRIC.metabolites.get_by_id('cpd00428_c') # 2-Oxopropanal 

CD630_35150 = cobra.Gene('CD630_35150')
CD630_35150.name = 'bifunctional UDP-N-acetylglucosamine pyrophosphorylase / Glucosamine-1-phosphate N-acetyltransferase [EC:2.7.7.23 2.3.1.157]'
CD630_35150.notes = {'annotation': 'KEGG'}

rxn04937_c = cobra.Reaction('rxn04937_c')
rxn04937_c.notes = {'citations': ['North, R. A., et al.. (2018). “Just a spoonful of sugar...”: import of sialic acid across bacterial cell membranes. Biophysical Reviews, 10(2), 219–227.'], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn04937_c.gene_reaction_rule = 'CD630_35150'
rxn04937_c.name = 'Lactaldehyde:NAD+ oxidoreductase'
rxn04937_c.lower_bound = -1000.
rxn04937_c.upper_bound = 1000.
rxn04937_c.add_metabolites({
    cpd00334_c: -1.0,
    cpd00003_c: -1.0,
    cpd00004_c: 1.0,
    cpd00067_c: 1.0,
    cpd00428_c: 1.0 
})
cd630_PATRIC.add_reactions([rxn04937_c])
new_rxns += 1
# Galactose omitted because of Nakamura, et al. (1982).

# Mannose import and utilization already intact...


In [13]:
# Carbohydrate import (fermentation steps taken care of in SCFA steps)
# Nakamura, et al. (1982). Carbohydrate Fermentation by Clostridium difficile. Microbial. Immunol. 26 (2); 107-111.

# Already included:
# D-Fructose 	C6H12O6 	cpd00082
# D-Glucose 	C6H12O6 	cpd00027
# D-Mannitol 	C6H14O6 	cpd00314
# Sorbitol 	C6H14O6 	cpd00588
# D-Ribose 	C5H10O5 	cpd00105 
# Salicin 	C13H18O7 	cpd01030
# Trehalose (TRHL) 	C12H22O11 	cpd00794_e 

cpd05161_e = cobra.Metabolite(
    'cpd05161_e',
    formula='C18H32O16',
    name='Melezitose',
    compartment='extracellular')
cd630_PATRIC.add_boundary(cpd05161_e, type='exchange', reaction_id='EX_cpd05161_e', lb=-1000.0, ub=1000.0)
new_rxns += 1
cpd05161_c = cobra.Metabolite(
    'cpd05161_c',
    formula='C18H32O16',
    name='Melezitose',
    compartment='cytosol')
cpd00067_e = cd630_PATRIC.metabolites.get_by_id('cpd00067_e') # H+
cpd00067_c = cd630_PATRIC.metabolites.get_by_id('cpd00067_c') # H+

rxn12619_c = cobra.Reaction('rxn12619_c')
rxn12619_c.gene_reaction_rule = 'CD630_24170 and CD630_24180'
rxn12619_c.name = 'Melezitose transport via proton symport'
rxn12619_c.lower_bound = -1000.
rxn12619_c.upper_bound = 1000.
rxn12619_c.add_metabolites({
    cpd00067_e: -1.0,
    cpd05161_e: -1.0,
    cpd00067_c: 1.0,
    cpd05161_c: 1.0 
})
cd630_PATRIC.add_reactions([rxn12619_c])
new_rxns += 1
cpd00001_c = cd630_PATRIC.metabolites.get_by_id('cpd00001_c') # H2O
cpd00002_c = cd630_PATRIC.metabolites.get_by_id('cpd00002_c') # ATP
cpd00008_c = cd630_PATRIC.metabolites.get_by_id('cpd00008_c') # ADP
cpd00009_c = cd630_PATRIC.metabolites.get_by_id('cpd00009_c') # Phosphate

rxn18524_c = cobra.Reaction('rxn18524_c')
rxn01620_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn01620_c.notes['citations'].append('Nakamura, et al. (1982). Carbohydrate Fermentation by Clostridium difficile. Microbial. Immunol. 26 (2); 107-111.')
rxn18524_c.name = 'ATP phosphohydrolase (disaccharide-importing)'
rxn18524_c.gene_reaction_rule = 'CD630_30130 and CD630_30140 and CD630_30150'
rxn18524_c.lower_bound = 0.
rxn18524_c.upper_bound = 1000.
rxn18524_c.add_metabolites({
    cpd05161_e: -1.0,
    cpd00001_c: -1.0,
    cpd00002_c: -1.0,
    cpd00008_c: 1.0,
    cpd00009_c: 1.0,
    cpd00067_c: 1.0,
    cpd05161_c: 1.0 
})
cd630_PATRIC.add_reactions([rxn18524_c])
new_rxns += 1
T4BV06_CLODI = cobra.Gene('T4BV06_CLODI')
T4BV06_CLODI.name = 'melA; alpha-galactosidase [EC 3.2.1.22]'
T4BV06_CLODI.notes = {'annotation': 'KEGG'}

# Cleavage to glucose and turanose the latter of which is an isomer of sucrose. 
cpd20885_c = cobra.Metabolite(
    'cpd20885_c',
    formula='C12H22O11',
    name='Turanose',
    compartment='cytosol')
cpd00027_c = cd630_PATRIC.metabolites.get_by_id('cpd00027_c') # D-Glucose

R01103_c = cobra.Reaction('R01103_c')
R01103_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'KEGG'}
R01103_c.notes['citations'].append('Nakamura, et al. (1982). Carbohydrate Fermentation by Clostridium difficile. Microbial. Immunol. 26 (2); 107-111.')
R01103_c.name = 'Raffinose galactohydrolase'
R01103_c.gene_reaction_rule = 'T4BV06_CLODI'
R01103_c.lower_bound = 0.
R01103_c.upper_bound = 1000.
R01103_c.add_metabolites({
    cpd00001_c: -1.0,
    cpd05161_c: -1.0,
    cpd00027_c: 1.0,
    cpd20885_c: 1.0 
})
cd630_PATRIC.add_reactions([R01103_c])
new_rxns += 1
# Turanose isomerization to sucrose - spontaneous
cpd00076_c = cd630_PATRIC.metabolites.get_by_id('cpd00076_c') # Sucrose
ID010_c = cobra.Reaction('ID010_c')
ID010_c.name = 'Turanose isomerization'
ID010_c.lower_bound = -1000.
ID010_c.upper_bound = 1000.
ID010_c.add_metabolites({
    cpd20885_c: -1.0,
    cpd00076_c: 1.0 
})
cd630_PATRIC.add_reactions([ID010_c])
new_rxns += 1

In [14]:
# Amino acid exchange and transport

# Branched chain - facilitated diffusion
cpd00067_e = cd630_PATRIC.metabolites.get_by_id('cpd00067_e') # H+ (extracellular)
cpd00067_c = cd630_PATRIC.metabolites.get_by_id('cpd00067_c') # H+ (cytosolic)

# Leucine
cpd00107_e = cobra.Metabolite(
    'cpd00107_e',
    formula='C6H13NO2',
    name='L-Leucine',
    compartment='extracellular')
cd630_PATRIC.add_boundary(cpd00107_e, type='exchange', reaction_id='EX_cpd00107_e', lb=-1000.0, ub=1000.0)
new_rxns += 1
cpd00107_c = cd630_PATRIC.metabolites.get_by_id('cpd00107_c') # Leucine (cytosol)

CD630_12590 = cobra.Gene('CD630_12590')
CD630_12590.name = 'brnQ-1; Branched-chain amino acid transport system carrier protein'
CD630_12590.notes = {'annotation': 'KEGG'}

rxn05243_c = cobra.Reaction('rxn05243_c')
rxn05243_c.gene_reaction_rule = 'CD630_12590'
rxn05243_c.name = 'L-Leucine hydrogen symport'
rxn05243_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED'}
rxn05243_c.notes['citations'] += 'Driessen, A. J., Ubbink-Kok, T., & Konings, W. N. (1988). Amino acid transport by membrane vesicles of an obligate anaerobic bacterium, Clostridium acetobutylicum. Journal of Bacteriology, 170(2), 817–820.'
rxn05243_c.lower_bound = 0.
rxn05243_c.upper_bound = 1000.
rxn05243_c.add_metabolites({
    cpd00107_e: -1.0,
    cpd00067_e: -1.0,
    cpd00107_c: 1.0,
    cpd00067_c: 1.0
})
cd630_PATRIC.add_reactions([rxn05243_c])
new_rxns += 1
# Valine
# Extracellular exchange and transport in place, needs correct GPR
cd630_PATRIC.reactions.get_by_id('rxn05669_c').gene_reaction_rule = 'CD630_12590'

# Isoleucine
# Extracellular exchange and tranport in place, needs correct GPR
cd630_PATRIC.reactions.get_by_id('rxn05244_c').gene_reaction_rule = 'CD630_12590'

#----------------#

# Aromatic - active transport
cpd00002_c = cd630_PATRIC.metabolites.get_by_id('cpd00002_c') # ATP
cpd00008_c = cd630_PATRIC.metabolites.get_by_id('cpd00008_c') # ADP
cpd00009_c = cd630_PATRIC.metabolites.get_by_id('cpd00009_c') # Phosphate
cpd00001_c = cd630_PATRIC.metabolites.get_by_id('cpd00001_c') # H2O

CD630_08730 = cobra.Gene('CD630_08730')
CD630_08730.name = 'putative ABC transport system substrate-binding protein'
CD630_08730.notes = {'annotation': 'KEGG'}
CD630_08760 = cobra.Gene('CD630_08760')
CD630_08760.name = 'putative ABC transport system substrate-binding protein'
CD630_08760.notes = {'annotation': 'KEGG'}


# Tyrosine
cpd00069_e = cobra.Metabolite(
    'cpd00069_e',
    formula='C9H11NO3',
    name='L-Tyrosine',
    compartment='extracellular')
cd630_PATRIC.add_boundary(cpd00069_e, type='exchange', reaction_id='EX_cpd00069_e', lb=-1000.0, ub=1000.0)
new_rxns += 1
cpd00069_c = cd630_PATRIC.metabolites.get_by_id('cpd00069_c') # Tyrosine (cytosolic)

rxn18579_c = cobra.Reaction('rxn18579_c') 
rxn18579_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED', 'subsystem': 'ABC transporter'}
rxn18579_c.notes['citations'] += 'Steglich M., et al. (2018). Convergent Loss of ABC Transporter Genes From Clostridioides difficile Genomes Is Associated With Impaired Tyrosine Uptake and p-Cresol Production. Frontiers in Microbiology. doi 10.3389/fmicb.2018.00901.'
rxn05243_c.gene_reaction_rule = 'CD630_08730 or CD630_08760'
rxn18579_c.name = 'L-Tyrosine ABC transport'
rxn18579_c.lower_bound = 0.
rxn18579_c.upper_bound = 1000.
rxn18579_c.add_metabolites({
    cpd00002_c: -1.0,
    cpd00001_c: -1.0,
    cpd00069_e: -1.0,
    cpd00008_c: 1.0,
    cpd00009_c: 1.0,
    cpd00067_c: 1.0,
    cpd00069_c: 1.0
})
cd630_PATRIC.add_reactions([rxn18579_c])
new_rxns += 1
# Phenylalanine
cpd00066_e = cobra.Metabolite(
    'cpd00066_e',
    formula='C9H11NO2',
    name='L-Phenylalanine',
    compartment='extracellular')
cd630_PATRIC.add_boundary(cpd00066_e, type='exchange', reaction_id='EX_cpd00066_e', lb=-1000.0, ub=1000.0)
new_rxns += 1
cpd00066_c = cd630_PATRIC.metabolites.get_by_id('cpd00066_c')

rxn18584_c = cobra.Reaction('rxn18584_c') 
rxn18584_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED', 'subsystem': 'ABC transporter'}
rxn18584_c.notes['citations'] += 'Steglich M., et al. (2018). Convergent Loss of ABC Transporter Genes From Clostridioides difficile Genomes Is Associated With Impaired Tyrosine Uptake and p-Cresol Production. Frontiers in Microbiology. doi 10.3389/fmicb.2018.00901.'
rxn18584_c.gene_reaction_rule = 'CD630_08730 or CD630_08760'
rxn18584_c.name = 'nonpolar-amino-acid-transporting ATPase'
rxn18584_c.lower_bound = 0.
rxn18584_c.upper_bound = 1000.
rxn18584_c.add_metabolites({
    cpd00002_c: -1.0,
    cpd00001_c: -1.0,
    cpd00066_e: -1.0,
    cpd00008_c: 1.0,
    cpd00009_c: 1.0,
    cpd00067_c: 1.0,
    cpd00066_c: 1.0
})
cd630_PATRIC.add_reactions([rxn18584_c])
new_rxns += 1
#----------------#

# Hydrophobic - active transport

# Proline (D and L)
cpd00129_e = cobra.Metabolite(
    'cpd00129_e',
    formula='C5H9NO2',
    name='L-Proline',
    compartment='extracellular')
cd630_PATRIC.add_boundary(cpd00129_e, type='exchange', reaction_id='EX_cpd00129_e', lb=-1000.0, ub=1000.0)
new_rxns += 1
cpd00129_c = cd630_PATRIC.metabolites.get_by_id('cpd00129_c') # L-Proline
# http://www.jbc.org/content/249/21/6965.full.pdf

rxn05165_c = cobra.Reaction('rxn05165_c') 
rxn05165_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED', 'subsystem': 'ABC transporter'}
rxn05165_c.notes['citations'] += ''
rxn05165_c.gene_reaction_rule = 'CD630_08730 or CD630_08760'
rxn05165_c.name = 'nonpolar-amino-acid-transporting ATPase'
rxn05165_c.lower_bound = 0.
rxn05165_c.upper_bound = 1000.
rxn05165_c.add_metabolites({
    cpd00002_c: -1.0,
    cpd00001_c: -1.0,
    cpd00129_e: -1.0,
    cpd00008_c: 1.0,
    cpd00009_c: 1.0,
    cpd00067_c: 1.0,
    cpd00129_c: 1.0
})
cd630_PATRIC.add_reactions([rxn05165_c])
new_rxns += 1
cpd00567_e = cobra.Metabolite(
    'cpd00567_e',
    formula='C5H9NO2',
    name='D-Proline',
    compartment='extracellular')
cd630_PATRIC.add_boundary(cpd00567_e, type='exchange', reaction_id='EX_cpd00567_e', lb=-1000.0, ub=1000.0)
new_rxns += 1
cpd00567_c = cd630_PATRIC.metabolites.get_by_id('cpd00567_c') # D-Proline
cpd00971_e = cd630_PATRIC.metabolites.get_by_id('cpd00971_e') # Na+ (extracellular)
cpd00971_c = cd630_PATRIC.metabolites.get_by_id('cpd00971_c') # Na+ (cytosolic)

# Facilitated diffusion for D isomer
CD630_06820 = cobra.Gene('CD630_06820')
CD630_06820.name = 'Putative sodium:solute symporter'
CD630_06820.notes = {'annotation': 'UniProt'}

ENOG4108HXH_c = cobra.Reaction('ENOG4108HXH_c') 
ENOG4108HXH_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'EggNOG'}
ENOG4108HXH_c.gene_reaction_rule = 'CD630_06820'
ENOG4108HXH_c.name = 'D-proline sodium symporter'
ENOG4108HXH_c.lower_bound = 0.
ENOG4108HXH_c.upper_bound = 1000.
ENOG4108HXH_c.add_metabolites({
    cpd00567_e: -1.0,
    cpd00971_e: -1.0,
    cpd00567_c: 1.0,
    cpd00971_c: 1.0
})
cd630_PATRIC.add_reactions([ENOG4108HXH_c])
new_rxns += 1

# Hydroxyproline
cpd29317_e = cobra.Metabolite(
    'cpd29317_e',
    formula='C5H9NO3',
    name='4-hydroxyproline',
    compartment='extracellular')
cd630_PATRIC.add_boundary(cpd29317_e, type='exchange', reaction_id='EX_cpd29317_e', lb=-1000.0, ub=1000.0)
new_rxns += 1
cpd29317_c = cobra.Metabolite(
    'cpd29317_c',
    formula='C5H9NO3',
    name='4-hydroxyproline',
    compartment='cytosol')

rxn05165_2_c = cobra.Reaction('rxn05165_2_c')
rxn05165_2_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'EggNOG', 'subsystem': 'ABC transporter'}
rxn05165_2_c.notes['citations'] += ''
rxn05165_2_c.name = '4-hydroxyproline ABC transporter'
rxn05165_2_c.lower_bound = 0.
rxn05165_2_c.upper_bound = 1000.
rxn05165_2_c.add_metabolites({
    cpd00002_c: -1.0,
    cpd00001_c: -1.0,
    cpd29317_e: -1.0,
    cpd00008_c: 1.0,
    cpd00009_c: 1.0,
    cpd00067_c: 1.0,
    cpd29317_c: 1.0
})
cd630_PATRIC.add_reactions([rxn05165_2_c])
new_rxns += 1

# Glycine
cpd00033_e = cobra.Metabolite(
    'cpd00033_e',
    formula='C2H5NO2',
    name='Glycine',
    compartment='extracellular')
cd630_PATRIC.add_boundary(cpd00033_e, type='exchange', reaction_id='EX_cpd00033_e', lb=-1000.0, ub=1000.0)
new_rxns += 1
cpd00033_c = cd630_PATRIC.metabolites.get_by_id('cpd00033_c') # Glycine

rxn10862_c = cobra.Reaction('rxn10862_c') 
rxn10862_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED', 'subsystem': 'ABC transporter'}
rxn10862_c.notes['citations'] += ''
rxn10862_c.gene_reaction_rule = 'CD630_08730 or CD630_08760'
rxn10862_c.name = 'ATP phosphohydrolase (polar-amino-acid-importing)'
rxn10862_c.lower_bound = 0.
rxn10862_c.upper_bound = 1000.
rxn10862_c.add_metabolites({
    cpd00002_c: -1.0,
    cpd00001_c: -1.0,
    cpd00033_e: -1.0,
    cpd00008_c: 1.0,
    cpd00009_c: 1.0,
    cpd00067_c: 1.0,
    cpd00033_c: 1.0
})
cd630_PATRIC.add_reactions([rxn10862_c])
new_rxns += 1
# Hydroxyproline -> D-proline conversion
cpd02431_c = cd630_PATRIC.metabolites.get_by_id('cpd02431_c') # 1-Pyrroline-5-carboxylate
cpd00006_c = cd630_PATRIC.metabolites.get_by_id('cpd00006_c') # NADP
cpd00005_c = cd630_PATRIC.metabolites.get_by_id('cpd00005_c') # NADPH

CD630_25690 = cobra.Gene('CD630_25690')
CD630_25690.name = '4-hydroxyproline epimerase'
CD630_25690.notes = {'annotation': 'KEGG'}

rxn32054_c = cobra.Reaction('rxn32054_c')
rxn32054_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn32054_c.notes['citations'] += 'Watanabe, S., Tanimoto, Y., Nishiwaki, H., & Watanabe, Y. (2015). Identification and Characterization of Bifunctional Proline Racemase/Hydroxyproline Epimerase from Archaea: Discrimination of Substrates and Molecular Evolution. PLoS ONE, 10(3), e0120349.'
rxn32054_c.name = '4-Hydroxy-L-proline:NADP+ 5-oxidoreductase'
rxn10862_c.gene_reaction_rule = 'CD630_25690'
rxn32054_c.lower_bound = 0.
rxn32054_c.upper_bound = 1000.
rxn32054_c.add_metabolites({
    cpd00006_c: -1.0,
    cpd29317_c: -1.0,
    cpd00005_c: 1.0,
    cpd00001_c: 1.0,
    cpd02431_c: 1.0
})
cd630_PATRIC.add_reactions([rxn32054_c])
new_rxns += 1
# rxn00931_c an rxn00929_c in place
# NADP + L-Proline <=> NADPH + 2.0 H+ + 1-Pyrroline-5-carboxylate

#------------------------------------------------------------------------------------------------#

# Non-Stickland substrates
# Transport and exchange of non-auxotrophic amino acids

# Tryptophan - facilitated diffusion already included

#----------------#

# Histidine
cpd00119_e = cobra.Metabolite(
    'cpd00119_e',
    formula='C6H9N3O2',
    name='L-Histidine',
    compartment='extracellular')
cpd00119_c = cd630_PATRIC.metabolites.get_by_id('cpd00119_c')

CD630_17760 = cobra.Gene('CD630_17760')
CD630_17760.name = ' amino acid family ABC transporter ATP-binding protein'
CD630_17760.notes = {'annotation': 'NCBI'}

rxn05164_c = cobra.Reaction('rxn05164_c') 
rxn05164_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED', 'subsystem': 'ABC transporter'}
rxn05164_c.gene_reaction_rule = 'CD630_17760'
rxn05164_c.name = 'ATP phosphohydrolase (polar-amino-acid-importing)'
rxn05164_c.lower_bound = 0.
rxn05164_c.upper_bound = 1000.
rxn05164_c.add_metabolites({
    cpd00002_c: -1.0,
    cpd00001_c: -1.0,
    cpd00119_e: -1.0,
    cpd00008_c: 1.0,
    cpd00009_c: 1.0,
    cpd00067_c: 1.0,
    cpd00119_c: 1.0
})
cd630_PATRIC.add_reactions([rxn05164_c])
new_rxns += 1
cd630_PATRIC.add_boundary(cpd00119_e, type='exchange', reaction_id='EX_cpd00119_e', lb=-1000.0, ub=1000.0)
new_rxns += 1
# Asparagine
cpd00132_e = cobra.Metabolite(
    'cpd00132_e',
    formula='C4H8N2O3',
    name='L-Asparagine',
    compartment='extracellular')
cpd00132_c = cd630_PATRIC.metabolites.get_by_id('cpd00132_c')

rxn10144_c = cobra.Reaction('rxn10144_c') 
rxn10144_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED', 'subsystem': 'ABC transporter'}
rxn10144_c.gene_reaction_rule = 'CD630_17760'
rxn10144_c.name = 'ATP phosphohydrolase (polar-amino-acid-importing)'
rxn10144_c.lower_bound = 0.
rxn10144_c.upper_bound = 1000.
rxn10144_c.add_metabolites({
    cpd00002_c: -1.0,
    cpd00001_c: -1.0,
    cpd00132_e: -1.0,
    cpd00008_c: 1.0,
    cpd00009_c: 1.0,
    cpd00067_c: 1.0,
    cpd00132_c: 1.0
})
cd630_PATRIC.add_reactions([rxn10144_c])
new_rxns += 1
cd630_PATRIC.add_boundary(cpd00132_e, type='exchange', reaction_id='EX_cpd00132_e', lb=-1000.0, ub=1000.0)
new_rxns += 1
# Serine
cpd00054_e = cobra.Metabolite(
    'cpd00054_e',
    formula='C3H7NO3',
    name='L-Serine',
    compartment='extracellular')
cpd00054_c = cd630_PATRIC.metabolites.get_by_id('cpd00054_c')

rxn10883_c = cobra.Reaction('rxn10883_c') 
rxn10883_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED', 'subsystem': 'ABC transporter'}
rxn10883_c.gene_reaction_rule = 'CD630_17760'
rxn10883_c.name = 'ATP phosphohydrolase (polar-amino-acid-importing)'
rxn10883_c.lower_bound = 0.
rxn10883_c.upper_bound = 1000.
rxn10883_c.add_metabolites({
    cpd00002_c: -1.0,
    cpd00001_c: -1.0,
    cpd00054_e: -1.0,
    cpd00008_c: 1.0,
    cpd00009_c: 1.0,
    cpd00067_c: 1.0,
    cpd00054_c: 1.0
})
cd630_PATRIC.add_reactions([rxn10883_c])
new_rxns += 1
cd630_PATRIC.add_boundary(cpd00054_e, type='exchange', reaction_id='EX_cpd00054_e', lb=-1000.0, ub=1000.0)
new_rxns += 1
# Threonine
cpd00161_e = cobra.Metabolite(
    'cpd00161_e',
    formula='C4H9NO3',
    name='L-Threonine',
    compartment='extracellular')
cpd00161_c = cd630_PATRIC.metabolites.get_by_id('cpd00161_c')

rxn05169_c = cobra.Reaction('rxn05169_c') 
rxn05169_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED', 'subsystem': 'ABC transporter'}
rxn05169_c.gene_reaction_rule = 'CD630_17760'
rxn05169_c.name = 'ATP phosphohydrolase (polar-amino-acid-importing)'
rxn05169_c.lower_bound = 0.
rxn05169_c.upper_bound = 1000.
rxn05169_c.add_metabolites({
    cpd00002_c: -1.0,
    cpd00001_c: -1.0,
    cpd00161_e: -1.0,
    cpd00008_c: 1.0,
    cpd00009_c: 1.0,
    cpd00067_c: 1.0,
    cpd00161_c: 1.0
})
cd630_PATRIC.add_reactions([rxn05169_c])
new_rxns += 1
cd630_PATRIC.add_boundary(cpd00161_e, type='exchange', reaction_id='EX_cpd00161_e', lb=-1000.0, ub=1000.0)
new_rxns += 1
# Glutamine
cpd00053_e = cobra.Metabolite(
    'cpd00053_e',
    formula='C5H10N2O3',
    name='L-Glutamine',
    compartment='extracellular')
cpd00053_c = cd630_PATRIC.metabolites.get_by_id('cpd00053_c')

rxn05155_c = cobra.Reaction('rxn05155_c') 
rxn05155_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED', 'subsystem': 'ABC transporter'}
rxn05155_c.gene_reaction_rule = 'CD630_17760'
rxn05155_c.name = 'ATP phosphohydrolase (polar-amino-acid-importing)'
rxn05155_c.lower_bound = 0.
rxn05155_c.upper_bound = 1000.
rxn05155_c.add_metabolites({
    cpd00002_c: -1.0,
    cpd00001_c: -1.0,
    cpd00053_e: -1.0,
    cpd00008_c: 1.0,
    cpd00009_c: 1.0,
    cpd00067_c: 1.0,
    cpd00053_c: 1.0
})
cd630_PATRIC.add_reactions([rxn05155_c])
new_rxns += 1
cd630_PATRIC.add_boundary(cpd00053_e, type='exchange', reaction_id='EX_cpd00053_e', lb=-1000.0, ub=1000.0)
new_rxns += 1

# Lysine transport
cpd00039_c = cd630_PATRIC.metabolites.get_by_id('cpd00039_c')
cpd00039_e = cd630_PATRIC.metabolites.get_by_id('cpd00039_e')

rxn05151_c = cobra.Reaction('rxn05151_c') 
rxn05151_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED', 'subsystem': 'ABC transporter'}
rxn05151_c.gene_reaction_rule = 'CD630_17760'
rxn05151_c.name = 'ATP phosphohydrolase (polar-amino-acid-importing)'
rxn05151_c.lower_bound = 0.
rxn05151_c.upper_bound = 1000.
rxn05151_c.add_metabolites({
    cpd00002_c: -1.0,
    cpd00001_c: -1.0,
    cpd00039_e: -1.0,
    cpd00008_c: 1.0,
    cpd00009_c: 1.0,
    cpd00067_c: 1.0,
    cpd00039_c: 1.0
})
cd630_PATRIC.add_reactions([rxn05151_c])
new_rxns += 1

#----------------#

# Aspartate (same as Glutamate)
cpd00041_e = cobra.Metabolite(
    'cpd00041_e',
    formula='C4H6NO4',
    name='L-Aspartate',
    compartment='extracellular')
cpd00041_c = cd630_PATRIC.metabolites.get_by_id('cpd00041_c')
cpd00971_e = cd630_PATRIC.metabolites.get_by_id('cpd00971_e') # NA+ (extracellular)
cpd00971_c = cd630_PATRIC.metabolites.get_by_id('cpd00971_c') # NA+ (extracellular)

rxn34357_c = cobra.Reaction('rxn34357_c') 
rxn34357_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED', 'subsystem': 'Cation symporter'}
rxn34357_c.gene_reaction_rule = '272563.8.peg.2286 or ( 272563.8.peg.1830 or 272563.8.peg.2818 )'
rxn34357_c.name = 'Na+/aspartate symport'
rxn34357_c.lower_bound = -1000.
rxn34357_c.upper_bound = 1000.
rxn34357_c.add_metabolites({
    cpd00041_e: -1.0,
    cpd00971_e: -1.0,
    cpd00041_c: 1.0,
    cpd00971_c: 1.0
})
cd630_PATRIC.add_reactions([rxn34357_c])
new_rxns += 1
cd630_PATRIC.add_boundary(cpd00041_e, type='exchange', reaction_id='EX_cpd00041_e', lb=-1000.0, ub=1000.0)
new_rxns += 1

#35 so far....

In [15]:
# Stickland fermentation - Oxidative steps

# Leucine
# L-Leucine:2-oxoglutarate aminotransferase 	rxn00806 in model already
cpd00107_c = cd630_PATRIC.metabolites.get_by_id('cpd00107_c') # L-Leucine
cpd00024_c = cd630_PATRIC.metabolites.get_by_id('cpd00024_c') # 2-Oxoglutarate
cpd00023_c = cd630_PATRIC.metabolites.get_by_id('cpd00023_c') # L-Glutamate
cpd00200_c = cd630_PATRIC.metabolites.get_by_id('cpd00200_c') # 4MOP (2-Oxoisocaproate)
cpd00010_c = cd630_PATRIC.metabolites.get_by_id('cpd00010_c') # CoA
cpd11621_c = cd630_PATRIC.metabolites.get_by_id('cpd11621_c') # Oxidizedferredoxin
cpd00011_c = cd630_PATRIC.metabolites.get_by_id('cpd00011_c') # CO2
cpd01882_c = cd630_PATRIC.metabolites.get_by_id('cpd01882_c') # Isovaleryl-CoA
cpd11620_c = cd630_PATRIC.metabolites.get_by_id('cpd11620_c') # Reducedferredoxin
cpd00067_c = cd630_PATRIC.metabolites.get_by_id('cpd00067_c') # H+

CD630_14930 = cobra.Gene('CD630_14930')
CD630_14930.name = 'ilvE; branched-chain-amino-acid aminotransferase [EC 1.2.1.25 ]'
CD630_14930.notes = {'annotation': 'UniProt'}

rxn14152_c = cobra.Reaction('rxn14152_c') 
rxn14152_c.name = '4-methyl-2-oxopentanoate oxidoreductase'
rxn14152_c.gene_reaction_rule = 'CD630_14930'
rxn14152_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn14152_c.notes['citations'] += 'Kim J, Darley D, & Buckel W. (2005). 2-Hydroxyisocaproyl-CoA dehydratase and its activator from Clostridium difficile. FEBS J. 272; 550-561.'
rxn14152_c.lower_bound = 0.
rxn14152_c.upper_bound = 1000.
rxn14152_c.add_metabolites({
    cpd00010_c: -1.0,
    cpd00200_c: -1.0,
    cpd11621_c: -2.0,
    cpd00011_c: 1.0,
    cpd00067_c: 1.0,
    cpd01882_c: 1.0,
    cpd11620_c: 2.0
})
cd630_PATRIC.add_reactions([rxn14152_c])

cpd00003_c = cd630_PATRIC.metabolites.get_by_id('cpd00003_c') # NAD
cpd00004_c = cd630_PATRIC.metabolites.get_by_id('cpd00004_c') # NADH
cpd00001_c = cd630_PATRIC.metabolites.get_by_id('cpd00001_c') # H2O
cpd00013_c = cd630_PATRIC.metabolites.get_by_id('cpd00013_c') # NH3

rxn00804_c = cobra.Reaction('rxn00804_c') 
rxn00804_c.name = 'Leucine aminotransferase'
rxn00804_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn00804_c.notes['citations'] += 'Kim J, Darley D, & Buckel W. (2005). 2-Hydroxyisocaproyl-CoA dehydratase and its activator from Clostridium difficile. FEBS J. 272; 550-561.'
rxn00804_c.lower_bound = 0.
rxn00804_c.upper_bound = 1000.
rxn00804_c.add_metabolites({
    cpd00001_c: -1.0,
    cpd00003_c: -1.0,
    cpd00107_c: -1.0,
    cpd00004_c: 1.0,
    cpd00013_c: 1.0,
    cpd00067_c: 1.0,
    cpd00200_c: 1.0
})
cd630_PATRIC.add_reactions([rxn00804_c])

CD630_01120 = cobra.Gene('CD630_01120')
CD630_01120.name = 'ptb; Phosphate butyryltransferase'
CD630_01120.notes = {'annotation': 'KEGG'}

cpd05178_c = cobra.Metabolite(
    'cpd05178_c',
    formula='C5H10O2',
    name='Isovalerate',
    compartment='cytosol')
cpd00002_c = cd630_PATRIC.metabolites.get_by_id('cpd00002_c') # ATP
cpd00008_c = cd630_PATRIC.metabolites.get_by_id('cpd00008_c') # ADP
cpd00009_c = cd630_PATRIC.metabolites.get_by_id('cpd00009_c') # Phosphate
R01174_1_c = cobra.Reaction('R01174_1_c') 
R01174_1_c.name = 'Phosphate butyryltransferase'
R01174_1_c.gene_reaction_rule = 'CD630_01120'
R01174_1_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'KEGG'}
R01174_1_c.notes['citations'] += 'Kim J, Darley D, & Buckel W. (2005). 2-Hydroxyisocaproyl-CoA dehydratase and its activator from Clostridium difficile. FEBS J. 272; 550-561.'
R01174_1_c.lower_bound = 0.
R01174_1_c.upper_bound = 1000.
R01174_1_c.add_metabolites({
    cpd01882_c: -1.0,
    cpd00008_c: -1.0,
    cpd00067_c: -1.0,
    cpd00009_c: -1.0,
    cpd00002_c: 1.0,
    cpd00010_c: 1.0,
    cpd05178_c: 1.0
})
cd630_PATRIC.add_reactions([R01174_1_c])

cpd05178_e = cobra.Metabolite(
    'cpd05178_e',
    formula='C5H10O2',
    name='Isovalerate',
    compartment='extracellular')
ID001_c = cobra.Reaction('ID001_c') 
ID001_c.name = 'Isovalerate diffusion'
ID001_c.gene_reaction_rule = ''
ID001_c.lower_bound = -1000.
ID001_c.upper_bound = 1000.
ID001_c.add_metabolites({
    cpd05178_c: -1.0,
    cpd05178_e: 1.0,
    
})
cd630_PATRIC.add_reactions([ID001_c])
cd630_PATRIC.add_boundary(cpd05178_e, type='exchange', reaction_id='EX_cpd05178_e', lb=-1000.0, ub=1000.0)

#---------------#

# Valine
cpd00156_c = cd630_PATRIC.metabolites.get_by_id('cpd00156_c') # L-Valine
cpd00123_c = cd630_PATRIC.metabolites.get_by_id('cpd00123_c') # 3-Methyl-2-oxobutanoate (3mob)

R01214_c = cobra.Reaction('R01214_c') 
R01214_c.name = 'L-valine aminotransferase'
R01214_c.gene_reaction_rule = 'CD630_14930'
R01214_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'KEGG'}
R01214_c.lower_bound = -1000.
R01214_c.upper_bound = 1000.
R01214_c.add_metabolites({
    cpd00156_c: -1.0,
    cpd00024_c: -1.0,
    cpd00023_c: 1.0,
    cpd00123_c: 1.0
})
cd630_PATRIC.add_reactions([R01214_c])

cpd00481_c = cd630_PATRIC.metabolites.get_by_id('cpd00481_c') # Isobutyryl-CoA
cpd11620_c = cd630_PATRIC.metabolites.get_by_id('cpd11620_c') # Reducedferredoxin
cpd11621_c = cd630_PATRIC.metabolites.get_by_id('cpd11621_c') # Oxidizedferredoxin

CD630_21990 = cobra.Gene('CD630_21990')
CD630_21990.name = 'Putative ferredoxin/flavodoxin oxidoreductase,alpha subunit'
CD630_21990.notes = {'annotation': 'UniProt'}
CD630_21991 = cobra.Gene('CD630_21991')
CD630_21991.name = 'Putative 4Fe-4S ferredoxin iron-sulfur binding domain-containing protein'
CD630_21991.notes = {'annotation': 'UniProt'}

rxn10618_c = cobra.Reaction('rxn10618_c') 
rxn10618_c.name = '2-oxoisovalerate ferredoxin reductase'
rxn10618_c.gene_reaction_rule = 'CD630_21990 and CD630_21991'
rxn10618_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn10618_c.notes['citations'] += 'Kim J, Darley D, & Buckel W. (2005). 2-Hydroxyisocaproyl-CoA dehydratase and its activator from Clostridium difficile. FEBS J. 272; 550-561.'
rxn10618_c.lower_bound = 0.
rxn10618_c.upper_bound = 1000.
rxn10618_c.add_metabolites({
    cpd00010_c: -1.0,
    cpd00123_c: -1.0,
    cpd11621_c: -2.0,
    cpd00011_c: 1.0,
    cpd00067_c: 1.0,
    cpd00481_c: 1.0,
    cpd11620_c: 2.0
})
cd630_PATRIC.add_reactions([rxn10618_c])

cd630_PATRIC.metabolites.get_by_id('cpd01711_c').formula = 'C4H8O2' # Isobutyrate (correct formula)
cpd01711_c = cd630_PATRIC.metabolites.get_by_id('cpd01711_c')

R01174_2_c = cobra.Reaction('R01174_2_c') 
R01174_2_c.name = 'Phosphate butyryltransferase'
R01174_2_c.gene_reaction_rule = 'CD630_01120'
R01174_2_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'KEGG'}
R01174_2_c.notes['citations'] += 'Kim J, Darley D, & Buckel W. (2005). 2-Hydroxyisocaproyl-CoA dehydratase and its activator from Clostridium difficile. FEBS J. 272; 550-561.'
R01174_2_c.lower_bound = 0.
R01174_2_c.upper_bound = 1000.
R01174_2_c.add_metabolites({
    cpd00481_c: -1.0,
    cpd00008_c: -1.0,
    cpd00067_c: -1.0,
    cpd00009_c: -1.0,
    cpd00002_c: 1.0,
    cpd00010_c: 1.0,
    cpd01711_c: 1.0
})
cd630_PATRIC.add_reactions([R01174_2_c])

cpd01711_e = cobra.Metabolite(
    'cpd01711_e',
    formula='C4H8O2',
    name='Isobutyrate',
    compartment='extracellular')
ID002_c = cobra.Reaction('ID002_c') 
ID002_c.name = 'Isobutyrate diffusion'
ID002_c.lower_bound = -1000.
ID002_c.upper_bound = 1000.
ID002_c.add_metabolites({
    cpd01711_c: -1.0,
    cpd01711_e: 1.0
})
cd630_PATRIC.add_reactions([ID002_c])
cd630_PATRIC.add_boundary(cpd01711_e, type='exchange', reaction_id='EX_cpd01711_e', lb=-1000.0, ub=1000.0)

#---------------#

# Isoleucine
cpd00322_c = cd630_PATRIC.metabolites.get_by_id('cpd00322_c') # L-Isoleucine
cpd00508_c = cd630_PATRIC.metabolites.get_by_id('cpd00508_c') # 3MOP (3-Methyl-2-oxopentanoate)

rxn01575_c = cobra.Reaction('rxn01575_c') 
rxn01575_c.name = 'L-isoleucine aminotransferase'
rxn01575_c.gene_reaction_rule = 'CD630_14930'
rxn01575_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn01575_c.notes['citations'] += 'Kim J, Darley D, & Buckel W. (2005). 2-Hydroxyisocaproyl-CoA dehydratase and its activator from Clostridium difficile. FEBS J. 272; 550-561.'
rxn01575_c.lower_bound = -1000.
rxn01575_c.upper_bound = 1000.
rxn01575_c.add_metabolites({
    cpd00322_c: -1.0,
    cpd00024_c: -1.0,
    cpd00023_c: 1.0,
    cpd00508_c: 1.0
})
cd630_PATRIC.add_reactions([rxn01575_c])

# Next reaction already in place:  rxn02269_c;  NAD + CoA + 3MOP --> NADH + CO2 + 2-Methylbutyryl-CoA

cpd00760_c = cd630_PATRIC.metabolites.get_by_id('cpd00760_c') # 2-Methylbutyryl-CoA
cpd19585_c = cobra.Metabolite(
    'cpd19585_c',
    formula='C5H10O2',
    name='2-Methylbutyrate',
    compartment='cytosol')
R01174_3_c = cobra.Reaction('R01174_3_c') 
R01174_3_c.name = 'Phosphate butyryltransferase'
R01174_3_c.gene_reaction_rule = 'CD630_01120'
R01174_3_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'KEGG'}
R01174_3_c.notes['citations'] += 'Kim J, Darley D, & Buckel W. (2005). 2-Hydroxyisocaproyl-CoA dehydratase and its activator from Clostridium difficile. FEBS J. 272; 550-561.'
R01174_3_c.lower_bound = 0.
R01174_3_c.upper_bound = 1000.
R01174_3_c.add_metabolites({
    cpd00760_c: -1.0,
    cpd00008_c: -1.0,
    cpd00067_c: -1.0,
    cpd00009_c: -1.0,
    cpd00002_c: 1.0,
    cpd00010_c: 1.0,
    cpd19585_c: 1.0
})
cd630_PATRIC.add_reactions([R01174_3_c])

cpd19585_e = cobra.Metabolite(
    'cpd19585_e',
    formula='C5H10O2',
    name='2-Methylbutyrate',
    compartment='extracellular')
ID003_c = cobra.Reaction('ID003_c') 
ID003_c.name = '2-Methylbutyrate diffusion'
ID003_c.lower_bound = -1000.
ID003_c.upper_bound = 1000.
ID003_c.add_metabolites({
    cpd19585_c: -1.0,
    cpd19585_e: 1.0
})
cd630_PATRIC.add_reactions([ID003_c])
cd630_PATRIC.add_boundary(cpd19585_e, type='exchange', reaction_id='EX_cpd19585_e', lb=-1000.0, ub=1000.0)

#---------------#

# Tyrosine
cpd00069_c = cd630_PATRIC.metabolites.get_by_id('cpd00069_c') # L-Tyrosine
cpd00868_c = cd630_PATRIC.metabolites.get_by_id('cpd00868_c') # p-hydroxyphenylpyruvate
#rxn00527_c = cobra.Reaction('rxn00527_c') 
#rxn00527_c.name = 'L-Tyrosine aminotransferase'
#rxn00527_c.gene_reaction_rule = 'CD630_14930'
#rxn00527_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
#rxn00527_c.notes['citations'] += 'Neumann-Schaal M., et al. (2015). Time-resolved amino acid uptake of Clostridium difficile 630Δerm and concomitant fermentation product and toxin formation. BMC Microbiology. 15(1); 1471-2180.'
#rxn00527_c.lower_bound = -1000.
#rxn00527_c.upper_bound = 1000.
#rxn00527_c.add_metabolites({
#    cpd00069_c: -1.0,
#    cpd00024_c: -1.0,
#    cpd00023_c: 1.0,
#    cpd00868_c: 1.0
#})
#cd630_PATRIC.add_reactions([rxn00527_c])

cpd03165_c = cobra.Metabolite(
    'cpd03165_c',
    formula='C29H39N7O18P3S',
    name='4-Hydroxyphenylacetyl-CoA',
    compartment='cytosol')
rxn10563_c = cobra.Reaction('rxn10563_c') 
rxn10563_c.name = 'Indolepyruvate ferredoxin oxidoreductase (hydroxyphenylpyvurate)'
rxn10563_c.gene_reaction_rule = 'CD630_21990 and CD630_21991'
rxn10563_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn10563_c.notes['citations'] += 'Neumann-Schaal M., et al. (2015). Time-resolved amino acid uptake of Clostridium difficile 630Δerm and concomitant fermentation product and toxin formation. BMC Microbiology. 15(1); 1471-2180.'
rxn10563_c.lower_bound = 0.
rxn10563_c.upper_bound = 1000.
rxn10563_c.add_metabolites({
    cpd00010_c: -1.0,
    cpd00868_c: -1.0,
    cpd11621_c: -2.0,
    cpd00011_c: 1.0,
    cpd03165_c: 1.0,
    cpd11620_c: 2.0
})
cd630_PATRIC.add_reactions([rxn10563_c])

cpd00489_c = cobra.Metabolite(
    'cpd00489_c',
    formula='C8H8O3',
    name='4-Hydroxyphenylacetate',
    compartment='cytosol')

R01174_4_c = cobra.Reaction('R01174_4_c') 
R01174_4_c.name = 'Phosphate butyryltransferase'
R01174_4_c.gene_reaction_rule = 'CD630_01120'
R01174_4_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'KEGG'}
R01174_4_c.notes['citations'] += 'Kim J, Darley D, & Buckel W. (2005). 2-Hydroxyisocaproyl-CoA dehydratase and its activator from Clostridium difficile. FEBS J. 272; 550-561.'
R01174_4_c.lower_bound = 0.
R01174_4_c.upper_bound = 1000.
R01174_4_c.add_metabolites({
    cpd03165_c: -1.0,
    cpd00008_c: -1.0,
    cpd00009_c: -1.0,
    cpd00002_c: 1.0,
    cpd00010_c: 1.0,
    cpd00489_c: 1.0
})
cd630_PATRIC.add_reactions([R01174_4_c])

cpd00489_e = cobra.Metabolite(
    'cpd00489_e',
    formula='C8H8O3',
    name='4-Hydroxyphenylacetate',
    compartment='extracellular')
ID004_c = cobra.Reaction('ID004_c') 
ID004_c.name = '4-Hydroxyphenylacetate diffusion'
ID004_c.lower_bound = -1000.
ID004_c.upper_bound = 1000.
ID004_c.add_metabolites({
    cpd00489_c: -1.0,
    cpd00489_e: 1.0
})
cd630_PATRIC.add_reactions([ID004_c])
cd630_PATRIC.add_boundary(cpd00489_e, type='exchange', reaction_id='EX_cpd00489_e', lb=-1000.0, ub=1000.0)

# p-Cresol formation and secretion
cpd01042_c = cobra.Metabolite(
    'cpd01042_c',
    formula='C7H8O',
    name='p-Cresol',
    compartment='cytosol')

CD630_01530 = cobra.Gene('CD630_01530')
CD630_01530.name = '4-hydroxyphenylacetate decarboxylase large subunit [EC:4.1.1.83]'
CD630_01530.notes = {'annotation': 'KEGG'}
CD630_01540 = cobra.Gene('CD630_01540')
CD630_01540.name = '4-hydroxyphenylacetate decarboxylase small subunit [EC:4.1.1.83]'
CD630_01540.notes = {'annotation': 'KEGG'}

rxn05061_c = cobra.Reaction('rxn05061_c') 
rxn05061_c.name = '4-hydroxyphenylacetate carboxy-lyase'
rxn05061_c.gene_reaction_rule = 'CD630_01530 and CD630_01540'
rxn05061_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn05061_c.notes['citations'] += 'Selmer, T., & Andrei, P. I. (2001). p-Hydroxyphenylacetate decarboxylase from Clostridium difficile. A novel glycyl radical enzyme catalysing the formation of p-cresol. European Journal of Biochemistry, 268(5), 1363–1372.'
rxn05061_c.lower_bound = -1000.
rxn05061_c.upper_bound = 1000.
rxn05061_c.add_metabolites({
    cpd00489_c: -1.0,
    cpd00011_c: 1.0,
    cpd01042_c: 1.0
})
cd630_PATRIC.add_reactions([rxn05061_c])

cpd01042_e = cobra.Metabolite(
    'cpd01042_e',
    formula='C7H8O',
    name='p-Cresol',
    compartment='extracellular')
ID005_c = cobra.Reaction('ID005_c') 
ID005_c.name = 'p-Cresol diffusion'
ID005_c.lower_bound = -1000.
ID005_c.upper_bound = 1000.
ID005_c.add_metabolites({
    cpd01042_c: -1.0,
    cpd01042_e: 1.0
})
cd630_PATRIC.add_reactions([ID005_c])
cd630_PATRIC.add_boundary(cpd01042_e, type='exchange', reaction_id='EX_cpd01042_e', lb=-1000.0, ub=1000.0)

#---------------#

# Phenylalanine
cpd00066_c = cd630_PATRIC.metabolites.get_by_id('cpd00066_c') # L-Phenylalanine
cpd00143_c = cd630_PATRIC.metabolites.get_by_id('cpd00143_c') # Phenylpyruvate
rxn21134_c = cobra.Reaction('rxn21134_c') 
rxn21134_c.name = 'L-phenylalanine aminotransferase'
rxn21134_c.gene_reaction_rule = 'CD630_14930'
rxn21134_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn21134_c.notes['citations'] += 'Kim J, Darley D, & Buckel W. (2005). 2-Hydroxyisocaproyl-CoA dehydratase and its activator from Clostridium difficile. FEBS J. 272; 550-561.'
rxn21134_c.lower_bound = -1000.
rxn21134_c.upper_bound = 1000.
rxn21134_c.add_metabolites({
    cpd00066_c: -1.0,
    cpd00024_c: -1.0,
    cpd00023_c: 1.0,
    cpd00143_c: 1.0
})
cd630_PATRIC.add_reactions([rxn21134_c])

cpd00452_c = cobra.Metabolite(
    'cpd00452_c',
    formula='C29H38N7O17P3S',
    name='Phenylacetyl-CoA',
    compartment='cytosol')

CD630_23800 = cobra.Gene('CD630_23800')
CD630_23800.name = 'Indole pyruvate ferredoxin/flavodoxin oxidoreductase'
CD630_23800.notes = {'annotation': 'UniProt'}

rxn10562_c = cobra.Reaction('rxn10562_c') 
rxn10562_c.name = 'indolepyruvate ferredoxin oxidoreductase'
rxn10562_c.gene_reaction_rule = 'CD630_23800'
rxn10562_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn10562_c.notes['citations'] += 'Kim J, Darley D, & Buckel W. (2005). 2-Hydroxyisocaproyl-CoA dehydratase and its activator from Clostridium difficile. FEBS J. 272; 550-561.'
rxn10562_c.lower_bound = 0.
rxn10562_c.upper_bound = 1000.
rxn10562_c.add_metabolites({
    cpd00010_c: -1.0,
    cpd00143_c: -1.0,
    cpd11621_c: -2.0,
    cpd00011_c: 1.0,
    cpd00067_c: 1.0,
    cpd00452_c: 1.0,
    cpd11620_c: 2.0
})
cd630_PATRIC.add_reactions([rxn10562_c])

cpd00430_c = cobra.Metabolite(
    'cpd00430_c',
    formula='C8H8O2',
    name='Phenylacetate', # PACT in ModelSEED
    compartment='cytosol')

R01174_5_c = cobra.Reaction('R01174_5_c') 
R01174_5_c.name = 'Phosphate butyryltransferase'
R01174_5_c.gene_reaction_rule = 'CD630_01120'
R01174_5_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'KEGG'}
R01174_5_c.notes['citations'] += 'Kim J, Darley D, & Buckel W. (2005). 2-Hydroxyisocaproyl-CoA dehydratase and its activator from Clostridium difficile. FEBS J. 272; 550-561.'
R01174_5_c.lower_bound = 0.
R01174_5_c.upper_bound = 1000.
R01174_5_c.add_metabolites({
    cpd00008_c: -1.0,
    cpd00009_c: -1.0,
    cpd00067_c: -1.0,
    cpd00452_c: -1.0,
    cpd00002_c: 1.0,
    cpd00010_c: 1.0,
    cpd00430_c: 1.0
})
cd630_PATRIC.add_reactions([R01174_5_c])

cpd00430_e = cobra.Metabolite(
    'cpd00430_e',
    formula='C8H8O2',
    name='Phenylacetate', # PACT
    compartment='extracellular')

rxn11336_c = cobra.Reaction('rxn11336_c') 
rxn11336_c.name = 'Phenylacetate diffusion'
rxn11336_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED'}
rxn11336_c.lower_bound = -1000.
rxn11336_c.upper_bound = 1000.
rxn11336_c.add_metabolites({
    cpd00430_c: -1.0,
    cpd00430_e: 1.0
})
cd630_PATRIC.add_reactions([rxn11336_c])

cd630_PATRIC.add_boundary(cpd00430_e, type='exchange', reaction_id='EX_cpd00430_e', lb=-1000.0, ub=1000.0)


Reaction identifier,EX_cpd00430_e
Name,Phenylacetate exchange
Memory address,0x07f8473911310
Stoichiometry,cpd00430_e <=> Phenylacetate <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [16]:
# Stickland Reactions - Reductive steps

# Frequently needed metabolites
cpd00001_c = cd630_PATRIC.metabolites.get_by_id('cpd00001_c') # H2O
cpd00003_c = cd630_PATRIC.metabolites.get_by_id('cpd00003_c') # NAD
cpd00004_c = cd630_PATRIC.metabolites.get_by_id('cpd00004_c') # NADH
cpd00010_c = cd630_PATRIC.metabolites.get_by_id('cpd00010_c') # CoA
cpd00013_c = cd630_PATRIC.metabolites.get_by_id('cpd00013_c') # NH3
cpd11421_c = cd630_PATRIC.metabolites.get_by_id('cpd11421_c') # trdrd
cpd11420_c = cd630_PATRIC.metabolites.get_by_id('cpd11420_c') # trdox
cpd00196_c = cd630_PATRIC.metabolites.get_by_id('cpd00196_c') # Acetylphosphate
cpd00067_e = cd630_PATRIC.metabolites.get_by_id('cpd00067_e') # H+
cpd00067_c = cd630_PATRIC.metabolites.get_by_id('cpd00067_c') # H+
cpd00011_c = cd630_PATRIC.metabolites.get_by_id('cpd00011_c') # CO2

#---------------#

# Proline
cpd00129_c = cd630_PATRIC.metabolites.get_by_id('cpd00129_c') # L-Proline
cd630_PATRIC.remove_reactions(['rxn02029_c']) # This is incorrect for C. difficile metabolism
cpd00339_e = cobra.Metabolite(
    'cpd00339_e',
    formula='C5H11NO2',
    name='5-Aminopentanoate',
    compartment='extracellular')
# D-Proline reductase (C. difficile)
cpd00339_c = cd630_PATRIC.metabolites.get_by_id('cpd00339_c') # 5-Aminopentanoate
rxn20606_c = cobra.Reaction('rxn20606_c') 
rxn20606_c.name = 'D-proline reductase'
rxn20606_c.gene_reaction_rule = 'CD630_32440'
rxn20606_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn20606_c.notes['citations'] += 'Bouillaut, L., Self, W. T., & Sonenshein, A. L. (2013). Proline-Dependent Regulation of Clostridium difficile Stickland Metabolism. Journal of Bacteriology, 195(4), 844–854.'
rxn20606_c.notes['citations'] += 'Jackson, S., Calos, M., Myers, A., & Self, W. T. (2006). Analysis of proline reduction in the nosocomial pathogen Clostridium difficile. Journal of Bacteriology.'
rxn20606_c.lower_bound = 0.
rxn20606_c.upper_bound = 1000.
rxn20606_c.add_metabolites({
    cpd00567_c: -1.0,
    cpd00004_c: -1.0,
    cpd00067_c: -1.0, # H+
    cpd00003_c: 1.0,
    cpd00339_c: 1.0
})
cd630_PATRIC.add_reactions([rxn20606_c])
# 5-Aminopentanoate (aka 5-aminovalerate) export
rxn12566_c = cobra.Reaction('rxn12566_c') 
rxn12566_c.name = '5-Aminopentanoate transport via proton symport'
rxn12566_c.lower_bound = 0.
rxn12566_c.upper_bound = 1000.
rxn12566_c.add_metabolites({
    cpd00339_c: -1.0,
    cpd00067_c: -1.0,
    cpd00067_e: 1.0,
    cpd00339_e: 1.0
})
cd630_PATRIC.add_reactions([rxn12566_c])
cd630_PATRIC.add_boundary(cpd00339_e, type='exchange', reaction_id='EX_cpd00339_e', lb=-1000.0, ub=1000.0)

#---------------#

# Glycine 
cpd00033_c = cd630_PATRIC.metabolites.get_by_id('cpd00033_c') # Glycine
rxn11676_c = cobra.Reaction('rxn11676_c') 
rxn11676_c.name = 'Glycine reductase'
rxn11676_c.gene_reaction_rule = 'CD630_23540'
rxn11676_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn11676_c.notes['citations'] += 'Bouillaut, L., Self, W. T., & Sonenshein, A. L. (2013). Proline-Dependent Regulation of Clostridium difficile Stickland Metabolism. Journal of Bacteriology, 195(4), 844–854.'
rxn11676_c.lower_bound = -1000.
rxn11676_c.upper_bound = 1000.
rxn11676_c.add_metabolites({
    cpd00009_c: -1.0,
    cpd00033_c: -1.0,
    cpd00067_c: -1.0,
    cpd11421_c: -1.0,
    cpd00001_c: 1.0,
    cpd00013_c: 1.0,
    cpd00196_c: 1.0,
    cpd11420_c: 1.0
})
cd630_PATRIC.add_reactions([rxn11676_c])

# Acetylphosphate --> Acetate conversion intact
# ATP:acetate phosphotransferase; rxn00225; 1 ADP [0] + 1 Acetylphosphate [0] -> 1 ATP [0] + 1 Acetate [0] 
cd630_PATRIC.reactions.get_by_id('rxn00225_c').upper_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00225_c').notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
cd630_PATRIC.reactions.get_by_id('rxn00225_c').notes['citations'] += 'Bouillaut, L., Self, W. T., & Sonenshein, A. L. (2013). Proline-Dependent Regulation of Clostridium difficile Stickland Metabolism. Journal of Bacteriology, 195(4), 844–854.'

#---------------#

# Leucine
cpd00200_c = cd630_PATRIC.metabolites.get_by_id('cpd00200_c') # 4-Methyl-2-oxopentanoate - 4MOP (2-Oxoisocaproate)
cpd01646_c = cd630_PATRIC.metabolites.get_by_id('cpd01646_c') # 2-Isopropylmalate == 2-Hydroxyisocaproate

CD630_03940 = cobra.Gene('CD630_03940')
CD630_03940.name = 'ldhA; D-lactate dehydrogenase [EC:1.1.1.28]'
CD630_03940.notes = {'annotation': 'PATRIC'}

rxn13022_c = cobra.Reaction('rxn13022_c') 
rxn13022_c.name = '3-isopropylmalate dehydrogenase'
rxn13022_c.gene_reaction_rule = 'CD630_03940'
rxn13022_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn13022_c.notes['citations'] += 'Kim, J., Darley, D., Selmer, T., & Buckel, W. (2006). Characterization of (R)-2-Hydroxyisocaproate Dehydrogenase and a Family III Coenzyme A Transferase Involved in Reduction of l-Leucine to Isocaproate by Clostridium difficile. Applied and Environmental Microbiology, 72(9), 6062–6069.'
rxn13022_c.lower_bound = -1000.
rxn13022_c.upper_bound = 1000.
rxn13022_c.add_metabolites({
    cpd00004_c: -1.0,
    cpd00011_c: -1.0,
    cpd00200_c: -1.0,
    cpd00003_c: 1.0,
    cpd01646_c: 1.0
})
cd630_PATRIC.add_reactions([rxn13022_c])

CD630_03950 = cobra.Gene('CD630_03950')
CD630_03950.name = 'hadA; Isocaprenoyl-CoA:2-hydroxyisocaproate CoA-transferase'
CD630_03950.notes = {'annotation': 'UniProt'}

cpd23040_c = cobra.Metabolite(
    'cpd23040_c',
    formula='C27H42N7O18P3S',
    name='2-hydroxyisocaproyl-CoA',
    compartment='cytosol')
# C27H46N7O18P3S would be an uncharged formula, otherwise its -4 charge
# C27H42N7O18P3S (original)

C21400_c = cobra.Metabolite(
    'C21400_c',
    formula='C27H46N7O17P3S',
    name='Isocaproyl-CoA',
    compartment='cytosol')

CD630_03980 = cobra.Gene('CD630_03980')
CD630_03980.name = 'hadC; (R)-2-hydroxyisocaproyl-CoA dehydratase beta subunit [EC:4.2.1.157]'
CD630_03980.notes = {'annotation': 'UniProt'}
CD630_03970 = cobra.Gene('CD630_03970')
CD630_03970.name = 'hadB; (R)-2-hydroxyisocaproyl-CoA dehydratase alpha subunit [EC:4.2.1.157]'
CD630_03970.notes = {'annotation': 'UniProt'}

K20025_c = cobra.Reaction('K20025_c') 
K20025_c.name = '(R)-2-hydroxyisocaproyl-CoA dehydratase'
K20025_c.gene_reaction_rule = 'CD630_03980 and CD630_03970'
K20025_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'KEGG'}
K20025_c.notes['citations'] += 'Britz ML & Wilkinson RG (1982). "Leucine dissimilation to isovaleric and isocaproic acids by cell suspensions of amino acid fermenting anaerobes: the Stickland reaction revisited." Can J Microbiol. 28(3);291-300.'
K20025_c.lower_bound = 0.
K20025_c.upper_bound = 1000.
K20025_c.add_metabolites({
    cpd00067_c: -6.0,
    cpd23040_c: -1.0,
    C21400_c: 1.0,
    cpd00001_c: 1.0
})
cd630_PATRIC.add_reactions([K20025_c])   

C21399_c = cobra.Metabolite(
    'C21399_c',
    formula='C6H12O2',
    name='Isocaproate',
    compartment='cytosol')

cpd00047_c = cd630_PATRIC.metabolites.get_by_id('cpd00047_c') # Formate

R11462_c = cobra.Reaction('R11462_c') 
R11462_c.name = '4-methylpentanoyl-CoA:(R)-2-hydroxy-4-methylpentanoate CoA-transferase [EC:2.8.3.24]'
R11462_c.gene_reaction_rule = 'CD630_03950'
R11462_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'KEGG'}
R11462_c.notes['citations'] += 'Kim, J., Darley, D., Selmer, T., & Buckel, W. (2006). Characterization of (R)-2-Hydroxyisocaproate Dehydrogenase and a Family III Coenzyme A Transferase Involved in Reduction of l-Leucine to Isocaproate by Clostridium difficile. Applied and Environmental Microbiology, 72(9), 6062–6069.'
R11462_c.lower_bound = -1000.
R11462_c.upper_bound = 1000.
R11462_c.add_metabolites({
    cpd00067_c: -1.0,
    C21400_c: -1.0,
    cpd01646_c: -1.0,
    C21399_c: 1.0,
    cpd23040_c: 1.0,
    cpd00047_c: 1.0
})
cd630_PATRIC.add_reactions([R11462_c])                                                                           

CD630_03960 = cobra.Gene('CD630_03960')
CD630_03960.name = 'hadI; 2-hydroxyisocaproyl-CoA dehydratase activator'
CD630_03960.notes = {'annotation': 'UniProt'}

C21090_c = cobra.Metabolite(
    'C21090_c',
    formula='C27H44N7O17P3S',
    name='Isocaprenoyl-CoA',
    compartment='cytosol')

R11076_c = cobra.Reaction('R11076_c') 
R11076_c.name = '(R)-2-hydroxyisocaproyl-CoA dehydratase'
R11076_c.gene_reaction_rule = 'CD630_03960 and CD630_03970 and CD630_03980'
R11076_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'KEGG'}
R11076_c.notes['citations'] += 'Kim, J., Darley, D., Selmer, T., & Buckel, W. (2006). Characterization of (R)-2-Hydroxyisocaproate Dehydrogenase and a Family III Coenzyme A Transferase Involved in Reduction of l-Leucine to Isocaproate by Clostridium difficile. Applied and Environmental Microbiology, 72(9), 6062–6069.'
R11076_c.lower_bound = -1000.
R11076_c.upper_bound = 1000.
R11076_c.add_metabolites({
    cpd00067_c: -4.0,
    cpd23040_c: -1.0,
    cpd00001_c: 1.0,
    C21090_c: 1.0
})
cd630_PATRIC.add_reactions([R11076_c])                                                                           

C21399_e = cobra.Metabolite(
    'C21399_e',
    formula='C6H12O2',
    name='Isocaproate',
    compartment='extracellular')

ID006_c = cobra.Reaction('ID006_c') 
ID006_c.name = 'Isocaproate diffusion' 
ID006_c.lower_bound = -1000.
ID006_c.upper_bound = 1000.
ID006_c.add_metabolites({
    C21399_e: -1.0,
    C21399_c: 1.0
})
cd630_PATRIC.add_reactions([ID006_c])

cd630_PATRIC.add_boundary(C21399_e, type='exchange', reaction_id='EX_C21399_e', lb=-1000.0, ub=1000.0)

#---------------#

# Phenylalanine
cpd00143_c = cd630_PATRIC.metabolites.get_by_id('cpd00143_c') # Phenylpyruvate
cpd07326_c = cobra.Metabolite(
    'cpd07326_c',
    formula='C9H7O2',
    name='Cinnamic acid',
    compartment='cytosol')

CD630_32370_1 = cobra.Gene('CD630_32370_1')
CD630_32370_1.name = '3-phenylpropionate/trans-cinnamate dioxygenase subunit alpha [EC:1.14.12.19]'
CD630_32370_1.notes = {'annotation': 'KEGG'}
CD630_32370_2 = cobra.Gene('CD630_32370_2')
CD630_32370_2.name = '3-phenylpropionate/trans-cinnamate dioxygenase subunit beta [EC:1.14.12.19]'
CD630_32370_2.notes = {'annotation': 'KEGG'}

R06782_c = cobra.Reaction('R06782_c') 
R06782_c.name = '3-phenylpropanoate dioxygenase'
R06782_c.gene_reaction_rule = 'CD630_32370_1 and CD630_32370_2'
R06782_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'KEGG'}
R06782_c.notes['citations'] += 'Kim, J., Darley, D., Selmer, T., & Buckel, W. (2004). Dehydration of (R)-2-hydroxyacyl-CoA to enoyl-CoA in the fermentation of α-amino acids by anaerobic bacteria. FEMS Microbiology Reviews. 28 (4). 455–468.'
R06782_c.lower_bound = -1000.
R06782_c.upper_bound = 1000.
R06782_c.add_metabolites({
    cpd00067_c: -2.0,
    cpd00143_c: -1.0,
    cpd00001_c: 1.0,
    cpd07326_c: 1.0
})
cd630_PATRIC.add_reactions([R06782_c])                                                                           

cpd03343_c = cobra.Metabolite(
    'cpd03343_c',
    formula='C9H9O2',
    name='Phenylpropanoate',
    compartment='cytosol')


K00529 = cobra.Gene('K00529')
K00529.name = '3-phenylpropanoate,NADH:oxygen oxidoreductase'
K00529.notes = {'annotation': 'KEGG'}

rxn01610_c = cobra.Reaction('rxn01610_c') 
rxn01610_c.name = 'phenylpropanoate:NAD+ delta2-oxidoreductase'
rxn01610_c.gene_reaction_rule = 'K00529'
rxn01610_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn01610_c.notes['citations'] += 'Kim, J., Darley, D., Selmer, T., & Buckel, W. (2004). Dehydration of (R)-2-hydroxyacyl-CoA to enoyl-CoA in the fermentation of α-amino acids by anaerobic bacteria. FEMS Microbiology Reviews. 28 (4). 455–468.'
rxn01610_c.lower_bound = 0.
rxn01610_c.upper_bound = 1000.
rxn01610_c.add_metabolites({
    cpd00004_c: -1.0,
    cpd00067_c: -1.0,
    cpd07326_c: -1.0,
    cpd00003_c: 1.0,
    cpd03343_c: 1.0
})
cd630_PATRIC.add_reactions([rxn01610_c])                                                                           

cpd03343_e = cobra.Metabolite(
    'cpd03343_e',
    formula='C9H9O2',
    name='Phenylpropanoate',
    compartment='extracellular')
ID007_c = cobra.Reaction('ID007_c') 
ID007_c.name = 'Phenylpropanoate diffusion'
ID007_c.lower_bound = -1000.
ID007_c.upper_bound = 1000.
ID007_c.add_metabolites({
    cpd03343_e: -1.0,
    cpd03343_c: 1.0
})
cd630_PATRIC.add_reactions([ID007_c])
cd630_PATRIC.add_boundary(cpd03343_e, type='exchange', reaction_id='EX_cpd03343_e', lb=-1000.0, ub=1000.0)

#---------------#

# Tyrosine
cpd00868_c = cd630_PATRIC.metabolites.get_by_id('cpd00868_c') # p-hydroxyphenylpyruvate
cpd00001_c = cd630_PATRIC.metabolites.get_by_id('cpd00001_c') 
cpd03170_c = cobra.Metabolite(
    'cpd03170_c',
    formula='C8H7O4',
    name='4-Hydroxymandelate',
    compartment='cytosol')

CD630_01550 = cobra.Gene('CD630_01550')
CD630_01550.name = 'hpdA; 4-hydroxyphenylacetate decarboxylase activating enzyme'
CD630_01550.notes = {'annotation': 'UniProt'}

rxn07124_c = cobra.Reaction('rxn07124_c') 
rxn07124_c.name = 'p-Hydroxyphenylacetate decarboxylase'
rxn07124_c.gene_reaction_rule = 'CD630_01550'
rxn07124_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn07124_c.notes['citations'] += 'Yu L, Blaser M, Andrei PI, Pierik AJ, & Selmer T. (2006). 4-Hydroxyphenylacetate Decarboxylases:  Properties of a Novel Subclass of Glycyl Radical Enzyme Systems. Biochemistry. 45 (31), 9584-9592.'
rxn07124_c.lower_bound = -1000.
rxn07124_c.upper_bound = 1000.
rxn07124_c.add_metabolites({
    cpd00001_c: -2.0,
    cpd00868_c: -1.0,
    cpd00011_c: 1.0,
    cpd03170_c: 1.0,
    cpd00067_c: 4.0
})
cd630_PATRIC.add_reactions([rxn07124_c])

cpd03170_e = cobra.Metabolite(
    'cpd03170_e',
    formula='C8H7O4',
    name='4-Hydroxymandelate',
    compartment='extracellular')
ID008_c = cobra.Reaction('ID008_c')
ID008_c.name = '2,3-Dihydroxyphenylpropanoate diffusion'
ID008_c.lower_bound = -1000.
ID008_c.upper_bound = 1000.
ID008_c.add_metabolites({
    cpd03170_e: -1.0,
    cpd03170_c: 1.0
})
cd630_PATRIC.add_reactions([ID008_c])

cd630_PATRIC.add_boundary(cpd03170_e, type='exchange', reaction_id='EX_cpd02501_e', lb=-1000.0, ub=1000.0)

#------------------#

# Possible other reductive substrates

# from Valine
#cpd00123_c = cd630_PATRIC.metabolites.get_by_id('cpd00123_c') # 3-Methyl-2-oxobutanoate - 3MOB

# from Isoleucine
#cpd00508_c = cd630_PATRIC.metabolites.get_by_id('cpd00508_c') # 3MOP (3-Methyl-2-oxopentanoate)

# 67 so far...

Reaction identifier,EX_cpd03170_e
Name,4-Hydroxymandelate exchange
Memory address,0x07f84738b8dd0
Stoichiometry,cpd03170_e <=> 4-Hydroxymandelate <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [17]:
# Fatty acid metabolism

# Succinate

# Exchange and import
cpd00036_e = cobra.Metabolite(
    'cpd00036_e',
    formula='C4H6O4',
    name='Succinate',
    compartment='extracellular')
cd630_PATRIC.metabolites.get_by_id('cpd00036_c').formula = 'C4H6O4' # fixes charge

cd630_PATRIC.add_boundary(cpd00036_e, type='exchange', reaction_id='EX_cpd00036_e', lb=-1000.0, ub=1000.0)

cpd00036_c = cd630_PATRIC.metabolites.get_by_id('cpd00036_c') # Succinate (cytosolic)
rxn09271_c = cobra.Reaction('rxn09271_c') # Diffusion
rxn09271_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn09271_c.notes['citations'] += 'S. Esquivel-Elizondo, Z. E. Ilhan, E. I. Garcia-Peña, R. Krajmalnik-Brown. (2017). Insights into Butyrate Production in a Controlled Fermentation System via Gene Predictions. mSystems. 2 (4); e00051-17.'
rxn09271_c.notes['citations'] += 'Ferreyra, J. A., Wu, K. J., Hryckowian, A. J., Bouley, D. M., Weimer, B. C., & Sonnenburg, J. L. (2014). Gut microbiota-produced succinate promotes C. difficile infection after antibiotic treatment or motility disturbance. Cell Host & Microbe, 16(6), 770–777.'
rxn09271_c.name = 'Succinate transport'
rxn09271_c.lower_bound = -1000.
rxn09271_c.upper_bound = 1000.
rxn09271_c.add_metabolites({
    cpd00036_e: -1.0,
    cpd00036_c: 1.0
})

# Succinate --> Succinyl-CoA
cpd00078_c = cd630_PATRIC.metabolites.get_by_id('cpd00078_c') # Succinyl-CoA
cpd00029_c = cd630_PATRIC.metabolites.get_by_id('cpd00029_c') # Acetate
cpd00022_c = cd630_PATRIC.metabolites.get_by_id('cpd00022_c') # Acetyl-CoA
cpd00067_c = cd630_PATRIC.metabolites.get_by_id('cpd00067_c') # H

CD630_21960 = cobra.Gene('CD630_21960')
CD630_21960.name = 'abfD; Putative succinyl-CoA synthetase-like ATP-binding protein'
CD630_21960.notes = {'annotation': 'UniProt'}

rxn25164_c = cobra.Reaction('rxn25164_c') 
rxn25164_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn25164_c.notes['citations'] += 'S. Esquivel-Elizondo, Z. E. Ilhan, E. I. Garcia-Peña, R. Krajmalnik-Brown. (2017). Insights into Butyrate Production in a Controlled Fermentation System via Gene Predictions. mSystems. 2 (4); e00051-17.'
rxn25164_c.notes['citations'] += 'Ferreyra, J. A., Wu, K. J., Hryckowian, A. J., Bouley, D. M., Weimer, B. C., & Sonnenburg, J. L. (2014). Gut microbiota-produced succinate promotes C. difficile infection after antibiotic treatment or motility disturbance. Cell Host & Microbe, 16(6), 770–777.'
rxn25164_c.name = 'Succinyl-CoA synthase'
rxn25164_c.gene_reaction_rule = 'CD630_21960'
rxn25164_c.lower_bound = -1000.
rxn25164_c.upper_bound = 1000.
rxn25164_c.add_metabolites({
    cpd00036_c: -1.0,
    cpd00022_c: -1.0,
    cpd00078_c: 1.0,
    cpd00029_c: 1.0,
    cpd00067_c: 2.0
})

# Succinyl-CoA --> 4-Oxobutanoate
cpd00199_c = cd630_PATRIC.metabolites.get_by_id('cpd00199_c') # 4-Oxobutanoate (succinate semialdehyde)
cpd00067_c = cd630_PATRIC.metabolites.get_by_id('cpd00067_c') # H+
cpd00005_c = cd630_PATRIC.metabolites.get_by_id('cpd00005_c') # NADPH
cpd00006_c = cd630_PATRIC.metabolites.get_by_id('cpd00006_c') # NADP
cpd00010_c = cd630_PATRIC.metabolites.get_by_id('cpd00010_c') # CoA

CD630_23420 = cobra.Gene('CD630_23420')
CD630_23420.name = 'sucD; Succinate-semialdehyde dehydrogenase'
CD630_23420.notes = {'annotation': 'UniProt'}

rxn16140_c = cobra.Reaction('rxn16140_c') 
rxn16140_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn16140_c.notes['citations'] += 'S. Esquivel-Elizondo, Z. E. Ilhan, E. I. Garcia-Peña, R. Krajmalnik-Brown. (2017). Insights into Butyrate Production in a Controlled Fermentation System via Gene Predictions. mSystems. 2 (4); e00051-17.'
rxn16140_c.notes['citations'] += 'Ferreyra, J. A., Wu, K. J., Hryckowian, A. J., Bouley, D. M., Weimer, B. C., & Sonnenburg, J. L. (2014). Gut microbiota-produced succinate promotes C. difficile infection after antibiotic treatment or motility disturbance. Cell Host & Microbe, 16(6), 770–777.'
rxn16140_c.name = 'Succinate semialdehyde dehydrogenase'
rxn16140_c.gene_reaction_rule = 'CD630_23420'
rxn16140_c.lower_bound = -1000.
rxn16140_c.upper_bound = 1000.
rxn16140_c.add_metabolites({
    cpd00078_c: -1.0,
    cpd00067_c: -1.0,
    cpd00005_c: -1.0,
    cpd00010_c: 1.0,
    cpd00006_c: 1.0,
    cpd00199_c: 1.0
})

# 4-hydroxybutyrate dehydrogenase: rxn01201_c
# 4-Oxobutanoate --> 4-Hydroxybutanoate

# 4-hydroxybutyrate CoA transferase: rxn03641_c
# 4-Hydroxybutanoate --> 4-Hydroxybutyryl-CoA

# 4-Hydroxybutyryl-CoA --> 3-Butenoyl-CoA
# 3-Butenoyl-CoA --> Crotonyl-CoA 
# or directly...
cpd07946_c = cd630_PATRIC.metabolites.get_by_id('cpd07946_c') # 4-Hydroxybutyryl-CoA
cpd00001_c = cd630_PATRIC.metabolites.get_by_id('cpd00001_c') # H2O
cpd00650_c = cd630_PATRIC.metabolites.get_by_id('cpd00650_c') # Crotonyl-CoA

CD630_23410 = cobra.Gene('CD630_23410')
CD630_23410.name = 'abfD; Gamma-aminobutyrate metabolism dehydratase/isomerase'
CD630_23410.notes = {'annotation': 'UniProt'}

rxn02167_c = cobra.Reaction('rxn02167_c') 
rxn02167_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn02167_c.notes['citations'] += 'S. Esquivel-Elizondo, Z. E. Ilhan, E. I. Garcia-Peña, R. Krajmalnik-Brown. (2017). Insights into Butyrate Production in a Controlled Fermentation System via Gene Predictions. mSystems. 2 (4); e00051-17.'
rxn02167_c.notes['citations'] += 'Ferreyra, J. A., Wu, K. J., Hryckowian, A. J., Bouley, D. M., Weimer, B. C., & Sonnenburg, J. L. (2014). Gut microbiota-produced succinate promotes C. difficile infection after antibiotic treatment or motility disturbance. Cell Host & Microbe, 16(6), 770–777.'
rxn02167_c.name = '4-hydroxybutyryl-CoA dehydratase'
rxn02167_c.gene_reaction_rule = 'CD630_23410'
rxn02167_c.lower_bound = -1000.
rxn02167_c.upper_bound = 1000.
rxn02167_c.add_metabolites({
    cpd07946_c: -1.0,
    cpd00001_c: 1.0,
    cpd00650_c: 1.0
})

CD630_10540 = cobra.Gene('CD630_10540')
CD630_10540.name = 'bcd2; Butyryl-CoA dehydrogenase'
CD630_10540.notes = {'annotation': 'UniProt'}

# Crotonyl-CoA --> Butyryl-CoA
cpd00120_c = cd630_PATRIC.metabolites.get_by_id('cpd00120_c') # Butyryl-CoA
cpd00728_c = cd630_PATRIC.metabolites.get_by_id('cpd00728_c') # 4-Hydroxybutanoate

rxn16586_c = cobra.Reaction('rxn16586_c') 
rxn16586_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn16586_c.notes['citations'] += 'S. Esquivel-Elizondo, Z. E. Ilhan, E. I. Garcia-Peña, R. Krajmalnik-Brown. (2017). Insights into Butyrate Production in a Controlled Fermentation System via Gene Predictions. mSystems. 2 (4); e00051-17.'
rxn16586_c.notes['citations'] += 'Ferreyra, J. A., Wu, K. J., Hryckowian, A. J., Bouley, D. M., Weimer, B. C., & Sonnenburg, J. L. (2014). Gut microbiota-produced succinate promotes C. difficile infection after antibiotic treatment or motility disturbance. Cell Host & Microbe, 16(6), 770–777.'
rxn16586_c.name = 'Butyryl-CoA dehydrgenase'
rxn16586_c.gene_reaction_rule = 'CD630_10540'
rxn16586_c.lower_bound = 0.
rxn16586_c.upper_bound = 1000.
rxn16586_c.add_metabolites({
    cpd00650_c: -1.0,
    cpd00005_c: -1.0,
    cpd00067_c: -1.0,
    cpd00006_c: 1.0,
    cpd00120_c: 1.0
})

CD630_23390 = cobra.Gene('CD630_23390')
CD630_23390.name = 'cat2; 4-hydroxybutyrate CoA-transferase'
CD630_23390.notes = {'annotation': 'KEGG'}

# Butyryl-CoA --> Butyrate
cpd00211_c = cd630_PATRIC.metabolites.get_by_id('cpd00211_c') # Butyrate
rxn13713_c = cobra.Reaction('rxn13713_c') 
rxn13713_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn13713_c.notes['citations'] += 'S. Esquivel-Elizondo, Z. E. Ilhan, E. I. Garcia-Peña, R. Krajmalnik-Brown. (2017). Insights into Butyrate Production in a Controlled Fermentation System via Gene Predictions. mSystems. 2 (4); e00051-17.'
rxn13713_c.notes['citations'] += 'Ferreyra, J. A., Wu, K. J., Hryckowian, A. J., Bouley, D. M., Weimer, B. C., & Sonnenburg, J. L. (2014). Gut microbiota-produced succinate promotes C. difficile infection after antibiotic treatment or motility disturbance. Cell Host & Microbe, 16(6), 770–777.'
rxn13713_c.name = '4-hydroxybutyrate CoA-transferase'
rxn13713_c.gene_reaction_rule = 'CD630_23390'
rxn13713_c.lower_bound = -1000.
rxn13713_c.upper_bound = 1000.
rxn13713_c.add_metabolites({
    cpd00120_c: -1.0,
    cpd00728_c: -1.0,
    cpd07946_c: 1.0,
    cpd00211_c: 1.0
})

# Butyryl-CoA --> Butanoylphosphate
cpd01662_c = cd630_PATRIC.metabolites.get_by_id('cpd01662_c') # Butanoylphosphate
cpd00009_c = cd630_PATRIC.metabolites.get_by_id('cpd00009_c') # Phosphate
cpd00010_c = cd630_PATRIC.metabolites.get_by_id('cpd00010_c') # CoA

rxn00871_c = cobra.Reaction('rxn00871_c') 
rxn00871_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn00871_c.notes['citations'] += 'S. Esquivel-Elizondo, Z. E. Ilhan, E. I. Garcia-Peña, R. Krajmalnik-Brown. (2017). Insights into Butyrate Production in a Controlled Fermentation System via Gene Predictions. mSystems. 2 (4); e00051-17.'
rxn00871_c.notes['citations'] += 'Ferreyra, J. A., Wu, K. J., Hryckowian, A. J., Bouley, D. M., Weimer, B. C., & Sonnenburg, J. L. (2014). Gut microbiota-produced succinate promotes C. difficile infection after antibiotic treatment or motility disturbance. Cell Host & Microbe, 16(6), 770–777.'
rxn00871_c.name = 'Phosphate butyrltransferase'
rxn00871_c.gene_reaction_rule = 'CD630_01120'
rxn00871_c.lower_bound = -1000.
rxn00871_c.upper_bound = 1000.
rxn00871_c.add_metabolites({
    cpd00009_c: -1.0,
    cpd00120_c: -1.0,
    cpd00010_c: 1.0,
    cpd01662_c: 1.0
})

# Butanoylphosphate --> Butyrate: rxn01236_c (butyrate kinase)

# Final export
cpd00211_e = cobra.Metabolite(
    'cpd00211_e',
    formula='C4H7O2',
    name='Butyrate',
    compartment='extracellular')

ID009_c = cobra.Reaction('ID009_c') # Diffusion
ID009_c.name = 'Butyrate transport'
ID009_c.lower_bound = -1000.
ID009_c.upper_bound = 1000.
ID009_c.add_metabolites({
    cpd00211_e: -1.0,
    cpd00211_c: 1.0
})

cd630_PATRIC.add_boundary(cpd00211_e, type='exchange', reaction_id='EX_cpd00211_e', lb=-1000.0, ub=1000.0)

cd630_PATRIC.add_reactions([rxn09271_c, rxn25164_c, rxn16140_c, rxn02167_c, rxn16586_c, rxn13713_c, rxn00871_c, ID009_c])

#----------------------------------------------------------------------------------------------#

# Acetate
cpd00029_e = cobra.Metabolite(
    'cpd00029_e',
    formula='C2H3O2',
    name='Acetate',
    compartment='extracellular')
cd630_PATRIC.add_boundary(cpd00029_e, type='exchange', reaction_id='EX_cpd00029_e', lb=-1000.0, ub=1000.0)
cpd00029_c = cd630_PATRIC.metabolites.get_by_id('cpd00029_c') # Acetate
cpd00067_e = cd630_PATRIC.metabolites.get_by_id('cpd00067_e') # H+
rxn08061_c = cobra.Reaction('rxn08061_c') 
rxn08061_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn08061_c.notes['citations'] += 'S. Esquivel-Elizondo, Z. E. Ilhan, E. I. Garcia-Peña, R. Krajmalnik-Brown. (2017). Insights into Butyrate Production in a Controlled Fermentation System via Gene Predictions. mSystems. 2 (4); e00051-17.'
rxn08061_c.name = 'Acetate transport via proton symport'
rxn08061_c.lower_bound = -1000.
rxn08061_c.upper_bound = 1000.
rxn08061_c.add_metabolites({
    cpd00029_e: -1.0,
    cpd00067_e: -1.0,
    cpd00067_c: 1.0,
    cpd00029_c: 1.0
})

# Acetate --> Acetyl-CoA in place already rxn03641_c

cpd00002_c = cd630_PATRIC.metabolites.get_by_id('cpd00002_c') # ATP
cpd00012_c = cd630_PATRIC.metabolites.get_by_id('cpd00012_c') # PPi
cpd00018_c = cd630_PATRIC.metabolites.get_by_id('cpd00018_c') # AMP
rxn30338_c = cobra.Reaction('rxn30338_c') 
rxn30338_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn30338_c.notes['citations'] += 'S. Esquivel-Elizondo, Z. E. Ilhan, E. I. Garcia-Peña, R. Krajmalnik-Brown. (2017). Insights into Butyrate Production in a Controlled Fermentation System via Gene Predictions. mSystems. 2 (4); e00051-17.'
rxn30338_c.name = 'Acetyl-CoA synthetase'
rxn30338_c.gene_reaction_rule = 'CD630_21960'
rxn30338_c.lower_bound = 0.
rxn30338_c.upper_bound = 1000.
rxn30338_c.add_metabolites({
    cpd00002_c: -1.0,
    cpd00010_c: -1.0,
    cpd00029_c: -1.0,
    cpd00012_c: 1.0,
    cpd00018_c: 1.0,
    cpd00022_c: 1.0,
    cpd00067_c: 1.0
})

cd630_PATRIC.add_reactions([rxn08061_c, rxn30338_c])

#----------------------------------------------------------------------------------------------#

# Lactate 
cpd00159_e = cobra.Metabolite(
    'cpd00159_e',
    formula='C3H5O3',
    name='L-Lactate',
    compartment='extracellular')
cd630_PATRIC.add_boundary(cpd00159_e, type='exchange', reaction_id='EX_cpd00159_e', lb=-1000.0, ub=1000.0)
cpd00159_c = cd630_PATRIC.metabolites.get_by_id('cpd00159_c') # L-Lactate
rxn05602_c = cobra.Reaction('rxn05602_c') 
rxn05602_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn05602_c.notes['citations'] += 'S. Esquivel-Elizondo, Z. E. Ilhan, E. I. Garcia-Peña, R. Krajmalnik-Brown. (2017). Insights into Butyrate Production in a Controlled Fermentation System via Gene Predictions. mSystems. 2 (4); e00051-17.'
rxn05602_c.name = 'L-lactate transport via proton symport'
rxn05602_c.lower_bound = -1000.
rxn05602_c.upper_bound = 1000.
rxn05602_c.add_metabolites({
    cpd00159_e: -1.0,
    cpd00067_e: -1.0,
    cpd00159_c: 1.0,
    cpd00067_c: 1.0
})
cpd00221_e = cobra.Metabolite(
    'cpd00221_e',
    formula='C3H5O3',
    name='D-Lactate',
    compartment='extracellular')
cd630_PATRIC.add_boundary(cpd00221_e, type='exchange', reaction_id='EX_cpd00221_e', lb=-1000.0, ub=1000.0)
cpd00221_c = cd630_PATRIC.metabolites.get_by_id('cpd00221_c') # D-Lactate
rxn10171_c = cobra.Reaction('rxn10171_c') 
rxn10171_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn10171_c.notes['citations'] += 'S. Esquivel-Elizondo, Z. E. Ilhan, E. I. Garcia-Peña, R. Krajmalnik-Brown. (2017). Insights into Butyrate Production in a Controlled Fermentation System via Gene Predictions. mSystems. 2 (4); e00051-17.'
rxn10171_c.name = 'D-lactate transport via proton symport'
rxn10171_c.lower_bound = -1000.
rxn10171_c.upper_bound = 1000.
rxn10171_c.add_metabolites({
    cpd00221_e: -1.0,
    cpd00067_e: -1.0,
    cpd00221_c: 1.0,
    cpd00067_c: 1.0
})

cd630_PATRIC.add_reactions([rxn05602_c, rxn10171_c])

#----------------------------------------------------------------------------------------------#

# Propionate formation
# https://onlinelibrary.wiley.com/doi/pdf/10.1111/1462-2920.13589

# Succinyl-CoA -> D-methylmalonyl-CoA
cpd00519_c = cd630_PATRIC.metabolites.get_by_id('cpd00519_c') # D-methylmalonyl-CoA
rxn00602_c = cobra.Reaction('rxn00602_c') 
rxn00602_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn00602_c.notes['citations'] += 'S. Esquivel-Elizondo, Z. E. Ilhan, E. I. Garcia-Peña, R. Krajmalnik-Brown. (2017). Insights into Butyrate Production in a Controlled Fermentation System via Gene Predictions. mSystems. 2 (4); e00051-17.'
rxn00602_c.name = 'Methylmalonyl-CoA isomerase'
rxn00602_c.gene_reaction_rule = 'CD630_10540'
rxn00602_c.lower_bound = -1000.
rxn00602_c.upper_bound = 1000.
rxn00602_c.add_metabolites({
    cpd00078_c: -1.0,
    cpd00519_c: 1.0
})

# rxn00678_c: D-methylmalonyl-CoA -> Propionyl-CoA

# Propionyl-CoA -> Propionate
cpd00141_c = cd630_PATRIC.metabolites.get_by_id('cpd00141_c') # Propionate
cpd00086_c = cd630_PATRIC.metabolites.get_by_id('cpd00086_c') # Propionyl-CoA
rxn00669_c = cobra.Reaction('rxn00669_c') 
rxn00669_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn00669_c.notes['citations'] += 'S. Esquivel-Elizondo, Z. E. Ilhan, E. I. Garcia-Peña, R. Krajmalnik-Brown. (2017). Insights into Butyrate Production in a Controlled Fermentation System via Gene Predictions. mSystems. 2 (4); e00051-17.'
rxn00669_c.name = 'Propanoate-CoA ligase'
rxn00669_c.gene_reaction_rule = 'CD630_01120'
rxn00669_c.lower_bound = -1000.
rxn00669_c.upper_bound = 1000.
rxn00669_c.add_metabolites({
    cpd00086_c: -1.0,
    cpd00009_c: -1.0,
    cpd00008_c: -1.0,
    cpd00141_c: 1.0,
    cpd00002_c: 1.0,
    cpd00010_c: 1.0
})

cpd00141_e = cobra.Metabolite(
    'cpd00141_e',
    formula='C3H5O2',
    name='Propionate',
    compartment='extracellular')
rxn09172_c = cobra.Reaction('rxn09172_c') 
rxn09172_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn09172_c.notes['citations'] += 'S. Esquivel-Elizondo, Z. E. Ilhan, E. I. Garcia-Peña, R. Krajmalnik-Brown. (2017). Insights into Butyrate Production in a Controlled Fermentation System via Gene Predictions. mSystems. 2 (4); e00051-17.'
rxn09172_c.name = 'Propionate diffusion'
rxn09172_c.lower_bound = -1000.
rxn09172_c.upper_bound = 1000.
rxn09172_c.add_metabolites({
    cpd00141_c: -1.0,
    cpd00141_e: 1.0
})

cd630_PATRIC.add_reactions([rxn00602_c, rxn00669_c, rxn09172_c])

cd630_PATRIC.add_boundary(cpd00141_e, type='exchange', reaction_id='EX_cpd00141_e', lb=-1000.0, ub=1000.0)


Reaction identifier,EX_cpd00141_e
Name,Propionate exchange
Memory address,0x07f847361c210
Stoichiometry,cpd00141_e <=> Propionate <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [18]:
# Membrane components

# Peptidoglycan subunits
cd630_PATRIC.reactions.get_by_id('rxn02009_c').lower_bound = 0.

cpd00002_c = cd630_PATRIC.metabolites.get_by_id('cpd00002_c') # ATP
cpd00731_c = cd630_PATRIC.metabolites.get_by_id('cpd00731_c') # Ala-Ala
cpd02861_c = cd630_PATRIC.metabolites.get_by_id('cpd02861_c') # UDP-N-acetylmuramoyl-L-alanyl-D-glutamyl-L-lysine
cpd00008_c = cd630_PATRIC.metabolites.get_by_id('cpd00008_c') # ADP
cpd00009_c = cd630_PATRIC.metabolites.get_by_id('cpd00009_c') # Phosphate
cpd00067_c = cd630_PATRIC.metabolites.get_by_id('cpd00067_c') # H+
cpd02862_c = cd630_PATRIC.metabolites.get_by_id('cpd02862_c') # UDPMurAc(oyl-L-Ala-D-gamma-Glu-L-Lys-D-Ala-D-Ala)
    
CD630_26550 = cobra.Gene('CD630_26550')
CD630_26550.name = 'murF; UDP-N-acetylmuramoyl-tripeptide--D-alanyl-D-alanine ligase'
CD630_26550.notes = {'annotation': 'KEGG'}
    
rxn03141_c = cobra.Reaction('rxn03141_c') 
rxn03141_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn03141_c.notes['citations'] += ''
rxn03141_c.name = 'UDP-N-acetylmuramoyl-tripeptide---D-alanyl-D-alanine ligase'
rxn03141_c.gene_reaction_rule = 'CD630_26550'
rxn03141_c.lower_bound = 0.
rxn03141_c.upper_bound = 1000.
rxn03141_c.add_metabolites({
    cpd00002_c: -1.0,
    cpd00731_c: -1.0,
    cpd02861_c: -1.0,
    cpd00008_c: 1.0,
    cpd00009_c: 1.0,
    cpd00067_c: 1.0,
    cpd02862_c: 1.0
})
cd630_PATRIC.add_reactions([rxn03141_c])


# Ultimately synthesizes both:
# cpd03495_c	Undecaprenyl-diphospho-N-acetylmuramoyl--N-acetylglucosamine-L-ala-D-glu-meso-2-6-diaminopimeloyl-D-ala-D-ala
# cpd03491_c	Undecaprenyl-diphospho-N-acetylmuramoyl-(N-acetylglucosamine)-L-alanyl-gamma-D-glutamyl-L-lysyl-D-alanyl-D-alanine
# to be included in peptidoglycan polymer

#----------------------------------------------------------------------------#

# Remove extraneous metabolites and associated reactions

# Macromolecular components of ModelSEED Gram+ cell wall
# Peptidoglycan polymer (n-1 subunits)
# Peptidoglycan polymer (n subunits)
# Stearoylcardiolipin (B. subtilis)
# Isoheptadecanoylcardiolipin (B. subtilis)
# Anteisoheptadecanoylcardiolipin (B. subtilis)
# Stearoyllipoteichoic acid (n=24), linked, unsubstituted
# Stearoyllipoteichoic acid (n=24), linked, D-alanine substituted
# Isoheptadecanoyllipoteichoic acid (n=24), linked, unsubstituted
# Isoheptadecanoyllipoteichoic acid (n=24), linked, N-acetyl-D-glucosamine
# Isoheptadecanoyllipoteichoic acid (n=24), linked, D-alanine substituted
# Anteisoheptadecanoyllipoteichoic acid (n=24), linked, D-alanine substituted
# Stearoyllipoteichoic acid (n=24), linked, N-acetyl-D-glucosamine
# Stearoyllipoteichoic acid (n=24), linked, glucose substituted
# Isoheptadecanoyllipoteichoic acid (n=24), linked, glucose substituted
# Anteisoheptadecanoyllipoteichoic acid (n=24), linked, glucose substituted
# Anteisoheptadecanoyllipoteichoic acid (n=24), linked, unsubstituted
# Anteisoheptadecanoyllipoteichoic acid (n=24), linked, N-acetyl-D-glucosamine
# glycerol teichoic acid (n=45), linked, unsubstituted
# glycerol teichoic acid (n=45), linked, D-ala substituted
# glycerol teichoic acid (n=45), linked, glucose substituted

macro = ['cpd15666_c','cpd15665_c','cpd15793_c','cpd15794_c','cpd15795_c','cpd15748_c','cpd15775_c', 'cpd15749_c','cpd15767_c','cpd15776_c','cpd15777_c','cpd15766_c','cpd15757_c','cpd15758_c', 'cpd15759_c','cpd15750_c','cpd15768_c','cpd15667_c','cpd15668_c','cpd15669_c']
cd630_PATRIC = removeAll(cd630_PATRIC, macro)


# B. subtilis-annotated cardiolipin molecules
# Stearoylcardiolipin (B. subtilis)
# Isoheptadecanoylcardiolipin (B. subtilis)
# Anteisoheptadecanoylcardiolipin (B. subtilis)

cardiolipin = ['cpd15793_c','cpd15794_c','cpd15795_c']
cd630_PATRIC = removeAll(cd630_PATRIC, cardiolipin)


cpd15793_c not found
cpd15794_c not found
cpd15795_c not found


In [19]:
# Add missing intracellular metabolites

cpd00314_c = cobra.Metabolite(
    'cpd00314_c',
    formula='C6H14O6',
    name='D-Mannitol',
    compartment='cytosol')

CD630_23310 = cobra.Gene('CD630_23310')
CD630_23310.name = 'mtlD; Mannitol-1-phosphate 5-dehydrogenase'
CD630_23310.notes = {'annotation': 'UniProt'}

cpd00003_c = cd630_PATRIC.metabolites.get_by_id('cpd00003_c') # NAD
cpd00004_c = cd630_PATRIC.metabolites.get_by_id('cpd00004_c') # NADH
cpd00067_c = cd630_PATRIC.metabolites.get_by_id('cpd00067_c') # H+
cpd00082_c = cd630_PATRIC.metabolites.get_by_id('cpd00082_c') # D-Fructose

rxn00629_c = cobra.Reaction('rxn00629_c')
rxn00629_c.gene_reaction_rule = 'CD630_23310'
rxn00629_c.name = 'D-Mannitol:NAD+ 2-oxidoreductase'
rxn00629_c.lower_bound = -1000.
rxn00629_c.upper_bound = 1000.
rxn00629_c.add_metabolites({
    cpd00003_c: -1.0,
    cpd00314_c: -1.0,
    cpd00004_c: 1.0,
    cpd00067_c: 1.0,
    cpd00082_c: 1.0 
})
cd630_PATRIC.add_reactions([rxn00629_c])

#-------------------------------------------------------------------------------------#

cpd00588_c = cobra.Metabolite(
    'cpd00588_c',
    formula='C6H14O6',
    name='Sorbitol',
    compartment='cytosol')

CD630_07680 = cobra.Gene('CD630_07680')
CD630_07680.name = 'srlD; Sorbitol 6-phosphate 2-dehydrogenase (Glucitol)'
CD630_07680.notes = {'annotation': 'UniProt'}

rxn00634_c = cobra.Reaction('rxn00634_c')
rxn00634_c.gene_reaction_rule = 'CD630_07680'
rxn00634_c.name = 'D-Glucitol:NAD+ 2-oxidoreductase'
rxn00634_c.lower_bound = -1000.
rxn00634_c.upper_bound = 1000.
rxn00634_c.add_metabolites({
    cpd00003_c: -1.0,
    cpd00588_c: -1.0,
    cpd00004_c: 1.0,
    cpd00067_c: 1.0,
    cpd00082_c: 1.0 
})
cd630_PATRIC.add_reactions([rxn00634_c])

#### Biomass composition

Adapted from...

In [20]:
# DNA replication

cpd00115_c = cd630_PATRIC.metabolites.get_by_id('cpd00115_c') # dATP
cpd00356_c = cd630_PATRIC.metabolites.get_by_id('cpd00356_c') # dCTP
cpd00357_c = cd630_PATRIC.metabolites.get_by_id('cpd00357_c') # TTP
cpd00241_c = cd630_PATRIC.metabolites.get_by_id('cpd00241_c') # dGTP
cpd00002_c = cd630_PATRIC.metabolites.get_by_id('cpd00002_c') # ATP
cpd00001_c = cd630_PATRIC.metabolites.get_by_id('cpd00001_c') # H2O

cpd00008_c = cd630_PATRIC.metabolites.get_by_id('cpd00008_c') # ADP
cpd00009_c = cd630_PATRIC.metabolites.get_by_id('cpd00009_c') # Phosphate
cpd00012_c = cd630_PATRIC.metabolites.get_by_id('cpd00012_c') # PPi

cpd17042_c = cobra.Metabolite(
    'cpd17042_c',
    formula='',
    name='DNA polymer',
    compartment='cytosol')
cpd17042_c.notes = {'Macromolecule'}

dna_rxn = cobra.Reaction('dna_rxn')
dna_rxn.name = 'DNA replication'
dna_rxn.notes = {'citations': [], 'type': 'biomass', 'annotation': 'none'}
dna_rxn.lower_bound = 0.
dna_rxn.upper_bound = 1000.
dna_rxn.add_metabolites({
    cpd00115_c: -1.0,
    cpd00356_c: -0.5,
    cpd00357_c: -1.0,
    cpd00241_c: -0.5,
    cpd00002_c: -4.0,
    cpd00001_c: -1.0,
    cpd17042_c: 1.0,
    cpd00008_c: 4.0,
    cpd00009_c: 4.0,
    cpd00012_c: 1.0
})
cd630_PATRIC.add_reactions([dna_rxn])


In [21]:
# RNA transcription

cpd00002_c = cd630_PATRIC.metabolites.get_by_id('cpd00002_c') # ATP
cpd00052_c = cd630_PATRIC.metabolites.get_by_id('cpd00052_c') # CTP
cpd00062_c = cd630_PATRIC.metabolites.get_by_id('cpd00062_c') # UTP
cpd00038_c = cd630_PATRIC.metabolites.get_by_id('cpd00038_c') # GTP
cpd00001_c = cd630_PATRIC.metabolites.get_by_id('cpd00001_c') # H2O

cpd00008_c = cd630_PATRIC.metabolites.get_by_id('cpd00008_c') # ADP
cpd00009_c = cd630_PATRIC.metabolites.get_by_id('cpd00009_c') # Phosphate
cpd00012_c = cd630_PATRIC.metabolites.get_by_id('cpd00012_c') # PPi

cpd17043_c = cobra.Metabolite(
    'cpd17043_c',
    formula='',
    name='RNA polymer',
    compartment='cytosol')
cpd17043_c.notes = {'Macromolecule'}

rna_rxn = cobra.Reaction('rna_rxn')
rna_rxn.notes = {'citations': [], 'type': 'biomass', 'annotation': 'none'}
rna_rxn.notes['citations'] += ''
rna_rxn.name = 'RNA transcription'
rna_rxn.lower_bound = 0.
rna_rxn.upper_bound = 1000.
rna_rxn.add_metabolites({
    cpd00002_c: -2.0,
    cpd00052_c: -0.5,
    cpd00062_c: -0.5,
    cpd00038_c: -0.5,
    cpd00001_c: -1.0,
    cpd17043_c: 1.0,
    cpd00008_c: 2.0,
    cpd00009_c: 2.0,
    cpd00012_c: 1.0
})
cd630_PATRIC.add_reactions([rna_rxn])


In [22]:
# Protein biosynthesis

# tRNA synthesis
cpd00035_c = cd630_PATRIC.metabolites.get_by_id('cpd00035_c') # L-Alanine


cpd00051_c = cd630_PATRIC.metabolites.get_by_id('cpd00051_c') # L-Arginine
cpd00132_c = cd630_PATRIC.metabolites.get_by_id('cpd00132_c') # L-Asparagine
cpd00041_c = cd630_PATRIC.metabolites.get_by_id('cpd00041_c') # L-Aspartate
cpd00084_c = cd630_PATRIC.metabolites.get_by_id('cpd00084_c') # L-Cysteine
cpd00053_c = cd630_PATRIC.metabolites.get_by_id('cpd00053_c') # L-Glutamine
cpd00023_c = cd630_PATRIC.metabolites.get_by_id('cpd00023_c') # L-Glutamate
cpd00033_c = cd630_PATRIC.metabolites.get_by_id('cpd00033_c') # Glycine
cpd00119_c = cd630_PATRIC.metabolites.get_by_id('cpd00119_c') # L-Histidine
cpd00322_c = cd630_PATRIC.metabolites.get_by_id('cpd00322_c') # L-Isoleucine
cpd00107_c = cd630_PATRIC.metabolites.get_by_id('cpd00107_c') # L-Leucine
cpd00039_c = cd630_PATRIC.metabolites.get_by_id('cpd00039_c') # L-Lysine
cpd00060_c = cd630_PATRIC.metabolites.get_by_id('cpd00060_c') # L-Methionine
cpd00066_c = cd630_PATRIC.metabolites.get_by_id('cpd00066_c') # L-Phenylalanine
cpd00129_c = cd630_PATRIC.metabolites.get_by_id('cpd00129_c') # L-Proline
cpd00054_c = cd630_PATRIC.metabolites.get_by_id('cpd00054_c') # L-Serine
cpd00161_c = cd630_PATRIC.metabolites.get_by_id('cpd00161_c') # L-Threonine
cpd00065_c = cd630_PATRIC.metabolites.get_by_id('cpd00065_c') # L-Tryptophan
cpd00069_c = cd630_PATRIC.metabolites.get_by_id('cpd00069_c') # L-Tyrosine
cpd00156_c = cd630_PATRIC.metabolites.get_by_id('cpd00156_c') # L-Valine
cpd00002_c = cd630_PATRIC.metabolites.get_by_id('cpd00002_c') # ATP
cpd00001_c = cd630_PATRIC.metabolites.get_by_id('cpd00001_c') # H2O

cpd00008_c = cd630_PATRIC.metabolites.get_by_id('cpd00008_c') # ADP
cpd00009_c = cd630_PATRIC.metabolites.get_by_id('cpd00009_c') # Phosphate

cpd17041_c = cobra.Metabolite(
    'cpd17041_c',
    formula='',
    name='Protein polymer',
    compartment='cytosol')
cpd17041_c.notes = {'Macromolecule'}

protein_rxn = cobra.Reaction('protein_rxn')
protein_rxn.notes = {'citations': [], 'type': 'biomass', 'annotation': 'none'}
protein_rxn.notes['citations'] += ''
protein_rxn.name = 'Protein biosynthesis'
protein_rxn.lower_bound = 0.
protein_rxn.upper_bound = 1000.
protein_rxn.add_metabolites({
    cpd00035_c: -0.5,
    cpd00051_c: -0.25,
    cpd00132_c: -0.5,
    cpd00041_c: -0.5,
    cpd00084_c: -0.05,
    cpd00053_c: -0.25,
    cpd00023_c: -0.5,
    cpd00033_c: -0.5,
    cpd00119_c: -0.05,
    cpd00322_c: -0.5,
    cpd00107_c: -0.5,
    cpd00039_c: -0.5,
    cpd00060_c: -0.25,
    cpd00066_c: -0.5,
    cpd00129_c: -0.25,
    cpd00054_c: -0.5,
    cpd00161_c: -0.5,
    cpd00065_c: -0.05,
    cpd00069_c: -0.25,
    cpd00156_c: -0.5,
    cpd00002_c: -20.0,
    cpd00001_c: -1.0,
    cpd17041_c: 1.0,
    cpd00008_c: 20.0,
    cpd00009_c: 20.0
})
cd630_PATRIC.add_reactions([protein_rxn])


In [23]:
# Cell wall synthesis

cpd02967_c = cd630_PATRIC.metabolites.get_by_id('cpd02967_c') # N-Acetyl-beta-D-mannosaminyl-1,4-N-acetyl-D-glucosaminyldiphosphoundecaprenol
cpd00402_c = cd630_PATRIC.metabolites.get_by_id('cpd00402_c') # CDPglycerol
cpd00046_c = cd630_PATRIC.metabolites.get_by_id('cpd00046_c') # CMP

cpd12894_c = cobra.Metabolite(
    'cpd12894_c',
    formula='',
    name='Teichoic acid',
    compartment='cytosol')
cpd12894_c.notes = {'type':'Macromolecule'}

CD630_02440 = cobra.Gene('CD630_02440')
CD630_02440.name = 'Putative CDP-glycerol:Poly(Glycerophosphate) glycerophosphotransferase'
CD630_02440.notes = {'annotation': 'UniProt'}

teichoicacid_rxn = cobra.Reaction('teichoicacid_rxn')
teichoicacid_rxn.notes = {'citations': [], 'type': 'biomass', 'annotation': 'none'}
teichoicacid_rxn.notes['citations'] += ''
teichoicacid_rxn.gene_reaction_rule = 'CD630_02440'
teichoicacid_rxn.name = 'Teichoic acid biosynthesis'
teichoicacid_rxn.lower_bound = 0.
teichoicacid_rxn.upper_bound = 1000.
teichoicacid_rxn.add_metabolites({
    cpd02967_c: -1.0,
    cpd00402_c: -1.0,
    cpd00046_c: 1.0,
    cpd12894_c: 1.0
})
cd630_PATRIC.add_reactions([teichoicacid_rxn])

#---------------------------------------#

# Peptidoglycan subunits
 # Undecaprenyl-diphospho-N-acetylmuramoyl--N-acetylglucosamine-L-ala-D-glu-meso-2-6-diaminopimeloyl-D-ala-D-ala (right)
cpd03495_c = cd630_PATRIC.metabolites.get_by_id('cpd03495_c')
# Undecaprenyl-diphospho-N-acetylmuramoyl-(N-acetylglucosamine)-L-alanyl-gamma-D-glutamyl-L-lysyl-D-alanyl-D-alanine (left)
cpd03491_c = cd630_PATRIC.metabolites.get_by_id('cpd03491_c') 

cpd00002_c = cd630_PATRIC.metabolites.get_by_id('cpd00002_c') # ATP
cpd00001_c = cd630_PATRIC.metabolites.get_by_id('cpd00001_c') # H2O

cpd02229_c = cd630_PATRIC.metabolites.get_by_id('cpd02229_c') # Bactoprenyl diphosphate
cpd00117_c = cd630_PATRIC.metabolites.get_by_id('cpd00117_c') # D-Alanine
cpd00008_c = cd630_PATRIC.metabolites.get_by_id('cpd00008_c') # ADP
cpd00009_c = cd630_PATRIC.metabolites.get_by_id('cpd00009_c') # Phosphate

cpd16661_c = cobra.Metabolite(
    'cpd16661_c',
    formula='',
    name='Peptidoglycan polymer',
    compartment='cytosol')
cpd16661_c.notes = {'type':'Macromolecule'}
                            
CD630_26510 = cobra.Gene('CD630_26510')
CD630_26510.name = 'murG; UDP-N-acetylglucosamine--N-acetylmuramyl-(pentapeptide) pyrophosphoryl-undecaprenol N-acetylglucosamine transferase'
CD630_26510.notes = {'annotation': 'UniProt'}

peptidoglycan_rxn = cobra.Reaction('peptidoglycan_rxn')
peptidoglycan_rxn.gene_reaction_rule = 'CD630_26510'
peptidoglycan_rxn.notes = {'citations': [], 'type': 'biomass', 'annotation': 'none'}
peptidoglycan_rxn.name = 'Peptidoglycan synthesis'
peptidoglycan_rxn.lower_bound = 0.
peptidoglycan_rxn.upper_bound = 1000.
peptidoglycan_rxn.add_metabolites({
    cpd03491_c: -1.0,
    cpd03495_c: -1.0,
    cpd00002_c: -4.0,
    cpd00001_c: -1.0,
    cpd16661_c: 1.0,
    cpd02229_c: 1.0,
    cpd00117_c: 0.5, # D-Alanine
    cpd00008_c: 4.0, # ADP
    cpd00009_c: 4.0 # Phosphate
})
cd630_PATRIC.add_reactions([peptidoglycan_rxn])

#---------------------------------------#

cellwall_c = cobra.Metabolite(
    'cellwall_c',
    formula='',
    name='Cell Wall polymer',
    compartment='cytosol')
cellwall_c.notes = {'type':'Macromolecule'}

cellwall_rxn = cobra.Reaction('cellwall_rxn')
cellwall_rxn.name = 'Cell wall biosynthesis'
cellwall_rxn.notes = {'citations': [], 'type': 'biomass', 'annotation': 'none'}
cellwall_rxn.lower_bound = 0.
cellwall_rxn.upper_bound = 1000.
cellwall_rxn.add_metabolites({
    cpd16661_c: -1.5,
    cpd12894_c: -0.05,
    cellwall_c: 1.0
})
cd630_PATRIC.add_reactions([cellwall_rxn])


In [24]:
# Lipid pool

cpd15543_c = cd630_PATRIC.metabolites.get_by_id('cpd15543_c') # Phosphatidylglycerophosphate ditetradecanoyl
cpd15545_c = cd630_PATRIC.metabolites.get_by_id('cpd15545_c') # Phosphatidylglycerophosphate dihexadecanoyl
cpd15540_c = cd630_PATRIC.metabolites.get_by_id('cpd15540_c') # Phosphatidylglycerol dioctadecanoyl
cpd15728_c = cd630_PATRIC.metabolites.get_by_id('cpd15728_c') # Diglucosyl-1,2 dipalmitoylglycerol
cpd15729_c = cd630_PATRIC.metabolites.get_by_id('cpd15729_c') # Diglucosyl-1,2 dimyristoylglycerol
cpd15737_c = cd630_PATRIC.metabolites.get_by_id('cpd15737_c') # Monoglucosyl-1,2 dipalmitoylglycerol
cpd15738_c = cd630_PATRIC.metabolites.get_by_id('cpd15738_c') # Monoglucosyl-1,2 dimyristoylglycerol

cpd11852_c = cobra.Metabolite(
    'cpd11852_c',
    formula='',
    name='Lipid Pool',
    compartment='cytosol')
cpd11852_c.notes = {'type':'Metabolite pool'}

lipid_rxn = cobra.Reaction('lipid_rxn')
lipid_rxn.notes = {'citations': [], 'type': 'biomass', 'annotation': 'none'}
lipid_rxn.notes['citations'] += ''
lipid_rxn.name = 'Lipid composition'
lipid_rxn.lower_bound = 0.
lipid_rxn.upper_bound = 1000.
lipid_rxn.add_metabolites({
    cpd15543_c: -0.005,
    cpd15545_c: -0.005,
    cpd15540_c: -0.005,
    cpd15728_c: -0.005,
    cpd15729_c: -0.005,
    cpd15737_c: -0.005,
    cpd15738_c: -0.005,
    cpd11852_c: 1.0
})
cd630_PATRIC.add_reactions([lipid_rxn])


In [25]:
# Ions, Vitamins, & Cofactors

# Vitamins
cpd00104_c = cd630_PATRIC.metabolites.get_by_id('cpd00104_c') # Biotin          MDM
cpd00644_c = cd630_PATRIC.metabolites.get_by_id('cpd00644_c') # Pantothenate    MDM
cpd00263_c = cd630_PATRIC.metabolites.get_by_id('cpd00263_c') # Pyridoxine      MDM
cpd00393_c = cd630_PATRIC.metabolites.get_by_id('cpd00393_c') #  folate
cpd00133_c = cd630_PATRIC.metabolites.get_by_id('cpd00133_c') #  nicotinamide
cpd00443_c = cd630_PATRIC.metabolites.get_by_id('cpd00443_c') #  p-aminobenzoic acid
cpd00220_c = cd630_PATRIC.metabolites.get_by_id('cpd00220_c') #  riboflavin  
cpd00305_c = cd630_PATRIC.metabolites.get_by_id('cpd00305_c') #  thiamin

# Ions
cpd00149_c = cd630_PATRIC.metabolites.get_by_id('cpd00149_c') # Cobalt
cpd00030_c = cd630_PATRIC.metabolites.get_by_id('cpd00030_c') # Manganese
cpd00254_c = cd630_PATRIC.metabolites.get_by_id('cpd00254_c') # Magnesium
cpd00971_c = cd630_PATRIC.metabolites.get_by_id('cpd00971_c') # Sodium
cpd00063_c = cd630_PATRIC.metabolites.get_by_id('cpd00063_c') # Calcium
cpd10515_c = cd630_PATRIC.metabolites.get_by_id('cpd10515_c') # Iron
cpd00205_c = cd630_PATRIC.metabolites.get_by_id('cpd00205_c') # Potassium
cpd00099_c = cd630_PATRIC.metabolites.get_by_id('cpd00099_c') # Chloride

# Cofactors
cpd00022_c = cd630_PATRIC.metabolites.get_by_id('cpd00022_c') # Acetyl-CoA
cpd00010_c = cd630_PATRIC.metabolites.get_by_id('cpd00010_c') # CoA
cpd00015_c = cd630_PATRIC.metabolites.get_by_id('cpd00015_c') # FAD
cpd00003_c = cd630_PATRIC.metabolites.get_by_id('cpd00003_c') # NAD
cpd00004_c = cd630_PATRIC.metabolites.get_by_id('cpd00004_c') # NADH
cpd00006_c = cd630_PATRIC.metabolites.get_by_id('cpd00006_c') # NADP
cpd00005_c = cd630_PATRIC.metabolites.get_by_id('cpd00005_c') # NADPH

# Energy molecules
cpd00002_c = cd630_PATRIC.metabolites.get_by_id('cpd00002_c') # ATP
cpd00008_c = cd630_PATRIC.metabolites.get_by_id('cpd00008_c') # ADP
cpd00009_c = cd630_PATRIC.metabolites.get_by_id('cpd00009_c') # Phosphate
cpd00012_c = cd630_PATRIC.metabolites.get_by_id('cpd00012_c') # PPi
cpd00038_c = cd630_PATRIC.metabolites.get_by_id('cpd00038_c') # GTP
cpd00031_c = cd630_PATRIC.metabolites.get_by_id('cpd00031_c') # GDP

#cpd00274_c = cd630_PATRIC.metabolites.get_by_id('cpd00274_c') # Citrulline
#cpd00907_c = cd630_PATRIC.metabolites.get_by_id('cpd00907_c') # alpha-D-glucose-1,6-biphosphate

cofactor_c = cobra.Metabolite(
    'cofactor_c',
    formula='',
    name='Cofactor Pool',
    compartment='cytosol')
cofactor_c.notes = {'type':'Metabolite pool'}

cofactor_rxn = cobra.Reaction('cofactor_rxn')
cofactor_rxn.notes = {'citations': [], 'type': 'biomass', 'annotation': 'none'}
cofactor_rxn.notes['citations'] += 'Karasawa T, Ikoma S, Yamakawa K, & Nakamura S. (1995). A defined growth medium for Clostridium difficile.Microbiology. 141(2); 371-5.'
cofactor_rxn.name = 'Cofactor Pool'
cofactor_rxn.lower_bound = 0.
cofactor_rxn.upper_bound = 1000.
cofactor_rxn.add_metabolites({
    cpd00104_c: -0.005, 
    cpd00644_c: -0.005, 
    cpd00263_c: -0.005, 
    cpd00393_c: -0.005,
    cpd00133_c: -0.005,
    cpd00443_c: -0.005,
    cpd00220_c: -0.005,
    cpd00305_c: -0.005,
    cpd00149_c: -0.005, 
    cpd00030_c: -0.005, 
    cpd00254_c: -0.005, 
    cpd00971_c: -0.005, 
    cpd00063_c: -0.005, 
    cpd10515_c: -0.005, 
    cpd00205_c: -0.005,
    cpd00099_c: -0.005, 
    cpd00022_c: -0.005,
    cpd00010_c: -0.0005,
    cpd00015_c: -0.0005,
    cpd00003_c: -0.005,
    cpd00004_c: -0.005,
    cpd00006_c: -0.005,
    cpd00005_c: -0.005,
    cpd00002_c: -0.005, 
    cpd00008_c: -0.005, 
    cpd00009_c: -0.5,
    cpd00012_c: -0.005, 
    cpd00038_c: -0.005, 
    cpd00031_c: -0.005, 

    cofactor_c: 1.0
})
cd630_PATRIC.add_reactions([cofactor_rxn])


In [26]:
# Final Biomass

cpd11416_c = cobra.Metabolite(
    'cpd11416_c',
    formula='',
    name='Biomass',
    compartment='cytosol')
cpd11416_c.notes = {'type':'Biomass'}

biomass_rxn = cobra.Reaction('biomass')
biomass_rxn.notes = {'citations': [], 'type': 'biomass', 'annotation': 'none'}
biomass_rxn.name = 'Biomass Reaction'
biomass_rxn.lower_bound = 0.
biomass_rxn.upper_bound = 1000.
biomass_rxn.add_metabolites({
    cpd17041_c: -0.4,  # Protein
    cpd17043_c: -0.15, # RNA
    cpd17042_c: -0.05, # DNA
    cpd11852_c: -0.05, # Lipid
    cellwall_c: -0.2,
    cofactor_c: -0.2,
    cpd00001_c: -20.0,
    cpd00002_c: -20.0,
    cpd00008_c: 20.0,
    cpd00009_c: 20.0,
    cpd11416_c: 1.0 # Biomass
})
cd630_PATRIC.add_reactions([biomass_rxn])

cd630_PATRIC.add_boundary(cpd11416_c, type='sink', reaction_id='EX_biomass', lb=0.0, ub=1000.0)

# Set new objective
cd630_PATRIC.objective = biomass_rxn


In [27]:
# Deal with extracellular metabolites without transporters          # sulfate     # CoA
for cpd in ['cpd10516_e', 'cpd01188_e', 'cpd00028_e', 'cpd11606_e', 'cpd00048_e', 'cpd00010_e']:
    try:
        cd630_PATRIC.metabolites.get_by_id(cpd).remove_from_model()
    except:
        pass

#---------------------------------------------------------------------------#

# Manually add necessary reactions

# Diffusion reactions

# H2O
cpd00001_e = cd630_PATRIC.metabolites.get_by_id('cpd00001_e')
cpd00001_c = cd630_PATRIC.metabolites.get_by_id('cpd00001_c')
rxn08686_c = cobra.Reaction('rxn08686_c') 
rxn08686_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED'}
rxn08686_c.name = 'H2O transport via diffusion'
rxn08686_c.lower_bound = -1000.
rxn08686_c.upper_bound = 1000.
rxn08686_c.add_metabolites({
    cpd00001_e: -1.0,
    cpd00001_c: 1.0
})
cd630_PATRIC.add_reactions([rxn08686_c])

# O2
cpd00007_e = cd630_PATRIC.metabolites.get_by_id('cpd00007_e')
cpd00007_c = cd630_PATRIC.metabolites.get_by_id('cpd00007_c')
rxn09031_c = cobra.Reaction('rxn09031_c') 
rxn09031_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED'}
rxn09031_c.name = 'Oxygen transport via diffusion'
rxn09031_c.lower_bound = -1000.
rxn09031_c.upper_bound = 1000.
rxn09031_c.add_metabolites({
    cpd00007_e: -1.0,
    cpd00007_c: 1.0
})
cd630_PATRIC.add_reactions([rxn09031_c])

# CO2
cpd00011_e = cd630_PATRIC.metabolites.get_by_id('cpd00011_e')
cpd00011_c = cd630_PATRIC.metabolites.get_by_id('cpd00011_c')
rxn05467_c = cobra.Reaction('rxn05467_c') 
rxn05467_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED'}
rxn05467_c.name = 'CO2 transport via diffusion'
rxn05467_c.lower_bound = -1000.
rxn05467_c.upper_bound = 1000.
rxn05467_c.add_metabolites({
    cpd00011_e: -1.0,
    cpd00011_c: 1.0
})
cd630_PATRIC.add_reactions([rxn05467_c])

# Ca2+
cpd00063_e = cd630_PATRIC.metabolites.get_by_id('cpd00063_e')
cpd00063_c = cd630_PATRIC.metabolites.get_by_id('cpd00063_c')
rxn08186_c = cobra.Reaction('rxn08186_c') 
rxn08186_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED'}
rxn08186_c.name = 'Calcium transport via diffusion'
rxn08186_c.lower_bound = -1000.
rxn08186_c.upper_bound = 1000.
rxn08186_c.add_metabolites({
    cpd00063_e: -1.0,
    cpd00063_c: 1.0
})
cd630_PATRIC.add_reactions([rxn08186_c])

# Mn2+
cpd00030_e = cd630_PATRIC.metabolites.get_by_id('cpd00030_e')
cpd00030_c = cd630_PATRIC.metabolites.get_by_id('cpd00030_c')
rxn08952_c = cobra.Reaction('rxn08952_c') 
rxn08952_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED'}
rxn08952_c.name = 'Manganese transport via diffusion'
rxn08952_c.lower_bound = -1000.
rxn08952_c.upper_bound = 1000.
rxn08952_c.add_metabolites({
    cpd00030_e: -1.0,
    cpd00030_c: 1.0
})
cd630_PATRIC.add_reactions([rxn08952_c])

#---------------------------------#

# Active transport

# ocdca
cpd01080_e = cd630_PATRIC.metabolites.get_by_id('cpd01080_e')
cpd01080_c = cd630_PATRIC.metabolites.get_by_id('cpd01080_c')
cpd00067_e = cd630_PATRIC.metabolites.get_by_id('cpd00067_e')
cpd00067_c = cd630_PATRIC.metabolites.get_by_id('cpd00067_c')

rxn10161_c = cobra.Reaction('rxn10161_c') 
rxn10161_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED'}
rxn10161_c.name = 'Octadecanoate transport via proton symport '
rxn10161_c.lower_bound = -1000.
rxn10161_c.upper_bound = 1000.
rxn10161_c.add_metabolites({
    cpd01080_e: -1.0,
    cpd00067_e: -1.0,
    cpd01080_c: 1.0,
    cpd00067_c: 1.0
})
cd630_PATRIC.add_reactions([rxn10161_c])

# Spermidine
cpd00264_e = cd630_PATRIC.metabolites.get_by_id('cpd00264_e')
cpd00264_c = cd630_PATRIC.metabolites.get_by_id('cpd00264_c')
cpd00002_c = cd630_PATRIC.metabolites.get_by_id('cpd00002_c')
cpd00008_c = cd630_PATRIC.metabolites.get_by_id('cpd00008_c')
cpd00009_c = cd630_PATRIC.metabolites.get_by_id('cpd00009_c')

rxn05175_c = cobra.Reaction('rxn05175_c') 
rxn05175_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED', 'subsystem': 'ABC transporter'}
rxn05175_c.name = 'Spermidine transport via ABC system'
rxn05175_c.lower_bound = 0.
rxn05175_c.upper_bound = 1000.
rxn05175_c.add_metabolites({
    cpd00001_c: -1.0,
    cpd00002_c: -1.0,
    cpd00264_e: -1.0,
    cpd00008_c: 1.0,
    cpd00009_c: 1.0,
    cpd00067_c: 1.0,
    cpd00264_c: 1.0
})
cd630_PATRIC.add_reactions([rxn05175_c])

# meso-2,6-Diaminopimelate
cpd00516_e = cd630_PATRIC.metabolites.get_by_id('cpd00516_e')
cpd00516_c = cd630_PATRIC.metabolites.get_by_id('cpd00516_c')

rxn08304_c = cobra.Reaction('rxn08304_c') 
rxn08304_c.notes = {'citations': [], 'type': 'transport', 'annotation': 'ModelSEED', 'subsystem': 'ABC transporter'}
rxn08304_c.name = 'M-diaminopimelic acid ABC transport'
rxn08304_c.lower_bound = 0.
rxn08304_c.upper_bound = 1000.
rxn08304_c.add_metabolites({
    cpd00001_c: -1.0,
    cpd00002_c: -1.0,
    cpd00516_e: -1.0,
    cpd00008_c: 1.0,
    cpd00009_c: 1.0,
    cpd00067_c: 1.0,
    cpd00516_c: 1.0
})
cd630_PATRIC.add_reactions([rxn08304_c])


In [28]:
# Check irreversibility of ATP consuming reactions

reversible = []
for rxn in list(cd630_PATRIC.metabolites.get_by_id('cpd00002_c').reactions):
    if rxn.reversibility == True:
        reversible.append(rxn.id)
print('Reversible reactions containing ATP: ' + str(len(reversible)))

# Set correct reversibility
cd630_PATRIC.reactions.get_by_id('rxn08298_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn01396_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn02503_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn01517_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn02262_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn08297_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn10257_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn10256_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00392_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00134_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00669_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn08295_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn01100_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn02314_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00337_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn01434_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn01917_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn01127_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn02175_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn01513_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn02489_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00547_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn01343_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00077_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn03536_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn01739_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00151_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn01807_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn01135_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn10255_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00772_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn08294_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn01607_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00690_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn03075_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn02362_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00303_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn10258_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn01671_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn05054_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn06672_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn09320_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00364_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn08647_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn04384_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn07584_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00695_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn02937_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn10253_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00615_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn01114_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn12510_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00124_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00181_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00440_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn03108_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn01219_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn10042_c').lower_bound = 0. # Proton motive force...
cd630_PATRIC.reactions.get_by_id('rxn00239_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn02484_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00770_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn03908_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn05814_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00105_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn07841_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn08296_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn01300_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn01951_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn08299_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn01492_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn01492_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00097_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn01509_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00780_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn01754_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn02155_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00545_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn01102_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn10254_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn05145_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00119_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn01199_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00410_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn08300_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn03540_c').lower_bound = 0.

cd630_PATRIC.reactions.get_by_id('rxn01236_c').upper_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn01987_c').upper_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00985_c').upper_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00148_c').upper_bound = 0.

Reversible reactions containing ATP: 87


In [29]:
# Check irreversibility of GTP consuming reactions

reversible = []
for rxn in list(cd630_PATRIC.metabolites.get_by_id('cpd00038_c').reactions):
    if rxn.reversibility == True:
        reversible.append(rxn.id)
print('Reversible reactions containing GTP: ' + str(len(reversible)))

# Fix bounds
cd630_PATRIC.reactions.get_by_id('rxn04413_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn03537_c').lower_bound = 0.
cd630_PATRIC.reactions.get_by_id('rxn00641_c').lower_bound = 0.


# GTP generation
cpd00002_c = cd630_PATRIC.metabolites.get_by_id('cpd00002_c')
cpd00031_c = cd630_PATRIC.metabolites.get_by_id('cpd00031_c')
cpd00008_c = cd630_PATRIC.metabolites.get_by_id('cpd00008_c')
cpd00038_c = cd630_PATRIC.metabolites.get_by_id('cpd00038_c')
rxn00237_c = cobra.Reaction('rxn00237_c') 
rxn00237_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED', 'subsystem': 'Nucleotide biosynthesis'}
rxn00237_c.name = 'GDP phosphotransferase'
rxn00237_c.lower_bound = 0.
rxn00237_c.upper_bound = 1000.
rxn05175_c.add_metabolites({
    cpd00031_c: -1.0,
    cpd00002_c: -1.0,
    cpd00008_c: 1.0,
    cpd00038_c: 1.0
})
cd630_PATRIC.add_reactions([rxn00237_c])


Reversible reactions containing GTP: 3


In [30]:
# Remaining minimal media components

# Ammonia (extracellular)
cpd00013_e = cobra.Metabolite(
    'cpd00013_e',
    formula='H4N',
    name='Ammonia',
    compartment='extracellular')
cd630_PATRIC.add_boundary(cpd00013_e, type='exchange', reaction_id='EX_cpd00013_e', lb=-1000.0, ub=1000.0)
cpd00013_c = cd630_PATRIC.metabolites.get_by_id('cpd00013_c')
rxn05466_c = cobra.Reaction('rxn05466_c') 
rxn05466_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn05466_c.name = 'Ammonia diffusion'
rxn05466_c.lower_bound = -1000.
rxn05466_c.upper_bound = 1000.
rxn05466_c.add_metabolites({
    cpd00013_e: -1.0,
    cpd00013_c: 1.0
})
cd630_PATRIC.add_reactions([rxn05466_c])

# Carbonate (extracellular)
cpd00242_c = cd630_PATRIC.metabolites.get_by_id('cpd00242_c')
cpd00242_e = cobra.Metabolite(
    'cpd00242_e',
    formula='CHO3',
    name='Carbonate',
    compartment='extracellular')
cd630_PATRIC.add_boundary(cpd00242_e, type='exchange', reaction_id='EX_cpd00242_e', lb=-1000.0, ub=1000.0)

rxn10826_c = cobra.Reaction('rxn10826_c') 
rxn10826_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn10826_c.name = 'Carbonate diffusion'
rxn10826_c.lower_bound = -1000.
rxn10826_c.upper_bound = 1000.
rxn10826_c.add_metabolites({
    cpd00242_e: -1.0,
    cpd00242_c: 1.0
})
cd630_PATRIC.add_reactions([rxn10826_c])


In [31]:
# FAD biosynthesis
cpd00002_c = cd630_PATRIC.metabolites.get_by_id('cpd00002_c') # ATP
cpd00050_c = cd630_PATRIC.metabolites.get_by_id('cpd00050_c') # FMN
cpd00012_c = cd630_PATRIC.metabolites.get_by_id('cpd00012_c') # PPi
cpd00015_c = cd630_PATRIC.metabolites.get_by_id('cpd00015_c') # FAD

try:
    cd630_PATRIC.reactions.get_by_id('rxn30210_c').remove_from_model()
except:
    pass

rxn30210_c = cobra.Reaction('rxn30210_c') 
rxn30210_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn30210_c.name = 'FAD synthetase'
rxn30210_c.gene_reaction_rule = '272563.8.peg.1382 and 272563.8.peg.1382'
rxn30210_c.lower_bound = 0.
rxn30210_c.upper_bound = 1000.
rxn30210_c.add_metabolites({
    cpd00002_c: -1.0,
    cpd00050_c: -1.0,
    cpd00012_c: 1.0,
    cpd00015_c: 1.0
})
cd630_PATRIC.add_reactions([rxn30210_c])

In [32]:
# Import of last missing vitamins

# Nicotinamide
cpd00133_e = cobra.Metabolite(
    'cpd00133_e',
    formula='C6H6N2O',
    name='Nicotinamide',
    compartment='extracellular')
cd630_PATRIC.add_boundary(cpd00133_e, type='exchange', reaction_id='EX_cpd00133_e', lb=-1000.0, ub=1000.0)
cpd00133_c = cd630_PATRIC.metabolites.get_by_id('cpd00133_c')
cpd00067_c = cd630_PATRIC.metabolites.get_by_id('cpd00067_c')
cpd00067_e = cd630_PATRIC.metabolites.get_by_id('cpd00067_e')
rxn11334_c = cobra.Reaction('rxn11334_c') 
rxn11334_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn11334_c.name = 'Nicotinamide symporter'
rxn11334_c.gene_reaction_rule = '272563.8.peg.164 or 272563.8.peg.1796'
rxn11334_c.lower_bound = 0.
rxn11334_c.upper_bound = 1000.
rxn11334_c.add_metabolites({
    cpd00133_e: -1.0,
    cpd00067_e: -1.0,
    cpd00133_c: 1.0,
    cpd00067_c: 1.0
})
cd630_PATRIC.add_reactions([rxn11334_c])

# p-Aminobenzoic acid (ABEE)
cpd00443_e = cobra.Metabolite(
    'cpd00443_e',
    formula='C7H6NO2',
    name='Nicotinamide',
    compartment='extracellular')
cd630_PATRIC.add_boundary(cpd00443_e, type='exchange', reaction_id='EX_cpd00443_e', lb=-1000.0, ub=1000.0)
cpd00443_c = cd630_PATRIC.metabolites.get_by_id('cpd00443_c') #  p-aminobenzoic acid
rxn10421_c = cobra.Reaction('rxn10421_c') 
rxn10421_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn10421_c.name = 'p-Aminobenzoic acid symporter'
rxn10421_c.gene_reaction_rule = '272563.8.peg.164 or 272563.8.peg.1796'
rxn10421_c.lower_bound = 0.
rxn10421_c.upper_bound = 1000.
rxn10421_c.add_metabolites({
    cpd00443_e: -1.0,
    cpd00067_e: -1.0,
    cpd00443_c: 1.0,
    cpd00067_c: 1.0
})
cd630_PATRIC.add_reactions([rxn10421_c])

# Folate, Thiamin, and Riboflavin in place already

In [33]:
# Check progress
cd630_PATRIC

Name,272563.8
Memory address,0x07f84b432b950
Number of metabolites,1306
Number of reactions,1182
Objective expression,-1.0*biomass_reverse_01e59 + 1.0*biomass
Compartments,"cytosol, extracellular"


In [34]:
cd630_PATRIC.slim_optimize()

0.0

In [35]:
cobra.io.write_sbml_model(cd630_PATRIC, 'data/cdiff_temp.sbml')

### Gapfilling (complete media)

In [17]:
# Dependencies
import numpy
import copy
import time
import cobra
from cobra.util import solver
from optlang.symbolics import Zero
from itertools import chain
from cobra.manipulation.delete import *
from cobra.flux_analysis.sampling import ACHRSampler

# pFBA gapfiller
def pfba_gapfill(model, reaction_bag, sample=True, extracellular='e', tasks=False):
    '''
    Function that utilizes iterations of pFBA solution with a universal reaction bag 
    in order to gapfill a model.
    
    Parameters
    ----------
    model : cobra.Model
        Model to be gapfilled
    reaction_bag : cobra.Model
        Reaction bag reference to use during gapfilling
    sample : bool
        Perform flux sampling to identify all possible active reactions from universal
    extracellular : string
        Label for extracellular compartment of model
    tasks : list
        List of strings representing reaction IDs that must be able to carry flux
    '''
    start_time = time.time()
    
    # Save some basic network info for downstream membership testing
    orig_rxn_ids = set([str(w.id) for w in model.reactions])
    orig_cpd_ids = set([str(x.id) for x in model.metabolites])
    univ_rxn_ids = set([str(y.id) for y in reaction_bag.reactions])
    
    # Find overlap in model and reaction bag
    overlap_rxn_ids = univ_rxn_ids.intersection(orig_rxn_ids)
    
    # Get model objective reaction ID
    obj = get_objective(model)

    # Modify universal reaction bag
    new_rxn_ids = set()
    print('Creating universal model')
    with reaction_bag as universal:

        # Remove overlapping reactions from universal bag, and reset objective if needed
        for rxn in overlap_rxn_ids: universal.reactions.get_by_id(rxn).remove_from_model()

        # Add model reactions to universal bag
        universal.add_reactions(list(copy.deepcopy(model.reactions)))
        
        # Set up pFBA objective
        print('Formulating pFBA objective')
        rxn_vars = ((rxn.forward_variable, rxn.reverse_variable) for rxn in universal.reactions if rxn.id not in orig_rxn_ids)
        rxn_vars = chain(*rxn_vars)
        orig_rxn_vars = ((rxn.forward_variable, rxn.reverse_variable) for rxn in universal.reactions if rxn.id in orig_rxn_ids)
        orig_rxn_vars = chain(*orig_rxn_vars)
        pfba_obj = universal.problem.Objective(Zero, direction='min', sloppy=True)
        universal.objective = pfba_obj
        universal.objective.set_linear_coefficients({x: 1.0 for x in rxn_vars})
        universal.objective.set_linear_coefficients({y: 0.0 for y in orig_rxn_vars})
        universal.solver.update()
        
        # Set minimum lower bound for previous objective
        universal.reactions.get_by_id(obj).lower_bound = 1.0
        
        # Set lower bounds for metaboloic tasks
        if tasks == True:
            for rxn in tasks:
                try:
                    universal.reactions.get_by_id(rxn).lower_bound = 0.001
                except:
                    continue
                
        # Run FBA and save solution
        solution = universal.optimize()
        
        # Identify reactions from universal that are now active
        if sample == True:
            print('Generating flux sampling object')
            optgp_object = ACHRSampler(universal)
            
            print('Sampling flux distribution for active reactions')
            flux_samples = optgp_object.sample(1000)
            for distribution in flux_samples.iterrows():
                for rxn in list(flux_samples.columns):
                    if rxn in orig_rxn_ids:
                        continue
                    elif abs(distribution[1][rxn]) > 1e-6:
                        new_rxn_ids |= set([rxn])
            
        else:
            print('Finding active reactions in single flux distribution')
            rxns = list(solution.fluxes.index)
            fluxes = list(solution.fluxes)
            for flux in range(0, len(fluxes)):
                if rxns[flux] in orig_rxn_ids:
                    continue
                elif abs(fluxes[flux]) > 1e-6:
                    new_rxn_ids |= set([rxns[flux]])
    
    # Screen new reaction IDs
    new_rxn_ids = new_rxn_ids.difference(orig_rxn_ids)
    
    # Get reactions and metabolites to be added to the model
    new_rxns = copy.deepcopy([reaction_bag.reactions.get_by_id(rxn) for rxn in new_rxn_ids])
    new_cpd_ids = set()
    for rxn in new_rxns: new_cpd_ids |= set([str(x.id) for x in list(rxn.metabolites)])
    new_cpd_ids = new_cpd_ids.difference(orig_cpd_ids)
    new_cpds = copy.deepcopy([reaction_bag.metabolites.get_by_id(cpd) for cpd in new_cpd_ids])
    
    # Create gapfilled model 
    new_model = copy.deepcopy(model)
    new_model.add_metabolites(new_cpds)
    new_model.add_reactions(new_rxns)
    
    # Identify extracellular metabolites still need exchanges
    new_exchanges = extend_exchanges(new_model, new_cpd_ids, extracellular)
    
    # Report to user
    new_rxn_ids = len(new_rxn_ids) + new_exchanges
    new_cpd_ids = len(new_cpd_ids)
    print('Gapfilled ' + str(new_rxn_ids) + ' reactions and ' + str(new_cpd_ids) + ' metabolites') 
    if new_model.slim_optimize() > 1e-6: print('Objective now carries flux')

    duration = str(round(((time.time() - start_time) / 60.0), 1)).split('.')
    mins = duration[0]
    secs = str(round(((float(duration[1]) / 10.0) + 60.0)))
    print('Finished in ' + mins + ' minutes and ' + secs + ' seconds')
    return new_model


# Adds missing exchanges for extracellulart metabolites
def extend_exchanges(model, cpd_ids, outside):
        
    model_exchanges = set()
    for rxn in model.reactions:
        if len(rxn.reactants) == 0 or len(rxn.products) == 0:
            if outside in [str(cpd.compartment) for cpd in rxn.metabolites]:
                model_exchanges |= set([rxn.id])

    new_ex_ids = 0
    cpds = [model.metabolites.get_by_id(cpd) for cpd in cpd_ids if model.metabolites.get_by_id(cpd).compartment == outside]
    for cpd in cpds:
        rxns = set([str(rxn.id) for rxn in cpds.reactions])
        if bool(rxns & model_exchanges) == False:
            new_id = 'EX_' + cpd.id
            model.add_boundary(cpd, type='exchange', reaction_id=new_id, lb=-1000.0, ub=1000.0)
            new_ex_ids += 1

    return new_ex_ids


# Returns the reaction ID of the objective reaction
def get_objective(model):
    
    if len(list(model.objective.variables)) == 0:
        raise IndexError('Model has no objective set.')
    
    expression = str(model.objective.expression).split()
    if 'reverse' in expression[0]:
        obj_id = expression[2].split('*')[-1]
    else:
        obj_id = expression[0].split('*')[-1]
            
    return obj_id



In [38]:
import copy
import cobra
import symengine

# pFBA gapfiller
def pfba_gapfill(model, objective, universal, extracellular='extracellular', ignore=[]):
    start_time = time.time()
    '''
    Parameters
    ----------
    model_file : str
        Model to be gapfilled
    objective : 
    universal_file : str
        Reaction bag reference
    iters : int
        Number of iterations to repeat gapfilling process
    extracellular : str
        Label for extracellular compartment of model
    '''
    # Create temporary models to modify for gapfilling
    temp_model = copy.deepcopy(model)
    temp_universal = copy.deepcopy(universal)

    # Save some basic network info for downstream membership testing
    target_rxns = set([str(x.id) for x in model.reactions])
    target_cpds = set([str(y.id) for y in model.metabolites])
    ref_rxns = set([str(z.id) for z in universal.reactions])
    shared_rxns = ref_rxns.intersection(target_rxns)
    
    # Define minimum objective value
    obj_val = temp_model.slim_optimize()
    obj_constraint = constrained_model.problem.Constraint(temp_model.objective.expression, lb=obj_val*0.01, ub=obj_val)
    temp_model.add_cons_vars([prev_obj_constraint])
    temp_model.solver.update()
                
    # Open exchanges are all open
    for rxn in temp_model.reactions:
        if len(rxn.reactants) == 0 or len(rxn.products) == 0:
            rxn.bounds = (min(rxn.lower_bound, -1000.), max(rxn.upper_bound, 1000.))
    for rxn in temp_universal.reactions:
        if len(rxn.reactants) == 0 or len(rxn.products) == 0:
            rxn.bounds = (min(rxn.lower_bound, -1000.), max(rxn.upper_bound, 1000.))
        
    # Remove overlapping reactions from universal bag, add model reactions to universal bag
    for rxn in shared_rxns: temp_universal.reactions.get_by_id(rxn).remove_from_model()
    temp_universal.add_reactions(list(copy.deepcopy(model.reactions)))
        
    # Set up pFBA objective
    pfba_expr = symengine.RealDouble(0)
    for rxn in temp_universal.reactions:
        if rxn.id in target_rxns:
            pfba_expr += 1.0 * rxn.forward_variable
            pfba_expr += 1.0 * rxn.reverse_variable
        else:
            pfba_expr += 0.0 * rxn.forward_variable
            pfba_expr += 0.0 * rxn.reverse_variable
        
    temp_universal.objective = temp_universal.problem.Objective(pfba_expr, direction='min', sloppy=True)
    temp_universal.solver.update()

    # Run FBA and save solution
    solution = temp_universal.optimize()
    active_rxns = set([rxn.id for rxn in temp_universal.reactions if abs(solution.fluxes[rxn.id]) > 1e-6])
    
    # Screen new reaction IDs
    new_rxns = active_rxns.difference(target_rxns)
    
    # Get reactions and metabolites to be added to the model
    new_rxns = copy.deepcopy([temp_universal.reactions.get_by_id(rxn) for rxn in new_rxns])
    new_cpds = set()
    for rxn in new_rxns: new_cpds |= set([str(x.id) for x in list(rxn.metabolites)]).difference(target_cpds)
    new_cpds = copy.deepcopy([temp_universal.metabolites.get_by_id(cpd) for cpd in new_cpds])
    
    # Create gapfilled model 
    new_model = copy.deepcopy(model)
    new_model.add_metabolites(new_cpds)
    new_model.add_reactions(new_rxns)
    
    # Identify extracellular metabolites still need exchanges
    new_exchs = 0
    model_exchanges = set()
    rxns = set([str(rxn.id) for rxn in model.reactions])
    for rxn in new_model.reactions:
        if len(rxn.reactants) == 0 or len(rxn.products) == 0:
            if extracellular in [str(cpd.compartment) for cpd in rxn.metabolites]:
                model_exchanges |= set([rxn.id])                
    for cpd in new_model.metabolites:
        if cpd.compartment != extracellular: continue
        current_rxns = set([x.id for x in cpd.reactions])
        if bool(current_rxns & model_exchanges) == False:
            new_id = 'EX_' + cpd.id
            new_model.add_boundary(cpd, type='exchange', reaction_id=new_id, lb=-1000.0, ub=1000.0)
            new_exchs += 1
    
    # Report to user
    print('\nGapfilled ' + str(len(new_rxns) + new_exchs) + ' reactions and ' + str(len(new_cpds)) + ' metabolites') 
    if new_model.slim_optimize() <= 1e-6: print('WARNING: Objective does not carries flux')
    
    return new_model


In [34]:
cd630_PATRIC = cobra.io.read_sbml_model('data/cdiff_temp.sbml')

In [35]:
# Read in curated universal reacton bag
universal = cobra.io.load_json_model('data/universal.json')

In [36]:
# Gap filling to biomass on complete media

# Set metabolic tasks
scfa_rxns = ['R01174_5_c','rxn05061_c','R01174_4_c','R01174_3_c','R01174_2_c','rxn01236_c',
             'R01174_1_c','rxn11676_c','rxn01610_c','rxn07124_c','rxn00669_c','rxn13713_c', 'R11462_c']

# Flux-sampling pFBA gapfill
cd630_PATRIC_gapfilled = pfba_ensemble_gapfill(cd630_PATRIC, universal, extracellular='extracellular', tasks=scfa_rxns)

Progress: 100%   
Gapfilled 30 reactions and 0 metabolites
Objective now carries flux
Finished in 18 minutes and 60.0 seconds


In [39]:
cd630_PATRIC_gapfilled

Name,272563.8
Memory address,0x07f1b7cb9e050
Number of metabolites,1306
Number of reactions,1212
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"cytosol, extracellular"


In [1]:
# Rerun gapfill until no new reactions were found, reseting the kernel each time

iter1 = ['rxn02269_c', 'rxn09142_c', 'rxn09145_c', 'rxn00620_c', 'rxn01105_c', 'rxn05293_c', 'rxn02286_c', 'rxn09147_c', 
         'rxn01953_c', 'rxn02011_c', 'rxn07466_c', 'rxn09429_c', 'rxn03852_c', 'rxn13140_c', 'rxn09065_c', 'rxn01520_c', 
         'rxn13914_c', 'rxn09063_c', 'rxn09348_c', 'rxn05744_c', 'rxn08040_c']
iter2 = ['rxn02269_c', 'rxn13139_c', 'rxn00620_c', 'rxn05293_c', 'rxn09341_c', 'rxn02286_c', 'rxn01953_c', 'rxn02011_c',
         'rxn07466_c', 'rxn09429_c', 'rxn09126_c', 'rxn09124_c', 'rxn13140_c', 'rxn09065_c', 'rxn01520_c', 'rxn00553_c',
         'rxn03852_c', 'rxn13914_c', 'rxn09063_c', 'rxn05744_c', 'rxn08040_c', 'rxn09163_c']
iter3 = ['rxn02269_c', 'rxn09145_c', 'rxn13139_c', 'rxn00620_c', 'rxn05293_c', 'rxn02286_c', 'rxn09147_c', 'rxn01953_c', 'rxn00605_c',
         'rxn02011_c', 'rxn00579_c', 'rxn07466_c', 'rxn09429_c', 'rxn03852_c', 'rxn13140_c', 'rxn09065_c', 'rxn03164_c', 'rxn01520_c',
         'rxn09063_c', 'rxn09348_c', 'rxn05744_c', 'rxn08040_c', 'rxn09163_c']
iter4 = ['rxn02269_c', 'rxn13139_c', 'rxn00620_c', 'rxn05293_c', 'rxn02286_c', 'rxn01953_c', 'rxn00605_c', 'rxn02011_c', 'rxn00579_c',
         'rxn07466_c', 'rxn09429_c', 'rxn09126_c', 'rxn09124_c', 'rxn13140_c', 'rxn09065_c', 'rxn01520_c', 'rxn03852_c', 'rxn13914_c',
         'rxn09063_c', 'rxn09348_c', 'rxn05744_c', 'rxn08040_c', 'rxn09163_c']

new_rxn_ids = set(iter1 + iter2 + iter3 + iter4)
print('Reactions to be added: ' + str(len(new_rxn_ids)))


Reactions to be added: 30


In [41]:
#cobra.io.write_sbml_model(cd630_PATRIC_gapfilled, 'data/cdiff_temp2.sbml')

##### Gapfilling to minimal media

In [40]:
# Function to change media condition based on a list of exchange reaction IDs
def changeMedia(full_model, components):
    model = copy.deepcopy(full_model)
    
    media = copy.copy(components)
    media = ['EX_' + x for x in media]
    
    for rxn in model.reactions:
        if len(list(rxn.products)) == 0:
            if rxn.id in media:
                model.reactions.get_by_id(rxn.id).lower_bound = -1000.0
                media.remove(rxn.id)
            else:
                model.reactions.get_by_id(rxn.id).lower_bound = 0.0
    if len(media) != 0:
        print(media)
    return model

# Undoes change media conditions
def unblockExch(model):
    exchanges = []
    for rxn in model.reactions:
        if len(rxn.reactants) == 0 or len(rxn.products) == 0:
            exchanges.append(rxn.id)
    
    for rxn in exchanges:
        model.reactions.get_by_id(rxn).lower_bound = -1000.0
        model.reactions.get_by_id(rxn).upper_bound = 1000.0
    
    # Keep anaerobic
    model.reactions.get_by_id('EX_cpd00532_e').lower_bound = 0.0
    model.reactions.get_by_id('EX_cpd00532_e').upper_bound = 0.0
            
    return model


# Actually prune all unused metabolites and reactions (innate function does not work)
def removeUnused(model):
    removed_cpd = set()
    removed_rxn = set()
    unused_current_cpd = 1
    unused_current_rxn = 1
    
    while unused_current_cpd != 0 or unused_current_rxn != 0:
        unused_cpd = prune_unused_metabolites(model)
        removed_cpd |= set(unused_cpd)
        unused_rxn = prune_unused_reactions(model)
        removed_rxn |= set(unused_rxn)
        
        unused_current_cpd = len(unused_cpd)
        unused_current_rxn = len(unused_rxn)
    
    print('Pruned ' + str(len(removed_cpd)) + ' metabolites from model')
    print('Pruned ' + str(len(removed_rxn)) + ' reactions from model')
        
    return(list(removed_cpd), list(removed_rxn))


In [41]:
# Karlsson et al. (1999). Microbiology.
mdm = ['cpd00001_e', # water
       'cpd00065_e', # L-Tryptophan
       'cpd00060_e', # L-Methionine
       'cpd00322_e', # L-Isoleucine
       'cpd00129_e', # L-Proline
       'cpd00156_e', # L-Valine
       'cpd00107_e', # L-Leucine
       'cpd00084_e', # L-Cysteine 
       'cpd00149_e', # Cobalt
       'cpd00099_e', # Chloride
       'cpd10515_e', # Iron
       'cpd00030_e', # Manganese
       'cpd00254_e', # Magnesium
       'cpd00063_e', # Calcium
       'cpd00205_e', # Potassium
       'cpd00009_e', # Phosphate
       'cpd00971_e', # Sodium
       'cpd00242_e', # Carbonate
       'cpd00104_e', # Biotin
       'cpd00644_e', # Pantothenate
       'cpd00263_e', # Pyridoxine
       'cpd00027_e'] # D-Glucose (Carbohydrate C-source)

# Theriot et al. (2013). Nature Communications.
ncmm = ['cpd00001_e', # water
        'cpd00104_e', # Biotin
        'cpd00644_e', # Pantothenate
        'cpd00263_e', # Pyridoxine
        'cpd00149_e', # Cobalt
        'cpd00099_e', # Chloride
        'cpd10515_e', # Iron
        'cpd00030_e', # Manganese
        'cpd00254_e', # Magnesium
        'cpd00063_e', # Calcium
        'cpd00205_e', # Potassium
        'cpd00009_e', # Phosphate
        'cpd00971_e', # Sodium
        'cpd00242_e', # Carbonate
        'cpd00322_e', # L-Isoleucine
        'cpd00129_e', # L-Proline
        'cpd00156_e', # L-Valine
        'cpd00107_e', # L-Leucine
        'cpd00084_e', # L-Cysteine 
        'cpd00065_e', # L-Tryptophan
        'cpd00060_e', # L-Methionine
        'cpd00119_e', # L-Histidine
        'cpd00033_e', # Glycine
        'cpd00051_e', # L-Arginine
        'cpd00161_e'] # L-Threonine


In [44]:
#cd630_PATRIC_gapfilled = cobra.io.read_sbml_model('data/cdiff_temp2.sbml')

In [42]:
# change to C. difficle minimal media
cd630_PATRIC_mdm = changeMedia(cd630_PATRIC_gapfilled, mdm)
cd630_PATRIC_min = pfba_ensemble_gapfill(cd630_PATRIC_mdm, universal, extracellular='extracellular')

Progress: 46.69%Failed to find new reactions twice in a row, ending search
Progress: 100%   
Gapfilled 38 reactions and 7 metabolites
Objective now carries flux
Completed in 8.0 minutes


In [2]:
iter1_mdm = ['rxn01014_c', 'rxn10175_c', 'rxn02476_c', 'rxn00777_c', 'rxn01359_c', 'rxn12676_c', 'rxn02928_c', 'rxn01538_c', 
             'rxn01182_c', 'rxn12239_c', 'rxn03167_c', 'rxn05039_c', 'rxn10484_c', 'rxn05291_c', 'rxn10030_c', 'rxn01256_c',
             'rxn01644_c', 'rxn08131_c', 'rxn00530_c', 'rxn00010_c', 'rxn10053_c', 'rxn01669_c']
iter2_mdm = ['rxn01014_c', 'rxn10175_c', 'rxn02476_c', 'rxn00777_c', 'rxn00970_c', 'rxn13251_c', 'rxn02928_c', 'rxn01538_c', 
             'rxn12239_c', 'rxn03167_c', 'rxn05039_c', 'rxn10484_c', 'rxn05291_c', 'rxn01256_c', 'rxn01644_c', 'rxn08131_c',
             'rxn00530_c', 'rxn00958_c', 'rxn00010_c', 'rxn10053_c', 'rxn01669_c','rxn01539_c']
iter3_mdm = ['rxn01014_c', 'rxn10175_c', 'rxn02476_c', 'rxn00777_c', 'rxn12676_c', 'rxn13251_c', 'rxn02928_c', 'rxn01538_c',
             'rxn01182_c', 'rxn12239_c', 'rxn03167_c', 'rxn05039_c', 'rxn10484_c', 'rxn10030_c', 'rxn01256_c', 'rxn01644_c',
             'rxn08131_c', 'rxn00530_c', 'rxn00010_c', 'rxn10053_c', 'rxn01669_c']

mdm_rxn_ids = set(iter1_mdm + iter2_mdm + iter3_mdm)
print('Reactions to be added: ' + str(len(mdm_rxn_ids)))


Reactions to be added: 26


In [43]:
# change to C. difficle no-carbohydrate minimal media
cd630_PATRIC_ncmm = changeMedia(cd630_PATRIC_min, ncmm)
cd630_PATRIC_min2 = pfba_ensemble_gapfill(cd630_PATRIC_ncmm, universal, extracellular='extracellular')

Progress: 13.34%Failed to find new reactions twice in a row, ending search
Progress: 100%   
Gapfilled 0 reactions and 0 metabolites
Objective now carries flux
Completed in 3.1 minutes


In [3]:
iter1_ncmm = ['rxn00285_c', 'rxn05227_c', 'rxn10036_c', 'rxn10840_c', 'rxn00341_c', 'rxn13012_c', 'rxn09119_c',
              'rxn10663_c']
iter2_ncmm = ['rxn00285_c', 'rxn05227_c', 'rxn10036_c', 'rxn10840_c', 'rxn00341_c', 'rxn13012_c', 'rxn09119_c',
              'rxn10663_c']

ncmm_rxn_ids= set(iter1_ncmm + iter2_ncmm)
print('Reactions to be added: ' + str(len(ncmm_rxn_ids)))

Reactions to be added: 8


In [4]:
gapfilled_rxn_ids = set(list(new_rxn_ids) + list(mdm_rxn_ids) + list(ncmm_rxn_ids))

In [50]:
# Gapfill model with union of pFBA gapfill iterations
new_rxns = copy.deepcopy([universal.reactions.get_by_id(rxn) for rxn in gapfilled_rxn_ids])
new_cpd_ids = set()
for rxn in new_rxns: new_cpd_ids |= set([str(x.id) for x in list(rxn.metabolites)])
orig_cpd_ids = set([str(y.id) for y in cd630_PATRIC.metabolites])
new_cpd_ids = new_cpd_ids.difference(orig_cpd_ids)
new_cpds = copy.deepcopy([universal.metabolites.get_by_id(cpd) for cpd in new_cpd_ids])

cd630_PATRIC_gapfilled = copy.deepcopy(cd630_PATRIC)
cd630_PATRIC_gapfilled.add_metabolites(new_cpds)
cd630_PATRIC_gapfilled.add_reactions(new_rxns)

trans, ex = missingRxn(cd630_PATRIC_gapfilled)
for x in ex:
    try:
        new_id = 'EX_' + x
        cpd = cd630_PATRIC_gapfilled.metabolites.get_by_id(x)
        cd630_PATRIC_gapfilled.add_boundary(cpd, type='exchange', reaction_id=new_id, lb=-1000.0, ub=1000.0)
    except ValueError:
        pass


2 extracellular metabolites are missing transport reactions


In [51]:
doubling(cd630_PATRIC_gapfilled)

'15.808 minutes'

In [52]:
test = changeMedia(cd630_PATRIC_gapfilled, mdm)
doubling(test)

'48.873 minutes'

In [53]:
test = changeMedia(cd630_PATRIC_gapfilled, ncmm)
doubling(test)

'50.831 minutes'

In [54]:
# Collect and label gapfilled reactions
for rxn in gapfilled_rxn_ids:
    cd630_PATRIC_gapfilled.reactions.get_by_id(rxn).notes = 'gapfilled'

In [55]:
cobra.io.write_sbml_model(cd630_PATRIC_gapfilled, 'data/cdf_temp3.sbml')

#### Fixing remaining mass-balancing reactions

In [56]:
# Removes reactions from unbalanced list that contain R-groups in their substrate formulas, or are charged imbalanced
def trueUnbalance(model, ignore=[]):
    imbal = []
    rxn_ids = [str(x.id) for x in model.reactions]
    
    for rxn in rxn_ids:
        test = list(model.reactions.get_by_id(rxn).check_mass_balance().keys())
        
        if rxn in ignore:
            continue
        elif len(test) == 0:
            continue
        elif test == ['charge']:
            continue
        elif 'EX_' in rxn:
            continue
            
        formula = ''.join([x.formula for x in list(model.reactions.get_by_id(rxn).metabolites)])
        if 'R' not in formula:
            imbal.append(str(rxn))
    
    formula_dict = {}
    for x in imbal:
        react = [i.formula for i in model.reactions.get_by_id(x).reactants]
        prod = [j.formula for j in model.reactions.get_by_id(x).products]
        if react == prod:
            imbal.remove(x)
            continue
        else:
            formula_dict[x] = [react, prod]
    
    print(str(len(formula_dict.keys())) + ' mass-unbalanced reactions')
    return(formula_dict)


In [57]:
#cd630_PATRIC_gapfilled = cobra.io.read_sbml_model('data/cdf_temp3.sbml')

In [58]:
true_imbal = trueUnbalance(cd630_PATRIC_gapfilled, ignore=['dna_rxn','rna_rxn','protein_rxn','teichoicacid_rxn','peptidoglycan_rxn','lipid_rxn','cofactor_rxn','biomass'])

6 mass-unbalanced reactions


In [59]:

cpd00106_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00106_c')
cpd00282_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00282_c')
cpd00036_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00036_c')
cpd00247_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00247_c')
try:
    cd630_PATRIC_gapfilled.reactions.get_by_id('rxn13251_c').remove_from_model()
except:
    pass
rxn13251_c = cobra.Reaction('rxn13251_c') 
rxn13251_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn13251_c.name = 'Fumarate dependent DHORD'
rxn13251_c.lower_bound = -1000.
rxn13251_c.upper_bound = 1000.
rxn13251_c.add_metabolites({
    cpd00106_c: -1.0,
    cpd00282_c: -1.0,
    cpd00067_c: -2.0,
    cpd00036_c: 1.0,
    cpd00247_c: 1.0
})
cd630_PATRIC_gapfilled.add_reactions([rxn13251_c])


In [60]:

cpd00067_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00067_c')
cpd00239_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00239_c')
cpd00822_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00822_c')
cpd00036_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00036_c')
cpd00135_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00135_c')
try:
    cd630_PATRIC_gapfilled.reactions.get_by_id('rxn05957_c').remove_from_model()
except:
    pass
rxn05957_c = cobra.Reaction('rxn05957_c') 
rxn05957_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn05957_c.name = 'O4-succinyl-L-homoserine:hydrogen sulfide S-(3-amino-3-carboxypropyl)transferase'
rxn05957_c.gene_reaction_rule = '272563.8.peg.1916'
rxn05957_c.lower_bound = 0.
rxn05957_c.upper_bound = 1000.
rxn05957_c.add_metabolites({
    cpd00067_c: -2.0,
    cpd00239_c: -1.0,
    cpd00822_c: -1.0,
    cpd00036_c: 1.0,
    cpd00135_c: 1.0
})
cd630_PATRIC_gapfilled.add_reactions([rxn05957_c])


In [61]:

cpd00002_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00002_c')
cpd00010_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00010_c')
cpd00036_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00036_c')
cpd00008_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00008_c')
cpd00009_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00009_c')
cpd00078_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00078_c')
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn00285_c').remove_from_model()
rxn00285_c = cobra.Reaction('rxn00285_c') 
rxn00285_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn00285_c.name = 'Succinate:CoA ligase (ADP-forming)'
rxn00285_c.lower_bound = 0.
rxn00285_c.upper_bound = 1000.
rxn00285_c.add_metabolites({
    cpd00002_c: -1.0,
    cpd00010_c: -1.0,
    cpd00036_c: -1.0,
    cpd00008_c: 1.0,
    cpd00009_c: 1.0,
    cpd00078_c: 1.0,
    cpd00067_c: 2.0
})
cd630_PATRIC_gapfilled.add_reactions([rxn00285_c])


In [62]:

cpd00001_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00001_c')
cpd03989_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd03989_c')
cpd00461_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00461_c')
cpd01711_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd01711_c')
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn03665_c').remove_from_model()
rxn03665_c = cobra.Reaction('rxn03665_c') 
rxn03665_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn03665_c.name = '2-Hydroxy-6-oxo-7-methylocta-2,4-dienoate acylhydrolase'
rxn03665_c.gene_reaction_rule = '272563.8.peg.3588'
rxn03665_c.lower_bound = 0.
rxn03665_c.upper_bound = 1000.
rxn03665_c.add_metabolites({
    cpd00001_c: -1.0,
    cpd03989_c: -1.0,
    cpd00461_c: 1.0,
    cpd01711_c: 1.0
})
cd630_PATRIC_gapfilled.add_reactions([rxn03665_c])


In [63]:

cpd00001_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00001_c')
cpd02731_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd02731_c')
cpd00036_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00036_c')
cpd00461_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00461_c')
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn01895_c').remove_from_model()
rxn01895_c = cobra.Reaction('rxn01895_c') 
rxn01895_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
rxn01895_c.name = '2-Hydroxy-6-oxonona-2,4-diene-1,9-dioate succinylhydrolase'
rxn01895_c.gene_reaction_rule = '272563.8.peg.3588'
rxn01895_c.lower_bound = 0.
rxn01895_c.upper_bound = 1000.
rxn01895_c.add_metabolites({  
    cpd00067_c: -1.0,
    cpd00001_c: -1.0,
    cpd02731_c: -1.0,
    cpd00036_c: 1.0,
    cpd00461_c: 1.0 
})
cd630_PATRIC_gapfilled.add_reactions([rxn01895_c])


In [64]:

C21400_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('C21400_c')
cpd01646_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd01646_c')
C21399_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('C21399_c')
cpd00047_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00047_c')
cpd23040_c = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd23040_c')
cd630_PATRIC_gapfilled.reactions.get_by_id('R11462_c').remove_from_model()
R11462_c = cobra.Reaction('R11462_c') 
R11462_c.notes = {'citations': [], 'type': 'metabolic', 'annotation': 'ModelSEED'}
R11462_c.name = '4-methylpentanoyl-CoA:(R)-2-hydroxy-4-methylpentanoate CoA-transferase [EC:2.8.3.24]'
R11462_c.gene_reaction_rule = 'CD630_03950'
R11462_c.lower_bound = 0.
R11462_c.upper_bound = 1000.
R11462_c.add_metabolites({  
    C21400_c: -1.0,
    cpd01646_c: -1.0,
    C21399_c: 1.0,
    cpd00047_c: 1.0,
    cpd23040_c: 1.0,
    cpd00067_c: 1.0
})
cd630_PATRIC_gapfilled.add_reactions([R11462_c])


In [65]:
# Identifies blocked reactions, 1% cutoff for fraction of optimum
def blockedReactions(model):
    
    with model as m:
        blocked = cobra.flux_analysis.variability.find_blocked_reactions(m)
        nogene_blocked = []
        nogene_gapfilled_blocked = []
        for rxn in blocked:
            if m.reactions.get_by_id(rxn).gene_reaction_rule == '':
                nogene_blocked.append(rxn)
                if m.reactions.get_by_id(rxn).notes == 'gapfilled':
                    nogene_gapfilled_blocked.append(rxn)
    
    print(str(len(blocked)) + ' total reactions are blocked')
    print(str(len(nogene_blocked)) + ' reactions without GPRs are blocked')
    print(str(len(nogene_gapfilled_blocked)) + ' gapfilled reactions without GPRs are blocked')
    
    return blocked, nogene_blocked, nogene_gapfilled_blocked


In [66]:
# Check for blocked reactions
blocked, no_gene, gapfilled_blocked = blockedReactions(cd630_PATRIC_gapfilled)

519 total reactions are blocked
33 reactions without GPRs are blocked
0 gapfilled reactions without GPRs are blocked


In [67]:
check = []
for x in gapfilled_rxn_ids:
    rxn = cd630_PATRIC_gapfilled.reactions.get_by_id(x)
    cpds = [str(y.id) for y in rxn.metabolites]
    if 'cpd00002_c' in cpds or 'cpd00038_c' in cpds:
        if rxn.lower_bound < -1e-6 and rxn.upper_bound > 1e-6:
            check.append(str(rxn.id))
print('Gapfilled reaction with ATP of GTP: ' + str(len(check)))

Gapfilled reaction with ATP of GTP: 8


In [70]:
# Correct gapfilled reaction reversibility
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn01182_c').lower_bound = 0.0
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn13012_c').lower_bound = 0.0 # really slows it down, consumes 6 ATP
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn10663_c').lower_bound = 0.0
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn10053_c').lower_bound = 0.0
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn13139_c').lower_bound = 0.0
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn13914_c').lower_bound = 0.0
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn09562_c').lower_bound = 0.0
cd630_PATRIC_gapfilled.reactions.rxn01953_c.bounds = (-1000, 0)
cd630_PATRIC_gapfilled.reactions.rxn00292_c.bounds = (-1000, 0)

In [71]:
test = changeMedia(cd630_PATRIC_gapfilled, ncmm)
doubling(test)

'5.322 hours'

In [72]:
test = changeMedia(cd630_PATRIC_gapfilled, mdm)
doubling(test)

'2.321 hours'

In [73]:
doubling(cd630_PATRIC_gapfilled)

'35.606 minutes'

In [74]:
# Remove reactions with no GPR that are gapfilled and don't have strong biological evidence
remove_blocked = ['EX_cpd11576_e','EX_cpd04097_e','EX_cpd01012_e','EX_cpd00058_e',
                  'EX_cpd11578_e','EX_cpd11579_e','EX_cpd00210_e','EX_cpd00532_e',
                  'EX_cpd00176_e','EX_cpd03724_e','EX_cpd03696_e','EX_cpd08023_e',
                  'EX_cpd01092_e','EX_cpd01262_e','EX_cpd11596_e','EX_cpd01914_e',
                  'EX_cpd00531_e','EX_cpd11575_e','EX_cpd09878_e','EX_cpd01329_e',
                  'rxn09030_c']

for rxn in remove_blocked:
    cd630_PATRIC_gapfilled.reactions.get_by_id(rxn).remove_from_model()
    

In [75]:
test = changeMedia(cd630_PATRIC_gapfilled, mdm)
doubling(test)

'2.321 hours'

In [76]:
doubling(cd630_PATRIC_gapfilled)

'35.606 minutes'

Edit SBML to ensure compatibility with Cobra read-in function

### Pruning model

In [77]:
# Identify potentially gapfilled reactions
def missingGPR(model, exclude=['dna_rxn','rna_rxn','protein_rxn','teichoicacid_rxn','peptidoglycan_rxn','lipid_rxn','cofactor_rxn','biomass']):
    gapfilled = []
    transport = findTransports(model)
    if not type(exclude) is list:
        exclude = [exclude]
        
    for index in model.reactions:
        if len(list(index.genes)) == 0:
            if not index in model.boundary:
                if not index.id in exclude or not index.id in transport:
                    gapfilled.append(index.id)
    
    if len(gapfilled) > 0:
        print(str(len(gapfilled)) + ' metabolic reactions not associated with genes')
    
    return gapfilled

# Check for missing transport and exchange reactions
def missingRxn(model, extracellular='extracellular'):

    transporters = set(findTransports(model))
    exchanges = set([str(rxn.id) for rxn in model.reactions if len(list(rxn.products)) == 0])
    
    missing_exchanges = []
    missing_transports = []
    
    for metabolite in model.metabolites:
        if not metabolite.compartment == extracellular:
            continue

        curr_rxns = set([x.id for x in list(metabolite.reactions)])
        
        if not bool(curr_rxns & transporters):
            missing_transports.append(metabolite.id)
        if not bool(curr_rxns & exchanges):
            missing_exchanges.append(metabolite.id)
    
    if len(missing_transports) != 0:
        print(str(len(missing_transports)) + ' extracellular metabolites are missing transport reactions')
    if len(missing_exchanges) != 0:
        print(str(len(missing_exchanges)) + ' extracellular metabolites are missing exchange reactions')
        
    return missing_transports, missing_exchanges


# Checks which cytosolic metabolites are generated for free (bacteria only)
def checkFreeMass(raw_model, cytosol='cytosol'):
    
    with raw_model as model:
        
        # Close all exchanges
        for index in model.boundary:
            model.reactions.get_by_id(index.id).lower_bound = 0.
        
        # Identify all metabolites that are produced within the network
        demand_metabolites = [x.reactants[0].id for x in model.demands if len(x.reactants) > 0] + [x.products[0].id for x in model.demands if len(x.products) > 0]

        free = []
        for index in model.metabolites: 
            if index.id in demand_metabolites:
                continue
            elif not index.compartment in cytosol:
                continue
            else:
                demand = model.add_boundary(index, type='demand')
                model.objective = demand
                obj_val = model.slim_optimize(error_value=0.)
                if obj_val > 1e-8:
                    free.append(index.id)
                model.remove_reactions([demand])
    
    if len(free) > 0:
        print(str(len(free)) + ' metabolites are generated for free')

    return(free)

# Check for mass and charge balance in reactions
def checkBalance(raw_model, exclude=['dna_rxn','rna_rxn','protein_rxn','teichoicacid_rxn','peptidoglycan_rxn','lipid_rxn','cofactor_rxn','biomass']):
    
    with raw_model as model:
        imbalanced = []
        mass_imbal = 0
        charge_imbal = 0
        elem_set = set()
        for metabolite in model.metabolites:
            try:
                elem_set |= set(metabolite.elements.keys())
            except:
                pass
        
        if len(elem_set) == 0:
            imbalanced = model.reactions
            mass_imbal = len(model.reactions)
            charge_imbal = len(model.reactions)
            print('No elemental data associated with metabolites!')
        
        else:
            if not type(exclude) is list: 
                exclude = [exclude]
            for index in model.reactions:
                if index in model.boundary or index.id in exclude:
                    continue

                else:
                    try:
                        test = index.check_mass_balance()
                    except ValueError:
                        continue

                    if len(list(test)) > 0:
                        #imbalanced.append(index.id)

                        #if 'charge' in test.keys():
                        #    charge_imbal += 1
                        if len(set(test.keys()).intersection(elem_set)) > 0:
                            mass_imbal += 1
                            imbalanced.append(index.id)

    if mass_imbal != 0:
        print(str(mass_imbal) + ' reactions are mass imbalanced')
    #if charge_imbal != 0:
    #    print(str(charge_imbal) + ' reactions are charge imbalanced')
    
    return(imbalanced)


def checkDetermination(model):
    if len(model.reactions) < len(model.metabolites):
        print('Model is overdetermined')
    if len(model.reactions) > len(model.metabolites):
        print('Model is underdetermined')


# Checks the quality of models by a couple metrics and returns problems
def checkQuality(model, exclude=['dna_rxn','rna_rxn','protein_rxn','teichoicacid_rxn','peptidoglycan_rxn','lipid_rxn','cofactor_rxn','biomass'], cytosol='cytosol'):
    
    checkDetermination(model)
    gaps = missingGPR(model, exclude)
    freemass = checkFreeMass(model, cytosol)
    balance = checkBalance(model, exclude)
    trans, exch = missingRxn(model)
    
    test = gaps + freemass + balance
    if len(test) == 0:
        print('No inconsistencies detected')

    return gaps, freemass, balance, trans, exch


# Prints the names of reactions or metabolites based on IDs
def printNames(model, id_list):
    for x in id_list:
        try:
            print(model.metabolites.get_by_id(x).name)
            continue
        except:
            pass
        
        print(model.reactions.get_by_id(x).name)

In [78]:
no_gpr, freemass, unbalanced, no_trans, no_exch = checkQuality(cd630_PATRIC_gapfilled)

Model is overdetermined
108 metabolic reactions not associated with genes
3 extracellular metabolites are missing transport reactions
20 extracellular metabolites are missing exchange reactions


In [79]:
# Fix no transporter reactions
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00034_e').remove_from_model()
cd630_PATRIC_gapfilled.reactions.get_by_id('EX_cpd00034_e').remove_from_model()

In [80]:
# Fix no exchanges

# L-methionine R-oxide
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd11576_e').remove_from_model()
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd11576_c').remove_from_model()
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn05614_c').remove_from_model()

# Pb
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd04097_e').remove_from_model()
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd04097_c').remove_from_model()
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn10344_c').remove_from_model()

# Cd+
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd01012_e').remove_from_model()
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd01012_c').remove_from_model()
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn05516_c').remove_from_model()

# Cu2+
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00058_e').remove_from_model()
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00058_c').remove_from_model()
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn10481_c').remove_from_model()
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn05528_c').remove_from_model()

# hexanesulfonate
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd11578_e').remove_from_model()
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd11578_c').remove_from_model()
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn05585_c').remove_from_model()

# ethanesulfonate
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd11579_e').remove_from_model()
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd11579_c').remove_from_model()
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn05552_c').remove_from_model()

# Taurine
cpd00210_e = cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00210_e')
cd630_PATRIC_gapfilled.add_boundary(cpd00210_e, type='exchange', reaction_id='EX_cpd00210_e', lb=-1000., ub=1000.)

# 2-keto-3-deoxygluconate
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00176_e').remove_from_model()
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00176_c').remove_from_model()
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn05681_c').remove_from_model()

# Ferrichrome
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd03724_e').remove_from_model()
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd03724_c').remove_from_model()
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn05188_c').remove_from_model()

# Ursin
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd03696_e').remove_from_model()
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn05501_c').remove_from_model()

# Allantoin
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd01092_e').remove_from_model()
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd01092_c').remove_from_model()
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn05682_c').remove_from_model()

# Amylotriose
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd01262_e').remove_from_model()
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd01262_c').remove_from_model()
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn05608_c').remove_from_model()

# butanesulfonate
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd11596_e').remove_from_model()
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd11596_c').remove_from_model()
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn05512_c').remove_from_model()

# L-Methionine S-oxide
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd01914_e').remove_from_model()
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd01914_c').remove_from_model()
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn05613_c').remove_from_model()

# Hg
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00531_e').remove_from_model()
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00531_c').remove_from_model()
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn10343_c').remove_from_model()

# MOPS
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd11575_e').remove_from_model()
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd11575_c').remove_from_model()
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn05620_c').remove_from_model()

# Sulfoacetate
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd09878_e').remove_from_model()
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd09878_c').remove_from_model()
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn05656_c').remove_from_model()

# Maltohexaose
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd01329_e').remove_from_model()
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd01329_c').remove_from_model()
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn10174_c').remove_from_model()

# methanesulfonate
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd08023_e').remove_from_model()
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd08023_c').remove_from_model()
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn05621_c').remove_from_model()

# O2-
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00532_e').remove_from_model()
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd00532_c').remove_from_model()
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn00206_c').remove_from_model()

# Molybdate
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd11574_e').remove_from_model()
cd630_PATRIC_gapfilled.metabolites.get_by_id('cpd11574_c').remove_from_model()
cd630_PATRIC_gapfilled.reactions.get_by_id('EX_cpd11574_e').remove_from_model()


In [81]:
no_gpr, freemass, unbalanced, no_trans, no_exch = checkQuality(cd630_PATRIC_gapfilled)

Model is overdetermined
108 metabolic reactions not associated with genes


In [82]:
# Correct obvious auxotrophies

# TTP
cd630_PATRIC_gapfilled.reactions.rxn10060_c.upper_bound = 0.0

# dAMP
cd630_PATRIC_gapfilled.reactions.rxn01507_c.lower_bound = -1000.0
cd630_PATRIC_gapfilled.reactions.rxn01507_c.upper_bound = 0.0

# dAMP
cd630_PATRIC_gapfilled.reactions.rxn01218_c.lower_bound = -1000.0
cd630_PATRIC_gapfilled.reactions.rxn01218_c.upper_bound = 0.0

# dGMP
cd630_PATRIC_gapfilled.reactions.rxn01445_c.lower_bound = -1000.0
cd630_PATRIC_gapfilled.reactions.rxn01445_c.upper_bound = 0.0


In [ ]:
# Fix last reaction reversibilities
cd630_PATRIC_gapfilled.reactions.rxn01538_c.lower_bound = 0.0 # ATP
cd630_PATRIC_gapfilled.reactions.rxn01105_c.lower_bound = 0.0 # GTP
cd630_PATRIC_gapfilled.reactions.rxn01105_c.lower_bound = 0.0 # GTP

# GTP biosynthesis
cd630_PATRIC_gapfilled.reactions.rxn13139_c.lower_bound = -1000.0
cd630_PATRIC_gapfilled.reactions.rxn13139_c.upper_bound = 0.0


In [ ]:
# Fix incorrect biosynthesis of limiting amino acids

# L-Proline - cpd00129_c
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn00931_c').lower_bound = 0.
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn00929_c').lower_bound = 0.

# L-Cysteine - cpd00084_c
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn12844_c').lower_bound = 0.
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn00650_c').lower_bound = 0.
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn09177_c').lower_bound = 0.

# L-Leucine - cpd00107_c
cd630_PATRIC_gapfilled.reactions.get_by_id('rxn12845_c').lower_bound = 0.

In [ ]:
# Close dipeptide exchange reactions
dipeptides = ['EX_cpd11585_e','EX_cpd11582_e','EX_cpd11586_e','EX_cpd01017_e','EX_cpd11591_e',
              'EX_cpd11593_e','EX_cpd15605_e','EX_cpd15603_e','EX_cpd11580_e','EX_cpd15606_e',
              'EX_cpd11583_e','EX_cpd11589_e','EX_cpd11588_e','EX_cpd15604_e','EX_cpd11592_e',
              'EX_cpd11587_e','EX_cpd11590_e','EX_cpd11581_e','EX_cpd11584_e']

for x in dipeptides:
    cd630_PATRIC_gapfilled.reactions.get_by_id(x).lower_bound = 0.0

In [83]:
unused_c, unused_r = removeUnused(cd630_PATRIC_gapfilled)

Pruned 90 metabolites from model
Pruned 7 reactions from model


In [84]:
# Removed orphaned reactions with no substrates
removed = 0
for rxn in cd630_PATRIC_gapfilled.reactions:
    if len(list(rxn.reactants)) == 0 and len(list(rxn.products)) == 0:
        cd630_PATRIC_gapfilled.reactions.get_by_id(rxn.id).remove_from_model()
        removed += 1
print('Found and removed ' + str(removed) + ' reactions')

Found and removed 0 reactions


In [85]:
# Check blocked reactions
blocked, nogene_blocked, nogene_gapfilled_blocked = blockedReactions(cd630_PATRIC_gapfilled)

481 total reactions are blocked
11 reactions without GPRs are blocked
0 gapfilled reactions without GPRs are blocked


In [ ]:
# Remove uninformative metabolites/reactions
remove_cpd = ['cpd11440_c', #fa6
              'cpd11438_c', #fa4
              'cpd00666_c', #Tartrate
              'cpd00158_c', #CELB
              'cpd11433_c', #fa12
              'cpd00392_c', #Phycite
              'cpd11430_c', #fa1
              'cpd11431_c', #fa11
              'cpd11436_c'] #fa3
for cpd in remove_cpd:
    try:
        metabolite = cd630_PATRIC_gapfilled.metabolites.get_by_id(cpd)
        cd630_PATRIC_gapfilled.remove_reactions(metabolite.reactions)
        cd630_PATRIC_gapfilled.remove_metabolites([metabolite])
    except KeyError:
        print(metabolite + ' not found')
        continue

In [ ]:
inactive_metabolites = [m for m in cd630_PATRIC_gapfilled.metabolites if len(m.reactions) == 0]
cd630_PATRIC_gapfilled.remove_metabolites(inactive_metabolites)

In [87]:
len(list(cd630_PATRIC_gapfilled.genes))

792

In [88]:
# Correct names
cd630_PATRIC_gapfilled.metabolites.cpd00443_e.name = 'ABEE'
cd630_PATRIC_gapfilled.reactions.rxn03185_c.name = 'alpha-N-acetylneuraminate alpha-2,8-sialyltransferase'

In [90]:
cd630_PATRIC_gapfilled

Name,iCdJ792
Memory address,0x07fd2281f0350
Number of metabolites,1179
Number of reactions,1194
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"cytosol, extracellular"


In [ ]:
# Add RefSeq annotations

patric_refseq = {}
with open('/home/matthew/Desktop/repos/Jenior_iCdJ794_2019/data/patric_refseq_genes.tsv') as ref:
    for line in ref:
        line = line.split()
        patric_refseq[line[0]] = line[1]
        
for gene in cd630_PATRIC_gapfilled.genes:
    if '.peg.' in gene.id:
        try:
            gene.annotation['refseq'] = patric_refseq[gene.id]
        except:
            continue

In [ ]:
# Add annotations

# Metabolites
for cpd in cd630_PATRIC_gapfilled.metabolites:
    if 'cpd' in cpd.id:
        new_id = str(cpd.id).split('_')
        if len(new_id) > 1:
            del new_id[-1]
        new_id = ''.join(new_id)
        cpd.annotation['seed.compound'] = new_id

cd630_PATRIC_gapfilled.metabolites.C21400_c.annotation['kegg'] = 'C21400'        
cd630_PATRIC_gapfilled.metabolites.C21399_c.annotation['kegg'] = 'C21399'        
cd630_PATRIC_gapfilled.metabolites.C21090_c.annotation['kegg'] = 'C21090'        
cd630_PATRIC_gapfilled.metabolites.C21399_e.annotation['kegg'] = 'C21399'        


# Reactions
for rxn in cd630_PATRIC_gapfilled.reactions:
    if 'rxn' in rxn.id or 'EX_' in rxn.id:
        new_id = str(rxn.id).split('_')
        if len(new_id) > 1:
            del new_id[-1]
        new_id = ''.join(new_id)
        rxn.annotation['seed.reaction'] = new_id
    elif 'R' in rxn.id:
        new_id = str(rxn.id).split('_')
        if len(new_id) > 1:
            del new_id[-1]
        new_id = ''.join(new_id)
        rxn.annotation['kegg.genes'] = new_id

cd630_PATRIC_gapfilled.reactions.SK_cpd11416_c.annotation['seed.reaction'] = 'SK_cpd11416'
cd630_PATRIC_gapfilled.reactions.ENOG4108HXH_c.annotation['eggnog'] = 'ENOG4108HXH'
cd630_PATRIC_gapfilled.reactions.COG3601_c.annotation['cog'] = 'COG3601'
cd630_PATRIC_gapfilled.reactions.K20025_c.annotation['kegg.genes'] = 'K20025'


# Genes
for gene in cd630_PATRIC_gapfilled.genes:
    if '.peg.' in gene.id:
        gene.annotation['patric'] = 'fig|' + gene.id
    elif 'CD630_' in gene.id:
        gene.annotation['uniprot'] = gene.id

cd630_PATRIC_gapfilled.genes.T4BV06_CLODI.annotation['kegg'] = 'T4BV06_CLODI'
cd630_PATRIC_gapfilled.genes.K00529.annotation['kegg'] = 'K00529'


In [ ]:
# Add SBOs

# Metabolites
for cpd in cd630_PATRIC_gapfilled.metabolites:
    cpd.annotation['sbo'] = 'SBO:0000247'

# Reactions
for rxn in cd630_PATRIC_gapfilled.reactions:
    substrates = list(rxn.metabolites)
    compartments = set([x.compartment for x in substrates])
    
    if 'EX_' in rxn.id:
        rxn.annotation['sbo'] = 'SBO:0000627' # exchange
    elif len(compartments) > 1:
        rxn.annotation['sbo'] = 'SBO:0000185' # transport
    else:
        rxn.annotation['sbo'] = 'SBO:0000176' # metabolic

# Biomass
cd630_PATRIC_gapfilled.reactions.dna_rxn.annotation['sbo'] = 'SBO:0000629'  
cd630_PATRIC_gapfilled.reactions.rna_rxn.annotation['sbo'] = 'SBO:0000629'  
cd630_PATRIC_gapfilled.reactions.protein_rxn.annotation['sbo'] = 'SBO:0000629'  
cd630_PATRIC_gapfilled.reactions.teichoicacid_rxn.annotation['sbo'] = 'SBO:0000629'  
cd630_PATRIC_gapfilled.reactions.peptidoglycan_rxn.annotation['sbo'] = 'SBO:0000629'  
cd630_PATRIC_gapfilled.reactions.cellwall_rxn.annotation['sbo'] = 'SBO:0000629'  
cd630_PATRIC_gapfilled.reactions.lipid_rxn.annotation['sbo'] = 'SBO:0000629'  
cd630_PATRIC_gapfilled.reactions.cofactor_rxn.annotation['sbo'] = 'SBO:0000629'  
cd630_PATRIC_gapfilled.reactions.biomass.annotation['sbo'] = 'SBO:0000629'  
cd630_PATRIC_gapfilled.reactions.SK_cpd11416_c.annotation['sbo'] = 'SBO:0000632'  


# Genes
for gene in cd630_PATRIC_gapfilled.genes:
    gene.annotation['sbo'] = 'SBO:0000243'


In [ ]:
from cobra.flux_analysis import flux_variability_analysis

def test_unbounded(model):
    
    fva_result = flux_variability_analysis(model, fraction_of_optimum=1.0)

    lower_bounds = numpy.asarray([rxn.lower_bound for rxn in model.reactions], dtype=float)
    upper_bounds = numpy.asarray([rxn.upper_bound for rxn in model.reactions], dtype=float)
    lower_bound = numpy.nanmedian(lower_bounds[lower_bounds != 0.0])
    upper_bound = numpy.nanmedian(upper_bounds[upper_bounds != 0.0])

    # Find those reactions whose flux is close to or outside of the median
    # upper or lower bound, i.e., appears unconstrained.
    unlimited_flux = fva_result.loc[
            numpy.isclose(fva_result["maximum"], upper_bound, atol=1e-7) |
            (fva_result["maximum"] > upper_bound) |
            numpy.isclose(fva_result["minimum"], lower_bound, atol=1e-7) |
            (fva_result["minimum"] < lower_bound)].index.tolist()
    
    saved = []
    for rxn in unlimited_flux:
        
        reaction = model.reactions.get_by_id(rxn) 
        substrates = set([x.id for x in reaction.metabolites])
        ignore = set(['cpd00003_c','cpd00004_c','cpd00005_c','cpd00006_c'])
        
        if 'EX_' in rxn:
            continue
        elif reaction.bounds != (-1000., 1000.): 
            continue
        elif 'transferase' in reaction.name:
            continue
        elif reaction.name.find('mutase') != -1:
            continue
        elif 'isomerase' in reaction.name:
            continue
        elif 'hydrolase' in reaction.name:
            continue 
        elif len(substrates & ignore) >= 1:
            continue
        else:
            saved.append(rxn)
        
    print('Potentially unbounded reactions in default medium: ' + str(len(saved)))
    return saved


In [ ]:
unlimited = test_unbounded(cd630_PATRIC_gapfilled)

# Correct reversibility to close potentially leaky cycles
cd630_PATRIC_gapfilled.reactions.rxn01322_c.bounds = (0., 1000.)
cd630_PATRIC_gapfilled.reactions.rxn05617_c.bounds = (0., 1000.)
cd630_PATRIC_gapfilled.reactions.rxn05607_c.bounds = (0., 1000.)
cd630_PATRIC_gapfilled.reactions.rxn05655_c.bounds = (0., 1000.)
cd630_PATRIC_gapfilled.reactions.rxn05496_c.bounds = (0., 1000.)
cd630_PATRIC_gapfilled.reactions.rxn05202_c.bounds = (0., 1000.)
cd630_PATRIC_gapfilled.reactions.rxn00606_c.bounds = (0., 1000.)
cd630_PATRIC_gapfilled.reactions.rxn10184_c.bounds = (0., 1000.)
cd630_PATRIC_gapfilled.reactions.rxn05494_c.bounds = (0., 1000.)
cd630_PATRIC_gapfilled.reactions.rxn05610_c.bounds = (0., 1000.)
cd630_PATRIC_gapfilled.reactions.rxn00575_c.bounds = (0., 1000.)
cd630_PATRIC_gapfilled.reactions.rxn05209_c.bounds = (-1000., 0.) # Na+ antiporter
cd630_PATRIC_gapfilled.reactions.rxn05226_c.bounds = (0., 1000.)
cd630_PATRIC_gapfilled.reactions.rxn02005_c.bounds = (0., 1000.)
cd630_PATRIC_gapfilled.reactions.rxn05518_c.bounds = (0., 1000.)
cd630_PATRIC_gapfilled.reactions.rxn29571_c.bounds = (0., 1000.) # Neu5Ac H+ symporter
cd630_PATRIC_gapfilled.reactions.rxn05486_c.bounds = (0., 1000.) # N-Acetylmannosamine H+ symporter
cd630_PATRIC_gapfilled.reactions.rxn10175_c.bounds = (0., 1000.)
cd630_PATRIC_gapfilled.reactions.rxn05485_c.bounds = (0., 1000.)

In [ ]:
# Correct last few gene IDs
corrected_genes = {'CD630_00450': '272563.120.peg.49', 
                   'CD630_01120': '272563.120.peg.124', 
                   'CD630_01530': '272563.120.peg.166', 
                   'CD630_01540': '272563.120.peg.167', 
                   'CD630_01550': '272563.120.peg.168', 
                   'CD630_01790': '272563.120.peg.195', 
                   'CD630_02440': '272563.8.peg.262', 
                   'CD630_03940': '272563.120.peg.414', 
                   'CD630_03950': '272563.120.peg.415', 
                   'CD630_03960': '272563.120.peg.416', 
                   'CD630_03970': '272563.120.peg.417', 
                   'CD630_03980': '272563.120.peg.418', 
                   'CD630_06820': '272563.8.peg.717', 
                   'CD630_07500': '272563.120.peg.772', 
                   'CD630_07680': '272563.120.peg.790', 
                   'CD630_08730': '272563.120.peg.897', 
                   'CD630_08760': '272563.120.peg.900', 
                   'CD630_10030': '272563.120.peg.1042', 
                   'CD630_10040': '272563.120.peg.1043', 
                   'CD630_10100': '272563.120.peg.1049', 
                   'CD630_10540': '272563.120.peg.1094', 
                   'CD630_12250': '272563.8.peg.1284', 
                   'CD630_12590': '272563.120.peg.1317', 
                   'CD630_14930': '272563.120.peg.1562', 
                   'CD630_17760': '272563.120.peg.1869', 
                   'CD630_21960': '272563.120.peg.2318', 
                   'CD630_21990': '272563.120.peg.2321', 
                   'CD630_21991': '272563.120.peg.2322', 
                   'CD630_22770': '272563.120.peg.2404', 
                   'CD630_23310': '272563.120.peg.2462', 
                   'CD630_23390': '272563.120.peg.2470', 
                   'CD630_23410': '272563.120.peg.2472', 
                   'CD630_23420': '272563.120.peg.2473', 
                   'CD630_23540': '272563.120.peg.2485', 
                   'CD630_23710': '272563.120.peg.2504', 
                   'CD630_23800': '272563.120.peg.2513', 
                   'CD630_24170': '272563.120.peg.2551', 
                   'CD630_24180': '272563.120.peg.2552', 
                   'CD630_25690': '272563.120.peg.2710', 
                   'CD630_26510': '272563.8.peg.2775', 
                   'CD630_26550': '272563.120.peg.2798', 
                   'CD630_30130': '272563.8.peg.3157', 
                   'CD630_30140': '272563.120.peg.3181', 
                   'CD630_30150': '272563.120.peg.3182', 
                   'CD630_32370_1': '272563.120.peg.3419', 
                   'CD630_32370_2': '272563.120.peg.3419', 
                   'CD630_32440': '272563.120.peg.3426', 
                   'CD630_35150': '272563.120.peg.3715',
                   'K00529': '272563.120.peg.1609',
                   'T4BV06_CLODI': '1151372.4.peg.1723'}
cobra.manipulation.modify.rename_genes(cd630_PATRIC_gapfilled, corrected_genes)

In [ ]:
# Name new model
cd630_PATRIC_gapfilled.name = 'Clostridium difficile 630'
cd630_PATRIC_gapfilled.id = 'iCdR' + str(len(cd630_PATRIC_gapfilled.genes))

In [ ]:
for rxn in cd630_PATRIC_gapfilled.reactions:
    fixed_gpr = rxn.gene_reaction_rule.replace('( ','').replace(' )','')
    rxn.gene_reaction_rule = fixed_gpr

In [ ]:
cobra.io.write_sbml_model(cd630_PATRIC_gapfilled, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG791.sbml')
cobra.io.save_json_model(cd630_PATRIC_gapfilled, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG791.json')

In [1]:
import cobra
iCdG791 = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG791.sbml')

In [2]:
# Gene annotations
patric_refseq = {}
with open('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/cd630_patric_refseq_genes.tsv') as ref:
    for line in ref:
        line = line.split()
        patric_refseq[line[0]] = line[1]
        
for gene in iCdG791.genes:
    if '.peg.' in gene.id:
        try:
            gene.annotation['refseq'] = patric_refseq[gene.id]
            gene.annotation['patric'] = 'fig|' + gene.id
        except:
            continue

In [3]:
# Metabolite annotations
for cpd in iCdG791.metabolites:
    if 'cpd' in cpd.id:
        new_id = str(cpd.id).split('_')
        if len(new_id) > 1:
            del new_id[-1]
        new_id = ''.join(new_id)
        cpd.annotation['seed.compound'] = new_id

iCdG791.metabolites.C21400_c.annotation['kegg'] = 'C21400'        
iCdG791.metabolites.C21399_c.annotation['kegg'] = 'C21399'        
iCdG791.metabolites.C21090_c.annotation['kegg'] = 'C21090'        
iCdG791.metabolites.C21399_e.annotation['kegg'] = 'C21399' 

In [4]:
# Reaction annotations
for rxn in iCdG791.reactions:
    if 'rxn' in rxn.id or 'EX_' in rxn.id:
        new_id = str(rxn.id).split('_')
        if len(new_id) > 1:
            del new_id[-1]
        new_id = ''.join(new_id)
        rxn.annotation['seed.reaction'] = new_id
    elif 'R' in rxn.id:
        new_id = str(rxn.id).split('_')
        if len(new_id) > 1:
            del new_id[-1]
        new_id = ''.join(new_id)
        rxn.annotation['kegg.genes'] = new_id

iCdG791.reactions.SK_cpd11416_c.annotation['seed.reaction'] = 'SK_cpd11416'
iCdG791.reactions.ENOG4108HXH_c.annotation['eggnog'] = 'ENOG4108HXH'
iCdG791.reactions.COG3601_c.annotation['cog'] = 'COG3601'
iCdG791.reactions.K20025_c.annotation['kegg.genes'] = 'K20025'

In [5]:
# Add SBOs

# Metabolites
for cpd in iCdG791.metabolites:
    cpd.annotation['sbo'] = 'SBO:0000247'

# Reactions
for rxn in iCdG791.reactions:
    substrates = list(rxn.metabolites)
    compartments = set([x.compartment for x in substrates])
    
    if 'EX_' in rxn.id:
        rxn.annotation['sbo'] = 'SBO:0000627' # exchange
    elif len(compartments) > 1:
        rxn.annotation['sbo'] = 'SBO:0000185' # transport
    else:
        rxn.annotation['sbo'] = 'SBO:0000176' # metabolic

# Biomass
iCdG791.reactions.dna_rxn.annotation['sbo'] = 'SBO:0000629'  
iCdG791.reactions.rna_rxn.annotation['sbo'] = 'SBO:0000629'  
iCdG791.reactions.protein_rxn.annotation['sbo'] = 'SBO:0000629'  
iCdG791.reactions.teichoicacid_rxn.annotation['sbo'] = 'SBO:0000629'  
iCdG791.reactions.peptidoglycan_rxn.annotation['sbo'] = 'SBO:0000629'  
iCdG791.reactions.cellwall_rxn.annotation['sbo'] = 'SBO:0000629'  
iCdG791.reactions.lipid_rxn.annotation['sbo'] = 'SBO:0000629'  
iCdG791.reactions.cofactor_rxn.annotation['sbo'] = 'SBO:0000629'  
iCdG791.reactions.biomass.annotation['sbo'] = 'SBO:0000629'  
iCdG791.reactions.SK_cpd11416_c.annotation['sbo'] = 'SBO:0000632'  

# Genes
for gene in iCdG791.genes:
    gene.annotation['sbo'] = 'SBO:0000243'


In [6]:
# Correct residual gene names
gene_name_fixes = {'272563.120.peg.1317':'Na(+)-dependent branched-chain amino acid transporter',
'272563.120.peg.772':'ABC transporter, substrate-binding protein (cluster 3, basic aa/glutamine/opines)',
'272563.120.peg.1043':'Fumarate hydratase class I, beta region (EC 4.2.1.2); L(+)-tartrate dehydratase beta subunit (EC 4.2.1.32) | fumB',
'272563.120.peg.1042':'Fumarate hydratase class I, alpha region (EC 4.2.1.2); L(+)-tartrate dehydratase alpha subunit (EC 4.2.1.32) | fumA',
'272563.120.peg.2504':'L-aspartate oxidase (EC 1.4.3.16) | nadB',
'272563.120.peg.195':'NAD-specific glutamate dehydrogenase (EC 1.4.1.2) | gluD',
'272563.120.peg.2551':'PTS system, glucitol/sorbitol-specific IIB component (EC 2.7.1.198) / PTS system, glucitol/sorbitol-specific IIC component 2 | srlE_2',
'272563.120.peg.2552':'PTS system, glucitol/sorbitol-specific IIC component | srlA',
'272563.120.peg.1049':'N-acetylgalactosamine-6-phosphate deacetylase | nagA',
'272563.120.peg.2404':'ribulose-5-phosphate 4-epimerase and related epimerases and aldolases',
'272563.120.peg.49':'Ribulose-5-phosphate 4-epimerase and related epimerases and aldolases',
'272563.120.peg.3715':'N-acetylglucosamine-1-phosphate uridyltransferase (EC 2.7.7.23) / Glucosamine-1-phosphate N-acetyltransferase (EC 2.3.1.157) | glmU',
'272563.120.peg.3181':'PTS system, IIB component',
'272563.8.peg.3157':'PTS system, IIC component',
'272563.120.peg.3182':'PTS system, IIA component',
'1151372.4.peg.1723':'Branched-chain amino acid aminotransferase (EC 2.6.1.42) | ilvE',
'272563.120.peg.900':'ABC transporter, substrate-binding protein',
'272563.120.peg.897':'ABC transporter, substrate-binding protein',
'272563.8.peg.717':'sodium-solute symporter, putative',
'272563.120.peg.2710':'Mannosylglycerate hydrolase (EC 3.2.1.170)',
'272563.120.peg.1869':'ABC transporter, ATP-binding protein (cluster 3, basic aa/glutamine/opines)',
'272563.120.peg.1562':'DNA-3-methyladenine glycosylase II (EC 3.2.2.21)',
'272563.120.peg.124':'Phosphate butyryltransferase (EC 2.3.1.19) | ptb',
'272563.120.peg.2322':'2-oxoglutarate/2-oxoacid ferredoxin oxidoreductase, delta subunit, ferredoxin-like 4Fe-4S binding protein (EC 1.2.7.-)',
'272563.120.peg.2321':'2-oxoglutarate/2-oxoacid ferredoxin oxidoreductase, alpha subunit (EC 1.2.7.-)',
'272563.120.peg.167':'4-hydroxyphenylacetate decarboxylase, small subunit (EC 4.1.1.83) | hpdC',
'272563.120.peg.166':'4-hydroxyphenylacetate decarboxylase, large subunit (EC 4.1.1.83) | hpdB',
'272563.120.peg.2513':'Indolepyruvate oxidoreductase subunit IorB (EC 1.2.7.8) | iorB',
'272563.120.peg.3426':'D-proline reductase, 45 kDa subunit (EC 1.21.4.1) / D-proline reductase, 23 kDa subunit (EC 1.21.4.1) | prdA',
'272563.120.peg.2485':'Glycine reductase component B beta subunit (EC 1.21.4.2) / Glycine reductase component B alpha subunit (EC 1.21.4.2) | grdE',
'272563.120.peg.414':'D-lactate dehydrogenase (EC 1.1.1.28) | ldhA',
'272563.120.peg.417':'2-hydroxyglutaryl-CoA dehydratase, D-component HgdA | hadB',
'272563.120.peg.418':'2-hydroxyglutaryl-CoA dehydratase, D-component HgdB | hadC',
'272563.120.peg.415':'Hydroxyproline dehydratase putative | hadA',
'272563.120.peg.416':'CoA-substrate-specific enzyme activase; 2-hydroxyglutaryl-CoA dehydratase activator, A-component | hadI',
'272563.120.peg.3419':'Proline racemase (EC 5.1.1.4) | prdF',
'272563.120.peg.1609':'NADH-dependent reduced ferredoxin:NADP+ oxidoreductase subunit A',
'272563.120.peg.168':'4-hydroxyphenylacetate decarboxylase activating enzyme | hpdA',
'272563.120.peg.2318':'Acetyl-CoA synthetase (ADP-forming) alpha and beta chains, putative',
'272563.120.peg.2473':'Succinate-semialdehyde dehydrogenase, CoA-dependent | sucD',
'272563.120.peg.2472':'4-hydroxybutanoyl-CoA dehydratase (EC 4.2.1.120) / Vinylacetyl-CoA Delta-isomerase (EC 5.3.3.3) | abfD',
'272563.120.peg.1094':'Electron bifurcating butyryl-CoA dehydrogenase (NAD+, ferredoxin) | bcd2',
'272563.120.peg.2470':'4-hydroxybutyrate coenzyme A transferase | abfT',
'272563.120.peg.2798':'UDP-N-acetylmuramoyl-tripeptide--D-alanyl-D-alanine ligase (EC 6.3.2.10) | murF',
'272563.8.peg.262':'hypothetical protein',
'272563.120.peg.2462':'Mannitol-1-phosphate 5-dehydrogenase (EC 1.1.1.17) | mtlD',
'272563.120.peg.790':'Sorbitol-6-phosphate 2-dehydrogenase (EC 1.1.1.140) | gutD'}

for gene in gene_name_fixes.keys():
    iCdG791.genes.get_by_id(gene).name = gene_name_fixes[gene]


In [7]:
# Save to files
cobra.io.write_sbml_model(iCdG791, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/temp/iCdG791.sbml')
cobra.io.save_json_model(iCdG791, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/temp/iCdG791.json')

In [1]:
# Deal with genes that have now reactions
import cobra
iCdG791 = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG791.sbml')

In [2]:
# Correct names

# Reactions
iCdG791.reactions.rxn01987_c.name = 'Ribokinase'
iCdG791.reactions.rxn05496_c.name = 'Amino acid transporter (ala-L)'
iCdG791.reactions.rxn05202_c.name = 'Xanthine ion-coupled transport;'
iCdG791.reactions.rxn12191_c.name = 'Choline monooxygenase (NADP)'
iCdG791.reactions.rxn05663_c.name = 'L-tryptophan reversible transport via proton symport'
iCdG791.reactions.rxn01213_c.name = 'Geranyl-diphosphate synthase'
iCdG791.reactions.rxn10770_c.name = 'Choline dehydrogenase'
iCdG791.reactions.rxn05616_c.name = 'Magnesium (+2) transport in via diffusion'
iCdG791.reactions.rxn05669_c.name = 'Amino acid transporter (val-L)'
iCdG791.reactions.rxn05244_c.name = 'Branched chain amino acid:H+ symporter (Isoleucine)'
iCdG791.reactions.EX_cpd00276_e.name = 'D-Glucosamine exchange'
iCdG791.reactions.rxn08783_c.name = 'D-lactate dehydrogenase'
iCdG791.reactions.rxn03463_c.name = 'alpha-2,8-sialyltransferase'
iCdG791.reactions.rxn03185_c.name = 'alpha-2,8-sialyltransferase'
iCdG791.reactions.rxn02687_c.name = '(R)-mandelate dehydrogenase'
iCdG791.reactions.rxn04779_c.name = ' N-cyclopropylmelamine deaminase'
iCdG791.reactions.rxn07875_c.name = 'geranal dehydrogenase'
iCdG791.reactions.rxn05127_c.name = 'gamma-glutamyl-gamma aminobutyric acid dehydrogenase'
iCdG791.reactions.rxn11610_c.name = '2-hydroxymuconic semialdehyde hydrolase'
iCdG791.reactions.rxn04781_c.name = 'N-cyclopropylammelide alkylamino hydrolase'
iCdG791.reactions.rxn03891_c.name = 'trans-hexaprenyltranstransferase'
iCdG791.reactions.rxn03762_c.name = 'CEA synthetase'
iCdG791.reactions.rxn03536_c.name = 'Adenosyl cobinamide kinase'
iCdG791.reactions.rxn03282_c.name = 'alcohol dehydrogenase'
iCdG791.reactions.rxn04723_c.name = '3-methylsalicylaldehyde dehydrogenase'
iCdG791.reactions.rxn07261_c.name = 'ethanolaminephosphotransferase'
iCdG791.reactions.rxn03540_c.name = 'Ado-cobyric acid synthase [glutamine hydrolyzing]'
iCdG791.reactions.rxn04712_c.name = 'naphthyl-2-methylsuccinate synthase'
iCdG791.reactions.rxn07622_c.name = 'dihydrophloroglucinol hydrolase'
iCdG791.reactions.rxn06905_c.name = 'TCOA hydrolase'
iCdG791.reactions.rxn04737_c.name = '2-naphthaldehyde dehydrogenase'
iCdG791.reactions.rxn03921_c.name = 'aldehyde dehydrogenase [EC:1.2.1.-]'
iCdG791.reactions.rxn03721_c.name = 'Parathion hydrolase'
iCdG791.reactions.rxn04727_c.name = '1-naphthaldehyde dehydrogenase'
iCdG791.reactions.rxn03537_c.name = 'Adenosyl cobinamide phosphate guanyltransferase'
iCdG791.reactions.rxn08352_c.name = '(E)-4-hydroxy-3-methylbut-2-en-1-yl diphosphate reductase'
iCdG791.reactions.rxn03135_c.name = 'Imidazole glycerol phosphate synthase amidotransferase'
iCdG791.reactions.rxn03837_c.name = 'aryldialkylphosphatase'
iCdG791.reactions.rxn06752_c.name = 'neuraminidase'
iCdG791.reactions.rxn04413_c.name = 'adenosylcobinamide kinase'
iCdG791.reactions.rxn04604_c.name = '(2E,4Z)-2-hydroxy-6-oxona-2,4-dienedioate succinylhydrolase'
iCdG791.reactions.rxn03005_c.name = 'N2-Formyl-N1-(5-phospho-D-ribosyl)glycinamide formyltransferase'
iCdG791.reactions.rxn07876_c.name = 'citronellal dehydrogenase'
iCdG791.reactions.rxn06394_c.name = 'neuraminidase'
iCdG791.reactions.rxn04780_c.name = 'N-cyclopropylammeline deaminase'
iCdG791.reactions.rxn03486_c.name = '2-hydroxymuconic semi-aldehyde (m-xylene)'
iCdG791.reactions.rxn07609_c.name = '2-hydroxy-6-oxo-6-(2-carboxyphenyl)-hexa-2,4-dienoate hydrolase'
iCdG791.reactions.rxn03538_c.name = 'Adenosylcobalamin 5-phosphate synthase'
iCdG791.reactions.rxn04238_c.name = '4-isopropenyl-2-ketocyclohexane-1-carboxyl-CoA hydrolase'
iCdG791.reactions.rxn01208_c.name = '2-Oxo-4-methyl-3-carboxypentanoate decarboxylation'
iCdG791.reactions.rxn03467_c.name = 'neuraminidase'
iCdG791.reactions.rxn06750_c.name = 'alpha-N-acetylneuraminate alpha-2,8-sialyltransferase'
iCdG791.reactions.rxn03892_c.name = 'all-trans-hexaprenyl-diphosphate synthase'
iCdG791.reactions.rxn07493_c.name = 'GDP-L-galactose phosphorylase'
iCdG791.reactions.rxn02362_c.name = 'cyclic 2,3-diphosphoglycerate synthase'
iCdG791.reactions.rxn01905_c.name = 'cysteine sulfinic acid deaminase'
iCdG791.reactions.rxn02604_c.name = 'vicianin beta-glucosidase'
iCdG791.reactions.rxn05126_c.name = '4-(gamma-L-glutamylamino)butanal:NAD+ oxidoreductase'
iCdG791.reactions.rxn04658_c.name = 'LysW-gamma-L-alpha-aminoadipate 6-semialdehyde:NADP+ 6-oxidoreductase (phosphorylating)'
iCdG791.reactions.rxn04744_c.name = '4-hydroxymethylsalicyaldehyde dehydrogenase'
iCdG791.reactions.rxn03873_c.name = '6-oxocyclohex-1-ene-1-carbonyl-CoA hydrolase (decyclizing)'
iCdG791.reactions.rxn07885_c.name = 'citronellal dehydrogenase'
iCdG791.reactions.rxn06760_c.name = 'neuraminidase'
iCdG791.reactions.rxn06759_c.name = 'alpha-N-acetylneuraminate alpha-2,8-sialyltransferase'
iCdG791.reactions.rxn11584_c.name = 'aryldialkylphosphatase'
iCdG791.reactions.rxn00970_c.name = 'Hydroxybenzoate synthase'
iCdG791.reactions.rxn05039_c.name = '5-amino-6-(5-phosphoribitylamino)uracil phosphatase'
iCdG791.reactions.rxn03365_c.name = 'R04916'
iCdG791.reactions.rxn03366_c.name = 'R04916'
iCdG791.reactions.rxn00568_c.name = 'Nitrite reductase (NAD(P)H)'

# Metabolites
iCdG791.metabolites.cpd00443_c.name = '4-Aminobenzoate'
iCdG791.metabolites.cpd00443_e.name = '4-Aminobenzoate'
iCdG791.metabolites.cpd00103_c.name = '5-Phospho-alpha-D-ribose 1-diphosphate'
iCdG791.metabolites.cpd00226_c.name = 'Hypoxanthine'
iCdG791.metabolites.cpd00104_c.name = 'Biotin'
iCdG791.metabolites.cpd00104_e.name = 'Biotin'
iCdG791.metabolites.cpd00202_c.name = '2-Isopentenyl diphosphate'
iCdG791.metabolites.cpd11421_c.name = 'Reduced thioredoxin'
iCdG791.metabolites.cpd11420_c.name = 'Oxidized thioredoxin'
iCdG791.metabolites.cpd00309_c.name = 'Xanthine'
iCdG791.metabolites.cpd00309_e.name = 'Xanthine'
iCdG791.metabolites.cpd00276_e.name = 'D-Glucosamine'
iCdG791.metabolites.cpd01080_c.name = 'Octadecanoic acid'
iCdG791.metabolites.cpd00050_c.name = 'Flavin mononucleotide'
iCdG791.metabolites.cpd00508_c.name = '(3S)-3-Methyl-2-oxopentanoate'
iCdG791.metabolites.cpd00644_c.name = '(R)-Pantothenate'
iCdG791.metabolites.cpd00226_e.name = 'Hypoxanthine'
iCdG791.metabolites.cpd01080_e.name = 'Octadecanoic acid'
iCdG791.metabolites.cpd02996_c.name = 'N-Acetyl-D-galactosaminyl-(N-acetylneuraminyl)-D-galactosyl-1,4- beta-D-glucosyl-N-acylsphingosine'
iCdG791.metabolites.cpd00361_c.name = '(S)-Acetoin'
iCdG791.metabolites.cpd00497_c.name = '(9-D-Ribosylxanthine)-5-phosphate'
iCdG791.metabolites.cpd11434_c.name = 'Anteiso-C17:0 CoA Anteisoheptadecanoyl-CoA'
iCdG791.metabolites.cpd02921_c.name = 'S-2-5-Amino-1-(5-phospho-D-ribosyl)imidazole-4-carboxamido-succinate'
iCdG791.metabolites.cpd00042_c.name = 'Glutathione'
iCdG791.metabolites.cpd11616_c.name = 'Aldehyde'
iCdG791.metabolites.cpd02678_c.name = '5-P-ribosyl-N-formylglycineamide'
iCdG791.metabolites.cpd02394_c.name = '5-Phosphoribosylglycinamide'
iCdG791.metabolites.cpd02591_c.name = '(2E,6E,10E,14E)-geranylfarnesyl diphosphate'
iCdG791.metabolites.cpd00200_c.name = '2-Oxoisocaproate'
iCdG791.metabolites.cpd00599_c.name = '2-hydroxybenzoate'
iCdG791.metabolites.cpd00853_c.name = '2,3-Bisphospho-D-glycerate'
iCdG791.metabolites.cpd11432_c.name = 'Iso-C17:0 CoA Isoheptadecanoyl-CoA'
iCdG791.metabolites.cpd11441_c.name = 'Iso-C16:0 CoA Isohexadecanoyl-CoA'
iCdG791.metabolites.cpd11435_c.name = 'Iso-C14:0 CoA Isotetradecanoyl-CoA'
iCdG791.metabolites.cpd11439_c.name = 'Anteiso-C15:0 CoA Anteisopentadecanoyl-CoA'
iCdG791.metabolites.cpd02972_c.name = 'alpha-N-Acetylneuraminyl-2,8-alpha-N-acetylneuraminyl-2,3-beta-D-galactosyl-R'
iCdG791.metabolites.cpd00794_e.name = 'D-Trehalose'
iCdG791.metabolites.cpd00794_c.name = 'D-Trehalose'
iCdG791.metabolites.cpd00668_c.name = '2-Acetolactate'
iCdG791.metabolites.cpd02631_c.name = '(3R,4S,5R)-3,4,5-Trihydroxy-1,2-cyclohexanedione'
iCdG791.metabolites.cpd02968_c.name = 'UDP-N-acetylmuramoyl-L-alanyl-D-glutamyl-6-carboxy-L-lysyl-D-alanyl-D-alanine'
iCdG791.metabolites.cpd00208_e.name = 'Lactose'
iCdG791.metabolites.cpd00208_c.name = 'Lactose'
iCdG791.metabolites.cpd00158_e.name = 'Cellobiose'
iCdG791.metabolites.cpd00644_e.name = 'Pantothenate'
iCdG791.metabolites.cpd02964_c.name = 'UDP-N-acetylmuramoyl-L-alanyl-D-gamma-glutamyl-meso-2-6-diaminopimelate'
iCdG791.metabolites.cpd01741_c.name = 'Dodecanoate'
iCdG791.metabolites.cpd00354_c.name = 'Nucleoside 5-diphosphate'
iCdG791.metabolites.cpd02920_c.name = '2-Amino-4-hydroxy-6-hydroxymethyl-7-8-dihydropteridinediphosphate'
iCdG791.metabolites.cpd00327_c.name = 'Stearoyl-CoA'
iCdG791.metabolites.cpd11437_c.name = 'Iso-C15:0 CoA Isopentadecanoyl-CoA'
iCdG791.metabolites.cpd00646_c.name = '4-Hydroxynitrobenzene'
iCdG791.metabolites.cpd04011_c.name = 'Diethylthiophosphoric acid'


In [3]:
test = 0
for x in iCdG791.genes:
    if (len(x.reactions)) == 0:
        test += 1
print('Reamining genes: ' + str(test))

Reamining genes: 278


In [4]:
iCdG791.slim_optimize()

89.76884522354938

In [5]:
# Remove those genes that only act on macromolecules
from cobra.manipulation.delete import remove_genes
genes_to_be_removed = ['272563.8.peg.14','272563.8.peg.24','272563.8.peg.3559','272563.8.peg.9','272563.8.peg.13','272563.8.peg.2710','272563.8.peg.2183','272563.8.peg.1314','272563.8.peg.2596','272563.8.peg.3838','272563.8.peg.3388','272563.8.peg.3083','272563.8.peg.10','272563.8.peg.348','272563.8.peg.1360','272563.8.peg.1361','272563.8.peg.3664','272563.8.peg.1339','272563.8.peg.3717','272563.8.peg.77','272563.8.peg.1373','272563.8.peg.113','272563.8.peg.3857','272563.8.peg.1312','272563.8.peg.1527','272563.8.peg.1570','272563.8.peg.78','272563.8.peg.3858','272563.8.peg.89','272563.8.peg.94','272563.8.peg.83','272563.8.peg.1235','272563.8.peg.2253','272563.8.peg.69','272563.8.peg.95','272563.8.peg.1383','272563.8.peg.80','272563.8.peg.103','272563.8.peg.1221','272563.8.peg.112','272563.8.peg.3837','272563.8.peg.120','272563.8.peg.119','272563.8.peg.96','272563.8.peg.93','272563.8.peg.1322','272563.8.peg.721','272563.8.peg.97','272563.8.peg.2685','272563.8.peg.88','272563.8.peg.91','272563.8.peg.74','272563.8.peg.92','272563.8.peg.111','272563.8.peg.2567','272563.8.peg.73','272563.8.peg.86','272563.8.peg.101','272563.8.peg.100','272563.8.peg.98','272563.8.peg.1318','272563.8.peg.2595','272563.8.peg.99','272563.8.peg.85','272563.8.peg.114','272563.8.peg.109','272563.8.peg.3833','272563.8.peg.110','272563.8.peg.87','272563.8.peg.1219','272563.8.peg.90','272563.8.peg.722','272563.8.peg.79','272563.8.peg.84','272563.8.peg.3839','272563.8.peg.102','272563.8.peg.3653']
remove_genes(iCdG791, genes_to_be_removed)

# Lead, cadmium, zinc and mercury transporters
genes_to_be_removed = ['272563.8.peg.620','272563.8.peg.1714','272563.8.peg.2961']
remove_genes(iCdG791, genes_to_be_removed)

# Pyruvate formate-lyase activating enzyme
genes_to_be_removed = ['272563.8.peg.795','272563.8.peg.3436','272563.8.peg.1176']
remove_genes(iCdG791, genes_to_be_removed)

# L(+)-tartrate dehydratase beta subunit (EC 4.2.1.32)
genes_to_be_removed = ['272563.8.peg.1055','272563.8.peg.1054']
remove_genes(iCdG791, genes_to_be_removed)

# Sulfur carrier protein ThiS
genes_to_be_removed = ['272563.8.peg.1781','272563.8.peg.1782']
remove_genes(iCdG791, genes_to_be_removed)

# Others
remove_genes(iCdG791, ['272563.8.peg.907'])  # Predicted molybdate-responsive regulator YvgK in bacilli
remove_genes(iCdG791, ['272563.8.peg.2617']) # Selenide,water dikinase (EC 2.7.9.3)
remove_genes(iCdG791, ['272563.8.peg.2586']) # Hypothetical radical SAM family enzyme in heat shock gene cluster
remove_genes(iCdG791, ['272563.8.peg.2223']) # Copper-translocating P-type ATPase (EC 3.6.3.4)
remove_genes(iCdG791, ['272563.8.peg.3769']) # Methionine synthase activation domain (EC 2.1.1.13)
remove_genes(iCdG791, ['272563.8.peg.472'])  # ornithine racemase (EC 5.1.1.12)
remove_genes(iCdG791, ['272563.8.peg.3730']) # tRNA(Ile)-lysidine synthetase

In [6]:
test = 0
for x in iCdG791.genes:
    if (len(x.reactions)) == 0:
        test += 1
print('Reamining genes: ' + str(test))

Reamining genes: 184


In [7]:
# Protein biosynthesis - correction to integrate genes that create respective tRNAs

cpd00002_c = iCdG791.metabolites.get_by_id('cpd00002_c') # ATP
cpd17043_c = iCdG791.metabolites.get_by_id('cpd17043_c') # RNA
cpd00067_c = iCdG791.metabolites.get_by_id('cpd00067_c') # H+
cpd00018_c = iCdG791.metabolites.get_by_id('cpd00018_c') # AMP
cpd00012_c = iCdG791.metabolites.get_by_id('cpd00012_c') # PPi

# L-Alanine
cpd00035_c = iCdG791.metabolites.get_by_id('cpd00035_c') 
cpd11770_c = cobra.Metabolite(
    'cpd11770_c',
    formula='',
    name='L-Alanyl-tRNA',
    compartment='cytosol')
cpd11770_c.annotation['sbo'] = 'SBO:0000247'
cpd11770_c.annotation['seed.compound'] = 'cpd11770'
rxn06300_c = cobra.Reaction('rxn06300_c')
rxn06300_c.gene_reaction_rule = '272563.8.peg.1347'
rxn06300_c.name = 'L-Alanine:tRNA(Ala) ligase (AMP-forming)'
rxn06300_c.bounds = (-1000., 1000.)
rxn06300_c.add_metabolites({
    cpd00035_c: -1.0, # amino acid
    cpd17043_c: -0.05, # tRNA
    cpd00002_c: -1.0, # ATP
    cpd11770_c: 1.0, # AA-tRNA
    cpd00018_c: 1.0, # AMP
    cpd00012_c: 1.0, # PPi
    cpd00067_c: 1.0 # H+
})
rxn06300_c.annotation['sbo'] = 'SBO:0000176'
rxn06300_c.annotation['seed.reaction'] = 'rxn06300'

# L-Arginine
cpd00051_c = iCdG791.metabolites.get_by_id('cpd00051_c') 
cpd12036_c = cobra.Metabolite(
    'cpd12036_c',
    formula='',
    name='L-Arginyl-tRNA',
    compartment='cytosol')
cpd12036_c.annotation['sbo'] = 'SBO:0000247'
cpd12036_c.annotation['seed.compound'] = 'cpd12036'
rxn06432_c = cobra.Reaction('rxn06432_c')
rxn06432_c.gene_reaction_rule = '272563.8.peg.746'
rxn06432_c.name = 'L-Arginine:tRNA(Arg) ligase (AMP-forming)'
rxn06432_c.bounds = (-1000., 1000.)
rxn06432_c.add_metabolites({
    cpd00051_c: -1.0, # amino acid
    cpd17043_c: -0.05, # tRNA
    cpd00002_c: -1.0, # ATP
    cpd12036_c: 1.0, # AA-tRNA
    cpd00018_c: 1.0, # AMP
    cpd00012_c: 1.0, # PPi
    cpd00067_c: 1.0 # H+
})
rxn06432_c.annotation['sbo'] = 'SBO:0000176'
rxn06432_c.annotation['seed.reaction'] = 'rxn06432'

# L-Asparagine
cpd00132_c = iCdG791.metabolites.get_by_id('cpd00132_c') 
cpd12313_c = cobra.Metabolite(
    'cpd12313_c',
    formula='',
    name='L-Asparagyl-tRNA',
    compartment='cytosol')
cpd12313_c.annotation['sbo'] = 'SBO:0000247'
cpd12313_c.annotation['seed.compound'] = 'cpd12313'
rxn06434_c = cobra.Reaction('rxn06434_c')
rxn06434_c.gene_reaction_rule = '272563.8.peg.2360'
rxn06434_c.name = 'L-Asparagine:tRNA(Asn) ligase (AMP-forming)'
rxn06434_c.bounds = (-1000., 1000.)
rxn06434_c.add_metabolites({
    cpd00132_c: -1.0, # amino acid
    cpd17043_c: -0.05, # tRNA
    cpd00002_c: -1.0, # ATP
    cpd12313_c: 1.0, # AA-tRNA
    cpd00018_c: 1.0, # AMP
    cpd00012_c: 1.0, # PPi
    cpd00067_c: 1.0 # H+
})
rxn06434_c.annotation['sbo'] = 'SBO:0000176'
rxn06434_c.annotation['seed.reaction'] = 'rxn06434'

# L-Aspartate
cpd00041_c = iCdG791.metabolites.get_by_id('cpd00041_c') 
cpd12226_c = cobra.Metabolite(
    'cpd12226_c',
    formula='',
    name='L-Aspartyl-tRNA',
    compartment='cytosol')
cpd12226_c.annotation['sbo'] = 'SBO:0000247'
cpd12226_c.annotation['seed.compound'] = 'cpd12226'
rxn06936_c = cobra.Reaction('rxn06936_c')
rxn06936_c.gene_reaction_rule = '272563.8.peg.2866'
rxn06936_c.name = 'L-Aspartate:tRNA(Asp) ligase (AMP-forming)'
rxn06936_c.bounds = (-1000., 1000.)
rxn06936_c.add_metabolites({
    cpd00041_c: -1.0, # amino acid
    cpd17043_c: -0.05, # tRNA
    cpd00002_c: -1.0, # ATP
    cpd12226_c: 1.0, # AA-tRNA
    cpd00018_c: 1.0, # AMP
    cpd00012_c: 1.0, # PPi
    cpd00067_c: 1.0 # H+
})
rxn06936_c.annotation['sbo'] = 'SBO:0000176'
rxn06936_c.annotation['seed.reaction'] = 'rxn06936'

# L-Cysteine
cpd00084_c = iCdG791.metabolites.get_by_id('cpd00084_c') 
cpd12255_c = cobra.Metabolite(
    'cpd12255_c_c',
    formula='',
    name='Cysteinyl-tRNA',
    compartment='cytosol')
cpd12255_c.annotation['sbo'] = 'SBO:0000247'
cpd12255_c.annotation['seed.compound'] = 'cpd12255'
rxn06435_c = cobra.Reaction('rxn06435_c')
rxn06435_c.gene_reaction_rule = '272563.8.peg.62'
rxn06435_c.name = 'L-Cysteine:tRNA(Cys) ligase (AMP-forming)'
rxn06435_c.bounds = (-1000., 1000.)
rxn06435_c.add_metabolites({
    cpd00084_c: -1.0, # amino acid
    cpd17043_c: -0.05, # tRNA
    cpd00002_c: -1.0, # ATP
    cpd12255_c: 1.0, # AA-tRNA
    cpd00018_c: 1.0, # AMP
    cpd00012_c: 1.0, # PPi
    cpd00067_c: 1.0 # H+
})
rxn06435_c.annotation['sbo'] = 'SBO:0000176'
rxn06435_c.annotation['seed.reaction'] = 'rxn06435'

# L-Glutamine - no gene
cpd00053_c = iCdG791.metabolites.get_by_id('cpd00053_c') 
cpd12060_c = cobra.Metabolite(
    'cpd12060_c',
    formula='',
    name='L-Glutaminyl-tRNA',
    compartment='cytosol')
cpd12060_c.annotation['sbo'] = 'SBO:0000247'
cpd12060_c.annotation['seed.compound'] = 'cpd12060'
rxn06437_c = cobra.Reaction('rxn06437_c')
rxn06437_c.gene_reaction_rule = '272563.120.peg.2173'
rxn06437_c.name = 'L-Glutamine:tRNA(Gln) ligase (AMP-forming)'
rxn06437_c.bounds = (-1000., 1000.)
rxn06437_c.add_metabolites({
    cpd00053_c: -1.0, # amino acid
    cpd17043_c: -0.05, # tRNA
    cpd00002_c: -1.0, # ATP
    cpd12060_c: 1.0, # AA-tRNA
    cpd00018_c: 1.0, # AMP
    cpd00012_c: 1.0, # PPi
    cpd00067_c: 1.0 # H+
})
rxn06437_c.annotation['sbo'] = 'SBO:0000176'
rxn06437_c.annotation['seed.reaction'] = 'rxn06437'

# L-Glutamate - no gene
cpd00023_c = iCdG791.metabolites.get_by_id('cpd00023_c') 
cpd12828_c = cobra.Metabolite(
    'cpd12828_c',
    formula='',
    name='L-Glutamyl-tRNA',
    compartment='cytosol')
cpd12828_c.annotation['sbo'] = 'SBO:0000247'
cpd12828_c.annotation['seed.compound'] = 'cpd12828'
rxn06436_c = cobra.Reaction('rxn06436_c')
rxn06436_c.gene_reaction_rule = '272563.120.peg.55'
rxn06436_c.name = 'L-Glutamate:tRNA(Gln) ligase (AMP-forming)'
rxn06436_c.bounds = (-1000., 1000.)
rxn06436_c.add_metabolites({
    cpd00023_c: -1.0, # amino acid
    cpd17043_c: -0.05, # tRNA
    cpd00002_c: -1.0, # ATP
    cpd12828_c: 1.0, # AA-tRNA
    cpd00018_c: 1.0, # AMP
    cpd00012_c: 1.0, # PPi
    cpd00067_c: 1.0 # H+
})
rxn06436_c.annotation['sbo'] = 'SBO:0000176'
rxn06436_c.annotation['seed.reaction'] = 'rxn06436'

# Glycine
cpd00033_c = iCdG791.metabolites.get_by_id('cpd00033_c') 
cpd12100_c = cobra.Metabolite(
    'cpd12100_c',
    formula='',
    name='Glycyl-tRNA',
    compartment='cytosol')
cpd12100_c.annotation['sbo'] = 'SBO:0000247'
cpd12100_c.annotation['seed.compound'] = 'cpd12100'
rxn06438_c = cobra.Reaction('rxn06438_c')
rxn06438_c.gene_reaction_rule = '272563.8.peg.2551 and 272563.8.peg.2552'
rxn06438_c.name = 'glycine:tRNA(Gly) ligase (AMP-forming)'
rxn06438_c.bounds = (-1000., 1000.)
rxn06438_c.add_metabolites({
    cpd00033_c: -1.0, # amino acid
    cpd17043_c: -0.05, # tRNA
    cpd00002_c: -1.0, # ATP
    cpd12100_c: 1.0, # AA-tRNA
    cpd00018_c: 1.0, # AMP
    cpd00012_c: 1.0, # PPi
    cpd00067_c: 1.0 # H+
})
rxn06438_c.annotation['sbo'] = 'SBO:0000176'
rxn06438_c.annotation['seed.reaction'] = 'rxn06438'

# L-Histidine
cpd00119_c = iCdG791.metabolites.get_by_id('cpd00119_c') 
cpd12228_c = cobra.Metabolite(
    'cpd12228_c',
    formula='',
    name='L-Histidyl-tRNA',
    compartment='cytosol')
cpd12228_c.annotation['sbo'] = 'SBO:0000247'
cpd12228_c.annotation['seed.compound'] = 'cpd12228'
rxn06439_c = cobra.Reaction('rxn06439_c')
rxn06439_c.gene_reaction_rule = '272563.8.peg.2867'
rxn06439_c.name = 'L-Histidine:tRNA(His) ligase (AMP-forming)'
rxn06439_c.bounds = (-1000., 1000.)
rxn06439_c.add_metabolites({
    cpd00119_c: -1.0, # amino acid
    cpd17043_c: -0.05, # tRNA
    cpd00002_c: -1.0, # ATP
    cpd12228_c: 1.0, # AA-tRNA
    cpd00018_c: 1.0, # AMP
    cpd00012_c: 1.0, # PPi
    cpd00067_c: 1.0 # H+
})
rxn06439_c.annotation['sbo'] = 'SBO:0000176'
rxn06439_c.annotation['seed.reaction'] = 'rxn06439'

# L-Isoleucine
cpd00322_c = iCdG791.metabolites.get_by_id('cpd00322_c') 
cpd12256_c = cobra.Metabolite(
    'cpd12256_c',
    formula='',
    name='L-Isoleucyl-tRNA',
    compartment='cytosol')
cpd12256_c.annotation['sbo'] = 'SBO:0000247'
cpd12256_c.annotation['seed.compound'] = 'cpd12256'
rxn06440_c = cobra.Reaction('rxn06440_c')
rxn06440_c.gene_reaction_rule = '272563.8.peg.2741'
rxn06440_c.name = 'L-Isoleucine:tRNA(Ile) ligase (AMP-forming)'
rxn06440_c.bounds = (-1000., 1000.)
rxn06440_c.add_metabolites({
    cpd00322_c: -1.0, # amino acid
    cpd17043_c: -0.05, # tRNA
    cpd00002_c: -1.0, # ATP
    cpd12256_c: 1.0, # AA-tRNA
    cpd00018_c: 1.0, # AMP
    cpd00012_c: 1.0, # PPi
    cpd00067_c: 1.0 # H+
})
rxn06440_c.annotation['sbo'] = 'SBO:0000176'
rxn06440_c.annotation['seed.reaction'] = 'rxn06440'

# L-Leucine
cpd00107_c = iCdG791.metabolites.get_by_id('cpd00107_c') 
cpd12003_c = cobra.Metabolite(
    'cpd12003_c',
    formula='',
    name='Leucyl-tRNA',
    compartment='cytosol')
cpd12003_c.annotation['sbo'] = 'SBO:0000247'
cpd12003_c.annotation['seed.compound'] = 'cpd12003'
rxn06441_c = cobra.Reaction('rxn06441_c')
rxn06441_c.gene_reaction_rule = '272563.8.peg.2644'
rxn06441_c.name = 'L-Leucine:tRNA(Leu) ligase (AMP-forming)'
rxn06441_c.bounds = (-1000., 1000.)
rxn06441_c.add_metabolites({
    cpd00107_c: -1.0, # amino acid
    cpd17043_c: -0.05, # tRNA
    cpd00002_c: -1.0, # ATP
    cpd12003_c: 1.0, # AA-tRNA
    cpd00018_c: 1.0, # AMP
    cpd00012_c: 1.0, # PPi
    cpd00067_c: 1.0 # H+
})
rxn06441_c.annotation['sbo'] = 'SBO:0000176'
rxn06441_c.annotation['seed.reaction'] = 'rxn06441'

# L-Lysine
cpd00039_c = iCdG791.metabolites.get_by_id('cpd00039_c') 
cpd01326_c = cobra.Metabolite(
    'cpd01326_c',
    formula='',
    name='L-Lysyl-tRNA',
    compartment='cytosol')
cpd01326_c.annotation['sbo'] = 'SBO:0000247'
cpd01326_c.annotation['seed.compound'] = 'cpd01326'
rxn06442_c = cobra.Reaction('rxn06442_c')
rxn06442_c.gene_reaction_rule = '272563.8.peg.3722'
rxn06442_c.name = 'L-Lysine:tRNALys ligase (AMP-forming)'
rxn06442_c.bounds = (-1000., 1000.)
rxn06442_c.add_metabolites({
    cpd00039_c: -1.0, # amino acid
    cpd17043_c: -0.05, # tRNA
    cpd00002_c: -1.0, # ATP
    cpd01326_c: 1.0, # AA-tRNA
    cpd00018_c: 1.0, # AMP
    cpd00012_c: 1.0, # PPi
    cpd00067_c: 1.0 # H+
})
rxn06442_c.annotation['sbo'] = 'SBO:0000176'
rxn06442_c.annotation['seed.reaction'] = 'rxn06442'

# L-Methionine
cpd00060_c = iCdG791.metabolites.get_by_id('cpd00060_c') 
cpd12105_c = cobra.Metabolite(
    'cpd12105_c',
    formula='',
    name='Methionyl-tRNA',
    compartment='cytosol')
cpd12105_c.annotation['sbo'] = 'SBO:0000247'
cpd12105_c.annotation['seed.compound'] = 'cpd12105'
rxn06443_c = cobra.Reaction('rxn06443_c')
rxn06443_c.gene_reaction_rule = '272563.8.peg.3708'
rxn06443_c.name = 'L-Methionine:tRNAMet ligase (AMP-forming)'
rxn06443_c.bounds = (-1000., 1000.)
rxn06443_c.add_metabolites({
    cpd00060_c: -1.0, # amino acid
    cpd17043_c: -0.05, # tRNA
    cpd00002_c: -1.0, # ATP
    cpd12105_c: 1.0, # AA-tRNA
    cpd00018_c: 1.0, # AMP
    cpd00012_c: 1.0, # PPi
    cpd00067_c: 1.0 # H+
})
rxn06443_c.annotation['sbo'] = 'SBO:0000176'
rxn06443_c.annotation['seed.reaction'] = 'rxn06443'

# L-Phenylalanine
cpd00066_c = iCdG791.metabolites.get_by_id('cpd00066_c') 
cpd12335_c = cobra.Metabolite(
    'cpd12335_c',
    formula='',
    name='Phenylalanyl-tRNA',
    compartment='cytosol')
cpd12335_c.annotation['sbo'] = 'SBO:0000247'
cpd12335_c.annotation['seed.compound'] = 'cpd12335'
rxn06444_c = cobra.Reaction('rxn06444_c')
rxn06444_c.gene_reaction_rule = '272563.8.peg.734 and 272563.8.peg.735'
rxn06444_c.name = 'L-Phenylalanine:tRNA(Ala) ligase (AMP-forming)'
rxn06444_c.bounds = (-1000., 1000.)
rxn06444_c.add_metabolites({
    cpd00066_c: -1.0, # amino acid
    cpd17043_c: -0.05, # tRNA
    cpd00002_c: -1.0, # ATP
    cpd12335_c: 1.0, # AA-tRNA
    cpd00018_c: 1.0, # AMP
    cpd00012_c: 1.0, # PPi
    cpd00067_c: 1.0 # H+
})
rxn06444_c.annotation['sbo'] = 'SBO:0000176'
rxn06444_c.annotation['seed.reaction'] = 'rxn06444'

# L-Proline
cpd00129_c = iCdG791.metabolites.get_by_id('cpd00129_c') 
cpd12164_c = cobra.Metabolite(
    'cpd12164_c',
    formula='',
    name='L-Prolyl-tRNA',
    compartment='cytosol')
cpd12164_c.annotation['sbo'] = 'SBO:0000247'
cpd12164_c.annotation['seed.compound'] = 'cpd12164'
rxn06445_c = cobra.Reaction('rxn06445_c')
rxn06445_c.gene_reaction_rule = '272563.120.peg.53'
rxn06445_c.name = 'L-proline:tRNA(Pro) ligase (AMP-forming)'
rxn06445_c.bounds = (-1000., 1000.)
rxn06445_c.add_metabolites({
    cpd00129_c: -1.0, # amino acid
    cpd17043_c: -0.05, # tRNA
    cpd00002_c: -1.0, # ATP
    cpd12164_c: 1.0, # AA-tRNA
    cpd00018_c: 1.0, # AMP
    cpd00012_c: 1.0, # PPi
    cpd00067_c: 1.0 # H+
})
rxn06445_c.annotation['sbo'] = 'SBO:0000176'
rxn06445_c.annotation['seed.reaction'] = 'rxn06445'

# L-Serine
cpd00054_c = iCdG791.metabolites.get_by_id('cpd00054_c') 
cpd12132_c = cobra.Metabolite(
    'cpd12132_c',
    formula='',
    name='L-Seryl-tRNA',
    compartment='cytosol')
cpd12132_c.annotation['sbo'] = 'SBO:0000247'
cpd12132_c.annotation['seed.compound'] = 'cpd12132'
rxn06446_c = cobra.Reaction('rxn06446_c')
rxn06446_c.gene_reaction_rule = '272563.8.peg.22 or 272563.8.peg.2968'
rxn06446_c.name = 'L-Serine:tRNA(Ser) ligase (AMP-forming)'
rxn06446_c.bounds = (-1000., 1000.)
rxn06446_c.add_metabolites({
    cpd00054_c: -1.0, # amino acid
    cpd17043_c: -0.05, # tRNA
    cpd00002_c: -1.0, # ATP
    cpd12132_c: 1.0, # AA-tRNA
    cpd00018_c: 1.0, # AMP
    cpd00012_c: 1.0, # PPi
    cpd00067_c: 1.0 # H+
})
rxn06446_c.annotation['sbo'] = 'SBO:0000176'
rxn06446_c.annotation['seed.reaction'] = 'rxn06446'

# L-Threonine
cpd00161_c = iCdG791.metabolites.get_by_id('cpd00161_c') 
cpd12229_c = cobra.Metabolite(
    'cpd12229_c',
    formula='',
    name='Threonyl-tRNA',
    compartment='cytosol')
cpd12229_c.annotation['sbo'] = 'SBO:0000247'
cpd12229_c.annotation['seed.compound'] = 'cpd12229'
rxn06447_c = cobra.Reaction('rxn06447_c')
rxn06447_c.gene_reaction_rule = '272563.8.peg.601'
rxn06447_c.name = 'L-Threonine:tRNA(Thr) ligase (AMP-forming)'
rxn06447_c.bounds = (-1000., 1000.)
rxn06447_c.add_metabolites({
    cpd00161_c: -1.0, # amino acid
    cpd17043_c: -0.05, # tRNA
    cpd00002_c: -1.0, # ATP
    cpd12229_c: 1.0, # AA-tRNA
    cpd00018_c: 1.0, # AMP
    cpd00012_c: 1.0, # PPi
    cpd00067_c: 1.0 # H+
})
rxn06447_c.annotation['sbo'] = 'SBO:0000176'
rxn06447_c.annotation['seed.reaction'] = 'rxn06447'

# L-Tryptophan
cpd00065_c = iCdG791.metabolites.get_by_id('cpd00065_c') 
cpd12336_c = cobra.Metabolite(
    'cpd12336_c',
    formula='',
    name='L-Tryptophanyl-tRNA',
    compartment='cytosol')
cpd12336_c.annotation['sbo'] = 'SBO:0000247'
cpd12336_c.annotation['seed.compound'] = 'cpd12336'
rxn06448_c = cobra.Reaction('rxn06448_c')
rxn06448_c.gene_reaction_rule = '272563.8.peg.2733'
rxn06448_c.name = 'L-Tryptophan -tRNA(Trp) ligase (AMP-forming)'
rxn06448_c.bounds = (-1000., 1000.)
rxn06448_c.add_metabolites({
    cpd00065_c: -1.0, # amino acid
    cpd17043_c: -0.05, # tRNA
    cpd00002_c: -1.0, # ATP
    cpd12336_c: 1.0, # AA-tRNA
    cpd00018_c: 1.0, # AMP
    cpd00012_c: 1.0, # PPi
    cpd00067_c: 1.0 # H+
})
rxn06448_c.annotation['sbo'] = 'SBO:0000176'
rxn06448_c.annotation['seed.reaction'] = 'rxn06448'

# L-Tyrosine
cpd00069_c = iCdG791.metabolites.get_by_id('cpd00069_c') 
cpd12194_c = cobra.Metabolite(
    'cpd12194_c',
    formula='',
    name='L-Tyrosyl-tRNA',
    compartment='cytosol')
cpd12194_c.annotation['sbo'] = 'SBO:0000247'
cpd12194_c.annotation['seed.compound'] = 'cpd12194'
rxn06280_c = cobra.Reaction('rxn06280_c')
rxn06280_c.gene_reaction_rule = '272563.8.peg.1597'
rxn06280_c.name = 'L-Tyrosine:tRNA(Tyr) ligase (AMP-forming)'
rxn06280_c.bounds = (-1000., 1000.)
rxn06280_c.add_metabolites({
    cpd00069_c: -1.0, # amino acid
    cpd17043_c: -0.05, # tRNA
    cpd00002_c: -1.0, # ATP
    cpd12194_c: 1.0, # AA-tRNA
    cpd00018_c: 1.0, # AMP
    cpd00012_c: 1.0, # PPi
    cpd00067_c: 1.0 # H+
})
rxn06280_c.annotation['sbo'] = 'SBO:0000176'
rxn06280_c.annotation['seed.reaction'] = 'rxn06280'

# L-Valine
cpd00156_c = iCdG791.metabolites.get_by_id('cpd00156_c') 
cpd12133_c = cobra.Metabolite(
    'cpd12133_c',
    formula='',
    name='L-Valyl-tRNA',
    compartment='cytosol')
cpd12133_c.annotation['sbo'] = 'SBO:0000247'
cpd12133_c.annotation['seed.compound'] = 'cpd12133'
rxn06449_c = cobra.Reaction('rxn06449_c')
rxn06449_c.gene_reaction_rule = '272563.8.peg.3409'
rxn06449_c.name = 'L-Valine:tRNAVal ligase (AMP-forming)'
rxn06449_c.bounds = (-1000., 1000.)
rxn06449_c.add_metabolites({
    cpd00156_c: -1.0, # amino acid
    cpd17043_c: -0.05, # RNA
    cpd00002_c: -1.0, # ATP
    cpd12133_c: 1.0, # AA-tRNA
    cpd00018_c: 1.0, # AMP
    cpd00012_c: 1.0, # PPi
    cpd00067_c: 1.0 # H+
})
rxn06449_c.annotation['sbo'] = 'SBO:0000176'
rxn06449_c.annotation['seed.reaction'] = 'rxn06449'

iCdG791.add_reactions([rxn06300_c,rxn06432_c,rxn06434_c,rxn06936_c,rxn06435_c,rxn06437_c,rxn06436_c,
                       rxn06438_c,rxn06439_c,rxn06440_c,rxn06441_c,rxn06442_c,rxn06443_c,rxn06444_c,
                       rxn06445_c,rxn06446_c,rxn06447_c,rxn06448_c,rxn06280_c,rxn06449_c])

iCdG791.genes.get_by_id('272563.120.peg.2173').name = 'Glutaminyl-tRNA synthetase (EC 6.1.1.18)'
iCdG791.genes.get_by_id('272563.120.peg.2173').annotation['sbo'] = 'SBO:0000243'
iCdG791.genes.get_by_id('272563.120.peg.2173').annotation['patric'] = 'fig|272563.120.peg.2173'

iCdG791.genes.get_by_id('272563.120.peg.55').name = 'Glutamyl-tRNA synthetase (EC 6.1.1.17)'
iCdG791.genes.get_by_id('272563.120.peg.55').annotation['sbo'] = 'SBO:0000243'
iCdG791.genes.get_by_id('272563.120.peg.55').annotation['patric'] = 'fig|272563.120.peg.55'

iCdG791.genes.get_by_id('272563.120.peg.53').name = 'Prolyl-tRNA synthetase (EC 6.1.1.15)'
iCdG791.genes.get_by_id('272563.120.peg.53').annotation['sbo'] = 'SBO:0000243'
iCdG791.genes.get_by_id('272563.120.peg.53').annotation['patric'] = 'fig|272563.120.peg.53'

#---------------------------------------------------------------------------#

iCdG791.reactions.protein_rxn.remove_from_model(remove_orphans=False)
cpd00001_c = iCdG791.metabolites.get_by_id('cpd00001_c') # H2O
cpd00008_c = iCdG791.metabolites.get_by_id('cpd00008_c') # ADP
cpd00009_c = iCdG791.metabolites.get_by_id('cpd00009_c') # Phosphate
cpd17041_c = iCdG791.metabolites.get_by_id('cpd17041_c') # Protein

# Create new reaction, now with tRNAs
protein_rxn = cobra.Reaction('protein_rxn')
protein_rxn.name = 'Protein biosynthesis'
protein_rxn.lower_bound = 0.
protein_rxn.upper_bound = 1000.
protein_rxn.add_metabolites({
    cpd11770_c: -0.264, # Alanine
    cpd12036_c: -0.160, # Arginine
    cpd12313_c: -0.322, # Asparagine
    cpd12226_c: -0.286, # Aspartate
    cpd12255_c: -0.062, # Cysteine
    cpd12060_c: -0.120, # Glutamine
    cpd12828_c: -0.367, # Glutamate
    cpd12100_c: -0.314, # Glycine
    cpd12228_c: -0.068, # Histidine
    cpd12256_c: -0.498, # Isoleucine
    cpd12003_c: -0.455, # Leucine
    cpd01326_c: -0.457, # Lysine
    cpd12105_c: -0.135, # Methionine
    cpd12335_c: -0.219, # Phenylalanine
    cpd12164_c: -0.137, # Proline
    cpd12132_c: -0.331, # Serine
    cpd12229_c: -0.248, # Threonine
    cpd12336_c: -0.032, # Tryptophan
    cpd12194_c: -0.208, # Tyrosine
    cpd12133_c: -0.328, # Valine
    cpd00002_c: -20.0,
    cpd00001_c: -1.0,
    cpd17041_c: 1.0,
    cpd00008_c: 20.0,
    cpd00009_c: 20.0
})

iCdG791.add_reactions([protein_rxn])
iCdG791.reactions.protein_rxn.annotation['sbo'] = 'SBO:0000629'


In [8]:
test = 0
for x in iCdG791.genes:
    if (len(x.reactions)) == 0:
        test += 1
print('Reamining genes: ' + str(test))

Reamining genes: 164


In [9]:
iCdG791.slim_optimize()

82.79962839526671

In [10]:
# Read in curated universal reacton bag and annotate
universal_GramPos = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/GramPos.sbml')

for cpd in universal_GramPos.metabolites:
    cpd.annotation['sbo'] = 'SBO:0000247'
    cpd.annotation['seed.compound'] = cpd.id.split('_')[0]    
    cpd.compartment = cpd.compartment.lower()

for rxn in universal_GramPos.reactions:
    rxn.annotation['sbo'] = 'SBO:0000176'
    rxn.annotation['seed.reaction'] = rxn.id.split('_')[0]


'GramPositive.modeltemplate' is not a valid SBML 'SId'.
No objective coefficients in model. Unclear what should be optimized


In [11]:
# Dipeptides
# Methionyl aminopeptidase and associated active transport + exchanges

# gly-glu-L 
rxn12634_c = universal_GramPos.reactions.rxn12634_c
rxn12634_c.lower_bound = 0.
rxn12634_c.gene_reaction_rule = '272563.8.peg.1673 or 272563.8.peg.106 or 272563.8.peg.1365 or 272563.8.peg.2607 or 272563.8.peg.2736 or 272563.8.peg.2464'
universal_GramPos.reactions.rxn12600_c.remove_from_model(remove_orphans=False)
rxn05534_c = universal_GramPos.reactions.rxn05534_c
rxn05534_c.gene_reaction_rule = '(272563.8.peg.896 or 272563.8.peg.2794 or 272563.8.peg.895) and 272563.8.peg.2798 and (272563.8.peg.894 or 272563.8.peg.2795) and (272563.8.peg.893 or 272563.8.peg.2796)'
iCdG791.add_reactions([rxn12634_c,rxn05534_c])
cpd11592_e = iCdG791.metabolites.cpd11592_e
iCdG791.add_boundary(cpd11592_e, type='exchange', reaction_id='EX_cpd11592_e', lb=-1000.0, ub=1000.0)

# met-L-ala-L
rxn12636_c = universal_GramPos.reactions.rxn12636_c
rxn12636_c.lower_bound = 0.
rxn12636_c.gene_reaction_rule = '272563.8.peg.1673 or 272563.8.peg.106 or 272563.8.peg.1365 or 272563.8.peg.2607 or 272563.8.peg.2736 or 272563.8.peg.2464'
universal_GramPos.reactions.rxn12544_c.remove_from_model(remove_orphans=False)
rxn05536_c = universal_GramPos.reactions.rxn05536_c
rxn05536_c.gene_reaction_rule = '(272563.8.peg.896 or 272563.8.peg.2794 or 272563.8.peg.895) and 272563.8.peg.2798 and (272563.8.peg.894 or 272563.8.peg.2795) and (272563.8.peg.893 or 272563.8.peg.2796)'
iCdG791.add_reactions([rxn12636_c,rxn05536_c])
cpd11590_e = iCdG791.metabolites.cpd11590_e
iCdG791.add_boundary(cpd11590_e, type='exchange', reaction_id='EX_cpd11590_e', lb=-1000.0, ub=1000.0)

# gly-pro-L 
rxn12638_c = universal_GramPos.reactions.rxn12638_c
rxn12638_c.lower_bound = 0.
rxn12638_c.gene_reaction_rule = '272563.8.peg.1673 or 272563.8.peg.106 or 272563.8.peg.1365 or 272563.8.peg.2607 or 272563.8.peg.2736 or 272563.8.peg.2464'
universal_GramPos.reactions.rxn12576_c.remove_from_model(remove_orphans=False)
rxn05538_c = universal_GramPos.reactions.rxn05538_c
rxn05538_c.gene_reaction_rule = '(272563.8.peg.896 or 272563.8.peg.2794 or 272563.8.peg.895) and 272563.8.peg.2798 and (272563.8.peg.894 or 272563.8.peg.2795) and (272563.8.peg.893 or 272563.8.peg.2796)'
iCdG791.add_reactions([rxn12638_c,rxn05538_c])
cpd11588_e = iCdG791.metabolites.cpd11588_e
iCdG791.add_boundary(cpd11588_e, type='exchange', reaction_id='EX_cpd11588_e', lb=-1000.0, ub=1000.0)

# ala-L-glu-L
rxn12640_c = universal_GramPos.reactions.rxn12640_c
rxn12640_c.lower_bound = 0.
rxn12640_c.gene_reaction_rule = '272563.8.peg.1673 or 272563.8.peg.106 or 272563.8.peg.1365 or 272563.8.peg.2607 or 272563.8.peg.2736 or 272563.8.peg.2464'
universal_GramPos.reactions.rxn12523_c.remove_from_model(remove_orphans=False)
rxn05541_c = universal_GramPos.reactions.rxn05541_c
rxn05541_c.gene_reaction_rule = '(272563.8.peg.896 or 272563.8.peg.2794 or 272563.8.peg.895) and 272563.8.peg.2798 and (272563.8.peg.894 or 272563.8.peg.2795) and (272563.8.peg.893 or 272563.8.peg.2796)'
iCdG791.add_reactions([rxn12640_c,rxn05541_c])
cpd11586_e = iCdG791.metabolites.cpd11586_e
iCdG791.add_boundary(cpd11586_e, type='exchange', reaction_id='EX_cpd11586_e', lb=-1000.0, ub=1000.0)

# ala-L-Thr-L
rxn12644_c = universal_GramPos.reactions.rxn12644_c
rxn12644_c.lower_bound = 0.
rxn12644_c.gene_reaction_rule = '272563.8.peg.1673 or 272563.8.peg.106 or 272563.8.peg.1365 or 272563.8.peg.2607 or 272563.8.peg.2736 or 272563.8.peg.2464'
universal_GramPos.reactions.rxn12616_c.remove_from_model(remove_orphans=False)
rxn05545_c = universal_GramPos.reactions.rxn05545_c
rxn05545_c.gene_reaction_rule = '(272563.8.peg.896 or 272563.8.peg.2794 or 272563.8.peg.895) and 272563.8.peg.2798 and (272563.8.peg.894 or 272563.8.peg.2795) and (272563.8.peg.893 or 272563.8.peg.2796)'
iCdG791.add_reactions([rxn12644_c,rxn05545_c])
cpd11582_e = iCdG791.metabolites.cpd11582_e
iCdG791.add_boundary(cpd11582_e, type='exchange', reaction_id='EX_cpd11582_e', lb=-1000.0, ub=1000.0)

# gly-asn-L
rxn12645_c = universal_GramPos.reactions.rxn12645_c
rxn12645_c.lower_bound = 0.
rxn12645_c.gene_reaction_rule = '272563.8.peg.1673 or 272563.8.peg.106 or 272563.8.peg.1365 or 272563.8.peg.2607 or 272563.8.peg.2736 or 272563.8.peg.2464'
universal_GramPos.reactions.rxn12542_c.remove_from_model(remove_orphans=False)
rxn05546_c = universal_GramPos.reactions.rxn05546_c
rxn05546_c.gene_reaction_rule = '(272563.8.peg.896 or 272563.8.peg.2794 or 272563.8.peg.895) and 272563.8.peg.2798 and (272563.8.peg.894 or 272563.8.peg.2795) and (272563.8.peg.893 or 272563.8.peg.2796)'
iCdG791.add_reactions([rxn12645_c,rxn05546_c])
cpd11581_e = iCdG791.metabolites.cpd11581_e 
iCdG791.add_boundary(cpd11581_e, type='exchange', reaction_id='EX_cpd11581_e', lb=-1000.0, ub=1000.0)


Reaction identifier,EX_cpd11581_e
Name,gly-asn-L exchange
Memory address,0x07f3496646dd0
Stoichiometry,cpd11581_e <=> gly-asn-L <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [12]:
# Remaining dipeptide import and Aminopeptidases in universal bag

cpd00002_c = iCdG791.metabolites.cpd00002_c # ATP
cpd00001_c = iCdG791.metabolites.cpd00001_c # H2O
cpd00008_c = iCdG791.metabolites.cpd00008_c # ADP
cpd00009_c = iCdG791.metabolites.cpd00009_c # Phosphate
cpd00067_c = iCdG791.metabolites.cpd00067_c # H

# L-Prolinylglycine
rxn08133_c = universal_GramPos.reactions.rxn08133_c
rxn08133_c.lower_bound = 0.
rxn08133_c.gene_reaction_rule = '272563.8.peg.1365 or 272563.8.peg.2607 or 272563.8.peg.2736 or 272563.8.peg.2464'
cpd15551_e = cobra.Metabolite(
    'cpd15551_e',
    formula='C7H11N2O3',
    name='L-Prolinylglycine',
    compartment='extracellular')
cpd15551_e.annotation['sbo'] = 'SBO:0000247'
cpd15551_e.annotation['seed.compound'] = 'cpd15551'
cpd15551_c = universal_GramPos.metabolites.cpd15551_c
rxn09189_c = cobra.Reaction('rxn09189_c')
rxn09189_c.gene_reaction_rule = '(272563.8.peg.896 or 272563.8.peg.2794 or 272563.8.peg.895) and 272563.8.peg.2798 and (272563.8.peg.894 or 272563.8.peg.2795) and (272563.8.peg.893 or 272563.8.peg.2796)'
rxn09189_c.name = 'L-Prolinylglycine (Pro-Gly) transport via ABC system'
rxn09189_c.bounds = (0., 1000.)
rxn09189_c.add_metabolites({
    cpd00001_c: -1.0, # H2O
    cpd00002_c: -1.0, # ATP
    cpd15551_e: -1.0, # L-Prolinylglycine
    cpd00008_c: 1.0, # ADP
    cpd00009_c: 1.0, # Pi
    cpd15551_c: 1.0, # L-Prolinylglycine
    cpd00067_c: 1.0 # H+
})
rxn09189_c.annotation['sbo'] = 'SBO:0000176'
rxn09189_c.annotation['seed.reaction'] = 'rxn09189'
iCdG791.add_reactions([rxn08133_c,rxn09189_c])
iCdG791.add_boundary(cpd15551_e, type='exchange', reaction_id='EX_cpd15551_e', lb=-1000.0, ub=1000.0)

# Gly-Cys
rxn12844_c = universal_GramPos.reactions.rxn12844_c
rxn12844_c.lower_bound = 0.
rxn12844_c.gene_reaction_rule = '272563.8.peg.1365 or 272563.8.peg.2607 or 272563.8.peg.2736 or 272563.8.peg.2464'
rxn12848_c = universal_GramPos.reactions.rxn12848_c
rxn12848_c.gene_reaction_rule = '(272563.8.peg.896 or 272563.8.peg.2794 or 272563.8.peg.895) and 272563.8.peg.2798 and (272563.8.peg.894 or 272563.8.peg.2795) and (272563.8.peg.893 or 272563.8.peg.2796)'
iCdG791.add_reactions([rxn12844_c,rxn12848_c])
cpd15603_e = iCdG791.metabolites.cpd15603_e
iCdG791.add_boundary(cpd15603_e, type='exchange', reaction_id='EX_cpd15603_e', lb=-1000.0, ub=1000.0)

# Gly-Leu
rxn12845_c = universal_GramPos.reactions.rxn12845_c
rxn12845_c.lower_bound = 0.
rxn12845_c.gene_reaction_rule = '272563.8.peg.1365 or 272563.8.peg.2607 or 272563.8.peg.2736 or 272563.8.peg.2464'
rxn12849_c = universal_GramPos.reactions.rxn12849_c
rxn12849_c.gene_reaction_rule = '(272563.8.peg.896 or 272563.8.peg.2794 or 272563.8.peg.895) and 272563.8.peg.2798 and (272563.8.peg.894 or 272563.8.peg.2795) and (272563.8.peg.893 or 272563.8.peg.2796)'
iCdG791.add_reactions([rxn12845_c,rxn12849_c])
cpd15604_e = iCdG791.metabolites.cpd15604_e
iCdG791.add_boundary(cpd15604_e, type='exchange', reaction_id='EX_cpd15604_e', lb=-1000.0, ub=1000.0)

# Gly-Phe
rxn12846_c = universal_GramPos.reactions.rxn12846_c
rxn12846_c.lower_bound = 0.
rxn12846_c.gene_reaction_rule = '272563.8.peg.1365 or 272563.8.peg.2607 or 272563.8.peg.2736 or 272563.8.peg.2464'
rxn12850_c = universal_GramPos.reactions.rxn12850_c
rxn12850_c.gene_reaction_rule = '(272563.8.peg.896 or 272563.8.peg.2794 or 272563.8.peg.895) and 272563.8.peg.2798 and (272563.8.peg.894 or 272563.8.peg.2795) and (272563.8.peg.893 or 272563.8.peg.2796)'
iCdG791.add_reactions([rxn12846_c,rxn12850_c])
cpd15605_e = iCdG791.metabolites.cpd15605_e
iCdG791.add_boundary(cpd15605_e, type='exchange', reaction_id='EX_cpd15605_e', lb=-1000.0, ub=1000.0)

# Gly-Try
rxn12847_c = universal_GramPos.reactions.rxn12847_c
rxn12847_c.lower_bound = 0.
rxn12847_c.gene_reaction_rule = '272563.8.peg.1365 or 272563.8.peg.2607 or 272563.8.peg.2736 or 272563.8.peg.2464'
rxn12851_c = universal_GramPos.reactions.rxn12851_c
rxn12851_c.gene_reaction_rule = '(272563.8.peg.896 or 272563.8.peg.2794 or 272563.8.peg.895) and 272563.8.peg.2798 and (272563.8.peg.894 or 272563.8.peg.2795) and (272563.8.peg.893 or 272563.8.peg.2796)'
iCdG791.add_reactions([rxn12851_c,rxn12847_c])
cpd15606_e = iCdG791.metabolites.cpd15606_e
iCdG791.add_boundary(cpd15606_e, type='exchange', reaction_id='EX_cpd15606_e', lb=-1000.0, ub=1000.0)


Reaction identifier,EX_cpd15606_e
Name,Gly-Tyr exchange
Memory address,0x07f3496862610
Stoichiometry,cpd15606_e <=> Gly-Tyr <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [13]:
test = 0
for x in iCdG791.genes:
    if (len(x.reactions)) == 0:
        test += 1
print('Reamining genes: ' + str(test))

Reamining genes: 150


In [14]:
iCdG791.slim_optimize()

82.98461354618698

In [15]:
# Dipeptide-associated reactions not in universal bag

cpd00002_c = iCdG791.metabolites.cpd00002_c # ATP
cpd00001_c = iCdG791.metabolites.cpd00001_c # H2O
cpd00008_c = iCdG791.metabolites.cpd00008_c # ADP
cpd00009_c = iCdG791.metabolites.cpd00009_c # Phosphate
cpd00067_c = iCdG791.metabolites.cpd00067_c # H+

# L-ala-L-asp
cpd29695_e = cobra.Metabolite(
    'cpd29695_e',
    formula='C7H11N2O5',
    name='L-ala-L-asp',
    compartment='extracellular')
cpd29695_e.annotation['sbo'] = 'SBO:0000247'
cpd29695_e.annotation['seed.compound'] = 'cpd29695'
iCdG791.add_boundary(cpd29695_e, type='exchange', reaction_id='EX_cpd29695_e', lb=-1000.0, ub=1000.0)
cpd29695_c = cobra.Metabolite(
    'cpd29695_c',
    formula='C7H11N2O5',
    name='L-ala-L-asp',
    compartment='cytosol')
cpd29695_c.annotation['sbo'] = 'SBO:0000247'
cpd29695_c.annotation['seed.compound'] = 'cpd29695'

# Transport
rxn30259_c = cobra.Reaction('rxn30259_c')
rxn30259_c.gene_reaction_rule = '(272563.8.peg.896 or 272563.8.peg.2794 or 272563.8.peg.895) and 272563.8.peg.2798 and (272563.8.peg.894 or 272563.8.peg.2795) and (272563.8.peg.893 or 272563.8.peg.2796)'
rxn30259_c.name = 'Dipeptide transport via ABC system (ala-asp)'
rxn30259_c.bounds = (0., 1000.)
rxn30259_c.add_metabolites({
    cpd29695_e: -1.0, # dipeptide
    cpd00002_c: -1.0, # ATP
    cpd00001_c: -1.0, # H2O
    cpd29695_c: 1.0, # dipeptide
    cpd00008_c: 1.0, # ADP
    cpd00009_c: 1.0, # Phosphate
    cpd00067_c: 1.0 # H+
})
rxn30259_c.annotation['sbo'] = 'SBO:0000176'
rxn30259_c.annotation['seed.reaction'] = 'rxn30259'
iCdG791.add_reactions([rxn30259_c])

# Hydrolysis
cpd00035_c = iCdG791.metabolites.cpd00035_c # Alanine
cpd00041_c = iCdG791.metabolites.cpd00041_c # Aspartate
rxn30237_c = cobra.Reaction('rxn30237_c')
rxn30237_c.gene_reaction_rule = '272563.8.peg.1365 or 272563.8.peg.2607 or 272563.8.peg.2736 or 272563.8.peg.2464'
rxn30237_c.name = 'aminopeptidase (ala-asp)'
rxn30237_c.bounds = (0., 1000.)
rxn30237_c.add_metabolites({
    cpd29695_c: -1.0, # dipeptide
    cpd00001_c: -1.0, # H2O
    cpd00035_c: 1.0, # peptide 1
    cpd00041_c: 1.0 # peptide 2
})
rxn30237_c.annotation['sbo'] = 'SBO:0000176'
rxn30237_c.annotation['seed.reaction'] = 'rxn30237'
iCdG791.add_reactions([rxn30237_c])

#------------------------------------------------------------------#

# L-ala-gly
cpd29694_e = cobra.Metabolite(
    'cpd29694_e',
    formula='C5H13N2O2',
    name='L-ala-gly',
    compartment='extracellular')
cpd29694_e.annotation['sbo'] = 'SBO:0000247'
cpd29694_e.annotation['seed.compound'] = 'cpd29694'
iCdG791.add_boundary(cpd29694_e, type='exchange', reaction_id='EX_cpd29694_e', lb=-1000.0, ub=1000.0)
cpd29694_c = cobra.Metabolite(
    'cpd29694_c',
    formula='C5H13N2O2',
    name='L-ala-gly',
    compartment='cytosol')
cpd29694_c.annotation['sbo'] = 'SBO:0000247'
cpd29694_c.annotation['seed.compound'] = 'cpd29694'

# Transport
rxn30262_c = cobra.Reaction('rxn30262_c')
rxn30262_c.gene_reaction_rule = '(272563.8.peg.896 or 272563.8.peg.2794 or 272563.8.peg.895) and 272563.8.peg.2798 and (272563.8.peg.894 or 272563.8.peg.2795) and (272563.8.peg.893 or 272563.8.peg.2796)'
rxn30262_c.name = 'Dipeptide transport via ABC system (ala-gly)'
rxn30262_c.bounds = (0., 1000.)
rxn30262_c.add_metabolites({
    cpd29694_e: -1.0, # dipeptide
    cpd00002_c: -1.0, # ATP
    cpd00001_c: -1.0, # H2O
    cpd29694_c: 1.0, # dipeptide
    cpd00008_c: 1.0, # ADP
    cpd00009_c: 1.0, # Phosphate
    cpd00067_c: 1.0 # H+
})
rxn30262_c.annotation['sbo'] = 'SBO:0000176'
rxn30262_c.annotation['seed.reaction'] = 'rxn30262'
iCdG791.add_reactions([rxn30262_c])

# Hydrolysis
cpd00035_c = iCdG791.metabolites.cpd00035_c # Alanine
cpd00033_c = iCdG791.metabolites.cpd00033_c # Glycine
rxn30238_c = cobra.Reaction('rxn30238_c')
rxn30238_c.gene_reaction_rule = '272563.8.peg.1365 or 272563.8.peg.2607 or 272563.8.peg.2736 or 272563.8.peg.2464'
rxn30238_c.name = 'Aminopeptidase (ala-gly)'
rxn30238_c.bounds = (0., 1000.)
rxn30238_c.add_metabolites({
    cpd29694_c: -1.0, # dipeptide
    cpd00001_c: -1.0, # H2O
    cpd00035_c: 1.0, # peptide 1
    cpd00033_c: 1.0 # peptide 2
})
rxn30238_c.annotation['sbo'] = 'SBO:0000176'
rxn30238_c.annotation['seed.reaction'] = 'rxn30238'
iCdG791.add_reactions([rxn30238_c])

#------------------------------------------------------------------#

# L-ala-L-his
cpd29697_e = cobra.Metabolite(
    'cpd29697_e',
    formula='C9H14N4O3',
    name='L-ala-L-his',
    compartment='extracellular')
cpd29697_e.annotation['sbo'] = 'SBO:0000247'
cpd29697_e.annotation['seed.compound'] = 'cpd29697'
iCdG791.add_boundary(cpd29697_e, type='exchange', reaction_id='EX_cpd29697_e', lb=-1000.0, ub=1000.0)
cpd29697_c = cobra.Metabolite(
    'cpd29697_c',
    formula='C9H14N4O3',
    name='L-ala-L-his',
    compartment='cytosol')
cpd29697_c.annotation['sbo'] = 'SBO:0000247'
cpd29697_c.annotation['seed.compound'] = 'cpd29697'

# Transport
rxn30264_c = cobra.Reaction('rxn30264_c')
rxn30264_c.gene_reaction_rule = '(272563.8.peg.896 or 272563.8.peg.2794 or 272563.8.peg.895) and 272563.8.peg.2798 and (272563.8.peg.894 or 272563.8.peg.2795) and (272563.8.peg.893 or 272563.8.peg.2796)'
rxn30264_c.name = 'Dipeptide transport via ABC system'
rxn30264_c.bounds = (0., 1000.)
rxn30264_c.add_metabolites({
    cpd29697_e: -1.0, # dipeptide
    cpd00002_c: -1.0, # ATP
    cpd00001_c: -1.0, # H2O
    cpd29697_c: 1.0, # dipeptide
    cpd00008_c: 1.0, # ADP
    cpd00009_c: 1.0, # Phosphate
    cpd00067_c: 1.0 # H+
})
rxn30264_c.annotation['sbo'] = 'SBO:0000176'
rxn30264_c.annotation['seed.reaction'] = 'rxn30264'
iCdG791.add_reactions([rxn30264_c])

# Hydrolysis
cpd00035_c = iCdG791.metabolites.cpd00035_c # Alanine
cpd00119_c = iCdG791.metabolites.cpd00119_c # Histidine
rxn30239_c = cobra.Reaction('rxn30239_c')
rxn30239_c.gene_reaction_rule = '272563.8.peg.1365 or 272563.8.peg.2607 or 272563.8.peg.2736 or 272563.8.peg.2464'
rxn30239_c.name = 'Aminopeptidase'
rxn30239_c.bounds = (0., 1000.)
rxn30239_c.add_metabolites({
    cpd29697_c: -1.0, # dipeptide
    cpd00001_c: -1.0, # H2O
    cpd00035_c: 1.0, # peptide 1
    cpd00119_c: 1.0 # peptide 2
})
rxn30239_c.annotation['sbo'] = 'SBO:0000176'
rxn30239_c.annotation['seed.reaction'] = 'rxn30239'
iCdG791.add_reactions([rxn30239_c])

#------------------------------------------------------------------#

# L-ala-L-leu
cpd29698_e = cobra.Metabolite(
    'cpd29698_e',
    formula='C9H14N4O3',
    name='L-ala-L-leu',
    compartment='extracellular')
cpd29698_e.annotation['sbo'] = 'SBO:0000247'
cpd29698_e.annotation['seed.compound'] = 'cpd29698'
iCdG791.add_boundary(cpd29698_e, type='exchange', reaction_id='EX_cpd29698_e', lb=-1000.0, ub=1000.0)
cpd29698_c = cobra.Metabolite(
    'cpd29698_c',
    formula='C9H14N4O3',
    name='L-ala-L-leu',
    compartment='cytosol')
cpd29698_c.annotation['sbo'] = 'SBO:0000247'
cpd29698_c.annotation['seed.compound'] = 'cpd29698'

# Transport
rxn30266_c = cobra.Reaction('rxn30266_c')
rxn30266_c.gene_reaction_rule = '(272563.8.peg.896 or 272563.8.peg.2794 or 272563.8.peg.895) and 272563.8.peg.2798 and (272563.8.peg.894 or 272563.8.peg.2795) and (272563.8.peg.893 or 272563.8.peg.2796)'
rxn30266_c.name = 'Dipeptide transport via ABC system'
rxn30266_c.bounds = (0., 1000.)
rxn30266_c.add_metabolites({
    cpd29698_e: -1.0, # dipeptide
    cpd00002_c: -1.0, # ATP
    cpd00001_c: -1.0, # H2O
    cpd29698_c: 1.0, # dipeptide
    cpd00008_c: 1.0, # ADP
    cpd00009_c: 1.0, # Phosphate
    cpd00067_c: 1.0 # H+
})
rxn30266_c.annotation['sbo'] = 'SBO:0000176'
rxn30266_c.annotation['seed.reaction'] = 'rxn30266'
iCdG791.add_reactions([rxn30266_c])

# Hydrolysis
cpd00035_c = iCdG791.metabolites.cpd00035_c # Alanine
cpd00107_c = iCdG791.metabolites.cpd00107_c # Leucine
rxn30240_c = cobra.Reaction('rxn30240_c')
rxn30240_c.gene_reaction_rule = '272563.8.peg.1365 or 272563.8.peg.2607 or 272563.8.peg.2736 or 272563.8.peg.2464'
rxn30240_c.name = 'Aminopeptidase'
rxn30240_c.bounds = (0., 1000.)
rxn30240_c.add_metabolites({
    cpd29698_c: -1.0, # dipeptide
    cpd00001_c: -1.0, # H2O
    cpd00035_c: 1.0, # peptide 1
    cpd00107_c: 1.0 # peptide 2
})
rxn30240_c.annotation['sbo'] = 'SBO:0000176'
rxn30240_c.annotation['seed.reaction'] = 'rxn30240'
iCdG791.add_reactions([rxn30240_c])

#------------------------------------------------------------------#

# L-ala-L-gln
cpd29696_e = cobra.Metabolite(
    'cpd29696_e',
    formula='C8H15N3O4',
    name='L-ala-L-gln',
    compartment='extracellular')
cpd29696_e.annotation['sbo'] = 'SBO:0000247'
cpd29696_e.annotation['seed.compound'] = 'cpd29696'
iCdG791.add_boundary(cpd29696_e, type='exchange', reaction_id='EX_cpd29696_e', lb=-1000.0, ub=1000.0)
cpd29696_c = cobra.Metabolite(
    'cpd29696_c',
    formula='C8H15N3O4',
    name='L-ala-L-gln',
    compartment='cytosol')
cpd29696_c.annotation['sbo'] = 'SBO:0000247'
cpd29696_c.annotation['seed.compound'] = 'cpd29696'

# Transport
rxn30268_c = cobra.Reaction('rxn30268_c')
rxn30268_c.gene_reaction_rule = '(272563.8.peg.896 or 272563.8.peg.2794 or 272563.8.peg.895) and 272563.8.peg.2798 and (272563.8.peg.894 or 272563.8.peg.2795) and (272563.8.peg.893 or 272563.8.peg.2796)'
rxn30268_c.name = 'Dipeptide transport via ABC system'
rxn30268_c.bounds = (0., 1000.)
rxn30268_c.add_metabolites({
    cpd29696_e: -1.0, # dipeptide
    cpd00002_c: -1.0, # ATP
    cpd00001_c: -1.0, # H2O
    cpd29696_c: 1.0, # dipeptide
    cpd00008_c: 1.0, # ADP
    cpd00009_c: 1.0, # Phosphate
    cpd00067_c: 1.0 # H+
})
rxn30268_c.annotation['sbo'] = 'SBO:0000176'
rxn30268_c.annotation['seed.reaction'] = 'rxn30268'
iCdG791.add_reactions([rxn30268_c])

# Hydrolysis
cpd00035_c = iCdG791.metabolites.cpd00035_c # Alanine
cpd00053_c = iCdG791.metabolites.cpd00053_c # Glutamine
rxn30241_c = cobra.Reaction('rxn30241_c')
rxn30241_c.gene_reaction_rule = '272563.8.peg.1365 or 272563.8.peg.2607 or 272563.8.peg.2736 or 272563.8.peg.2464'
rxn30241_c.name = 'Aminopeptidase'
rxn30241_c.bounds = (0., 1000.)
rxn30241_c.add_metabolites({
    cpd29696_c: -1.0, # dipeptide
    cpd00001_c: -1.0, # H2O
    cpd00035_c: 1.0, # peptide 1
    cpd00053_c: 1.0 # peptide 2
})
rxn30241_c.annotation['sbo'] = 'SBO:0000176'
rxn30241_c.annotation['seed.reaction'] = 'rxn30241'
iCdG791.add_reactions([rxn30241_c])

#------------------------------------------------------------------#

# L-ala-L-thr
cpd29699_e = cobra.Metabolite(
    'cpd29699_e',
    formula='C7H14N2O4',
    name='L-ala-L-thr',
    compartment='extracellular')
cpd29699_e.annotation['sbo'] = 'SBO:0000247'
cpd29699_e.annotation['seed.compound'] = 'cpd29699'
iCdG791.add_boundary(cpd29699_e, type='exchange', reaction_id='EX_cpd29699_e', lb=-1000.0, ub=1000.0)
cpd29699_c = cobra.Metabolite(
    'cpd29699_c',
    formula='C7H14N2O4',
    name='L-ala-L-thr',
    compartment='cytosol')
cpd29699_c.annotation['sbo'] = 'SBO:0000247'
cpd29699_c.annotation['seed.compound'] = 'cpd29699'

# Transport
rxn30270_c = cobra.Reaction('rxn30270_c')
rxn30270_c.gene_reaction_rule = '(272563.8.peg.896 or 272563.8.peg.2794 or 272563.8.peg.895) and 272563.8.peg.2798 and (272563.8.peg.894 or 272563.8.peg.2795) and (272563.8.peg.893 or 272563.8.peg.2796)'
rxn30270_c.name = 'Dipeptide transport via ABC system'
rxn30270_c.bounds = (0., 1000.)
rxn30270_c.add_metabolites({
    cpd29699_e: -1.0, # dipeptide
    cpd00002_c: -1.0, # ATP
    cpd00001_c: -1.0, # H2O
    cpd29699_c: 1.0, # dipeptide
    cpd00008_c: 1.0, # ADP
    cpd00009_c: 1.0, # Phosphate
    cpd00067_c: 1.0 # H+
})
rxn30270_c.annotation['sbo'] = 'SBO:0000176'
rxn30270_c.annotation['seed.reaction'] = 'rxn30270'
iCdG791.add_reactions([rxn30270_c])

# Hydrolysis
cpd00035_c = iCdG791.metabolites.cpd00035_c # Alanine
cpd00161_c = iCdG791.metabolites.cpd00161_c # Threonine
rxn30242_c = cobra.Reaction('rxn30242_c')
rxn30242_c.gene_reaction_rule = '272563.8.peg.1365 or 272563.8.peg.2607 or 272563.8.peg.2736 or 272563.8.peg.2464'
rxn30242_c.name = 'Aminopeptidase'
rxn30242_c.bounds = (0., 1000.)
rxn30242_c.add_metabolites({
    cpd29699_c: -1.0, # dipeptide
    cpd00001_c: -1.0, # H2O
    cpd00035_c: 1.0, # peptide 1
    cpd00161_c: 1.0 # peptide 2
})
rxn30242_c.annotation['sbo'] = 'SBO:0000176'
rxn30242_c.annotation['seed.reaction'] = 'rxn30242'
iCdG791.add_reactions([rxn30242_c])

#------------------------------------------------------------------#

# gly-L-met
cpd29693_e = cobra.Metabolite(
    'cpd29693_e',
    formula='C7H14N2O3S',
    name='gly-L-met',
    compartment='extracellular')
cpd29693_e.annotation['sbo'] = 'SBO:0000247'
cpd29693_e.annotation['seed.compound'] = 'cpd29693'
iCdG791.add_boundary(cpd29693_e, type='exchange', reaction_id='EX_cpd29693_e', lb=-1000.0, ub=1000.0)
cpd29693_c = cobra.Metabolite(
    'cpd29693_c',
    formula='C7H14N2O3S',
    name='gly-L-met',
    compartment='cytosol')
cpd29693_c.annotation['sbo'] = 'SBO:0000247'
cpd29693_c.annotation['seed.compound'] = 'cpd29693'

# Transport
rxn30274_c = cobra.Reaction('rxn30274_c')
rxn30274_c.gene_reaction_rule = '(272563.8.peg.896 or 272563.8.peg.2794 or 272563.8.peg.895) and 272563.8.peg.2798 and (272563.8.peg.894 or 272563.8.peg.2795) and (272563.8.peg.893 or 272563.8.peg.2796)'
rxn30274_c.name = 'Dipeptide transport via ABC system'
rxn30274_c.bounds = (0., 1000.)
rxn30274_c.add_metabolites({
    cpd29693_e: -1.0, # dipeptide
    cpd00002_c: -1.0, # ATP
    cpd00001_c: -1.0, # H2O
    cpd29693_c: 1.0, # dipeptide
    cpd00008_c: 1.0, # ADP
    cpd00009_c: 1.0, # Phosphate
    cpd00067_c: 1.0 # H+
})
rxn30274_c.annotation['sbo'] = 'SBO:0000176'
rxn30274_c.annotation['seed.reaction'] = 'rxn30274'
iCdG791.add_reactions([rxn30274_c])

# Hydrolysis
cpd00033_c = iCdG791.metabolites.cpd00033_c # Glycine
cpd00060_c = iCdG791.metabolites.cpd00060_c # Methionine
rxn30244_c = cobra.Reaction('rxn30244_c')
rxn30244_c.gene_reaction_rule = '272563.8.peg.1365 or 272563.8.peg.2607 or 272563.8.peg.2736 or 272563.8.peg.2464'
rxn30244_c.name = 'Aminopeptidase'
rxn30244_c.bounds = (0., 1000.)
rxn30244_c.add_metabolites({
    cpd29693_c: -1.0, # dipeptide
    cpd00001_c: -1.0, # H2O
    cpd00033_c: 1.0, # peptide 1
    cpd00060_c: 1.0 # peptide 2
})
rxn30244_c.annotation['sbo'] = 'SBO:0000176'
rxn30244_c.annotation['seed.reaction'] = 'rxn30244'
iCdG791.add_reactions([rxn30244_c])

#------------------------------------------------------------------#

# gly-L-asn
cpd29690_e = cobra.Metabolite(
    'cpd29690_e',
    formula='C6H11N3O4',
    name='gly-L-asn',
    compartment='extracellular')
cpd29690_e.annotation['sbo'] = 'SBO:0000247'
cpd29690_e.annotation['seed.compound'] = 'cpd29690'
iCdG791.add_boundary(cpd29690_e, type='exchange', reaction_id='EX_cpd29690_e', lb=-1000.0, ub=1000.0)
cpd29690_c = cobra.Metabolite(
    'cpd29690_c',
    formula='C6H11N3O4',
    name='gly-L-asn',
    compartment='cytosol')
cpd29690_c.annotation['sbo'] = 'SBO:0000247'
cpd29690_c.annotation['seed.compound'] = 'cpd29690'

# Transport
rxn30276_c = cobra.Reaction('rxn30276_c')
rxn30276_c.gene_reaction_rule = '(272563.8.peg.896 or 272563.8.peg.2794 or 272563.8.peg.895) and 272563.8.peg.2798 and (272563.8.peg.894 or 272563.8.peg.2795) and (272563.8.peg.893 or 272563.8.peg.2796)'
rxn30276_c.name = 'Dipeptide transport via ABC system'
rxn30276_c.bounds = (0., 1000.)
rxn30276_c.add_metabolites({
    cpd29690_e: -1.0, # dipeptide
    cpd00002_c: -1.0, # ATP
    cpd00001_c: -1.0, # H2O
    cpd29690_c: 1.0, # dipeptide
    cpd00008_c: 1.0, # ADP
    cpd00009_c: 1.0, # Phosphate
    cpd00067_c: 1.0 # H+
})
rxn30276_c.annotation['sbo'] = 'SBO:0000176'
rxn30276_c.annotation['seed.reaction'] = 'rxn30276'
iCdG791.add_reactions([rxn30276_c])

# Hydrolysis
cpd00033_c = iCdG791.metabolites.cpd00033_c # Glycine
cpd00132_c = iCdG791.metabolites.cpd00132_c # Asparagine

rxn30245_c = cobra.Reaction('rxn30245_c')
rxn30245_c.gene_reaction_rule = '272563.8.peg.1365 or 272563.8.peg.2607 or 272563.8.peg.2736 or 272563.8.peg.2464'
rxn30245_c.name = 'Aminopeptidase'
rxn30245_c.bounds = (0., 1000.)
rxn30245_c.add_metabolites({
    cpd29690_c: -1.0, # dipeptide
    cpd00001_c: -1.0, # H2O
    cpd00033_c: 1.0, # peptide 1
    cpd00132_c: 1.0 # peptide 2
})
rxn30245_c.annotation['sbo'] = 'SBO:0000176'
rxn30245_c.annotation['seed.reaction'] = 'rxn30245'
iCdG791.add_reactions([rxn30245_c])

#------------------------------------------------------------------#

# gly-L-gln
cpd29691_e = cobra.Metabolite(
    'cpd29691_e',
    formula='C7H13N3O4',
    name='gly-L-gln',
    compartment='extracellular')
cpd29691_e.annotation['sbo'] = 'SBO:0000247'
cpd29691_e.annotation['seed.compound'] = 'cpd29691'
iCdG791.add_boundary(cpd29691_e, type='exchange', reaction_id='EX_cpd29691_e', lb=-1000.0, ub=1000.0)
cpd29691_c = cobra.Metabolite(
    'cpd29691_c',
    formula='C7H13N3O4',
    name='gly-L-gln',
    compartment='cytosol')
cpd29691_c.annotation['sbo'] = 'SBO:0000247'
cpd29691_c.annotation['seed.compound'] = 'cpd29691'

# Transport
rxn30278_c = cobra.Reaction('rxn30278_c')
rxn30278_c.gene_reaction_rule = '(272563.8.peg.896 or 272563.8.peg.2794 or 272563.8.peg.895) and 272563.8.peg.2798 and (272563.8.peg.894 or 272563.8.peg.2795) and (272563.8.peg.893 or 272563.8.peg.2796)'
rxn30278_c.name = 'Dipeptide transport via ABC system'
rxn30278_c.bounds = (0., 1000.)
rxn30278_c.add_metabolites({
    cpd29691_e: -1.0, # dipeptide
    cpd00002_c: -1.0, # ATP
    cpd00001_c: -1.0, # H2O
    cpd29691_c: 1.0, # dipeptide
    cpd00008_c: 1.0, # ADP
    cpd00009_c: 1.0, # Phosphate
    cpd00067_c: 1.0 # H+
})
rxn30278_c.annotation['sbo'] = 'SBO:0000176'
rxn30278_c.annotation['seed.reaction'] = 'rxn30278'
iCdG791.add_reactions([rxn30278_c])

# Hydrolysis
cpd00033_c = iCdG791.metabolites.cpd00033_c # Glycine
cpd00053_c = iCdG791.metabolites.cpd00053_c # Glutamine
rxn30246_c = cobra.Reaction('rxn30246_c')
rxn30246_c.gene_reaction_rule = '272563.8.peg.1365 or 272563.8.peg.2607 or 272563.8.peg.2736 or 272563.8.peg.2464'
rxn30246_c.name = 'Aminopeptidase'
rxn30246_c.bounds = (0., 1000.)
rxn30246_c.add_metabolites({
    cpd29691_c: -1.0, # dipeptide
    cpd00001_c: -1.0, # H2O
    cpd00033_c: 1.0, # peptide 1
    cpd00053_c: 1.0 # peptide 2
})
rxn30246_c.annotation['sbo'] = 'SBO:0000176'
rxn30246_c.annotation['seed.reaction'] = 'rxn30246'
iCdG791.add_reactions([rxn30246_c])

#------------------------------------------------------------------#

# L-met-L-ala
cpd29700_e = cobra.Metabolite(
    'cpd29700_e',
    formula='C8H16N2O3S',
    name='L-met-L-ala',
    compartment='extracellular')
cpd29700_e.annotation['sbo'] = 'SBO:0000247'
cpd29700_e.annotation['seed.compound'] = 'cpd29700'
iCdG791.add_boundary(cpd29700_e, type='exchange', reaction_id='EX_cpd29700_e', lb=-1000.0, ub=1000.0)
cpd29700_c = cobra.Metabolite(
    'cpd29700_c',
    formula='C8H16N2O3S',
    name='L-met-L-ala',
    compartment='cytosol')
cpd29700_c.annotation['sbo'] = 'SBO:0000247'
cpd29700_c.annotation['seed.compound'] = 'cpd29700'

# Transport
rxn30280_c = cobra.Reaction('rxn30280_c')
rxn30280_c.gene_reaction_rule = '(272563.8.peg.896 or 272563.8.peg.2794 or 272563.8.peg.895) and 272563.8.peg.2798 and (272563.8.peg.894 or 272563.8.peg.2795) and (272563.8.peg.893 or 272563.8.peg.2796)'
rxn30280_c.name = 'Dipeptide transport via ABC system'
rxn30280_c.bounds = (0., 1000.)
rxn30280_c.add_metabolites({
    cpd29700_e: -1.0, # dipeptide
    cpd00002_c: -1.0, # ATP
    cpd00001_c: -1.0, # H2O
    cpd29700_c: 1.0, # dipeptide
    cpd00008_c: 1.0, # ADP
    cpd00009_c: 1.0, # Phosphate
    cpd00067_c: 1.0 # H+
})
rxn30280_c.annotation['sbo'] = 'SBO:0000176'
rxn30280_c.annotation['seed.reaction'] = 'rxn30280'
iCdG791.add_reactions([rxn30280_c])

# Hydrolysis
cpd00060_c = iCdG791.metabolites.cpd00060_c # Methionine
cpd00035_c = iCdG791.metabolites.cpd00035_c # Alanine
rxn30247_c = cobra.Reaction('rxn30247_c')
rxn30247_c.gene_reaction_rule = '272563.8.peg.1365 or 272563.8.peg.2607 or 272563.8.peg.2736 or 272563.8.peg.2464'
rxn30247_c.name = 'Aminopeptidase'
rxn30247_c.bounds = (0., 1000.)
rxn30247_c.add_metabolites({
    cpd29700_c: -1.0, # dipeptide
    cpd00001_c: -1.0, # H2O
    cpd00060_c: 1.0, # peptide 1
    cpd00035_c: 1.0 # peptide 2
})
rxn30247_c.annotation['sbo'] = 'SBO:0000176'
rxn30247_c.annotation['seed.reaction'] = 'rxn30247'
iCdG791.add_reactions([rxn30247_c])


In [16]:
# ATP synthase
iCdG791.reactions.rxn10042_c.remove_from_model(remove_orphans=False)

cpd00002_c = iCdG791.metabolites.cpd00002_c # ATP
cpd00001_c = iCdG791.metabolites.cpd00001_c # H2O
cpd00008_c = iCdG791.metabolites.cpd00008_c # ADP
cpd00009_c = iCdG791.metabolites.cpd00009_c # Phosphate
cpd00067_c = iCdG791.metabolites.cpd00067_c # H

cpd00067_p = cobra.Metabolite(
    'cpd00067_p',
    formula='H',
    name='H+',
    compartment='periplasm')
cpd00067_p.annotation['sbo'] = 'SBO:0000247'
cpd00067_p.annotation['seed.compound'] = 'cpd00067'

cpd00982_c = iCdG791.metabolites.cpd00982_c # FADH2
cpd00015_c = iCdG791.metabolites.cpd00015_c # FAD

rxn12212_c = cobra.Reaction('rxn12212_c')
rxn12212_c.gene_reaction_rule = '272563.120.peg.1189 and 272563.8.peg.1199 and 272563.120.peg.1185 and 272563.120.peg.1186 and 272563.120.peg.1188 and 272563.120.peg.1187'
rxn12212_c.name = 'FAD oxidation (electron transport complex)'
rxn12212_c.bounds = (-1000., 1000.)
rxn12212_c.add_metabolites({
    cpd00982_c: -1.0, # FADH2
    cpd00015_c: 1.0, # FAD
    cpd00067_p: 2.0 # H+
})
rxn12212_c.annotation['sbo'] = 'SBO:0000176'
rxn12212_c.annotation['seed.reaction'] = 'rxn12212'
iCdG791.add_reactions([rxn12212_c])

cpd00003_c = iCdG791.metabolites.cpd00003_c # NAD
cpd00004_c = iCdG791.metabolites.cpd00004_c # NADH

rxn12213_c = cobra.Reaction('rxn12213_c')
rxn12213_c.gene_reaction_rule = '272563.120.peg.1189 and 272563.8.peg.1199 and 272563.120.peg.1185 and 272563.120.peg.1186 and 272563.120.peg.1188 and 272563.120.peg.1187'
rxn12213_c.name = 'FAD oxidation (electron transport complex)'
rxn12213_c.bounds = (-1000., 1000.)
rxn12213_c.add_metabolites({
    cpd00004_c: -1.0, # FADH2
    cpd00003_c: 1.0, # FAD
    cpd00067_p: 2.0 # H+
})
rxn12213_c.annotation['sbo'] = 'SBO:0000176'
rxn12213_c.annotation['seed.reaction'] = 'rxn12213'

iCdG791.genes.get_by_id('272563.120.peg.1189').name = 'Ion-translocating oxidoreductase complex subunit A | rnfA'
iCdG791.genes.get_by_id('272563.120.peg.1189').annotation['sbo'] = 'SBO:0000243'
iCdG791.genes.get_by_id('272563.120.peg.1189').annotation['patric'] = 'fig|272563.120.peg.1189'

iCdG791.genes.get_by_id('272563.8.peg.1199').name = 'Ion-translocating oxidoreductase complex subunit B | rnfB'
iCdG791.genes.get_by_id('272563.8.peg.1199').annotation['sbo'] = 'SBO:0000243'
iCdG791.genes.get_by_id('272563.8.peg.1199').annotation['patric'] = 'fig|272563.8.peg.1199'

iCdG791.genes.get_by_id('272563.120.peg.1185').name = 'Ion-translocating oxidoreductase complex subunit C | rnfC'
iCdG791.genes.get_by_id('272563.120.peg.1185').annotation['sbo'] = 'SBO:0000243'
iCdG791.genes.get_by_id('272563.120.peg.1185').annotation['patric'] = 'fig|272563.120.peg.1185'

iCdG791.genes.get_by_id('272563.120.peg.1186').name = 'Ion-translocating oxidoreductase complex subunit D | rnfD'
iCdG791.genes.get_by_id('272563.120.peg.1186').annotation['sbo'] = 'SBO:0000243'
iCdG791.genes.get_by_id('272563.120.peg.1186').annotation['patric'] = 'fig|272563.120.peg.1186'

iCdG791.genes.get_by_id('272563.120.peg.1188').name = 'Ion-translocating oxidoreductase complex subunit E | rnfE'
iCdG791.genes.get_by_id('272563.120.peg.1188').annotation['sbo'] = 'SBO:0000243'
iCdG791.genes.get_by_id('272563.120.peg.1188').annotation['patric'] = 'fig|272563.120.peg.1188'

iCdG791.genes.get_by_id('272563.120.peg.1187').name = 'Ion-translocating oxidoreductase complex subunit G | rnfG'
iCdG791.genes.get_by_id('272563.120.peg.1187').annotation['sbo'] = 'SBO:0000243'
iCdG791.genes.get_by_id('272563.120.peg.1187').annotation['patric'] = 'fig|272563.120.peg.1187'

rxn10042_c = cobra.Reaction('rxn10042_c')
rxn10042_c.gene_reaction_rule = '272563.8.peg.3633 and 272563.8.peg.3632 and 272563.8.peg.3636 and 272563.8.peg.3635 and 272563.8.peg.3634 and (272563.8.peg.3640 or 272563.8.peg.3641)'
rxn10042_c.name = 'ATP synthase'
rxn10042_c.bounds = (0., 1000.)
rxn10042_c.add_metabolites({
    cpd00008_c: -1.0, # ADP
    cpd00009_c: -0.05, # Pi
    cpd00067_p: -1.0, # H+
    cpd00001_c: 1.0, # H20
    cpd00002_c: 1.0, # ATP
    cpd00067_c: 1.0 # H+
})
rxn10042_c.annotation['sbo'] = 'SBO:0000176'
rxn10042_c.annotation['seed.reaction'] = 'rxn10042'
iCdG791.add_reactions([rxn10042_c])


In [17]:
test = 0
for x in iCdG791.genes:
    if (len(x.reactions)) == 0:
        test += 1
print('Reamining genes: ' + str(test))

Reamining genes: 143


In [18]:
iCdG791.slim_optimize()

75.01730567597323

In [19]:
# PTS systems

# Salicin-specific
rxn05647_c = iCdG791.reactions.rxn05647_c
iCdG791.reactions.rxn05647_c.remove_from_model(remove_orphans=False)
rxn05647_c.lower_bound = 0.
rxn05647_c.gene_reaction_rule = '272563.8.peg.408 or 272563.8.peg.854 or 272563.8.peg.3282 or 272563.8.peg.3260 or 272563.8.peg.3241 or 272563.8.peg.3270'
iCdG791.add_reactions([rxn05647_c])

# Fructose-specific
rxn05560_c = iCdG791.reactions.rxn05560_c
iCdG791.reactions.rxn05560_c.remove_from_model(remove_orphans=False)
rxn05560_c.lower_bound = 0.
rxn05560_c.gene_reaction_rule = '(272563.8.peg.2383 or 272563.8.peg.224 or 272563.8.peg.3279) and 272563.8.peg.226 and 272563.8.peg.225'
iCdG791.add_reactions([rxn05560_c])
rxn10175_c = iCdG791.reactions.rxn10175_c
iCdG791.reactions.rxn10175_c.remove_from_model(remove_orphans=False)
rxn10175_c.lower_bound = 0.
rxn10175_c.gene_reaction_rule = '(272563.8.peg.2383 or 272563.8.peg.224 or 272563.8.peg.3279) and 272563.8.peg.226 and 272563.8.peg.225'
iCdG791.add_reactions([rxn10175_c])

# Mannitol-specific
rxn05617_c = iCdG791.reactions.rxn05617_c
iCdG791.reactions.rxn05617_c.remove_from_model(remove_orphans=False)
rxn05617_c.lower_bound = 0.
rxn05617_c.gene_reaction_rule = '(272563.8.peg.2449 or 272563.8.peg.3804) and 272563.8.peg.2450 and 272563.8.peg.2451'
iCdG791.add_reactions([rxn05617_c])

# Maltose-specific
rxn05607_c = iCdG791.reactions.rxn05607_c
iCdG791.reactions.rxn05607_c.remove_from_model(remove_orphans=False)
rxn05607_c.lower_bound = 0.
rxn05607_c.gene_reaction_rule = '272563.8.peg.3174 or 272563.8.peg.2632 or 272563.8.peg.1403'
iCdG791.add_reactions([rxn05607_c])

# Galactitol-specific
rxn05567_c = iCdG791.reactions.rxn05567_c
iCdG791.reactions.rxn05567_c.remove_from_model(remove_orphans=False)
rxn05567_c.lower_bound = 0.
rxn05567_c.gene_reaction_rule = '(272563.8.peg.50 or 272563.8.peg.2396) and (272563.8.peg.2395 or 272563.8.peg.1137 or 272563.8.peg.2443 or 272563.8.peg.51) and (272563.8.peg.52 or 272563.8.peg.2394 or 272563.8.peg.2442 or 272563.8.peg.2444)'
iCdG791.add_reactions([rxn05567_c])

# Sorbitol-specific
rxn10184_c = iCdG791.reactions.rxn10184_c
iCdG791.reactions.rxn10184_c.remove_from_model(remove_orphans=False)
rxn10184_c.lower_bound = 0.
rxn10184_c.gene_reaction_rule = '(272563.8.peg.804 or 272563.8.peg.2532) and (272563.8.peg.801 or 272563.8.peg.2536)'
iCdG791.add_reactions([rxn10184_c])

# Mannose-specific
rxn05610_c = iCdG791.reactions.rxn05610_c
iCdG791.reactions.rxn05610_c.remove_from_model(remove_orphans=False)
rxn05610_c.lower_bound = 0.
rxn05610_c.gene_reaction_rule = '(272563.8.peg.3431 or 272563.8.peg.304 or 272563.8.peg.515 or 272563.8.peg.1128) and (272563.8.peg.307 or 272563.8.peg.516 or 272563.8.peg.3432 or 272563.8.peg.1130 or 272563.8.peg.305) and (272563.8.peg.308 or 272563.8.peg.3430 or 272563.8.peg.517 or 272563.8.peg.1131) and (272563.8.peg.518 or 272563.8.peg.1132 or 272563.8.peg.309 or 272563.8.peg.3429)'
iCdG791.add_reactions([rxn05610_c])

# Glucose-specific
rxn05226_c = iCdG791.reactions.rxn05226_c
iCdG791.reactions.rxn05226_c.remove_from_model(remove_orphans=False)
rxn05226_c.lower_bound = 0.
rxn05226_c.gene_reaction_rule = '(272563.8.peg.3202 or 272563.8.peg.3247 or 272563.8.peg.3171 or 272563.8.peg.2634) and (272563.8.peg.2791 or 272563.8.peg.2790) and (272563.8.peg.3174 or 272563.8.peg.2632 or 272563.8.peg.1403)'
iCdG791.add_reactions([rxn05226_c])

# Cellobiose-specific
rxn05518_c = iCdG791.reactions.rxn05518_c
iCdG791.reactions.rxn05518_c.remove_from_model(remove_orphans=False)
rxn05518_c.lower_bound = 0.
rxn05518_c.gene_reaction_rule = '(272563.8.peg.3225 or 272563.8.peg.3012 or 272563.8.peg.153 or 272563.8.peg.901 or 272563.8.peg.3820 or 272563.8.peg.3192 or 272563.8.peg.3825) and (272563.8.peg.3224 or 272563.8.peg.3009 or 272563.8.peg.151 or 272563.8.peg.3822 or 272563.8.peg.902) and (272563.8.peg.3823 or 272563.8.peg.3226 or 272563.8.peg.152 or 272563.8.peg.3013 or 272563.8.peg.1679)'
iCdG791.add_reactions([rxn05518_c])


In [20]:
test = 0
for x in iCdG791.genes:
    if (len(x.reactions)) == 0:
        test += 1
print('Reamining genes: ' + str(test))

Reamining genes: 71


In [21]:
iCdG791.slim_optimize()

71.1778745591431

In [22]:
# Final reactions needed
cpd00320_c = cobra.Metabolite(
    'cpd00320_c',
    formula='C4H6NO4',
    name='D-Aspartate',
    compartment='cytosol')
cpd00320_c.annotation['sbo'] = 'SBO:0000247'
cpd00320_c.annotation['seed.compound'] = 'cpd00320'
cpd00320_e = cobra.Metabolite(
    'cpd00320_e',
    formula='C4H6NO4',
    name='D-Aspartate',
    compartment='extracellular')
cpd00320_e.annotation['sbo'] = 'SBO:0000247'
cpd00320_e.annotation['seed.compound'] = 'cpd00320'
cpd00002_c = iCdG791.metabolites.cpd00002_c # ATP
cpd00001_c = iCdG791.metabolites.cpd00001_c # H2O
cpd00008_c = iCdG791.metabolites.cpd00008_c # ADP
cpd00009_c = iCdG791.metabolites.cpd00009_c # Phosphate
cpd00067_c = iCdG791.metabolites.cpd00067_c # H
rxn05178_c = cobra.Reaction('rxn05178_c')
rxn05178_c.name = 'D-Aspartate-ABC transport'
rxn05178_c.bounds = (-1000., 1000.)
rxn05178_c.add_metabolites({
    cpd00320_e: -1.0, 
    cpd00002_c: -1.0, 
    cpd00001_c: -1.0, 
    cpd00008_c: 1.0, 
    cpd00009_c: 1.0, 
    cpd00067_c: 1.0, 
    cpd00320_c: 1.0 
})
rxn05178_c.annotation['sbo'] = 'SBO:0000176'
rxn05178_c.annotation['seed.reaction'] = 'rxn05178'
iCdG791.add_reactions([rxn05178_c])
cpd00041_c = iCdG791.metabolites.cpd00041_c
rxn00348_c = cobra.Reaction('rxn00348_c')
rxn00348_c.gene_reaction_rule = '272563.8.peg.2370 or 272563.8.peg.2133'
rxn00348_c.name = 'aspartate racemase'
rxn00348_c.bounds = (-1000., 1000.)
rxn00348_c.add_metabolites({
    cpd00041_c: -1.0, # L-aspartate
    cpd00320_c: 1.0 # D-aspartate
})
rxn00348_c.annotation['sbo'] = 'SBO:0000176'
rxn00348_c.annotation['seed.reaction'] = 'rxn00348'
iCdG791.add_reactions([rxn00348_c])
iCdG791.add_boundary(cpd00320_e, type='exchange', reaction_id='EX_cpd00320_e', lb=-1000.0, ub=1000.0)


cpd02711_e = cobra.Metabolite(
    'cpd02711_e',
    formula='C6H8O9P',
    name='2-keto-3-deoxygluconate',
    compartment='extracellular')
cpd02711_c = iCdG791.metabolites.cpd02711_c
cpd00067_e = iCdG791.metabolites.cpd00067_e
cpd00067_c = iCdG791.metabolites.cpd00067_c
rxn05681_c = cobra.Reaction('rxn05681_c')
rxn05681_c.gene_reaction_rule = '272563.8.peg.3148 or 272563.8.peg.2999'
rxn05681_c.name = '2-dehydro-3-deoxy-D-gluconate transport via proton symport, reversible'
rxn05681_c.bounds = (-1000., 1000.)
rxn05681_c.add_metabolites({
    cpd02711_e: -1.0,
    cpd00067_e: -1.0,
    cpd00067_c: 1.0,
    cpd02711_c: 1.0 
})
rxn05681_c.annotation['sbo'] = 'SBO:0000176'
rxn05681_c.annotation['seed.reaction'] = 'rxn05681'
iCdG791.add_reactions([rxn05681_c])
iCdG791.add_boundary(cpd02711_e, type='exchange', reaction_id='EX_cpd02711_e', lb=-1000.0, ub=1000.0)


cpd00025_c = iCdG791.metabolites.cpd00025_c
cpd00067_c = iCdG791.metabolites.cpd00067_c
cpd00001_c = iCdG791.metabolites.cpd00001_c
rxn00008_c = cobra.Reaction('rxn00008_c')
rxn00008_c.gene_reaction_rule = '272563.8.peg.1708'
rxn00008_c.name = 'Mn(II):hydrogen-peroxide oxidoreductase'
rxn00008_c.bounds = (0., 1000.)
rxn00008_c.add_metabolites({
    cpd00025_c: -1.0, # H202
    cpd00067_c: -2.0, # H+
    cpd00001_c: 2.0 # H20
})
rxn00008_c.annotation['sbo'] = 'SBO:0000176'
rxn00008_c.annotation['seed.reaction'] = 'rxn00008'
iCdG791.add_reactions([rxn00008_c])

cpd00038_c = iCdG791.metabolites.cpd00038_c # GTP
cpd00031_c = iCdG791.metabolites.cpd00031_c # GDP
cpd00009_c = iCdG791.metabolites.cpd00009_c # Pi
cpd00067_c = iCdG791.metabolites.cpd00067_c # H+
cpd00001_c = iCdG791.metabolites.cpd00001_c # H20
cpd10515_c = iCdG791.metabolites.cpd10515_c # Fe
cpd10515_e = iCdG791.metabolites.cpd10515_e # Fe
rxn34423_c = cobra.Reaction('rxn34423_c')
rxn34423_c.gene_reaction_rule = '272563.8.peg.3427 or 272563.8.peg.1551 or 272563.8.peg.1593'
rxn34423_c.name = 'ferrous iron uptake (GTP-dependent), plasma membrane'
rxn34423_c.bounds = (0., 1000.)
rxn34423_c.add_metabolites({
    cpd00001_c: -1.0,
    cpd00038_c: -1.0,
    cpd10515_e: -1.0,
    cpd00009_c: 1.0,
    cpd00031_c: 1.0,
    cpd00067_c: 1.0,
    cpd10515_c: 1.0
})
rxn34423_c.annotation['sbo'] = 'SBO:0000176'
rxn34423_c.annotation['seed.reaction'] = 'rxn34423'
iCdG791.add_reactions([rxn34423_c])


In [23]:
# Other miscellaneous genes
iCdG791.reactions.rxn05172_c.gene_reaction_rule = '(272563.8.peg.1554 or 272563.8.peg.3135) and (272563.8.peg.1556 or 272563.8.peg.2482) and (272563.8.peg.3134 or 272563.8.peg.1555 or 272563.8.peg.1052)'
iCdG791.reactions.rxn05298_c.gene_reaction_rule = '272563.8.peg.2286 or 272563.8.peg.2818 or 272563.8.peg.1830'
iCdG791.reactions.rxn00337_c.gene_reaction_rule = '272563.8.peg.1389 or 272563.8.peg.2161'
iCdG791.reactions.rxn05209_c.gene_reaction_rule = '272563.8.peg.2793 or 272563.8.peg.1732 or 272563.8.peg.473 or 272563.8.peg.2861'
iCdG791.reactions.rxn01522_c.gene_reaction_rule = '(272563.8.peg.2194 or 272563.8.peg.3328) and (272563.8.peg.2181 or 272563.8.peg.2187) and (272563.8.peg.2188 or 272563.8.peg.2195)'
iCdG791.reactions.rxn00414_c.gene_reaction_rule = '(272563.8.peg.3759 or 272563.8.peg.3761) and (272563.8.peg.3760 or 272563.8.peg.3762)'
iCdG791.reactions.rxn20606_c.gene_reaction_rule = '272563.120.peg.3426 and (272563.8.peg.3394 or 272563.8.peg.3392) and (272563.8.peg.3396 or 272563.8.peg.3393)'
iCdG791.reactions.rxn00785_c.gene_reaction_rule = '(272563.8.peg.3625 or 272563.8.peg.2439) and (272563.8.peg.2438 or 272563.8.peg.3624)'
iCdG791.reactions.rxn01200_c.gene_reaction_rule = '(272563.8.peg.3625 or 272563.8.peg.2439) and (272563.8.peg.2438 or 272563.8.peg.3624)'
iCdG791.reactions.rxn01301_c.gene_reaction_rule = '272563.8.peg.1224 or 272563.8.peg.2531 or 272563.8.peg.1655'
iCdG791.reactions.rxn00908_c.gene_reaction_rule = '272563.8.peg.240 and 272563.8.peg.766'
iCdG791.reactions.rxn01225_c.gene_reaction_rule = '272563.8.peg.3332 or 272563.8.peg.1740'
iCdG791.reactions.rxn03136_c.gene_reaction_rule = '272563.8.peg.513 or 272563.8.peg.237'
iCdG791.reactions.rxn13022_c.gene_reaction_rule = '272563.8.peg.1041 and 272563.8.peg.1042'
iCdG791.reactions.rxn03886_c.gene_reaction_rule = '272563.8.peg.805'
iCdG791.reactions.rxn00178_c.gene_reaction_rule = '272563.8.peg.1111 or 272563.8.peg.2800'
iCdG791.reactions.rxn08817_c.gene_reaction_rule = '272563.8.peg.1645'
iCdG791.reactions.rxn08849_c.gene_reaction_rule = '272563.8.peg.1645'
iCdG791.reactions.rxn08842_c.gene_reaction_rule = '272563.8.peg.1645'
iCdG791.reactions.rxn08819_c.gene_reaction_rule = '272563.8.peg.1645'
iCdG791.reactions.rxn08818_c.gene_reaction_rule = '272563.8.peg.1645'
iCdG791.reactions.rxn08840_c.gene_reaction_rule = '272563.8.peg.1645'
iCdG791.reactions.rxn08848_c.gene_reaction_rule = '272563.8.peg.1645'
iCdG791.reactions.rxn08851_c.gene_reaction_rule = '272563.8.peg.1645'
iCdG791.reactions.rxn08846_c.gene_reaction_rule = '272563.8.peg.1645'
iCdG791.reactions.rxn08820_c.gene_reaction_rule = '272563.8.peg.1645'
iCdG791.reactions.rxn08847_c.gene_reaction_rule = '272563.8.peg.1645'
iCdG791.reactions.rxn08850_c.gene_reaction_rule = '272563.8.peg.1645'
iCdG791.reactions.rxn08821_c.gene_reaction_rule = '272563.8.peg.1645'
iCdG791.reactions.rxn08843_c.gene_reaction_rule = '272563.8.peg.1645'
iCdG791.reactions.rxn08839_c.gene_reaction_rule = '272563.8.peg.1645'
iCdG791.reactions.rxn08844_c.gene_reaction_rule = '272563.8.peg.1645'
iCdG791.reactions.rxn08822_c.gene_reaction_rule = '272563.8.peg.1645'
iCdG791.reactions.rxn08845_c.gene_reaction_rule = '272563.8.peg.1645'
iCdG791.reactions.rxn08841_c.gene_reaction_rule = '272563.8.peg.1645'
iCdG791.reactions.rxn08838_c.gene_reaction_rule = '272563.8.peg.1645'
iCdG791.reactions.rxn08823_c.gene_reaction_rule = '272563.8.peg.1645'
iCdG791.reactions.rxn00693_c.gene_reaction_rule = '272563.8.peg.3768 or 272563.8.peg.1815'
iCdG791.reactions.rxn00143_c.gene_reaction_rule = '272563.8.peg.2697 or 272563.8.peg.2734'
iCdG791.reactions.rxn01519_c.gene_reaction_rule = '272563.8.peg.2521'
iCdG791.reactions.rxn00493_c.gene_reaction_rule = '272563.8.peg.2315 or 272563.8.peg.2624'
iCdG791.reactions.rxn02320_c.gene_reaction_rule = '272563.8.peg.2315 or 272563.8.peg.2624'
iCdG791.reactions.rxn00527_c.gene_reaction_rule = '272563.8.peg.2315 or 272563.8.peg.2624'
iCdG791.reactions.rxn01270_c.gene_reaction_rule = '272563.8.peg.2315 or 272563.8.peg.2624'
iCdG791.reactions.rxn02320_c.gene_reaction_rule = '272563.8.peg.1625'
iCdG791.reactions.rxn03167_c.gene_reaction_rule = '272563.8.peg.1521'
iCdG791.reactions.rxn00549_c.gene_reaction_rule = '272563.8.peg.1250'
iCdG791.reactions.rxn08040_c.gene_reaction_rule = '272563.8.peg.818'
iCdG791.reactions.rxn05555_c.gene_reaction_rule = '272563.8.peg.1726 and (272563.8.peg.1725 or 272563.8.peg.1724) and 272563.8.peg.1727'
iCdG791.reactions.rxn05183_c.gene_reaction_rule = '(272563.8.peg.1563 or 272563.8.peg.1730) and 272563.8.peg.1562'
iCdG791.reactions.rxn00146_c.gene_reaction_rule = '272563.120.peg.414 or 272563.8.peg.414'


In [24]:
# Glucose ABC transport

cpd00002_c = iCdG791.metabolites.cpd00002_c # ATP
cpd00001_c = iCdG791.metabolites.cpd00001_c # H2O
cpd00008_c = iCdG791.metabolites.cpd00008_c # ADP
cpd00009_c = iCdG791.metabolites.cpd00009_c # Phosphate
cpd00067_c = iCdG791.metabolites.cpd00067_c # H+
cpd00027_e = iCdG791.metabolites.cpd00027_e # Glucose
cpd00079_c = iCdG791.metabolites.cpd00079_c # Glucose 6-p

# Transport
rxn05147_c = cobra.Reaction('rxn05147_c')
rxn05147_c.gene_reaction_rule = '(272563.8.peg.3202 or 272563.8.peg.3247 or 272563.8.peg.3171 or 272563.8.peg.2634) and (272563.8.peg.2791 or 272563.8.peg.2790) and (272563.8.peg.3174 or 272563.8.peg.2632 or 272563.8.peg.1403)'
rxn05147_c.name = 'D-Glucose-ABC transport'
rxn05147_c.bounds = (0., 1000.)
rxn05147_c.add_metabolites({
    cpd00027_e: -1.0, # g
    cpd00002_c: -1.0, # ATP
    cpd00001_c: -1.0, # H2O
    cpd00079_c: 1.0, # g 6-p
    cpd00008_c: 1.0, # ADP
    cpd00009_c: 1.0, # Phosphate
    cpd00067_c: 1.0 # H+
})
rxn05147_c.annotation['sbo'] = 'SBO:0000176'
rxn05147_c.annotation['seed.reaction'] = 'rxn05147'
iCdG791.add_reactions([rxn05147_c])


In [25]:
test = 0
for x in iCdG791.genes:
    if (len(x.reactions)) == 0:
        test += 1
print('Reamining genes: ' + str(test))

Reamining genes: 0


In [26]:
iCdG791.slim_optimize()

97.22639015406979

In [27]:
# Rename GENRE
iCdG791.id = 'iCdG' + str(len(iCdG791.genes))

In [144]:
# Add additional annotation information

# BiGG
BiGG_rxn_ids = {}
with open('/home/mjenior/Desktop/repos/rxns_BiGG_ids.txt', 'r') as bigg_rxns:
    for line in bigg_rxns:
        line = line.split('=')
        modelseed = line[0]
        bigg = line[1].strip().split(',')
        if len(bigg) == 1: bigg = bigg[0]
        BiGG_rxn_ids[modelseed] = bigg
BiGG_cpd_ids = {}
with open('/home/mjenior/Desktop/repos/cpds_BiGG_ids.txt', 'r') as bigg_rxns:
    for line in bigg_rxns:
        line = line.split('=')
        modelseed = line[0]
        bigg = line[1].strip().split(',')
        if len(bigg) == 1: bigg = bigg[0]
        BiGG_cpd_ids[modelseed] = bigg

# KEGG
KEGG_rxn_ids = {}
with open('/home/mjenior/Desktop/repos/rxns_KEGG_ids.txt', 'r') as kegg_rxns:
    for line in kegg_rxns:
        line = line.split('=')
        modelseed = line[0]
        kegg = line[1].strip().split(',')
        if len(kegg) == 1: kegg = kegg[0]
        KEGG_rxn_ids[modelseed] = kegg
KEGG_cpd_ids = {}
with open('/home/mjenior/Desktop/repos/cpds_KEGG_ids.txt', 'r') as kegg_rxns:
    for line in kegg_rxns:
        line = line.split('=')
        modelseed = line[0]
        kegg = line[1].strip().split(',')
        if len(kegg) == 1: kegg = kegg[0]
        KEGG_cpd_ids[modelseed] = kegg


In [30]:
# Add annotations
bigg_rxn_success = 0
bigg_cpd_success = 0
kegg_rxn_success = 0
kegg_cpd_success = 0

for rxn in iCdG791.reactions:
    new_id = rxn.id.split('_')[0]
    try:
        bigg_id = BiGG_rxn_ids[new_id]
        iCdG791.reactions.get_by_id(rxn.id).annotation['bigg.metabolite'] = bigg_id
        bigg_rxn_success += 1
    except:
        pass
    try:
        kegg_id = KEGG_rxn_ids[new_id]
        iCdG791.reactions.get_by_id(rxn.id).annotation['kegg.compound'] = kegg_id
        kegg_rxn_success += 1
    except:
        pass

for cpd in iCdG791.metabolites:
    new_id = cpd.id.split('_')[0]
    try:
        bigg_id = BiGG_cpd_ids[new_id]
        iCdG791.metabolites.get_by_id(cpd.id).annotation['bigg.metabolite'] = bigg_id
        bigg_cpd_success += 1
    except:
        pass
    try:
        kegg_id = KEGG_cpd_ids[new_id]
        iCdG791.metabolites.get_by_id(cpd.id).annotation['kegg.compound'] = kegg_id
        kegg_cpd_success += 1
    except:
        pass
    

In [31]:
# Removed blocked, no-gene reactions
# Lysophospholipase L1 (periplasm)
lysophospholipase = ['rxn08807_c','rxn08796_c','rxn08802_c','rxn08814_c','rxn08801_c','rxn08798_c','rxn08804_c','rxn08812_c','rxn08803_c','rxn08813_c','rxn08806_c','rxn08810_c','rxn08805_c','rxn08811_c','rxn08800_c','rxn08816_c']
for rxn in lysophospholipase: iCdG791.reactions.get_by_id(rxn).remove_from_model(remove_orphans=True)

In [34]:
iCdG791

Name,iCdG706
Memory address,0x07f349aff0850
Number of metabolites,1182
Number of reactions,1205
Number of groups,0
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"cytosol, extracellular, periplasm"


In [35]:
# Save model
cobra.io.write_sbml_model(iCdG791, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/temp/iCdG706.sbml')
cobra.io.save_json_model(iCdG791, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/temp/iCdG706.json')

In [1]:
# Read in again for gap-filling
import cobra
iCdG706 = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/temp/iCdG706.sbml')

In [21]:
# Custom gap-filler
import copy
import cobra
import symengine

# pFBA gapfiller
def pfba_gapfill(model, universal, obj='biomass', demand='none', extracellular='extracellular'):
    '''
    Parameters
    ----------
    model_file : model
        Model to be gapfilled
    universal_file : model
        Reaction bag reference
    obj : str
        Name of objective reaction
    demand : str
        Demand metabolite ID, circumvents objective
    extracellular : str
        Label for extracellular compartment of model
    '''

    # Save some basic network info for downstream membership testing
    target_rxns = set([str(x.id) for x in model.reactions])
    target_cpds = set([str(y.id) for y in model.metabolites])
    ref_rxns = set([str(z.id) for z in universal.reactions])
    shared_rxns = ref_rxns.intersection(target_rxns)
        
    # Remove overlapping reactions from universal bag, add model reactions to universal bag
    temp_universal = copy.deepcopy(universal)
    for rxn in shared_rxns: temp_universal.reactions.get_by_id(rxn).remove_from_model()
    temp_universal.add_reactions(list(copy.deepcopy(model.reactions)))
    
    # Set objective
    if demand != 'none':
        cpd = temp_universal.metabolites.get_by_id(demand)
        demand = cobra.Reaction('demand')
        demand.bounds = (1., 1000.)
        demand.add_metabolites({cpd: -1.0})
        temp_universal.add_reactions([demand])
        temp_universal.objective = demand.id
    else:
        temp_universal.objective = obj
    obj_constraint = temp_universal.problem.Constraint(temp_universal.objective.expression, lb=10., ub=1000.)
    temp_universal.add_cons_vars([obj_constraint])
    temp_universal.solver.update()
    
    # Test to make sure objective flux is acheived with addition of reaction bag
    solution = temp_universal.optimize()
    if solution.objective_value <= 1e-6: 
        raise ValueError('ERROR: No objective flux possible with current reaction set.')    
    
    ## Set up pFBA objective
    pfba_expr = symengine.RealDouble(0)
    for rxn in temp_universal.reactions:
        if rxn.id in target_rxns:
            pfba_expr += 0. * rxn.forward_variable
            pfba_expr += 0. * rxn.reverse_variable
        else:
            pfba_expr += 1. * rxn.forward_variable
            pfba_expr += 1. * rxn.reverse_variable
    temp_universal.objective = temp_universal.problem.Objective(pfba_expr, direction='min', sloppy=True)
    temp_universal.solver.update()

    # Run FBA and save solution
    solution = temp_universal.optimize()
    active_rxns = set([rxn.id for rxn in temp_universal.reactions if abs(solution.fluxes[rxn.id]) > 1e-6])
    
    # Screen new reaction IDs
    new_rxns = active_rxns.difference(target_rxns)
    new_rxns = new_rxns.difference(set(['demand']))
    print('New reactions: ' + str(len(new_rxns)))
    print('New objective value: ' + str(solution.objective_value))
    
    return new_rxns
    

In [7]:
# Read in curated universal reacton bag and annotate
universal_GramPos = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/GramPos.sbml')
universal_GramPos.reactions.rxn13782_c.remove_from_model(remove_orphans=True)
universal_GramPos.reactions.rxn13783_c.remove_from_model(remove_orphans=True)
universal_GramPos.reactions.rxn13784_c.remove_from_model(remove_orphans=True)
#universal_GramPos.reactions.rxn11920_c.name = 'Poly-beta-hydroxybutyrate (PHB) synthease'
universal_GramPos.reactions.rxn12188_c.name = '2-deoxyglucose-6-phosphatase'

for cpd in universal_GramPos.metabolites:
    cpd.annotation['sbo'] = 'SBO:0000247'
    cpd.annotation['seed.compound'] = cpd.id.split('_')[0]    
    cpd.compartment = cpd.compartment.lower()

for rxn in universal_GramPos.reactions:
    rxn.annotation['sbo'] = 'SBO:0000176'
    rxn.annotation['seed.reaction'] = rxn.id.split('_')[0]

'GramPositive.modeltemplate' is not a valid SBML 'SId'.
No objective coefficients in model. Unclear what should be optimized


In [40]:
mdm = ['EX_cpd00001_e', # water
       'EX_cpd00065_e', # L-Tryptophan
       'EX_cpd00060_e', # L-Methionine
       'EX_cpd00322_e', # L-Isoleucine
       'EX_cpd00129_e', # L-Proline
       'EX_cpd00156_e', # L-Valine
       'EX_cpd00107_e', # L-Leucine
       'EX_cpd00084_e', # L-Cysteine 
       'EX_cpd00149_e', # Cobalt
       'EX_cpd00099_e', # Chloride
       'EX_cpd10515_e', # Iron
       'EX_cpd00030_e', # Manganese
       'EX_cpd00254_e', # Magnesium
       'EX_cpd00063_e', # Calcium
       'EX_cpd00205_e', # Potassium
       'EX_cpd00009_e', # Phosphate
       'EX_cpd00971_e', # Sodium
       'EX_cpd00242_e', # Carbonate
       'EX_cpd00104_e', # Biotin
       'EX_cpd00644_e', # Pantothenate
       'EX_cpd00263_e', # Pyridoxine
       'EX_cpd00027_e'] # D-Glucose (Carbohydrate C-source)

# Theriot et al. (2013). Nature Communications.
ncmm = ['EX_cpd00001_e', # water
        'EX_cpd00104_e', # Biotin
        'EX_cpd00644_e', # Pantothenate
        'EX_cpd00263_e', # Pyridoxine
        'EX_cpd00149_e', # Cobalt
        'EX_cpd00099_e', # Chloride
        'EX_cpd10515_e', # Iron
        'EX_cpd00030_e', # Manganese
        'EX_cpd00254_e', # Magnesium
        'EX_cpd00063_e', # Calcium
        'EX_cpd00205_e', # Potassium
        'EX_cpd00009_e', # Phosphate
        'EX_cpd00971_e', # Sodium
        'EX_cpd00242_e', # Carbonate
        'EX_cpd00322_e', # L-Isoleucine
        'EX_cpd00129_e', # L-Proline
        'EX_cpd00156_e', # L-Valine
        'EX_cpd00107_e', # L-Leucine
        'EX_cpd00084_e', # L-Cysteine 
        'EX_cpd00065_e', # L-Tryptophan
        'EX_cpd00060_e', # L-Methionine
        'EX_cpd00119_e', # L-Histidine
        'EX_cpd00033_e', # Glycine
        'EX_cpd00051_e', # L-Arginine
        'EX_cpd00161_e'] # L-Threonine


In [8]:
# Change media and gapfill - MDM
for rxn in iCdG706.boundary:
    if not rxn.id in mdm:
        rxn.bounds = (0., 1000.)
    else:
        rxn.bounds = (-1000., 1000.)
test = iCdG706.slim_optimize()
print('Pre-gapfill objective value: ' + str(test))

gapfilled_rxns = pfba_gapfill(iCdG706, universal_GramPos) # Biomass alone
gapfilled_rxns1 = pfba_gapfill(iCdG706, universal_GramPos, demand='cpd17043_c') # RNA polymer
gapfilled_rxns2 = pfba_gapfill(iCdG706, universal_GramPos, demand='cpd17041_c') # Protein polymer
gapfilled_rxns3 = pfba_gapfill(iCdG706, universal_GramPos, demand='cpd12894_c') # Teichoic acid
gapfilled_rxns4 = pfba_gapfill(iCdG706, universal_GramPos, demand='cpd16661_c') # Peptidoglycan polymer
gapfilled_rxns5 = pfba_gapfill(iCdG706, universal_GramPos, demand='cellwall_c') # Cell Wall polymer
gapfilled_rxns6 = pfba_gapfill(iCdG706, universal_GramPos, demand='cpd11852_c') # Lipid Pool
gapfilled_rxns7 = pfba_gapfill(iCdG706, universal_GramPos, demand='cofactor_c') # Cofactor Pool
gapfilled_rxns8 = pfba_gapfill(iCdG706, universal_GramPos, demand='cpd17042_c') # DNA polymer

mdm_gapfilled_rxns = gapfilled_rxns1.union(gapfilled_rxns2).union(gapfilled_rxns3).union(gapfilled_rxns4).union(gapfilled_rxns5).union(gapfilled_rxns6).union(gapfilled_rxns7).union(gapfilled_rxns8).union(gapfilled_rxns)
print('Total reactions to be added: ' + str(len(mdm_gapfilled_rxns)))

rxns_to_be_added = []
for rxn in mdm_gapfilled_rxns: 
    rxns_to_be_added.append(copy.deepcopy(universal_GramPos.reactions.get_by_id(rxn)))
iCdG706.add_reactions(rxns_to_be_added)
test = iCdG706.slim_optimize()
print('Post-gapfill objective value: ' + str(test))


New reactions: 1
New objective value: 0.0025000000001482686


In [36]:
# Change media and gapfill - NCMM
for rxn in iCdG706.boundary:
    if not rxn.id in ncmm:
        rxn.bounds = (0., 1000.)
    else:
        rxn.bounds = (-1000., 1000.)
test = iCdG706.slim_optimize()
print('Pre-gapfill objective value: ' + str(test))

gapfilled_rxns = pfba_gapfill(iCdG706, universal_GramPos) # Biomass alone
gapfilled_rxns1 = pfba_gapfill(iCdG706, universal_GramPos, demand='cpd17043_c') # RNA polymer
gapfilled_rxns2 = pfba_gapfill(iCdG706, universal_GramPos, demand='cpd17041_c') # Protein polymer
gapfilled_rxns3 = pfba_gapfill(iCdG706, universal_GramPos, demand='cpd12894_c') # Teichoic acid
gapfilled_rxns4 = pfba_gapfill(iCdG706, universal_GramPos, demand='cpd16661_c') # Peptidoglycan polymer
gapfilled_rxns5 = pfba_gapfill(iCdG706, universal_GramPos, demand='cellwall_c') # Cell Wall polymer
gapfilled_rxns6 = pfba_gapfill(iCdG706, universal_GramPos, demand='cpd11852_c') # Lipid Pool
gapfilled_rxns7 = pfba_gapfill(iCdG706, universal_GramPos, demand='cofactor_c') # Cofactor Pool
gapfilled_rxns8 = pfba_gapfill(iCdG706, universal_GramPos, demand='cpd17042_c') # DNA polymer

ncmm_gapfilled_rxns = gapfilled_rxns1.union(gapfilled_rxns2).union(gapfilled_rxns3).union(gapfilled_rxns4).union(gapfilled_rxns5).union(gapfilled_rxns6).union(gapfilled_rxns7).union(gapfilled_rxns8).union(gapfilled_rxns)
print('Total reactions to be added: ' + str(len(ncmm_gapfilled_rxns)))

rxns_to_be_added = []
for rxn in ncmm_gapfilled_rxns: 
    rxns_to_be_added.append(copy.deepcopy(universal_GramPos.reactions.get_by_id(rxn)))
iCdG706.add_reactions(rxns_to_be_added)
test = iCdG706.slim_optimize()
print('Post-gapfill objective value: ' + str(test))


Pre-gapfill objective value: 12.559386622123775
New reactions: 0
New objective value: 3.9164191226586654e-14
New reactions: 0
New objective value: 10.000000000000194
New reactions: 0
New objective value: 9.999999999999883
New reactions: 0
New objective value: 10.000000000000401
New reactions: 2
New objective value: 61.93957115009697
New reactions: 4
New objective value: 385.581871345032
New reactions: 0
New objective value: 10.0
New reactions: 0
New objective value: 10.0
New reactions: 0
New objective value: 10.000000000000016
Total reactions to be added: 5
Post-gapfill objective value: 44.875570765519235


In [75]:
# Correct names and possible gene annotations
all_gapfilled_rxns = ncmm_gapfilled_rxns.union(mdm_gapfilled_rxns)

iCdG706.reactions.rxn09655_c.name = 'Trehalose transport via proton symport'
iCdG706.reactions.rxn09658_c.name = 'Sucrose transport via proton symport'
iCdG706.reactions.rxn10183_c.name = 'N-acetyl-D-mannosamine transport via PTS'

iCdG706.reactions.EX_cpd00208_e.name = 'Lactose exchange'

gene = cobra.Gene('272563.120.peg.1564')
gene.annotation['patric'] = 'fig|272563.120.peg.1564'
gene.annotation['sbo'] = 'SBO:0000243'
iCdG706.reactions.rxn00504_c.gene_reaction_rule = '272563.120.peg.1564'
iCdG706.genes.get_by_id('272563.120.peg.1564').name = 'Pyrroline-5-carboxylate reductase | proC'

iCdG706.reactions.rxn09655_c.gene_reaction_rule = '(272563.8.peg.2383 or 272563.8.peg.224 or 272563.8.peg.3279) and 272563.8.peg.226 and 272563.8.peg.225'
iCdG706.reactions.rxn09658_c.gene_reaction_rule = '(272563.8.peg.2383 or 272563.8.peg.224 or 272563.8.peg.3279) and 272563.8.peg.226 and 272563.8.peg.225'
iCdG706.reactions.rxn10183_c.gene_reaction_rule = '(272563.8.peg.3202 or 272563.8.peg.3247 or 272563.8.peg.3171 or 272563.8.peg.2634) and (272563.8.peg.2790 or 272563.8.peg.2791)'


In [78]:
# Rename GENRE
iCdG706.id = 'iCdG' + str(len(iCdG706.genes))

In [79]:
for rxn in iCdG706.boundary:
    if not rxn.id in mdm:
        rxn.bounds = (0., 1000.)
    else:
        rxn.bounds = (-1000., 1000.)

In [80]:
iCdG706

Name,iCdG707
Memory address,0x07f40ef778210
Number of metabolites,1184
Number of reactions,1214
Number of groups,0
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"cytosol, extracellular, periplasm"


In [84]:
# Add annotations
bigg_rxn_success = 0
bigg_cpd_success = 0
kegg_rxn_success = 0
kegg_cpd_success = 0

for rxn in iCdG706.reactions:
    new_id = rxn.id.split('_')[0]
    try:
        bigg_id = BiGG_rxn_ids[new_id]
        iCdG706.reactions.get_by_id(rxn.id).annotation['bigg.metabolite'] = bigg_id
        bigg_rxn_success += 1
    except:
        pass
    try:
        kegg_id = KEGG_rxn_ids[new_id]
        iCdG706.reactions.get_by_id(rxn.id).annotation['kegg.compound'] = kegg_id
        kegg_rxn_success += 1
    except:
        pass

for cpd in iCdG706.metabolites:
    new_id = cpd.id.split('_')[0]
    try:
        bigg_id = BiGG_cpd_ids[new_id]
        iCdG706.metabolites.get_by_id(cpd.id).annotation['bigg.metabolite'] = bigg_id
        bigg_cpd_success += 1
    except:
        pass
    try:
        kegg_id = KEGG_cpd_ids[new_id]
        iCdG706.metabolites.get_by_id(cpd.id).annotation['kegg.compound'] = kegg_id
        kegg_cpd_success += 1
    except:
        pass

In [73]:
# Label tRNA reactions as Biomass
iCdG706.reactions.rxn06300_c.annotation['sbo'] = 'SBO:0000629'
iCdG706.reactions.rxn06432_c.annotation['sbo'] = 'SBO:0000629'
iCdG706.reactions.rxn06434_c.annotation['sbo'] = 'SBO:0000629'
iCdG706.reactions.rxn06936_c.annotation['sbo'] = 'SBO:0000629'
iCdG706.reactions.rxn06435_c.annotation['sbo'] = 'SBO:0000629'
iCdG706.reactions.rxn06437_c.annotation['sbo'] = 'SBO:0000629'
iCdG706.reactions.rxn06436_c.annotation['sbo'] = 'SBO:0000629'
iCdG706.reactions.rxn06438_c.annotation['sbo'] = 'SBO:0000629'
iCdG706.reactions.rxn06439_c.annotation['sbo'] = 'SBO:0000629'
iCdG706.reactions.rxn06440_c.annotation['sbo'] = 'SBO:0000629'
iCdG706.reactions.rxn06441_c.annotation['sbo'] = 'SBO:0000629'
iCdG706.reactions.rxn06442_c.annotation['sbo'] = 'SBO:0000629'
iCdG706.reactions.rxn06443_c.annotation['sbo'] = 'SBO:0000629'
iCdG706.reactions.rxn06444_c.annotation['sbo'] = 'SBO:0000629'
iCdG706.reactions.rxn06445_c.annotation['sbo'] = 'SBO:0000629'
iCdG706.reactions.rxn06446_c.annotation['sbo'] = 'SBO:0000629'
iCdG706.reactions.rxn06447_c.annotation['sbo'] = 'SBO:0000629'
iCdG706.reactions.rxn06448_c.annotation['sbo'] = 'SBO:0000629'
iCdG706.reactions.rxn06280_c.annotation['sbo'] = 'SBO:0000629'
iCdG706.reactions.rxn06449_c.annotation['sbo'] = 'SBO:0000629'


In [75]:
# Save model
cobra.io.write_sbml_model(iCdG706, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/temp/iCdG707.sbml')
cobra.io.save_json_model(iCdG706, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/temp/iCdG707.json')

In [111]:
import cobra
iCdG707 = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG707.sbml')
for rxn in iCdG707.boundary: rxn.bounds = (-1000., 1000.)

In [2]:
from cobra.flux_analysis import parsimonious

# Function to calculate doubling time from objective value
def doublingTime(model):
    with model as m:
        if m.slim_optimize(error_value=0.) < 1e-6:
            print('GENRE has no objective flux')
        else:
            growth = (1. / float(m.slim_optimize())) * 3600.
            print(str(round(growth, 2)) + ' minutes doubling time')


# Identifies blocked reactions, 1% cutoff for fraction of optimum
def blockedReactions(model):
    
    with model as m:
        blocked = cobra.flux_analysis.variability.find_blocked_reactions(m)
        nogene_blocked = []
        for rxn in blocked:
            if m.reactions.get_by_id(rxn).gene_reaction_rule == '':
                nogene_blocked.append(rxn)

    #print(str(len(blocked)) + ' total reactions are blocked')
    fraction = (float(len(blocked)) / float(len(model.reactions))) * 100.
    fraction = round(fraction, 2)
    print(str(fraction) + '% reactions are blocked')
    
    return blocked


# Identify potentially gapfilled reactions, checks against pFBA solution
def missingGPR(model, exclude=['biomass']):
    
    noGene = []        
    for rxn in model.reactions:
        if len(list(rxn.genes)) == 0:
            if rxn.id not in exclude:
                if len(list(rxn.reactants)) == 0 or len(list(rxn.products)) == 0:
                    exclude.append(rxn.id)
                    continue
                else:
                    noGene.append(rxn.id)
    
    solution = parsimonious.pfba(model)
    active_rxns = set([rxn.id for rxn in model.reactions if abs(solution.fluxes[rxn.id]) > 1e-5])
    active_rxns = active_rxns.difference(set(exclude))
    noGene_active = set(noGene).intersection(active_rxns)

    fraction = float(len(model.reactions)) - float(len(exclude))
    fraction = (float(len(noGene)) / fraction) * 100.
    fraction = round(fraction, 2)
    print(str(fraction) + '% reactions without GPRs')
    
    fraction = (float(len(noGene_active)) / float(len(active_rxns))) * 100.
    fraction = round(fraction, 2)
    print(str(fraction) + '% of reactions used in pFBA solution have no GPR')
    
    return noGene_active


# Checks which cytosolic metabolites are generated for free (bacteria only)
def checkFreeMass(model, cytosol='cytosol'):

    free = []
    with model as m:
    
        # Close all exchanges
        for rxn in m.boundary: m.reactions.get_by_id(rxn.id).lower_bound = 0.
    
        # Create demand for each reaction and optimize individually
        reset_rxn = m.reactions[0].id
        for cpd in m.metabolites: 
            if cpd.compartment == cytosol:
                demand = cobra.Reaction('demand')
                demand.bounds = (0., 1000.)
                demand.add_metabolites({cpd: -1.0})
                m.add_reactions([demand])
                m.objective = demand
                obj_val = m.slim_optimize()
                if obj_val > 1e-8: free.append(cpd.id)
                m.objective = reset_rxn
                m.remove_reactions([demand])
    
    fraction = (float(len(free)) / float(len(model.metabolites))) * 100.
    fraction = round(fraction, 2)
    print(str(fraction) + '% metabolites are generated for free')

    return(free)


# Check for mass and charge balance in reactions
def checkBalance(model, exclude=['biomass']):
    
    with model as m:

        elements = set()
        for cpd in m.metabolites:
            try:
                elements |= set(cpd.elements.keys())
            except:
                pass
        
        massImbal = []
        failed = 0
        if len(elements) == 0:
            print('No elemental data associated with metabolites!')
            failed = 1
        else:
            for rxn in m.reactions:
                if rxn in m.boundary or rxn.id in exclude: 
                    continue
                elif rxn.annotation['sbo'] == 'SBO:0000629':
                    continue

                try:
                    test = rxn.check_mass_balance()
                except ValueError:
                    continue

                if len(list(test)) > 0:
                    if len(set(test.keys()).intersection(elements)) > 0: massImbal.append(rxn.id)
                        
    if failed != 1:
        fraction = (float(len(massImbal)) / float(len(model.reactions))) * 100.
        fraction = round(fraction, 2)
        print(str(fraction) + '% reactions are mass imbalanced')
        
    return massImbal


def basicCheck(model):
    
    # Determination
    if len(model.reactions) < len(model.metabolites): 
        print('GENRE is overdetermined')
    elif len(model.reactions) > len(model.metabolites):
        print('GENRE is underdetermined')
    else:
        pass
    
    # Compartments
    print('GENRE has ' + str(len(model.compartments.keys())) + ' compartment(s)')
    
    # Genes
    if len(model.genes) == 0: 
        print('GENRE has no gene data')
    else:
        print('GENRE has ' + str(len(model.genes)) + ' genes')
          
    # Growth
    doublingTime(model)

# Quicker way to read in models
import pickle
def read_model(fileName, obj='none'):
    
    fileType = fileName.split('.')[-1]
    
    if fileType == 'sbml' or fileType == 'xml':
        model = cobra.io.read_sbml_model(fileName)
    elif fileType == 'json':
        model = cobra.io.load_json_model(fileName)
    elif fileType == 'yaml':
        model = cobra.io.load_yaml_model(fileName)
    elif fileType == 'mat':
        model = cobra.io.load_matlab_model(fileName)
    elif fileType == 'pkl':
        model = pickle.load(open(fileName, 'rb'))
    else:
        raise TypeError('Unrecognized file extension')
    
    if obj != 'none': model.objective = obj
    for rxn in model.boundary: rxn.bounds = (-1000., 1000.)
        
    return model

In [36]:
basicCheck(iCdG707)
iCdG707_noGPRblocked = blockedReactions(iCdG707)
iCdG707_free = checkFreeMass(iCdG707)
iCdG707_massImbal = checkBalance(iCdG707, exclude=['biomass'])
iCdG707_nogene = missingGPR(iCdG707, exclude=['biomass'])

GENRE is underdetermined
GENRE has 3 compartment(s)
GENRE has 707 genes
22.3 minutes doubling time
37.31% reactions are blocked
0.49% reactions are mass imbalanced
13.57% reactions without GPRs
24.82% of reactions used in pFBA solution have no GPR


In [166]:
# Correct mass imbalances

cpd00001_c = iCdG707.metabolites.cpd00001_c # H2O
cpd29694_c = iCdG707.metabolites.cpd29694_c # L-ala-gly
cpd00033_c = iCdG707.metabolites.cpd00033_c # Glycine
cpd00035_c = iCdG707.metabolites.cpd00035_c # L-Alanine
cpd00067_c = iCdG707.metabolites.cpd00067_c # H+

iCdG707.reactions.rxn30238_c.remove_from_model()

rxn30238_c = cobra.Reaction('rxn30238_c') 
rxn30238_c.name = 'Aminopeptidase (ala-gly)'
rxn30238_c.gene_reaction_rule = '272563.8.peg.1365 or 272563.8.peg.2607 or 272563.8.peg.2736 or 272563.8.peg.2464'
rxn30238_c.lower_bound = 0.
rxn30238_c.upper_bound = 1000.
rxn30238_c.add_metabolites({
    cpd00001_c: -2.0,
    cpd29694_c: -1.0,
    cpd00035_c: 1.0,
    cpd00033_c: 1.0,
    cpd00067_c: 5.0
})
iCdG707.add_reactions([rxn30238_c])

#----------------------------------------------------------------------#

cpd00982_c = iCdG707.metabolites.cpd00982_c # FADH2
cpd00015_c = iCdG707.metabolites.cpd00015_c # FAD
cpd00067_p = iCdG707.metabolites.cpd00067_p # H+

iCdG707.reactions.rxn12212_c.remove_from_model()

rxn12212_c = cobra.Reaction('rxn12212_c') 
rxn12212_c.name = 'FAD oxidation (electron transport complex)'
rxn12212_c.gene_reaction_rule = '272563.120.peg.1189 and 272563.8.peg.1199 and 272563.120.peg.1185 and 272563.120.peg.1186 and 272563.120.peg.1188 and 272563.120.peg.1187'
rxn12212_c.lower_bound = -1000.
rxn12212_c.upper_bound = 1000.
rxn12212_c.add_metabolites({
    cpd00982_c: -1.0,
    cpd00015_c: 1.0,
    cpd00067_p: 3.0
})
iCdG707.add_reactions([rxn12212_c])

#----------------------------------------------------------------------#

cpd00106_c = iCdG707.metabolites.cpd00106_c # Fumarate
cpd00791_c = iCdG707.metabolites.cpd00791_c # ProtoporphyrinogenIX
cpd00036_c = iCdG707.metabolites.cpd00036_c # Succinate
cpd01476_c = iCdG707.metabolites.cpd01476_c # Protoporphyrin

iCdG707.reactions.rxn09180_c.remove_from_model()

rxn09180_c = cobra.Reaction('rxn09180_c') 
rxn09180_c.name = 'protoporphyrinogen oxidase (anaerobic)'
rxn09180_c.lower_bound = -1000.
rxn09180_c.upper_bound = 1000.
rxn09180_c.add_metabolites({
    cpd00067_p: -6.0,
    cpd00106_c: -3.0,
    cpd00791_c: -1.0,
    cpd00036_c: 3.0,
    cpd01476_c: 1.0
})
iCdG707.add_reactions([rxn09180_c])

#----------------------------------------------------------------------#

cpd00008_c = iCdG707.metabolites.cpd00008_c # ADP
cpd00009_c = iCdG707.metabolites.cpd00009_c # Pi
cpd00002_c = iCdG707.metabolites.cpd00002_c # ATP

iCdG707.reactions.rxn10042_c.remove_from_model()

rxn10042_c = cobra.Reaction('rxn10042_c') 
rxn10042_c.name = 'ATP synthase'
rxn10042_c.gene_reaction_rule = '272563.8.peg.3633 and 272563.8.peg.3632 and 272563.8.peg.3636 and 272563.8.peg.3635 and 272563.8.peg.3634 and ( 272563.8.peg.3640 or 272563.8.peg.3641 )'
rxn10042_c.lower_bound = 0.
rxn10042_c.upper_bound = 1000.
rxn10042_c.add_metabolites({
    cpd00008_c: -1.0,
    cpd00009_c: -1.0,
    cpd00067_p: -1.0,
    cpd00001_c: 1.0,
    cpd00002_c: 1.0
})
iCdG707.add_reactions([rxn10042_c])

#----------------------------------------------------------------------#

cpd00027_e = iCdG707.metabolites.cpd00027_e # Glucose
cpd00079_c = iCdG707.metabolites.cpd00079_c # D-glucose-6-phosphate

iCdG707.reactions.rxn05147_c.remove_from_model()

rxn05147_c = cobra.Reaction('rxn05147_c') 
rxn05147_c.name = 'D-Glucose-ABC transport'
rxn12212_c.gene_reaction_rule = '272563.8.peg.3633 and 272563.8.peg.3632 and 272563.8.peg.3636 and 272563.8.peg.3635 and 272563.8.peg.3634 and ( 272563.8.peg.3640 or 272563.8.peg.3641 )'
rxn05147_c.lower_bound = 0.
rxn05147_c.upper_bound = 1000.
rxn05147_c.add_metabolites({
    cpd00002_c: -1.0,
    cpd00027_e: -1.0,
    cpd00008_c: 1.0,
    cpd00067_c: 1.0,
    cpd00079_c: 1.0
})
iCdG707.add_reactions([rxn05147_c])

#----------------------------------------------------------------------#

iCdG707.metabolites.cpd29698_c.formula = 'C9H18N2O3'
iCdG707.metabolites.cpd29698_e.formula = 'C9H18N2O3'


In [ ]:
# Unblock Lactose catabolism

cpd00208_c = iCdG707.metabolites.cpd00208_c # Lactose
cpd00001_c = iCdG707.metabolites.cpd00001_c # H2O
cpd00027_c = iCdG707.metabolites.cpd00027_c # Glucose

cpd00108_c = cobra.Metabolite(
    'cpd00108_c',
    formula='C6H12O6',
    name='Galactose',
    compartment='cytosol')
cpd00108_c.annotation['sbo'] = 'SBO:0000247'
cpd00108_c.annotation['seed.compound'] = 'cpd00108'
cpd00108_c.annotation['bigg.metabolite'] = 'gal'
cpd00108_c.annotation['kegg.compound'] = 'C00124'

rxn00816_c = cobra.Reaction('rxn00816_c')
rxn00816_c.name = 'Beta-galactosidase'
rxn00816_c.lower_bound = 0.
rxn00816_c.upper_bound = 1000.
rxn00816_c.add_metabolites({
    cpd00208_c: -1.0,
    cpd00001_c: -1.0,
    cpd00027_c: 1.0,
    cpd00108_c: 1.0
})
rxn00816_c.annotation['sbo'] = ''
rxn00816_c.annotation['seed.reaction'] = 'rxn00816'
rxn00816_c.annotation['kegg.reaction'] = 'R01100'
rxn00816_c.annotation['bigg.reaction'] = 'LACZ'
iCdG707.add_reactions([rxn00816_c])

cpd00002_c = iCdG707.metabolites.cpd00002_c # ATP
cpd00008_c = iCdG707.metabolites.cpd00008_c # ADP
cpd00067_c = iCdG707.metabolites.cpd00067_c # H
cpd00348_c = iCdG707.metabolites.cpd00348_c # galactose-1-phosphate 

rxn00808_c = cobra.Reaction('rxn00808_c')
rxn00808_c.name = 'ATP:D-galactose 1-phosphotransferase'
rxn00808_c.lower_bound = 0.
rxn00808_c.upper_bound = 1000.
rxn00808_c.add_metabolites({
    cpd00002_c: -1.0,
    cpd00108_c: -1.0,
    cpd00008_c: 1.0,
    cpd00067_c: 1.0,
    cpd00348_c: 1.0
})
rxn00808_c.annotation['sbo'] = ''
rxn00808_c.annotation['seed.reaction'] = 'rxn00808'
rxn00808_c.annotation['kegg.reaction'] = 'R01092'
rxn00808_c.annotation['bigg.reaction'] = 'GAL1'
iCdG707.add_reactions([rxn00808_c])



In [ ]:
# Unblock Salicin catabolism

cpd01553_c = iCdG707.metabolites.cpd01553_c # Saligenin

cpd01553_e = cobra.Metabolite(
    'cpd01553_e',
    formula='C7H8O2',
    name='Saligenin',
    compartment='extracellular')
cpd01553_e.annotation['sbo'] = 'SBO:0000247'
cpd01553_e.annotation['seed.compound'] = 'cpd01553'
cpd01553_e.annotation['bigg.metabolite'] = '2hymeph'
cpd01553_e.annotation['kegg.compound'] = 'C02323'
iCdG707.add_boundary(cpd01553_e, type='exchange', reaction_id='EX_cpd01553_e', lb=-1000.0, ub=1000.0)
iCdG707.reactions.EX_cpd01553_e.name = 'Saligenin exchange'
iCdG707.reactions.EX_cpd01553_e.annotation['sbo'] = 'SBO:0000627'

cpd00067_c = iCdG707.metabolites.cpd00067_c # H
cpd00067_e = iCdG707.metabolites.cpd00067_e # H

rxn05472_c = cobra.Reaction('rxn05472_c')
rxn05472_c.name = '2-(hydroxymethyl)phenol transport in/out via proton symport'
rxn05472_c.lower_bound = -1000.
rxn05472_c.upper_bound = 1000.
rxn05472_c.add_metabolites({
    cpd00067_e: -1.0,
    cpd01553_e: -1.0,
    cpd00067_c: 1.0,
    cpd01553_c: 1.0
})
rxn05472_c.annotation['sbo'] = 'SBO:0000185'
rxn05472_c.annotation['seed.reaction'] = 'rxn05472'
rxn05472_c.annotation['bigg.reaction'] = '2HXMPt6'
iCdG707.add_reactions([rxn05472_c])


In [ ]:
# Correct Melezitose catabolism
remove_gene = [iCdG707.genes.get_by_id('1151372.4.peg.1723')]
cobra.manipulation.delete.remove_genes(iCdG707, remove_gene)

cpd00001_c = iCdG707.metabolites.cpd00001_c # H2O
cpd05161_c = iCdG707.metabolites.cpd05161_c # Melezitose
cpd00027_c = iCdG707.metabolites.cpd00027_c # Glucose
cpd20885_c = iCdG707.metabolites.cpd20885_c # Turanose

R01103_c = cobra.Reaction('R01103_c')
R01103_c.name = 'Melezitose galactohydrolase'
R01103_c.lower_bound = 0.
R01103_c.upper_bound = 1000.
R01103_c.add_metabolites({
    cpd00001_c: -1.0,
    cpd05161_c: -1.0,
    cpd00027_c: 1.0,
    cpd20885_c: 1.0
})
R01103_c.annotation['kegg.reaction'] = 'R01103'
R01103_c.annotation['sbo'] = 'SBO:0000176'
iCdG707.add_reactions([R01103_c])

iCdG707.id = 'iCdG' + str(len(iCdG707.genes))

In [141]:
mdm = ['EX_cpd00001_e', # water
       'EX_cpd00065_e', # L-Tryptophan
       'EX_cpd00060_e', # L-Methionine
       'EX_cpd00322_e', # L-Isoleucine
       'EX_cpd00129_e', # L-Proline
       'EX_cpd00156_e', # L-Valine
       'EX_cpd00107_e', # L-Leucine
       'EX_cpd00084_e', # L-Cysteine 
       'EX_cpd00149_e', # Cobalt
       'EX_cpd00099_e', # Chloride
       'EX_cpd10515_e', # Iron
       'EX_cpd00030_e', # Manganese
       'EX_cpd00254_e', # Magnesium
       'EX_cpd00063_e', # Calcium
       'EX_cpd00205_e', # Potassium
       'EX_cpd00009_e', # Phosphate
       'EX_cpd00971_e', # Sodium
       'EX_cpd00242_e', # Carbonate
       'EX_cpd00104_e', # Biotin
       'EX_cpd00644_e', # Pantothenate
       'EX_cpd00263_e', # Pyridoxine
       'EX_cpd00027_e'] # D-Glucose (Carbohydrate C-source)

for rxn in iCdG707.boundary:
    if not rxn.id in mdm:
        rxn.bounds = (0., 1000.)
    else:
        rxn.bounds = (-1000., 1000.)

In [167]:
# Add annotations
bigg_rxn_success = 0
bigg_cpd_success = 0
kegg_rxn_success = 0
kegg_cpd_success = 0

for rxn in iCdG707.reactions:
    new_id = rxn.id.split('_')[0]
    try:
        bigg_id = BiGG_rxn_ids[new_id]
        iCdG707.reactions.get_by_id(rxn.id).annotation['bigg.metabolite'] = bigg_id
        bigg_rxn_success += 1
    except:
        pass
    try:
        kegg_id = KEGG_rxn_ids[new_id]
        iCdG707.reactions.get_by_id(rxn.id).annotation['kegg.compound'] = kegg_id
        kegg_rxn_success += 1
    except:
        pass

for cpd in iCdG707.metabolites:
    new_id = cpd.id.split('_')[0]
    try:
        bigg_id = BiGG_cpd_ids[new_id]
        iCdG707.metabolites.get_by_id(cpd.id).annotation['bigg.metabolite'] = bigg_id
        bigg_cpd_success += 1
    except:
        pass
    try:
        kegg_id = KEGG_cpd_ids[new_id]
        iCdG707.metabolites.get_by_id(cpd.id).annotation['kegg.compound'] = kegg_id
        kegg_cpd_success += 1
    except:
        pass

In [169]:
iCdG707.slim_optimize()

70.05819375319561

In [168]:
uniprot_gene_ids = {}
ec_gene_ids = {}
refseq_gene_ids = {}
kegg_gene_ids = {}
patric_genes = set()

with open('/home/mjenior/Desktop/repos/uniprot_Cd630.tab', 'r') as gene_annotation:
    header = gene_annotation.readline()
    for line in gene_annotation:
        line = line.split()
        uniprot_gene_ids[line[4]] = line[0]
        ec_gene_ids[line[4]] = line[1]
        refseq_gene_ids[line[4]] = line[2]
        kegg_gene_ids[line[4]] = line[3]
        patric_genes |= set([line[4]])

success = 0
for gene in iCdG707.genes:
    if not gene.id in patric_genes: continue
    
    success += 1
    gene.annotation['uniprot'] = uniprot_gene_ids[gene.id]
    gene.annotation['kegg.genes'] = kegg_gene_ids[gene.id]
    
    if refseq_gene_ids[gene.id] != 'none':
        gene.annotation['refseq'] = refseq_gene_ids[gene.id]
    
    if ec_gene_ids[gene.id] != 'none':
        gene.annotation['ec_number'] = ec_gene_ids[gene.id]
    

In [7]:
# Final annotation fixes

# no reaction annotation 
iCdG707.reactions.rxn12212_c.annotation['seed.reaction'] = 'rxn12212'

# no seed annotation
iCdG707.metabolites.cpd02711_e.annotation['seed.compound'] = 'cpd02711'

# No kegg reaction
for rxn in ["R03018_c","R01103_c","R01174_1_c","R01214_c","R01174_2_c","R01174_3_c","R01174_4_c","R01174_5_c","K20025_c","R11076_c","R06782_c","R11462_c"]:
    iCdG707.reactions.get_by_id(rxn).annotation['kegg.reaction'] = rxn.split('_')[0]

# no seed annotation
for rxn in ["rxn30238_c","rxn12212_c","rxn09180_c","rxn10042_c","rxn05147_c"]:
    iCdG707.reactions.get_by_id(rxn).annotation['seed.reaction'] = rxn.split('_')[0]

# SBO
iCdG707.metabolites.cpd02711_e.annotation['sbo'] = 'SBO:0000247'
for rxn in ["rxn30238_c","rxn12212_c","rxn09180_c","rxn10042_c","rxn05147_c"]:
    iCdG707.reactions.get_by_id(rxn).annotation['sbo'] = 'SBO:0000629'
iCdG707.genes.get_by_id('272563.120.peg.1564').annotation['sbo'] = 'SBO:0000243'
for rxn in ["rxn30270_c","rxn05546_c","rxn09658_c","rxn09189_c","rxn30274_c","rxn05534_c","rxn10183_c","rxn09655_c","rxn12848_c","rxn05536_c","rxn30276_c","rxn05538_c","rxn05147_c","rxn12849_c","rxn30278_c","rxn05541_c","rxn12850_c","rxn30280_c","rxn05545_c","rxn12851_c","rxn30264_c","rxn05178_c","rxn30268_c","rxn30259_c","rxn05681_c","rxn30266_c","rxn30262_c","rxn34423_c"]:
    iCdG707.reactions.get_by_id(rxn).annotation['sbo'] = 'SBO:0000185'

In [8]:
# Save model
cobra.io.write_sbml_model(iCdG707, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG706.sbml')
cobra.io.save_json_model(iCdG707, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG706.json')

In [4]:
import cobra
from cobra.flux_analysis import parsimonious

# Function to calculate doubling time from objective value
def doublingTime(model):
    with model as m:
        if m.slim_optimize(error_value=0.) < 1e-6:
            print('GENRE has no objective flux')
        else:
            growth = (1. / float(m.slim_optimize())) * 3600.
            print(str(round(growth, 2)) + ' minutes doubling time')


# Identifies blocked reactions, 1% cutoff for fraction of optimum
def blockedReactions(model):
    
    with model as m:
        blocked = cobra.flux_analysis.variability.find_blocked_reactions(m)
        nogene_blocked = []
        for rxn in blocked:
            if m.reactions.get_by_id(rxn).gene_reaction_rule == '':
                nogene_blocked.append(rxn)

    #print(str(len(blocked)) + ' total reactions are blocked')
    fraction = (float(len(blocked)) / float(len(model.reactions))) * 100.
    fraction = round(fraction, 2)
    print(str(fraction) + '% reactions are blocked')
    
    return blocked


# Identify potentially gapfilled reactions, checks against pFBA solution
def missingGPR(model):
    
    noGene = []
    exclude = []
    for rxn in model.reactions:
        if len(list(rxn.genes)) == 0:
            if rxn.annotation['sbo'] != 'SBO:0000629':
                if rxn in model.boundary:
                    exclude.append(rxn.id)
                    continue
                else:
                    noGene.append(rxn.id)
    
    solution = parsimonious.pfba(model)
    active_rxns = set([rxn.id for rxn in model.reactions if abs(solution.fluxes[rxn.id]) > 1e-5])
    active_rxns = active_rxns.difference(set(exclude))
    noGene_active = set(noGene).intersection(active_rxns)

    fraction = float(len(model.reactions)) - float(len(exclude))
    fraction = (float(len(noGene)) / fraction) * 100.
    fraction = round(fraction, 2)
    print(str(fraction) + '% reactions without GPRs')
    
    fraction = (float(len(noGene_active)) / float(len(active_rxns))) * 100.
    fraction = round(fraction, 2)
    print(str(fraction) + '% of reactions used in pFBA solution have no GPR')
    
    return noGene_active


# Checks which cytosolic metabolites are generated for free (bacteria only)
def checkFreeMass(model, cytosol='cytosol'):

    free = []
    with model as m:
    
        # Close all exchanges
        for rxn in m.boundary: m.reactions.get_by_id(rxn.id).lower_bound = 0.
    
        # Create demand for each reaction and optimize individually
        reset_rxn = m.reactions[0].id
        for cpd in m.metabolites: 
            if cpd.compartment == cytosol:
                demand = cobra.Reaction('demand')
                demand.bounds = (0., 1000.)
                demand.add_metabolites({cpd: -1.0})
                m.add_reactions([demand])
                m.objective = demand
                obj_val = m.slim_optimize()
                if obj_val > 1e-8: free.append(cpd.id)
                m.objective = reset_rxn
                m.remove_reactions([demand])
    
    fraction = (float(len(free)) / float(len(model.metabolites))) * 100.
    fraction = round(fraction, 2)
    print(str(fraction) + '% metabolites are generated for free')

    return(free)


# Check for mass and charge balance in reactions
def checkBalance(model):
    
    with model as m:

        elements = set()
        for cpd in m.metabolites:
            try:
                elements |= set(cpd.elements.keys())
            except:
                pass
        
        massImbal = []
        failed = 0
        if len(elements) == 0:
            print('No elemental data associated with metabolites!')
            failed = 1
        else:
            for rxn in m.reactions:
                if rxn.annotation['sbo'] == 'SBO:0000629': 
                    continue
                elif rxn in m.boundary:
                    continue

                try:
                    test = rxn.check_mass_balance()
                except ValueError:
                    continue

                if len(list(test)) > 0:
                    if len(set(test.keys()).intersection(elements)) > 0: massImbal.append(rxn.id)
                        
    if failed != 1:
        fraction = (float(len(massImbal)) / float(len(model.reactions))) * 100.
        fraction = round(fraction, 2)
        print(str(fraction) + '% reactions are mass imbalanced')
        
    return massImbal


def basicCheck(model):
    
    # Determination
    if len(model.reactions) < len(model.metabolites): 
        print('GENRE is overdetermined')
    elif len(model.reactions) > len(model.metabolites):
        print('GENRE is underdetermined')
    else:
        pass
    
    # Compartments
    print('GENRE has ' + str(len(model.compartments.keys())) + ' compartment(s)')
    
    # Genes
    if len(model.genes) == 0: 
        print('GENRE has no gene data')
    else:
        print('GENRE has ' + str(len(model.genes)) + ' genes')
    no_rxns = []
    for gene in model.genes:
        if len(gene.reactions) == 0:
            no_rxns.append(gene.id)
    if len(no_rxns) > 0:
          print('GENRE has ' + str(len(no_rxns)) + ' genes that are not associated with reactions')
            
    # Growth
    doublingTime(model)
    
    return no_rxns

# Quicker way to read in models
import pickle
def read_model(fileName, obj='none'):
    
    fileType = fileName.split('.')[-1]
    
    if fileType == 'sbml' or fileType == 'xml':
        model = cobra.io.read_sbml_model(fileName)
    elif fileType == 'json':
        model = cobra.io.load_json_model(fileName)
    elif fileType == 'yaml':
        model = cobra.io.load_yaml_model(fileName)
    elif fileType == 'mat':
        model = cobra.io.load_matlab_model(fileName)
    elif fileType == 'pkl':
        model = pickle.load(open(fileName, 'rb'))
    else:
        raise TypeError('Unrecognized file extension')
    
    if obj != 'none': model.objective = obj
    for rxn in model.boundary: rxn.bounds = (-1000., 1000.)
        
    return model


def save_model(model, fileName, media='none'):
    
    if media != 'none':
        media_exchanges = ['EX_' + x for x in media]
        for rxn in model.boundary:
            if not rxn.id in media_exchanges:
                rxn.bounds = (0., 1000.)
            else:
                rxn.bounds = (-1000., 1000.)
    else:
        for rxn in model.boundary:
            rxn.bounds = (-1000., 1000.)
    
    sbml_fileName = fileName + '.sbml'
    cobra.io.write_sbml_model(model, sbml_fileName)
    
    json_fileName = fileName + '.json'
    cobra.io.save_json_model(model, json_fileName)
    
mdm = ['cpd00001_e', # water
       'cpd00065_e', # L-Tryptophan
       'cpd00060_e', # L-Methionine
       'cpd00322_e', # L-Isoleucine
       'cpd00129_e', # L-Proline
       'cpd00156_e', # L-Valine
       'cpd00107_e', # L-Leucine
       'cpd00084_e', # L-Cysteine 
       'cpd00149_e', # Cobalt
       'cpd00099_e', # Chloride
       'cpd10515_e', # Iron
       'cpd00030_e', # Manganese
       'cpd00254_e', # Magnesium
       'cpd00063_e', # Calcium
       'cpd00205_e', # Potassium
       'cpd00009_e', # Phosphate
       'cpd00971_e', # Sodium
       'cpd00242_e', # Carbonate
       'cpd00104_e', # Biotin
       'cpd00644_e', # Pantothenate
       'cpd00263_e', # Pyridoxine
       'cpd00027_e'] # D-Glucose (Carbohydrate C-source)


In [ ]:

iCdG706 = read_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG706.sbml')

# Neu5Ac --> N-Acetyl-Neuraminic acid
iCdG706.metabolites.cpd00232_c.name = 'N-Acetyl-Neuraminic acid'

# D-Tagatose
cpd00589_e = cobra.Metabolite(
    'cpd00589_e',
    formula='C6H12O6',
    name='D-Tagatose',
    compartment='extracellular')
cpd00589_e.annotation['sbo'] = 'SBO:0000247'
cpd00589_e.annotation['seed.compound'] = 'cpd00589'
cpd00589_e.annotation['bigg.metabolite'] = 'tag__D'
cpd00589_e.annotation['kegg.compound'] = 'C00795'
iCdG706.add_boundary(cpd00589_e, type='exchange', reaction_id='EX_cpd00589_e', lb=-1000.0, ub=1000.0)
iCdG706.reactions.EX_cpd01553_e.name = 'Tagatose exchange'
iCdG706.reactions.EX_cpd01553_e.annotation['sbo'] = 'SBO:0000627'

cpd00805_c = iCdG706.metabolites.cpd00805_c # D-Tagatose 6-phosphate
cpd00002_c = iCdG706.metabolites.cpd00002_c # ATP
cpd00008_c = iCdG706.metabolites.cpd00008_c # ADP
cpd00067_c = iCdG706.metabolites.cpd00067_c # H

rxn02094_c = cobra.Reaction('rxn02094_c')
rxn02094_c.name = 'ATP:D-tagatose 6-phosphotransferase'
rxn02094_c.gene_reaction_rule = '( 645463.3.peg.3332 or 645463.3.peg.303 or 645463.3.peg.464 or 645463.3.peg.998 ) and ( 645463.3.peg.306 or 645463.3.peg.465 or 645463.3.peg.3333 or 645463.3.peg.1000 or 645463.3.peg.304 ) and ( 645463.3.peg.307 or 645463.3.peg.3331 or 645463.3.peg.466 or 645463.3.peg.1001 ) and ( 645463.3.peg.467 or 645463.3.peg.1002 or 645463.3.peg.308 or 645463.3.peg.3330 )'
rxn02094_c.lower_bound = 0.
rxn02094_c.upper_bound = 1000.
rxn02094_c.add_metabolites({
    cpd00589_e: -1.0,
    cpd00002_c: -1.0,
    cpd00008_c: 1.0,
    cpd00067_c: 1.0,
    cpd00805_c: 1.0
})
rxn02094_c.annotation['sbo'] = 'SBO:0000185' # transport
rxn02094_c.annotation['seed.reaction'] = 'rxn02094'
rxn02094_c.annotation['kegg.reaction'] = 'R02927'
rxn02094_c.annotation['bigg.reaction'] = 'TAGptspp'
iCdG706.add_reactions([rxn02094_c])


# N-Acetyl-D-Galactosamine
cpd27607_e = cobra.Metabolite(
    'cpd27607_e',
    formula='C8H15NO6',
    name='N-Acetyl-D-Galactosamine',
    compartment='extracellular')
cpd27607_e.annotation['sbo'] = 'SBO:0000247'
cpd27607_e.annotation['seed.compound'] = 'cpd27607'
cpd27607_e.annotation['bigg.metabolite'] = 'acgal'
cpd27607_e.annotation['kegg.compound'] = 'C01074'

iCdG706.add_boundary(cpd27607_e, type='exchange', reaction_id='EX_cpd27607_e', lb=-1000.0, ub=1000.0)
iCdG706.reactions.EX_cpd27607_e.name = 'N-Acetyl-D-Galactosamine exchange'
iCdG706.reactions.EX_cpd27607_e.annotation['sbo'] = 'SBO:0000627'

cpd15390_c = cobra.Metabolite(
    'cpd15390_c',
    formula='C8H14NO9P',
    name='N-Acetyl-D-Galactosamine-1-phosphate',
    compartment='cytosol')
cpd15390_c.annotation['sbo'] = 'SBO:0000247'
cpd15390_c.annotation['seed.compound'] = 'cpd15390'
cpd15390_c.annotation['bigg.metabolite'] = 'acgal1p'
cpd15390_c.annotation['kegg.compound'] = 'C18060'

cpd00002_c = iCdG706.metabolites.cpd00002_c # ATP
cpd00008_c = iCdG706.metabolites.cpd00008_c # ADP
cpd00067_c = iCdG706.metabolites.cpd00067_c # H

rxn46837_c = cobra.Reaction('rxn46837_c')
rxn46837_c.name = 'ATP:N-acetyl-D-galactosamine 1-phosphotransferase'
rxn46837_c.gene_reaction_rule = '645463.3.peg.2802 and 645463.3.peg.3106 and 645463.3.peg.3106'
rxn46837_c.lower_bound = 0.
rxn46837_c.upper_bound = 1000.
rxn46837_c.add_metabolites({
    cpd00002_c: -1.0,
    cpd27607_e: -1.0,
    cpd00008_c: 1.0,
    cpd00067_c: 1.0,
    cpd15390_c: 1.0
})
rxn46837_c.annotation['sbo'] = 'SBO:0000185' # transport
rxn46837_c.annotation['seed.reaction'] = 'rxn46837'
rxn46837_c.annotation['kegg.reaction'] = 'R08967'
rxn46837_c.annotation['bigg.reaction'] = 'ACGALK'
iCdG706.add_reactions([rxn46837_c])

cpd00062_c = iCdG706.metabolites.cpd00062_c # UTP
cpd00012_c = iCdG706.metabolites.cpd00012_c # Phosphate
cpd00175_c = iCdG706.metabolites.cpd00175_c # UDP-N-acetyl-D-galactosamine

rxn17713_c = cobra.Reaction('rxn17713_c')
rxn17713_c.name = 'UTP-monosaccharide-1-phosphate uridylyltransferase'
rxn17713_c.lower_bound = 0.
rxn17713_c.upper_bound = 1000.
rxn17713_c.add_metabolites({
    cpd00062_c: -1.0,
    cpd15390_c: -1.0,
    cpd00012_c: 1.0,
    cpd00175_c: 1.0
})
rxn17713_c.annotation['sbo'] = 'SBO:0000176' # metabolic
rxn17713_c.annotation['seed.reaction'] = 'rxn17713'
rxn17713_c.annotation['kegg.reaction'] = 'R10183'
rxn17713_c.annotation['bigg.reaction'] = 'GALT'
iCdG706.add_reactions([rxn17713_c])

save_model(iCdG706, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG706', media=mdm)


In [2]:
iCdG706 = read_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG706.sbml')

remove_genes = basicCheck(iCdG706)
iCdG706_noGPRblocked = blockedReactions(iCdG706)
iCdG706_free = checkFreeMass(iCdG706)
iCdG706_massImbal = checkBalance(iCdG706)
iCdG706_nogene = missingGPR(iCdG706)

GENRE is underdetermined
GENRE has 3 compartment(s)
GENRE has 725 genes
GENRE has 6 genes that are not associated with reactions
20.68 minutes doubling time
36.39% reactions are blocked
0.0% metabolites are generated for free
0.0% reactions are mass imbalanced
13.39% reactions without GPRs
22.26% of reactions used in pFBA solution have no GPR


In [3]:
remove_genes = [iCdG706.genes.get_by_id(x) for x in remove_genes]
cobra.manipulation.delete.remove_genes(iCdG706, remove_genes)

In [4]:
iCdG706.id = 'iCdG' + str(len(iCdG706.genes))

In [5]:
iCdG706

Name,iCdG719
Memory address,0x07f3fb459f810
Number of metabolites,1189
Number of reactions,1223
Number of groups,0
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"cytosol, extracellular, periplasm"


In [6]:
save_model(iCdG706, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG719', media=mdm)

### Matching Biolog Results

In [206]:
iCdG719 = read_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG719.sbml')

In [207]:
import copy

# Read in curated universal reacton bag and annotate
universal_GramPos = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/GramPos.sbml')
universal_GramPos.reactions.rxn13782_c.remove_from_model(remove_orphans=True)
universal_GramPos.reactions.rxn13783_c.remove_from_model(remove_orphans=True)
universal_GramPos.reactions.rxn13784_c.remove_from_model(remove_orphans=True)
#universal_GramPos.reactions.rxn11920_c.name = 'Poly-beta-hydroxybutyrate (PHB) synthease'
universal_GramPos.reactions.rxn12188_c.name = '2-deoxyglucose-6-phosphatase'

for cpd in universal_GramPos.metabolites:
    cpd.annotation['sbo'] = 'SBO:0000247'
    cpd.annotation['seed.compound'] = cpd.id.split('_')[0]    
    cpd.compartment = cpd.compartment.lower()

for rxn in universal_GramPos.reactions:
    rxn.annotation['sbo'] = 'SBO:0000176'
    rxn.annotation['seed.reaction'] = rxn.id.split('_')[0]

'GramPositive.modeltemplate' is not a valid SBML 'SId'.
No objective coefficients in model. Unclear what should be optimized


In [208]:
# Ala-ala
cpd00002_c = iCdG719.metabolites.cpd00002_c # ATP
cpd00001_c = iCdG719.metabolites.cpd00001_c # H2O
cpd00008_c = iCdG719.metabolites.cpd00008_c # ADP
cpd00009_c = iCdG719.metabolites.cpd00009_c # Phosphate
cpd00067_c = iCdG719.metabolites.cpd00067_c # H+
cpd00731_c = iCdG719.metabolites.cpd00731_c # Ala-ala
cpd00731_e = cobra.Metabolite(
    'cpd00731_e',
    formula='C6H12N2O3',
    name='Ala-ala',
    compartment='extracellular')
cpd00731_e.annotation['sbo'] = 'SBO:0000247'
cpd00731_e.annotation['seed.compound'] = 'cpd00731'
cpd00731_e.annotation['bigg.metabolite'] = ['alaala', 'diala-L']
cpd00731_e.annotation['kegg.compound'] = 'C00993'

# Transport
rxn18658_c = cobra.Reaction('rxn18658_c')
rxn18658_c.gene_reaction_rule = '(272563.8.peg.896 or 272563.8.peg.2794 or 272563.8.peg.895) and 272563.8.peg.2798 and (272563.8.peg.894 or 272563.8.peg.2795) and (272563.8.peg.893 or 272563.8.peg.2796)'
rxn18658_c.name = 'Dipeptide transport via ABC system'
rxn18658_c.bounds = (0., 1000.)
rxn18658_c.add_metabolites({
    cpd00731_e: -1.0, # dipeptide
    cpd00002_c: -1.0, # ATP
    cpd00001_c: -1.0, # H2O
    cpd00731_c: 1.0, # dipeptide
    cpd00008_c: 1.0, # ADP
    cpd00009_c: 1.0, # Phosphate
    cpd00067_c: 1.0 # H+
})
rxn18658_c.annotation['sbo'] = 'SBO:0000176'
rxn18658_c.annotation['seed.reaction'] = 'rxn18658'
rxn18658_c.annotation['bigg.reaction'] = 'ALAALAabcpp'
iCdG719.add_reactions([rxn18658_c])

cpd00731_e = iCdG719.metabolites.cpd00731_e
iCdG719.add_boundary(cpd00731_e, type='exchange', reaction_id='EX_cpd00731_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00731_e.annotation['sbo'] = 'SBO:0000627'

# Hydrolysis
iCdG719.reactions.rxn07466_c.gene_reaction_rule = '272563.8.peg.1365 or 272563.8.peg.2607 or 272563.8.peg.2736 or 272563.8.peg.2464'


In [209]:
# Phosphorylated hexose sugars
rxn05554_c = copy.deepcopy(universal_GramPos.reactions.rxn05554_c) # Fructose-6-phosphate
rxn05562_c = copy.deepcopy(universal_GramPos.reactions.rxn05562_c) # Glucose-1-phosphate
rxn05563_c = copy.deepcopy(universal_GramPos.reactions.rxn05563_c) # Glucose-6-phosphate
iCdG719.add_reactions([rxn05554_c,rxn05562_c,rxn05563_c])

iCdG719.reactions.rxn05554_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05554_c.annotation['seed.reaction'] = 'rxn05554'
iCdG719.reactions.rxn05554_c.annotation['bigg.reaction'] = 'F6Pt6_i2'

iCdG719.reactions.rxn05562_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05562_c.annotation['seed.reaction'] = 'rxn05562'
iCdG719.reactions.rxn05562_c.annotation['bigg.reaction'] = 'G1Pt6_2'

iCdG719.reactions.rxn05563_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05563_c.annotation['seed.reaction'] = 'rxn05563'
iCdG719.reactions.rxn05563_c.annotation['bigg.reaction'] = 'G6Pt6_2'

cpd00072_e = iCdG719.metabolites.cpd00072_e
iCdG719.add_boundary(cpd00072_e, type='exchange', reaction_id='EX_cpd00072_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00072_e.annotation['sbo'] = 'SBO:0000627'

cpd00089_e = iCdG719.metabolites.cpd00089_e
iCdG719.add_boundary(cpd00089_e, type='exchange', reaction_id='EX_cpd00089_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00089_e.annotation['sbo'] = 'SBO:0000627'

cpd00079_e = iCdG719.metabolites.cpd00079_e
iCdG719.add_boundary(cpd00079_e, type='exchange', reaction_id='EX_cpd00079_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00079_e.annotation['sbo'] = 'SBO:0000627'

# Galactose
rxn05162_c = copy.deepcopy(universal_GramPos.reactions.rxn05162_c)
iCdG719.add_reactions([rxn05162_c])
iCdG719.reactions.rxn05162_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05162_c.annotation['seed.reaction'] = ''
iCdG719.reactions.rxn05162_c.annotation['bigg.reaction'] = ''
cpd00108_e = iCdG719.metabolites.cpd00108_e
iCdG719.add_boundary(cpd00108_e, type='exchange', reaction_id='EX_cpd00108_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.rxn18658_c.annotation['sbo'] = 'SBO:0000627'

In [210]:
# Intracellular organic acids

# 4-Hydroxybenzoate
rxn12549_c = copy.deepcopy(universal_GramPos.reactions.rxn12549_c)
iCdG719.add_reactions([rxn12549_c])
iCdG719.reactions.rxn12549_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn12549_c.annotation['seed.reaction'] = 'rxn12549'
iCdG719.reactions.rxn12549_c.annotation['bigg.reaction'] = '4HBZt'
cpd00136_e = iCdG719.metabolites.cpd00136_e
iCdG719.add_boundary(cpd00136_e, type='exchange', reaction_id='EX_cpd00136_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00136_e.annotation['sbo'] = 'SBO:0000627'

# 2-Hydroxybutyrate
rxn12585_c = copy.deepcopy(universal_GramPos.reactions.rxn12585_c) 
iCdG719.add_reactions([rxn12585_c])
iCdG719.reactions.rxn12585_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn12585_c.annotation['seed.reaction'] = 'rxn12585'
iCdG719.reactions.rxn12585_c.annotation['bigg.reaction'] = '2HBt2'
cpd03561_e = iCdG719.metabolites.cpd03561_e
iCdG719.add_boundary(cpd03561_e, type='exchange', reaction_id='EX_cpd03561_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd03561_e.annotation['sbo'] = 'SBO:0000627'

# Fumarate
rxn05561_c = copy.deepcopy(universal_GramPos.reactions.rxn05561_c) 
iCdG719.add_reactions([rxn05561_c])
iCdG719.reactions.rxn05561_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05561_c.annotation['seed.reaction'] = 'rxn05561'
iCdG719.reactions.rxn05561_c.annotation['bigg.reaction'] = ['FUMt2r','FUMt6','U207']
cpd00106_e = iCdG719.metabolites.cpd00106_e
iCdG719.add_boundary(cpd00106_e, type='exchange', reaction_id='EX_cpd00106_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00106_e.annotation['sbo'] = 'SBO:0000627'

# Acetoacetate
rxn10960_c = copy.deepcopy(universal_GramPos.reactions.rxn10960_c) 
iCdG719.add_reactions([rxn10960_c])
iCdG719.reactions.rxn10960_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn10960_c.annotation['seed.reaction'] = 'rxn10960'
iCdG719.reactions.rxn10960_c.annotation['bigg.reaction'] = ['ACACtex','U126']
cpd00142_e = iCdG719.metabolites.cpd00142_e
iCdG719.add_boundary(cpd00142_e, type='exchange', reaction_id='EX_cpd00142_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00142_e.annotation['sbo'] = 'SBO:0000627'

# Shikimate
rxn10827_c = copy.deepcopy(universal_GramPos.reactions.rxn10827_c) 
iCdG719.add_reactions([rxn10827_c])
iCdG719.reactions.rxn10827_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn10827_c.annotation['seed.reaction'] = 'rxn10827'
iCdG719.reactions.rxn10827_c.annotation['bigg.reaction'] = ['SKMt2','SKMt2pp']
iCdG719.reactions.rxn10827_c.name = 'Shikimate transport in via proton symport'
cpd00383_e = iCdG719.metabolites.cpd00383_e
iCdG719.add_boundary(cpd00383_e, type='exchange', reaction_id='EX_cpd00383_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00383_e.annotation['sbo'] = 'SBO:0000627'

# Formate
rxn05559_c = copy.deepcopy(universal_GramPos.reactions.rxn05559_c) 
iCdG719.add_reactions([rxn05559_c])
iCdG719.reactions.rxn05559_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05559_c.annotation['seed.reaction'] = 'rxn05559'
iCdG719.reactions.rxn05559_c.annotation['bigg.reaction'] = ['FORt2','FORt2pp']
cpd00047_e = iCdG719.metabolites.cpd00047_e
iCdG719.add_boundary(cpd00047_e, type='exchange', reaction_id='EX_cpd00047_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00047_e.annotation['sbo'] = 'SBO:0000627'

# Glycolate
rxn05470_c = copy.deepcopy(universal_GramPos.reactions.rxn05470_c) 
iCdG719.add_reactions([rxn05470_c])
iCdG719.reactions.rxn05470_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05470_c.annotation['seed.reaction'] = 'rxn05470'
iCdG719.reactions.rxn05470_c.annotation['bigg.reaction'] = ['GLYCLTt2r','GLYCLTt2rpp']
cpd00139_e = iCdG719.metabolites.cpd00139_e
iCdG719.add_boundary(cpd00139_e, type='exchange', reaction_id='EX_cpd00139_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00139_e.annotation['sbo'] = 'SBO:0000627'

# GABA
rxn05564_c = copy.deepcopy(universal_GramPos.reactions.rxn05564_c) 
iCdG719.add_reactions([rxn05564_c])
iCdG719.reactions.rxn05564_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05564_c.annotation['seed.reaction'] = 'rxn05564'
iCdG719.reactions.rxn05564_c.annotation['bigg.reaction'] = ['ABUTt2','ABUTt2pp','ABUTt2r','GABAt2','PUT4_4','UGA4_1']
cpd00281_e = iCdG719.metabolites.cpd00281_e
iCdG719.add_boundary(cpd00281_e, type='exchange', reaction_id='EX_cpd00281_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00281_e.annotation['sbo'] = 'SBO:0000627'

In [211]:
# Hydroxylamine
rxn05703_c = copy.deepcopy(universal_GramPos.reactions.rxn05703_c) 
iCdG719.add_reactions([rxn05703_c])
iCdG719.reactions.rxn05703_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05703_c.annotation['seed.reaction'] = 'rxn05703'
cpd00165_e = iCdG719.metabolites.cpd00165_e
iCdG719.add_boundary(cpd00165_e, type='exchange', reaction_id='EX_cpd00165_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00165_e.annotation['sbo'] = 'SBO:0000627'

In [212]:
# Nucleotide analogs

# Deoxyinosine
rxn09687_c = copy.deepcopy(universal_GramPos.reactions.rxn09687_c) 
iCdG719.add_reactions([rxn09687_c])
iCdG719.reactions.rxn09687_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn09687_c.annotation['seed.reaction'] = 'rxn09687'
iCdG719.reactions.rxn09687_c.annotation['bigg.reaction'] = ['DINSt2','DINSt2pp','U180','U196']
cpd03279_e = iCdG719.metabolites.cpd03279_e
iCdG719.add_boundary(cpd03279_e, type='exchange', reaction_id='EX_cpd03279_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd03279_e.annotation['sbo'] = 'SBO:0000627'

# Inosine
rxn05316_c = copy.deepcopy(universal_GramPos.reactions.rxn05316_c) 
iCdG719.add_reactions([rxn05316_c])
iCdG719.reactions.rxn05316_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05316_c.annotation['seed.reaction'] = 'rxn05316'
iCdG719.reactions.rxn05316_c.annotation['bigg.reaction'] = ['INSt2','INSt2pp','INSt2r','INSt2rpp','U177','U194']
cpd00246_e = iCdG719.metabolites.cpd00246_e
iCdG719.add_boundary(cpd00246_e, type='exchange', reaction_id='EX_cpd00246_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00246_e.annotation['sbo'] = 'SBO:0000627'

# Guanosine
rxn05204_c = copy.deepcopy(universal_GramPos.reactions.rxn05204_c) 
iCdG719.add_reactions([rxn05204_c])
iCdG719.reactions.rxn05204_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05204_c.annotation['seed.reaction'] = 'rxn05204'
iCdG719.reactions.rxn05204_c.annotation['bigg.reaction'] = ['GSNt2','GSNt2m','GSNt2pp','GSNt2r','U175','U192']
cpd00311_e = iCdG719.metabolites.cpd00311_e
iCdG719.add_boundary(cpd00311_e, type='exchange', reaction_id='EX_cpd00311_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00311_e.annotation['sbo'] = 'SBO:0000627'

# Cytidine
rxn05198_c = copy.deepcopy(universal_GramPos.reactions.rxn05198_c) 
iCdG719.add_reactions([rxn05198_c])
iCdG719.reactions.rxn05198_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05198_c.annotation['seed.reaction'] = 'rxn05198'
iCdG719.reactions.rxn05198_c.annotation['bigg.reaction'] = ['CYTDt2','CYTDt2pp','CYTDt2r','CYTDt2rpp','U176','U186','U193']
cpd00367_e = iCdG719.metabolites.cpd00367_e
iCdG719.add_boundary(cpd00367_e, type='exchange', reaction_id='EX_cpd00367_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00367_e.annotation['sbo'] = 'SBO:0000627'

# Deoxyguanosine
rxn09688_c = copy.deepcopy(universal_GramPos.reactions.rxn09688_c) 
iCdG719.add_reactions([rxn09688_c])
iCdG719.reactions.rxn09688_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn09688_c.annotation['seed.reaction'] = 'rxn09688'
iCdG719.reactions.rxn09688_c.annotation['bigg.reaction'] = ['DGSNt2','DGSNt2pp','U181','U197']
cpd00277_e = iCdG719.metabolites.cpd00277_e
iCdG719.add_boundary(cpd00277_e, type='exchange', reaction_id='EX_cpd00277_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00277_e.annotation['sbo'] = 'SBO:0000627'

# Adenosine
rxn05318_c = copy.deepcopy(universal_GramPos.reactions.rxn05318_c) 
iCdG719.add_reactions([rxn05318_c])
iCdG719.reactions.rxn05318_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05318_c.annotation['seed.reaction'] = 'rxn05318'
iCdG719.reactions.rxn05318_c.annotation['bigg.reaction'] = ['ADNt2','ADNt2pp','ADNt2r','ADNt2rpp','U174','U185','U191']
cpd00182_e = iCdG719.metabolites.cpd00182_e
iCdG719.add_boundary(cpd00182_e, type='exchange', reaction_id='EX_cpd00182_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00182_e.annotation['sbo'] = 'SBO:0000627'

# Deoxycytidine
rxn05205_c = copy.deepcopy(universal_GramPos.reactions.rxn05205_c) 
iCdG719.add_reactions([rxn05205_c])
iCdG719.reactions.rxn05205_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05205_c.annotation['seed.reaction'] = 'rxn05205'
iCdG719.reactions.rxn05205_c.annotation['bigg.reaction'] = ['DCYTt2','DCYTt2pp','U183','U189','U199']
cpd00654_e = iCdG719.metabolites.cpd00654_e
iCdG719.add_boundary(cpd00654_e, type='exchange', reaction_id='EX_cpd00654_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00654_e.annotation['sbo'] = 'SBO:0000627'

# Deoxyuridine
rxn09685_c = copy.deepcopy(universal_GramPos.reactions.rxn09685_c) 
iCdG719.add_reactions([rxn09685_c])
iCdG719.reactions.rxn09685_c.name = 'Deoxyuridine symport'
iCdG719.reactions.rxn09685_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn09685_c.annotation['seed.reaction'] = 'rxn09685'
iCdG719.reactions.rxn09685_c.annotation['bigg.reaction'] = ['DURIt2','DURIt2pp','U184','U190','U200']
cpd00412_e = iCdG719.metabolites.cpd00412_e
iCdG719.add_boundary(cpd00412_e, type='exchange', reaction_id='EX_cpd00412_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00412_e.annotation['sbo'] = 'SBO:0000627'

# Deoxyadenosine
rxn05317_c = copy.deepcopy(universal_GramPos.reactions.rxn05317_c) 
iCdG719.add_reactions([rxn05317_c])
iCdG719.reactions.rxn05317_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05317_c.annotation['seed.reaction'] = 'rxn05317'
iCdG719.reactions.rxn05317_c.annotation['bigg.reaction'] = ['DADNt2','DADNt2pp','U182','U188','U198']
cpd00438_e = iCdG719.metabolites.cpd00438_e
iCdG719.add_boundary(cpd00438_e, type='exchange', reaction_id='EX_cpd00438_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00438_e.annotation['sbo'] = 'SBO:0000627'

In [213]:
# Additional amino acids

# Citrulline
rxn05674_c = copy.deepcopy(universal_GramPos.reactions.rxn05674_c) 
iCdG719.add_reactions([rxn05674_c])
iCdG719.reactions.rxn05674_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05674_c.annotation['seed.reaction'] = 'rxn05674'
iCdG719.reactions.rxn05674_c.annotation['bigg.reaction'] = ['CITR_Lt2','CITR_Lt2ex','CITR_Lt2pp']
cpd00274_e = iCdG719.metabolites.cpd00274_e
iCdG719.add_boundary(cpd00274_e, type='exchange', reaction_id='EX_cpd00274_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00274_e.annotation['sbo'] = 'SBO:0000627'

# D-Glutamate
rxn05672_c = copy.deepcopy(universal_GramPos.reactions.rxn05672_c) 
iCdG719.add_reactions([rxn05672_c])
iCdG719.reactions.rxn05672_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05672_c.annotation['seed.reaction'] = 'rxn05672'
iCdG719.reactions.rxn05672_c.annotation['bigg.reaction'] = 'GLU-Dt2'
cpd00186_e = iCdG719.metabolites.cpd00186_e
iCdG719.add_boundary(cpd00186_e, type='exchange', reaction_id='EX_cpd00186_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00186_e.annotation['sbo'] = 'SBO:0000627'

#--------------------------------------------------------------------------#

# Cystathionine
cpd00424_e = cobra.Metabolite(
    'cpd00424_e',
    formula='C7H14N2O4S',
    name='Cystathionine',
    compartment='extracellular')
cpd00424_e.annotation['sbo'] = 'SBO:0000247'
cpd00424_e.annotation['seed.compound'] = 'cpd00424'
cpd00424_e.annotation['bigg.metabolite'] = 'CST_c'
cpd00424_e.annotation['kegg.compound'] = 'C00542'
iCdG719.add_boundary(cpd00424_e, type='exchange', reaction_id='EX_cpd00424_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00424_e.annotation['sbo'] = 'SBO:0000627'

cpd00424_c = iCdG719.metabolites.cpd00424_c
rxn10398_c = cobra.Reaction('rxn10398_c')
rxn10398_c.name = 'cystathione diffusion'
rxn10398_c.bounds = (-1000., 1000.)
rxn10398_c.add_metabolites({
    cpd00424_e: -1.0,
    cpd00424_c: 1.0 
})
rxn10398_c.annotation['sbo'] = 'SBO:0000176'
rxn10398_c.annotation['seed.reaction'] = 'rxn10398'
rxn10398_c.annotation['bigg.reaction'] = 'CYSTtp'
iCdG719.add_reactions([rxn10398_c])
iCdG719.reactions.rxn10398_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn10398_c.annotation['seed.reaction'] = 'rxn10398'
iCdG719.reactions.rxn10398_c.annotation['bigg.reaction'] = 'CYSTtp'

# LL-2,6-Diaminopimelate
cpd00504_e = cobra.Metabolite(
    'cpd00504_e',
    formula='C7H14N2O4',
    name='LL-2,6-Diaminopimelate',
    compartment='extracellular')
cpd00504_e.annotation['sbo'] = 'SBO:0000247'
cpd00504_e.annotation['seed.compound'] = 'cpd00504'
cpd00504_e.annotation['bigg.metabolite'] = '26dap_LL'
cpd00504_e.annotation['kegg.compound'] = 'C00666'
iCdG719.add_boundary(cpd00504_e, type='exchange', reaction_id='EX_cpd00504_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00504_e.annotation['sbo'] = 'SBO:0000627'

cpd00001_c = iCdG719.metabolites.cpd00001_c
cpd00002_c = iCdG719.metabolites.cpd00002_c
cpd00008_c = iCdG719.metabolites.cpd00008_c
cpd00009_c = iCdG719.metabolites.cpd00009_c
cpd00067_c = iCdG719.metabolites.cpd00067_c
cpd00504_c = iCdG719.metabolites.cpd00504_c

ID011_c = cobra.Reaction('ID011_c')
ID011_c.name = 'ATP phosphohydrolase (polar-amino-acid-importing)'
ID011_c.gene_reaction_rule = '272563.120.peg.772'
ID011_c.bounds = (0., 1000.)
ID011_c.add_metabolites({
    cpd00001_c: -1.0,
    cpd00002_c: -1.0,
    cpd00504_e: -1.0,
    cpd00504_c: 1.0,
    cpd00008_c: 1.0,
    cpd00009_c: 1.0,
    cpd00067_c: 1.0 
})
iCdG719.add_reactions([ID011_c])
iCdG719.reactions.ID011_c.annotation['sbo'] = 'SBO:0000185'

In [ ]:
# Guanine
rxn05203_c = copy.deepcopy(universal_GramPos.reactions.rxn05203_c) 
rxn05203_c.name = 'Guanine symport'
rxn05203_c.annotation['sbo'] = 'SBO:0000185'
rxn05203_c.annotation['seed.reaction'] = 'rxn05203'
rxn05203_c.annotation['bigg.reaction'] = ['FCY21_3','FCY22_3','FCY2_3','GUAt2','GUAt2pp','GUAt2r','YGL186C_3']

iCdG719.add_reactions([rxn05203_c])
cpd00207_e = iCdG719.metabolites.cpd00207_e
iCdG719.add_boundary(cpd00207_e, type='exchange', reaction_id='EX_cpd00207_e', lb=-1000.0, ub=1000.0)
iCdG719.reactions.EX_cpd00207_e.annotation['sbo'] = 'SBO:0000627'


In [214]:
iCdG719_rmGenes = basicCheck(iCdG719)
iCdG719_noGPRblocked = blockedReactions(iCdG719)
iCdG719_free = checkFreeMass(iCdG719)
iCdG719_massImbal = checkBalance(iCdG719)
iCdG719_nogene = missingGPR(iCdG719)

GENRE is underdetermined
GENRE has 3 compartment(s)
GENRE has 719 genes
18.93 minutes doubling time
34.22% reactions are blocked
0.0% metabolites are generated for free
0.0% reactions are mass imbalanced
15.2% reactions without GPRs
24.23% of reactions used in pFBA solution have no GPR


In [215]:
save_model(iCdG719, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG719', media=mdm)

### Integrate CCDS

In [77]:
iCdG719 = read_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG719.json')

In [3]:
cd630_genes = {}
with open('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/PATRIC/Cd630.PATRIC.genes.format.fasta','r') as fasta:
    for line in fasta:
        cd630_genes[line.split('|')[0]] = line.split('|')[1]

In [4]:
success = 0
for gene in iCdG719.genes:
    try:
        gene.annotation['ccds'] = cd630_genes[gene.id]
        success += 1
    except KeyError:
            pass
print('Genes found: ' + str(success))

Genes found: 653


In [5]:
import copy

# Read in curated universal reacton bag and annotate
universal_GramPos = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/GramPos.sbml')
universal_GramPos.reactions.rxn13782_c.remove_from_model(remove_orphans=True)
universal_GramPos.reactions.rxn13783_c.remove_from_model(remove_orphans=True)
universal_GramPos.reactions.rxn13784_c.remove_from_model(remove_orphans=True)
#universal_GramPos.reactions.rxn11920_c.name = 'Poly-beta-hydroxybutyrate (PHB) synthease'
universal_GramPos.reactions.rxn12188_c.name = '2-deoxyglucose-6-phosphatase'

for cpd in universal_GramPos.metabolites:
    cpd.annotation['sbo'] = 'SBO:0000247'
    cpd.annotation['seed.compound'] = cpd.id.split('_')[0]    
    cpd.compartment = cpd.compartment.lower()
    if cpd.compartment == 'c':
        cpd.compartment = 'cytosol'
    elif cpd.compartment == 'e':
        cpd.compartment = 'extracellular'

for rxn in universal_GramPos.reactions:
    rxn.annotation['sbo'] = 'SBO:0000176'
    rxn.annotation['seed.reaction'] = rxn.id.split('_')[0]


'GramPositive.modeltemplate' is not a valid SBML 'SId'.
No objective coefficients in model. Unclear what should be optimized


In [81]:
# Fix bounds
#iCdR700.reactions.rxn00926_c.bounds = (0.,1000.) # tough...
iCdG719.reactions.rxn01021_c.bounds = (0.,1000.)
iCdG719.reactions.rxn00143_c.bounds = (0.,1000.)

# Adenine synthesis
rxn00131_c = copy.deepcopy(universal_GramPos.reactions.rxn00131_c) 
iCdG719.add_reactions([rxn00131_c])
iCdG719.reactions.rxn00131_c.bounds = (0., 1000.)
iCdG719.reactions.rxn00131_c.annotation['sbo'] = 'SBO:0000176'
iCdG719.reactions.rxn00131_c.annotation['seed.reaction'] = 'rxn00131'
iCdG719.reactions.rxn00131_c.annotation['bigg.reaction'] = 'AMP-NUCLEOSID-RXN'
iCdG719.reactions.rxn00131_c.annotation['kegg.reaction'] = 'R00182'

In [82]:
# Fix last genes
geneDict = {'645463.3.peg.3332':'272563.8.peg.3431', '645463.3.peg.3330':'272563.8.peg.3429', '645463.3.peg.3333':'272563.8.peg.3432', 
            '645463.3.peg.467':'272563.8.peg.518', '645463.3.peg.1000':'272563.8.peg.1130', '645463.3.peg.303':'272563.8.peg.304', 
            '645463.3.peg.304':'272563.8.peg.305', '645463.3.peg.466':'272563.8.peg.517', '645463.3.peg.998':'272563.8.peg.1128', 
            '645463.3.peg.465':'272563.8.peg.516', '645463.3.peg.1002':'272563.8.peg.1132', '645463.3.peg.306':'272563.8.peg.307', 
            '645463.3.peg.1001':'272563.8.peg.1131', '645463.3.peg.308':'272563.8.peg.309', '645463.3.peg.464':'272563.8.peg.515', 
            '645463.3.peg.307':'272563.8.peg.308', '645463.3.peg.3331':'272563.8.peg.3430', '645463.3.peg.2802':'272563.8.peg.2882',
            '645463.3.peg.3106':'272563.8.peg.3245'}

cobra.manipulation.modify.rename_genes(iCdG719, geneDict)

In [83]:
# Include missing genes

str630_only = ['272563.8.peg.1','272563.8.peg.2','272563.8.peg.3','272563.8.peg.4','272563.8.peg.5',
               '272563.8.peg.6','272563.8.peg.7','272563.8.peg.8','272563.8.peg.220','272563.8.peg.229','272563.8.peg.230','272563.8.peg.233','272563.8.peg.259','272563.8.peg.260','272563.8.peg.261','272563.8.peg.262','272563.8.peg.352','272563.8.peg.376','272563.8.peg.377','272563.8.peg.392','272563.8.peg.400','272563.8.peg.402','272563.8.peg.407','272563.8.peg.427','272563.8.peg.429','272563.8.peg.449','272563.8.peg.450','272563.8.peg.451','272563.8.peg.456','272563.8.peg.458','272563.8.peg.464','272563.8.peg.476','272563.8.peg.477','272563.8.peg.530','272563.8.peg.536','272563.8.peg.595','272563.8.peg.605','272563.8.peg.611','272563.8.peg.615','272563.8.peg.633','272563.8.peg.638','272563.8.peg.650','272563.8.peg.651','272563.8.peg.662','272563.8.peg.679','272563.8.peg.685','272563.8.peg.713','272563.8.peg.895','272563.8.peg.944','272563.8.peg.945','272563.8.peg.946','272563.8.peg.949','272563.8.peg.950','272563.8.peg.951','272563.8.peg.953','272563.8.peg.954','272563.8.peg.955','272563.8.peg.956','272563.8.peg.957','272563.8.peg.958','272563.8.peg.959','272563.8.peg.960','272563.8.peg.961','272563.8.peg.962','272563.8.peg.963','272563.8.peg.964','272563.8.peg.965','272563.8.peg.966','272563.8.peg.967','272563.8.peg.968','272563.8.peg.969','272563.8.peg.970','272563.8.peg.971','272563.8.peg.973','272563.8.peg.974','272563.8.peg.975','272563.8.peg.976','272563.8.peg.977','272563.8.peg.978','272563.8.peg.980','272563.8.peg.981','272563.8.peg.982','272563.8.peg.984','272563.8.peg.985','272563.8.peg.986','272563.8.peg.987','272563.8.peg.988','272563.8.peg.993','272563.8.peg.1007','272563.8.peg.1008','272563.8.peg.1009','272563.8.peg.1021','272563.8.peg.1025','272563.8.peg.1026','272563.8.peg.1027','272563.8.peg.1057','272563.8.peg.1146','272563.8.peg.1154','272563.8.peg.1157','272563.8.peg.1158','272563.8.peg.1159','272563.8.peg.1161','272563.8.peg.1162','272563.8.peg.1163','272563.8.peg.1164','272563.8.peg.1165','272563.8.peg.1166','272563.8.peg.1168','272563.8.peg.1172','272563.8.peg.1300','272563.8.peg.1380','272563.8.peg.1571','272563.8.peg.1622','272563.8.peg.1733','272563.8.peg.1798','272563.8.peg.1827','272563.8.peg.1828','272563.8.peg.1854','272563.8.peg.1910','272563.8.peg.1937','272563.8.peg.1952','272563.8.peg.1953','272563.8.peg.1954','272563.8.peg.1979','272563.8.peg.1980','272563.8.peg.1983','272563.8.peg.1998','272563.8.peg.2080','272563.8.peg.2100','272563.8.peg.2101','272563.8.peg.2103','272563.8.peg.2110','272563.8.peg.2114','272563.8.peg.2116','272563.8.peg.2141','272563.8.peg.2144','272563.8.peg.2146','272563.8.peg.2297','272563.8.peg.2335','272563.8.peg.2336','272563.8.peg.2337','272563.8.peg.2340','272563.8.peg.2399','272563.8.peg.2402','272563.8.peg.2407','272563.8.peg.2409','272563.8.peg.2414','272563.8.peg.2418','272563.8.peg.2640','272563.8.peg.2650','272563.8.peg.2651','272563.8.peg.2729','272563.8.peg.2730','272563.8.peg.2731','272563.8.peg.2732','272563.8.peg.2752','272563.8.peg.2790','272563.8.peg.2793','272563.8.peg.2849','272563.8.peg.2915','272563.8.peg.3020','272563.8.peg.3021','272563.8.peg.3022','272563.8.peg.3026','272563.8.peg.3038','272563.8.peg.3039','272563.8.peg.3040','272563.8.peg.3054','272563.8.peg.3059','272563.8.peg.3060','272563.8.peg.3061','272563.8.peg.3062','272563.8.peg.3063','272563.8.peg.3066','272563.8.peg.3068','272563.8.peg.3069','272563.8.peg.3070','272563.8.peg.3071','272563.8.peg.3072','272563.8.peg.3073','272563.8.peg.3076','272563.8.peg.3077','272563.8.peg.3078','272563.8.peg.3079','272563.8.peg.3080','272563.8.peg.3081','272563.8.peg.3082','272563.8.peg.3084','272563.8.peg.3086','272563.8.peg.3087','272563.8.peg.3088','272563.8.peg.3089','272563.8.peg.3090','272563.8.peg.3091','272563.8.peg.3092','272563.8.peg.3096','272563.8.peg.3161','272563.8.peg.3200','272563.8.peg.3265','272563.8.peg.3289','272563.8.peg.3291','272563.8.peg.3292','272563.8.peg.3293','272563.8.peg.3294','272563.8.peg.3299','272563.8.peg.3399','272563.8.peg.3455','272563.8.peg.3457','272563.8.peg.3481','272563.8.peg.3485','272563.8.peg.3487','272563.8.peg.3488','272563.8.peg.3494','272563.8.peg.3496','272563.8.peg.3497','272563.8.peg.3500','272563.8.peg.3504','272563.8.peg.3505','272563.8.peg.3531','272563.8.peg.3538','272563.8.peg.3542','272563.8.peg.3548','272563.8.peg.3550','272563.8.peg.3808','272563.8.peg.3851']

cd630_PATRIC = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/PATRIC/272563.8.xml')
for cpd in cd630_PATRIC.metabolites:
    if cpd.compartment == 'c':
        cpd.compartment = 'cytosol'
    elif cpd.compartment == 'e':
        cpd.compartment = 'extracellular'

new_rxns = []
for gene in str630_only:
    try:
        rxns = [x.id for x in cd630_PATRIC.genes.get_by_id(gene).reactions]
        for x in rxns:
            try:
                rxn = iCdG719.reactions.get_by_id(x)
            except:
                new_rxns.append(x)
                pass
    except:
        pass
    
print(new_rxns)

'272563.8' is not a valid SBML 'SId'.


['rxn05547_c', 'rxn05540_c', 'rxn05535_c', 'rxn05543_c', 'rxn05537_c', 'rxn05539_c', 'rxn05542_c', 'rxn05533_c', 'rxn05544_c']


In [84]:
# Add missing reactions

rxn05542_c = copy.deepcopy(cd630_PATRIC.reactions.rxn05542_c) 
iCdG719.add_reactions([rxn05542_c])
iCdG719.reactions.rxn05542_c.bounds = (0., 1000.)
iCdG719.reactions.rxn05542_c.annotation['sbo'] = 'SBO:0000185' # transport
iCdG719.reactions.rxn05542_c.annotation['seed.reaction'] = 'rxn05542'
iCdG719.reactions.rxn05542_c.annotation['bigg.reaction'] = 'DIPEPabc4'

cpd11585_e = iCdG719.metabolites.cpd11585_e
iCdG719.add_boundary(cpd11585_e, type='exchange', reaction_id='EX_cpd11585_e', lb=0.0, ub=1000.0)
iCdG719.reactions.EX_cpd11585_e.name = 'Ala-Gly exchange'
iCdG719.reactions.EX_cpd11585_e.annotation['sbo'] = 'SBO:0000627'

rxn12641_c = copy.deepcopy(cd630_PATRIC.reactions.rxn12641_c) 
iCdG719.add_reactions([rxn12641_c])
iCdG719.reactions.rxn12641_c.name = 'Ala-Gly aminopeptidase'
iCdG719.reactions.rxn12641_c.bounds = (0., 1000.)
iCdG719.reactions.rxn12641_c.annotation['sbo'] = 'SBO:0000176' # metabolic
iCdG719.reactions.rxn12641_c.annotation['seed.reaction'] = 'rxn12641'
iCdG719.reactions.rxn12641_c.annotation['bigg.reaction'] = 'MEAMP1(ala-gly)'

#-------------------------------------------------------------------------#

rxn05547_c = copy.deepcopy(cd630_PATRIC.reactions.rxn05547_c) 
iCdG719.add_reactions([rxn05547_c])
iCdG719.reactions.rxn05547_c.name = 'oligopeptide-transporting ATPase'
iCdG719.reactions.rxn05547_c.bounds = (0., 1000.)
iCdG719.reactions.rxn05547_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05547_c.annotation['seed.reaction'] = 'rxn05547'
iCdG719.reactions.rxn05547_c.annotation['bigg.reaction'] = 'DIPEPabc9'

cpd11580_e = iCdG719.metabolites.cpd11580_e
iCdG719.add_boundary(cpd11580_e, type='exchange', reaction_id='EX_cpd11580_e', lb=0.0, ub=1000.0)
iCdG719.reactions.EX_cpd11580_e.name = 'Gly-Gln exchange'
iCdG719.reactions.EX_cpd11580_e.annotation['sbo'] = 'SBO:0000627'

rxn12646_c = copy.deepcopy(cd630_PATRIC.reactions.rxn12646_c) 
iCdG719.add_reactions([rxn12646_c])
iCdG719.reactions.rxn12646_c.name = 'Gly-Gln aminopeptidase'
iCdG719.reactions.rxn12646_c.bounds = (0., 1000.)
iCdG719.reactions.rxn12646_c.annotation['sbo'] = 'SBO:0000176'
iCdG719.reactions.rxn12646_c.annotation['seed.reaction'] = 'rxn12646'
iCdG719.reactions.rxn12646_c.annotation['bigg.reaction'] = 'DIPEPabc2'

#-------------------------------------------------------------------------#

rxn05540_c = copy.deepcopy(cd630_PATRIC.reactions.rxn05540_c) 
iCdG719.add_reactions([rxn05540_c])
iCdG719.reactions.rxn05540_c.bounds = (0., 1000.)
iCdG719.reactions.rxn05540_c.name = 'oligopeptide-transporting ATPase'
iCdG719.reactions.rxn05540_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05540_c.annotation['seed.reaction'] = 'rxn05540'
iCdG719.reactions.rxn05540_c.annotation['bigg.reaction'] = 'DIPEPabc2'

cpd11587_e = iCdG719.metabolites.cpd11587_e
iCdG719.add_boundary(cpd11587_e, type='exchange', reaction_id='EX_cpd11587_e', lb=0.0, ub=1000.0)
iCdG719.reactions.EX_cpd11587_e.name = 'Ala-Gln exchange'
iCdG719.reactions.EX_cpd11587_e.annotation['sbo'] = 'SBO:0000627'

rxn12639_c = copy.deepcopy(cd630_PATRIC.reactions.rxn12639_c) 
iCdG719.add_reactions([rxn12639_c])
iCdG719.reactions.rxn12639_c.name = 'Ala-Gln aminopeptidase'
iCdG719.reactions.rxn12639_c.bounds = (0., 1000.)
iCdG719.reactions.rxn12639_c.annotation['sbo'] = 'SBO:0000176'
iCdG719.reactions.rxn12639_c.annotation['seed.reaction'] = 'rxn12639'
iCdG719.reactions.rxn12639_c.annotation['bigg.reaction'] = 'MEAMP1(ala-gln)'

#-------------------------------------------------------------------------#

rxn05537_c = copy.deepcopy(cd630_PATRIC.reactions.rxn05537_c) 
iCdG719.add_reactions([rxn05537_c])
iCdG719.reactions.rxn05537_c.bounds = (0., 1000.)
iCdG719.reactions.rxn05537_c.name = 'oligopeptide-transporting ATPase'
iCdG719.reactions.rxn05537_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05537_c.annotation['seed.reaction'] = 'rxn05537'
iCdG719.reactions.rxn05537_c.annotation['bigg.reaction'] = 'DIPEPabc13'

cpd11589_e = iCdG719.metabolites.cpd11589_e
iCdG719.add_boundary(cpd11589_e, type='exchange', reaction_id='EX_cpd11589_e', lb=0.0, ub=1000.0)
iCdG719.reactions.EX_cpd11589_e.name = 'Gly-Asp exchange'
iCdG719.reactions.EX_cpd11589_e.annotation['sbo'] = 'SBO:0000627'

rxn12637_c = copy.deepcopy(cd630_PATRIC.reactions.rxn12637_c) 
iCdG719.add_reactions([rxn12637_c])
iCdG719.reactions.rxn12637_c.name = 'Gly-Asp aminopeptidase'
iCdG719.reactions.rxn12637_c.bounds = (0., 1000.)
iCdG719.reactions.rxn12637_c.annotation['sbo'] = 'SBO:0000176'
iCdG719.reactions.rxn12637_c.annotation['seed.reaction'] = 'rxn12637'
iCdG719.reactions.rxn12637_c.annotation['bigg.reaction'] = ['AMPEP13','MEAMP1(gly-asp)']

#-------------------------------------------------------------------------#

rxn05543_c = copy.deepcopy(cd630_PATRIC.reactions.rxn05543_c) 
iCdG719.add_reactions([rxn05543_c])
iCdG719.reactions.rxn05543_c.bounds = (0., 1000.)
iCdG719.reactions.rxn05543_c.name = 'oligopeptide-transporting ATPase'
iCdG719.reactions.rxn05543_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05543_c.annotation['seed.reaction'] = 'rxn05543'
iCdG719.reactions.rxn05543_c.annotation['bigg.reaction'] = 'DIPEPabc5'

cpd11584_e = iCdG719.metabolites.cpd11584_e
iCdG719.add_boundary(cpd11584_e, type='exchange', reaction_id='EX_cpd11584_e', lb=0.0, ub=1000.0)
iCdG719.reactions.EX_cpd11584_e.name = 'Ala-His exchange'
iCdG719.reactions.EX_cpd11584_e.annotation['sbo'] = 'SBO:0000627'

rxn12642_c = copy.deepcopy(cd630_PATRIC.reactions.rxn12642_c) 
iCdG719.add_reactions([rxn12642_c])
iCdG719.reactions.rxn12642_c.name = 'Ala-His aminopeptidase'
iCdG719.reactions.rxn12642_c.bounds = (0., 1000.)
iCdG719.reactions.rxn12642_c.annotation['sbo'] = 'SBO:0000176' # metabolic
iCdG719.reactions.rxn12642_c.annotation['seed.reaction'] = 'rxn12642'
iCdG719.reactions.rxn12642_c.annotation['bigg.reaction'] = 'MEAMP1(ala-his)'

#-------------------------------------------------------------------------#

rxn05539_c = copy.deepcopy(cd630_PATRIC.reactions.rxn05539_c) 
iCdG719.add_reactions([rxn05539_c])
iCdG719.reactions.rxn05539_c.bounds = (0., 1000.)
iCdG719.reactions.rxn05539_c.name = 'oligopeptide-transporting ATPase'
iCdG719.reactions.rxn05539_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05539_c.annotation['seed.reaction'] = 'rxn05539'
iCdG719.reactions.rxn05539_c.annotation['bigg.reaction'] = ['CGLYabcpp','DIPEPabc15']

cpd01017_e = iCdG719.metabolites.cpd01017_e
iCdG719.add_boundary(cpd01017_e, type='exchange', reaction_id='EX_cpd01017_e', lb=0.0, ub=1000.0)
iCdG719.reactions.EX_cpd01017_e.name = 'Cys-Gly exchange'
iCdG719.reactions.EX_cpd01017_e.annotation['sbo'] = 'SBO:0000627'

rxn00650_c = copy.deepcopy(cd630_PATRIC.reactions.rxn00650_c) 
iCdG719.add_reactions([rxn00650_c])
iCdG719.reactions.rxn00650_c.name = 'Cys-Gly aminopeptidase'
iCdG719.reactions.rxn00650_c.bounds = (0., 1000.)
iCdG719.reactions.rxn00650_c.annotation['sbo'] = 'SBO:0000176' # metabolic
iCdG719.reactions.rxn00650_c.annotation['seed.reaction'] = 'rxn00650'
iCdG719.reactions.rxn00650_c.annotation['bigg.reaction'] = ['AMPTASECG','TRIA']
iCdG719.reactions.rxn00650_c.annotation['kegg.reaction'] = 'R00899'

rxn00350_c = copy.deepcopy(cd630_PATRIC.reactions.rxn00350_c) 
iCdG719.add_reactions([rxn00350_c])
iCdG719.reactions.rxn00350_c.bounds = (0., 1000.)
iCdG719.reactions.rxn00350_c.annotation['sbo'] = 'SBO:0000176' # metabolic
iCdG719.reactions.rxn00350_c.annotation['seed.reaction'] = 'rxn00350'
iCdG719.reactions.rxn00350_c.annotation['bigg.reaction'] = ['GGTAe2','GTHRDHpp']
iCdG719.reactions.rxn00350_c.annotation['kegg.reaction'] = 'R00494'

#-------------------------------------------------------------------------#

rxn05535_c = copy.deepcopy(cd630_PATRIC.reactions.rxn05535_c) 
iCdG719.add_reactions([rxn05535_c])
iCdG719.reactions.rxn05535_c.bounds = (0., 1000.)
iCdG719.reactions.rxn05535_c.name = 'oligopeptide-transporting ATPase'
iCdG719.reactions.rxn05535_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05535_c.annotation['seed.reaction'] = 'rxn05535'
iCdG719.reactions.rxn05535_c.annotation['bigg.reaction'] = 'DIPEPabc11'

cpd11591_e = iCdG719.metabolites.cpd11591_e
iCdG719.add_boundary(cpd11591_e, type='exchange', reaction_id='EX_cpd11591_e', lb=0.0, ub=1000.0)
iCdG719.reactions.EX_cpd11591_e.name = 'Gly-Met exchange'
iCdG719.reactions.EX_cpd11591_e.annotation['sbo'] = 'SBO:0000627'

rxn12635_c = copy.deepcopy(cd630_PATRIC.reactions.rxn12635_c) 
iCdG719.add_reactions([rxn12635_c])
iCdG719.reactions.rxn12635_c.name = 'Gly-Met aminopeptidase'
iCdG719.reactions.rxn12635_c.bounds = (0., 1000.)
iCdG719.reactions.rxn12635_c.annotation['sbo'] = 'SBO:0000176' # metabolic
iCdG719.reactions.rxn12635_c.annotation['seed.reaction'] = 'rxn12635'
iCdG719.reactions.rxn12635_c.annotation['bigg.reaction'] = 'MEAMP1(gly-met)'

#-------------------------------------------------------------------------#

rxn05544_c = copy.deepcopy(cd630_PATRIC.reactions.rxn05544_c) 
iCdG719.add_reactions([rxn05544_c])
iCdG719.reactions.rxn05544_c.bounds = (0., 1000.)
iCdG719.reactions.rxn05544_c.name = 'oligopeptide-transporting ATPase'
iCdG719.reactions.rxn05544_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05544_c.annotation['seed.reaction'] = 'rxn05544'
iCdG719.reactions.rxn05544_c.annotation['bigg.reaction'] = 'DIPEPabc6'

cpd11583_e = iCdG719.metabolites.cpd11583_e
iCdG719.add_boundary(cpd11583_e, type='exchange', reaction_id='EX_cpd11583_e', lb=0.0, ub=1000.0)
iCdG719.reactions.EX_cpd11583_e.name = 'Ala-Leu exchange'
iCdG719.reactions.EX_cpd11583_e.annotation['sbo'] = 'SBO:0000627'

rxn12643_c = copy.deepcopy(cd630_PATRIC.reactions.rxn12643_c) 
iCdG719.add_reactions([rxn12643_c])
iCdG719.reactions.rxn12643_c.name = 'Ala-Leu aminopeptidase'
iCdG719.reactions.rxn12643_c.bounds = (0., 1000.)
iCdG719.reactions.rxn12643_c.annotation['sbo'] = 'SBO:0000176' # metabolic
iCdG719.reactions.rxn12643_c.annotation['seed.reaction'] = 'rxn12643'
iCdG719.reactions.rxn12643_c.annotation['bigg.reaction'] = 'MEAMP1(ala-leu)'

#-------------------------------------------------------------------------#

rxn05533_c = copy.deepcopy(cd630_PATRIC.reactions.rxn05533_c) 
iCdG719.add_reactions([rxn05533_c])
iCdG719.reactions.rxn05533_c.bounds = (0., 1000.)
iCdG719.reactions.rxn05533_c.name = 'oligopeptide-transporting ATPase'
iCdG719.reactions.rxn05533_c.annotation['sbo'] = 'SBO:0000185'
iCdG719.reactions.rxn05533_c.annotation['seed.reaction'] = 'rxn05533'
iCdG719.reactions.rxn05533_c.annotation['bigg.reaction'] = 'DIPEPabc1'

cpd11593_e = iCdG719.metabolites.cpd11593_e
iCdG719.add_boundary(cpd11593_e, type='exchange', reaction_id='EX_cpd11593_e', lb=0.0, ub=1000.0)
iCdG719.reactions.EX_cpd11593_e.name = 'Ala-Asp exchange'
iCdG719.reactions.EX_cpd11593_e.annotation['sbo'] = 'SBO:0000627'

rxn12633_c = copy.deepcopy(cd630_PATRIC.reactions.rxn12633_c) 
iCdG719.add_reactions([rxn12633_c])
iCdG719.reactions.rxn12633_c.name = 'Ala-Asp aminopeptidase'
iCdG719.reactions.rxn12633_c.bounds = (0., 1000.)
iCdG719.reactions.rxn12633_c.annotation['sbo'] = 'SBO:0000176' # metabolic
iCdG719.reactions.rxn12633_c.annotation['seed.reaction'] = 'rxn12633'
iCdG719.reactions.rxn12633_c.annotation['bigg.reaction'] = 'MEAMP1(ala-asp)'


In [87]:
iCdG719.id = 'iCdG' + str(len(iCdG719.genes))

In [88]:
iCdG719

Name,iCdG700
Memory address,0x07f3aff278550
Number of metabolites,1235
Number of reactions,1308
Number of groups,0
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"cytosol, extracellular, periplasm"


In [93]:
mdm = ['cpd00001_e', # water
       'cpd00065_e', # L-Tryptophan
       'cpd00060_e', # L-Methionine
       'cpd00322_e', # L-Isoleucine
       'cpd00129_e', # L-Proline
       'cpd00156_e', # L-Valine
       'cpd00107_e', # L-Leucine
       'cpd00084_e', # L-Cysteine 
       'cpd00149_e', # Cobalt
       'cpd00099_e', # Chloride
       'cpd10515_e', # Iron
       'cpd00030_e', # Manganese
       'cpd00254_e', # Magnesium
       'cpd00063_e', # Calcium
       'cpd00205_e', # Potassium
       'cpd00009_e', # Phosphate
       'cpd00971_e', # Sodium
       'cpd00242_e', # Carbonate
       'cpd00104_e', # Biotin
       'cpd00644_e', # Pantothenate
       'cpd00263_e', # Pyridoxine
       'cpd00027_e'] # D-Glucose (Carbohydrate C-source)
mdm = ['EX_' + x for x in mdm]
for rxn in iCdG719.boundary:
    rxn.bounds = (-1000., 1000.)
    if not rxn.id in mdm:
        rxn.bounds = (0., 1000.)

In [94]:
iCdG719.slim_optimize()

80.96607010298041

In [95]:
save_model(iCdG719, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG700')

In [5]:
iCdG700 = read_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG700.json')

In [3]:
iCdG700_rmGenes = basicCheck(iCdG700)
iCdG700_noGPRblocked = blockedReactions(iCdG700)
iCdG700_free = checkFreeMass(iCdG700)
iCdG700_massImbal = checkBalance(iCdG700)
iCdG700_nogene = missingGPR(iCdG700)

GENRE is underdetermined
GENRE has 3 compartment(s)
GENRE has 700 genes
18.93 minutes doubling time
33.49% reactions are blocked
0.0% metabolites are generated for free
0.0% reactions are mass imbalanced
15.1% reactions without GPRs
25.66% of reactions used in pFBA solution have no GPR


In [7]:
for cpd in iCdG700.metabolites: cpd.annotation['sbo'] = 'SBO:0000247'
iCdG700.reactions.rxn18658_c.annotation['sbo'] = 'SBO:0000185'

In [16]:
cd630_genes = {}
with open('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/PATRIC/Cd630.PATRIC.genes.format.fasta','r') as fasta:
    for line in fasta:
        cd630_genes[line.split('|')[0]] = line.split('|')[1]
        
pepto630_genes = {}
with open('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/PATRIC/Pepto630.PATRIC.genes.format.fasta','r') as fasta:
    for line in fasta:
        pepto630_genes[line.split('|')[0]] = line.split('|')[1]        
        

In [13]:
success = 0
failed = []
for gene in iCdG700.genes:
    try:
        gene.annotation['ccds'] = cd630_genes[gene.id]
        success += 1
    except KeyError:
        failed.append(gene.id)
        pass
print('Genes found: ' + str(success))

Genes found: 653


In [17]:
success = 0
failed = []
for gene in iCdG700.genes:
    try:
        gene.annotation['ccds'] = pepto630_genes[gene.id]
        success += 1
    except KeyError:
        failed.append(gene.id)
        pass
print('Genes found: ' + str(success))

Genes found: 47


In [21]:
iCdG700.metabolites.cpd12255_c_c.id = 'cpd12255_c'
iCdG700.reactions.rxn00139_c.bounds = (-1000., 1000.)

In [22]:
cobra.io.write_sbml_model(iCdG700, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG700.sbml')
cobra.io.save_json_model(iCdG700, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG700.json')

In [123]:
iCdG700 = read_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG700.json')

In [1]:
# Create Peptoclostridium draft GENRE
import cobra
import mackinac
mackinac.modelseed.ms_client.url = 'http://p3.theseed.org/services/ProbModelSEED/'
mackinac.workspace.ws_client.url = 'http://p3.theseed.org/services/Workspace'
mackinac.genome.patric_url = 'https://www.patricbrc.org/api/'

# PATRIC user information
mackinac.get_token('mljenior')
# password: matrix54

patric password: ········


'mljenior@patricbrc.org'

In [62]:
genome_id = '272563.8'
template_id = '/chenry/public/modelsupport/templates/GramPosModelTemplate'
media_id = '/chenry/public/modelsupport/media/Complete'
file_id = '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/PATRIC/Cd630.draft.json'
strain_id = 'Clostridium difficile 630'

mackinac.reconstruct_modelseed_model(genome_id, template_reference=template_id)
mackinac.gapfill_modelseed_model(genome_id, media_reference=media_id)
mackinac.optimize_modelseed_model(genome_id)
patric_draft = mackinac.create_cobra_model_from_modelseed_model(genome_id)
patric_draft.id = strain_id

cobra.io.save_json_model(patric_draft, file_id)

In [69]:
patric_draft

Name,Clostridium difficile 630
Memory address,0x07f651b295350
Number of metabolites,1235
Number of reactions,1136
Number of groups,0
Objective expression,1.0*bio1 - 1.0*bio1_reverse_b18f7
Compartments,"c, e"


In [77]:
draft_genes = set([x.id for x in patric_draft.genes])
current_genes = set([y.id for y in iCdG700.genes])

new_genes = draft_genes.difference(current_genes)

current_rxns = set([z.id for z in iCdG700.reactions])
save_genes = []
for x in new_genes:
    rxns = set([i.id for i in patric_draft.genes.get_by_id(x).reactions])
    
    if len(rxns.intersection(current_rxns)) > 0:
        save_genes.append(x)
save_genes = set(save_genes)
print(str(len(save_genes)) + ' genes')
print(save_genes)

7 genes
{'272563.8.peg.1781', '272563.8.peg.3769', '272563.8.peg.795', '272563.8.peg.1176', '272563.8.peg.2251', '272563.8.peg.1782', '272563.8.peg.3436'}


In [125]:
# Integrate to existing reactions
iCdG700.reactions.rxn00157_c.gene_reaction_rule = '( ( 272563.8.peg.3435 or 272563.8.peg.1175 or 272563.8.peg.796 ) or ( 272563.8.peg.795 or 272563.8.peg.1176 or 272563.8.peg.3436 ) )'
iCdG700.genes.get_by_id('272563.8.peg.795').name = 'Pyruvate formate-lyase activating enzyme (EC 1.97.1.4)'
iCdG700.genes.get_by_id('272563.8.peg.1176').name = 'Pyruvate formate-lyase activating enzyme (EC 1.97.1.4)'
iCdG700.genes.get_by_id('272563.8.peg.3436').name = 'Pyruvate formate-lyase activating enzyme (EC 1.97.1.4)'

iCdG700.reactions.rxn00119_c.gene_reaction_rule += ' or 272563.8.peg.2251'
iCdG700.genes.get_by_id('272563.8.peg.2251').name = 'Uridylate kinase (EC 2.7.4.-)'

iCdG700.reactions.rxn09310_c.gene_reaction_rule = '272563.8.peg.1781 and 272563.8.peg.1782'
iCdG700.genes.get_by_id('272563.8.peg.1781').name = 'Sulfur carrier protein ThiS subunit'
iCdG700.genes.get_by_id('272563.8.peg.1782').name = 'Sulfur carrier protein ThiS subunit'

iCdG700.reactions.rxn00693_c.gene_reaction_rule += ' or 272563.8.peg.3769'
iCdG700.genes.get_by_id('272563.8.peg.3769').name = 'Methionine synthase activation domain (EC 2.1.1.13)'


In [126]:
uniprot_gene_ids = {}
ec_gene_ids = {}
refseq_gene_ids = {}
kegg_gene_ids = {}
patric_genes = set()

with open('/home/mjenior/Desktop/repos/uniprot_Cd630.tab', 'r') as gene_annotation:
    header = gene_annotation.readline()
    for line in gene_annotation:
        line = line.split()
        uniprot_gene_ids[line[4]] = line[0]
        ec_gene_ids[line[4]] = line[1]
        refseq_gene_ids[line[4]] = line[2]
        kegg_gene_ids[line[4]] = line[3]
        patric_genes |= set([line[4]])

cd630_genes = {}
with open('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/PATRIC/Cd630.PATRIC.genes.format.fasta','r') as fasta:
    for line in fasta:
        cd630_genes[line.split('|')[0]] = line.split('|')[1]

In [127]:
# Correct labels and annotation data
for gene in iCdG700.genes:
    gene.annotation['sbo'] = 'SBO:0000243'
    
    gene.annotation['patric'] = 'fig|' + gene.id
    try:
        gene.annotation['uniprot'] = uniprot_gene_ids[gene.id]
    except:
        pass
    try:
        gene.annotation['kegg.genes'] = kegg_gene_ids[gene.id]
    except:
        pass
    try:
        gene.annotation['refseq'] = refseq_gene_ids[gene.id]
    except:
        pass
    try:
        gene.annotation['ec_number'] = ec_gene_ids[gene.id]
    except:
        pass
    try:
        gene.annotation['ccds'] = cd630_genes[gene.id]
    except:
        pass
    
for rxn in patric_draft.reactions:
    rxn.annotation['sbo'] = 'SBO:0000185'
    
for cpd in patric_draft.metabolites:
    cpd.annotation['sbo'] = 'SBO:0000247'
    
    if cpd.compartment == 'c':
        cpd.compartment = 'cytosol'
    elif cpd.compartment == 'e':
        cpd.compartment = 'extracellular'
    

In [128]:
pepto_only = []
pepto_dict = {}
with open('/home/mjenior/Desktop/pepto630_vs_cd630.patric_id.tsv', 'r') as alignment:
    header = alignment.readline()
    for line in alignment:
        line = line.split()
        if line[1] == 'none':
            pepto_only.append(line[0])
        else:
            pepto_dict[line[0]] = line[1]
            

In [129]:
iCdG700_peptogenes = []
for x in iCdG700.genes:
    if '272563.120.' in x.id:
        iCdG700_peptogenes.append(x.id)
iCdG700_peptogenes = set(iCdG700_peptogenes)

replace_genes = {}
for x in iCdG700_peptogenes:
    replace_genes[x] = pepto_dict[x]
    
cobra.manipulation.modify.rename_genes(iCdG700, replace_genes)

In [131]:
iCdG700.id = 'iCdG' + str(len(iCdG700.genes))

In [132]:
iCdG700

Name,iCdG692
Memory address,0x07f64ce6c61d0
Number of metabolites,1235
Number of reactions,1308
Number of groups,0
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"cytosol, extracellular, periplasm"


In [133]:
iCdG700_rmGenes = basicCheck(iCdG700)
iCdG700_noGPRblocked = blockedReactions(iCdG700)
iCdG700_free = checkFreeMass(iCdG700)
iCdG700_massImbal = checkBalance(iCdG700)
iCdG700_nogene = missingGPR(iCdG700)

GENRE is underdetermined
GENRE has 3 compartment(s)
GENRE has 692 genes
18.48 minutes doubling time
33.49% reactions are blocked
0.0% metabolites are generated for free
0.0% reactions are mass imbalanced
14.92% reactions without GPRs
24.63% of reactions used in pFBA solution have no GPR


In [135]:
mdm = ['cpd00001_e', # water
       'cpd00065_e', # L-Tryptophan
       'cpd00060_e', # L-Methionine
       'cpd00322_e', # L-Isoleucine
       'cpd00129_e', # L-Proline
       'cpd00156_e', # L-Valine
       'cpd00107_e', # L-Leucine
       'cpd00084_e', # L-Cysteine 
       'cpd00149_e', # Cobalt
       'cpd00099_e', # Chloride
       'cpd10515_e', # Iron
       'cpd00030_e', # Manganese
       'cpd00254_e', # Magnesium
       'cpd00063_e', # Calcium
       'cpd00205_e', # Potassium
       'cpd00009_e', # Phosphate
       'cpd00971_e', # Sodium
       'cpd00242_e', # Carbonate
       'cpd00104_e', # Biotin
       'cpd00644_e', # Pantothenate
       'cpd00263_e', # Pyridoxine
       'cpd00027_e'] # D-Glucose (Carbohydrate C-source)
mdm = ['EX_' + x for x in mdm]
for rxn in iCdG700.boundary:
    rxn.bounds = (-1000., 1000.)
    if not rxn.id in mdm:
        rxn.bounds = (0., 1000.)

In [137]:
cobra.io.write_sbml_model(iCdG700, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG692.sbml')
cobra.io.save_json_model(iCdG700, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG692.json')